<a href="https://colab.research.google.com/github/rickrari/Creating-a-Custom-spaCy-NER-Model-to-recognize-lowercase-path-/blob/main/PATH_NER_DataCollection_Codebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Background**

The Port Authority Trans Hudson (PATH) is a subway system connecting Manhattan to Hudson and Essex Counties in New Jersey. The Hudson & Manhattan Railroad (H&M), a private company, built and began operating the system in 1908. Like most privately owned passenger rail systems in the US, competition from private automobiles and rising infrastructure costs pushed H&M into bankruptcy in the 1950s. The governors of New York and New Jersey prevented the railroad from falling apart by having the Port Authority (PA) take it over in 1962 as part of a deal that let the PA build the World Trade Center ([Citizens Budget Committee - CBC, 2014](https://cbcny.org/sites/default/files/REPORT_PATH_04242014.pdf)).

The PA is an independent authority responsible for multiple forms infrastructure including; the region’s airports, several bridges and, and most of the region’s ports. The PA does not receive tax revenue and instead funds its infrastructure from user fees. The PA uses surplus user fees from net revenue positive infrastructure, like the airports or the tunnels and bridges, to pay for net revenue negative infrastructure.

Like almost all subway systems, PATH requires more money to run than it can take in from fares or other sources. However, unlike any other subways or metros in the US, PATH does not receive local tax support, instead PATH is funded entirely by the PA. PATH costs loses about $400 million a year, limiting the PA’s ability to invest in other forms of infrastructure. (NYT, 2019). Discussion over how to solve PATH’s funding problems have gone on for decades, with many organizations suggesting that PATH benefit from local taxes ([CBC, 2014](https://www.nytimes.com/2019/01/02/nyregion/path-train-nj-nyc.html)). In 2014 the PA itself suggested PATH should receive some local tax funding ([Port Authority, 2014](https://www.governor.ny.gov/sites/default/files/atoms/files/SpecialPanelReporttotheGovernors2014-12-26FINAL.pdf)). If PATH is ever to receive local tax support, PA leaders need to better understand how the public and regional leaders view the agency.

PATH serves a region with multiple newspapers that report on development and local policies. Analyzing text from these newspapers can help PATH better understand how leaders in the region view the agency, identify potential allies, and look for communities with which it needs to improve relations.

However, most textual analysis in python requires lemmatizing (the process of breaking words down to their root) and standardizing all text as lowercase ([Krisel, Topic Modeling, 2024](https://github.com/rskrisel/topic_modeling_workshop/blob/main/topic_modeling_tomotopy_workshop.ipynb)). This presents a problem for analyzing data related to PATH, because while python can recognize PATH as an organization when it is written in capital letters, it cannot recognize ‘path’ (written in lowercase letters). This paper will demonstrate this issue and show how to solve it by training a custom Named Entity Recognition (NER) model using spaCy, an open-source software library for advanced natural language processing.


**Data Collection**

This paper collected data from Factiva, a Dow Jones owned tool which aggregates data from thousands of media publications. The author had access to Factiva through his affiliation with Columbia University.

To find articles that mention PATH specifically, the author searched “Port Authority Trans Hudson” and limited results to articles from the New York Times and The Bergen Record – two leading newspapers that focus on the New York-New Jersey region that PATH serves. This search returned 273 results from January 1980 through December 2024. Searching for just “PATH” from these sources returns 112,191 results, most of which have nothing to do with the PATH subway, providing further evidence for a custom tool which can distinguish between PATH and the common word path.

The author collected the first 100 articles from the search results by setting the display option to “Full Article/Report plus indexing” and the duplicates option to “identical”. Then from the print view window the author clicked control+u to view and copy the html code for all 100 articles (following the technique outlined in [Krisel, Factiva 2024](https://github.com/rskrisel/factiva_dataframe/blob/main/Create_spreadsheet_Factiva_data.ipynb)). The rest of the coding in this Colab codebook is also adapted from the Krisel Factiva resource unless otherwise noted.

In [1]:
import glob # used to find all the file paths that match a specified pattern
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
path = '/content/drive/MyDrive/PythonCourse/FinalProject' # replace with your path

In [ ]:
pwd

In [ ]:
mkdir $path/factiva

After collecting the HTML data from Factiva using the above outlined technique, the author then saved the contents of the article’s html code as the variable “html_code” using the below code. Note, the data of the 100 articles is extensive and may take several seconds to scroll through.

In [6]:
html_code= """

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html>
<head>
	<meta http-equiv="Content-Type" content="text/html; charset=UTF-8"/>
	<title>Factiva</title>
<script>window.ddjskey='D428D51E28968797BC27FB9153435D';window.ddoptions={};</script><script type='text/javascript' src='/datadome/tags.js' async></script>
<link rel="stylesheet" type="text/css" media="all" href="/css/ui.dotcom1390300ui4sr.ashx" />
<link rel="stylesheet" type="text/css" media="all" href="/css/DotComHeadlines1390300ui4sr.ashx" />
<link rel="stylesheet" type="text/css" media="all" href="/css/fcp/print1390300ui4sr.ashx" />

<script type="text/javascript" src="/gen/modernizr.underscore0390300ui4sr.ashx"></script>
<script type="text/javascript" src="/jquery/jquery.bundle0390300ui4sr.ashx"></script>
<script type="text/javascript" src="/xlib/x_combined0390300ui4sr.ashx"></script>
<script type="text/javascript" src="/gen/RecordGenericOD0390300ui4sr.ashx"></script>
<script type="text/javascript" src="/gen/ui.dotcom0390300ui4sr.ashx"></script>
<script type="text/javascript" src="/controls/search/jquery.overlay.1.30390300ui4sr.ashx"></script>

</head>
<body class=''><a id="skip-main" class="skip-main"  href="#PageBaseForm">Skip to main content</a>
<form name="LinkForm" id="LinkForm" method="post"><input type="hidden" id="_XFORMSTATE" name="_XFORMSTATE" value="" /><input type="hidden" id="_XFORMSESSSTATE" name="_XFORMSESSSTATE" value="" /><div id="LinkFormExElem"></div></form>
<script type="text/javascript" src="../gen/funcTwo.js"></script>
<div id="navcontainer" class="fcpNavContainer">
<table cellpadding="0" cellspacing="0" border="0" width="100%">
<tr>
<td class="factivalogo"><h1>Dow Jones Factiva</h1></td>
<td class="djrlogo" align="right"><span>Dow Jones</span></td>
</tr>
</table>
</div>
<form name="PageBaseForm" method="post" action="/hp/printsavews.aspx?ppstype=Article&amp;pp=Print&amp;hc=All" id="PageBaseForm">
<div>
<input type="hidden" name="PageScriptManager_HiddenField" id="PageScriptManager_HiddenField" value="" />
<input type="hidden" name="_XFORMSESSSTATE" id="_XFORMSESSSTATE" value="H4sIAAAAAAAEAHWSW3eaQBDHv0rOPi/JzLLAsn2pKGpt4qVUE3M5HkDOiakKUTRthe/eWfQhL2WBw8785/ZbTqBPU81uXoubYrfalvv4mH3sr+N98ZtxVJ4+oWZxplBmIrZAoLRkooSlQDqW7ae+n6KXLJOUcVezbMt4E4Jmh26zowqOBg7GXX7s3wouKLVmy3KTbzl8CcJvgwdOAqNmAMpltfF70u04bUeFQTdAEK7XchQoP2iFbREKCG2wJXTRB68L5AlaAAiVoMep4NNyKgeq/y0jFSBVhUBVpWbpYW1ts93XNF8fNskqvs6WB8YdzUJ3NBL9WXvSn427vckgkvNHbzTvTCeMq2ZKpGFrLnyzeToJzXxmpl7mZZpvzMiifqm5TywQGyCkQzIjjSs1+lw4jcn0QeHA7UZFcg2U1704hWe8yC/BgixKM0J5ZQGBhgr5VU4HIexLAJJ7mJdRGe/KbEmN2Jq11mtSSKM452+Q2yFIFwBCAk3ECahSqqJTBxDw+UI0r8u3ufHMk5n5WMPusF4lVIKapHI4jVoGBjSMewboE0Mp2QuxAxOzSTb0AxAHduqFQ32KfupnNh+3Yewf7nGOr7BfiO4Rtsc//veweBy+/z283a02SZHOcaKSOz+fRen96jZYPtxO80TE6/608Hbpr9UAxwsR/IjUwu5U4pnVNRWCc0kLz23V9T+SojjdCgMAAA==" />
<input type="hidden" name="_XFORMSTATE" id="_XFORMSTATE" value="H4sIAAAAAAAEAK1cbXPaypL+K1OuW5UPS3Il8e582JJB2MQYcSU5rGtrq3ZAAyiARPRih5zkv+/TI+FI+JgV6J46SQDbPc/M9PTzdE/Lf6nXf+H/euNaramNa6XWub7ylysvioPQm/NNJHg4X/GXaHdV615fDR5H1lVNU6+vnF+7IIyZ84sn8QrfG++Z+esT3sch96Ps9Spxo8DP3vAw9uYbgVehyH524cn3L2K240t6NdsES/yzTTaxtxWux/GztTd/MLY3j5NQyHfjX4ttjFdzaZn+9vfxIvtSGKVfMn/9h/yThyFHT+1liNI3Bzhvh8oAFkeFWSxJE2tXxx+1TSuJF61rFZ806V39ultTm9f//Vf6+RVjg00ShnsWLFiPVm4ahGu2CEKmu3wbMd1zxQes4cZlAy/cXtXIZvP6yg39a9oO71l88sVLlE3l0/jJGSj4T1O0htLVVLHubvC2/l3uGa3GVU1VCYkGANr1lfCvAPYv5fpKT01c/a7hzft2ftf+YB+LF/ZEgL2IhYK7e4l8HvhLEcUetnsHz/H8JcHWTsG2jJ7yOlxLbYl1a0nvv0nY2MpyqN83k0c9MhzHsGzmmMy5M5jRHzqm9ZndcJctuBevmOezZfAsQn8r/Jiw10tirysdTRHfOmt6H12M/chMHvvjbhlyF0vKJrpzx+x9FAvpFY1yXqF2FVVrioW6o91sX+wVx3byGAlZjT36uzBYeDGf4Whx38UHrgj5bocV8Xgs3BqzhfdTRGwYR+whOKx1s+RE2kpHaQi30yAAm8sncmQnPxGddf+pqox+csHnMbuDi0csDiRi8v07oGY8Zpw5YbDZiD17SCKRbMmDeoHviznGTOS0WiWn1QScpph1mnjb6l4+rSM7hf3ZcJ9mYWBTTHJzb7mKM38S4bM3F2wYriNmIfaEEYFvlwav0qAqDdrsVAFfsFMAzz2XjQP8nLhmfcHjVfSZ3Zqj/o1h3dZYXx8PjREbmY9Dm5B3SiJvKE21I3jTo/fzy5Ef2ckjJ6exEBd5AlrEa2O7Q7zXsQn0FSfkHsjSjjn8CGfjLpgFa+Fjk+ZB4sfYCHwtCLcgg6E/R7CNKAg48DqaZrfkNDVVQ2TpzFd4q/LLp3lkpxBd6ayDzUbSqxz+AyecPsLhed4zexXsdhkjgBRLo1ZptC6N5lVCnbeTR+2QVCHi55K3zMUCxAXNw/RoTUd5GYookux24202+JaIzrjcEWzqnNhiz4wFvkGedrUkSRMkpQ1Ibbxt/6g0tbyd/NTugo0kjAGHAMKxh5b4iNcrTEK4bMr38oirJ/k5P5Kiqk3RURd0NiuclCM7xbgrt8OLwcnhEucCcitMNvuUQpYhDoQrMZ/k5dxYGEqpi9asjrd8fTnmIzt5zAO4Rl88e3CZAbDXXg+0EFtadanrBp4vJPCSbE3UhHPWaJASmK0uBn5spxiW2BTUy272CCxgshvo0WvJBgg1qqJADzEmQZdkZgymaS3R0L4ToyZVQBfs5EE/cH/F45j71/CJjbf1oCc+El+xHg/B0bQbE/4Hekn2xZAqjlFDpWNUv9xRju3koVvc2zDkVDu2FIj7MU6loHjJvVAmHpGYJzKH+swcsAUkdADJt8WPAG9EakMydZRFqnR+Jwn6j7JUWvirIzStQxFQXCpQj80U+JnmoL8mgjseRSCAeCXYLJmvGRAjPYiQVAp/vj9sz0mWLoyranWhqVtaVqUC/IKZPHwp+BZ86208wI4SQWd3xTcxe6EDDPh/s/on2Tc3bhMBpC1UTiFPvRz+kZkCjSVQc4j1chpWQgFo6GMG/DWi3iWzGptSluMEyXyFScL37mSSf9gOrSQvK02ljRxAbcsc4Oflp+XITiHf+WR/kpRL+4CjEwmSFOwGAmidHvNesN0mUh89gIVB2Cuo9GwiJVkYACg8qhQexSKsMpGCncKxD3hIO2P7wUskZQOkEOEd48AAMk663KmpR2IPofc25NEum0dJbsb45Niq6hM/LavMo2CnwM3bOORrqegoA/qChUfac/CuvtjwPVTFDB+9BGmse/CiFd9F2VRKUjZBgDxQpTyoa5WmkreTn8p0JUIhQxPIeePSJIhABG0AG4jNpoZpIpaFHEKPXBDi3HcPEylL4Q0k812hdGJKxPaXT+TITlEvpclbFpQcUCNUnx/gTFAgqMndMncC0WAKjoQ2CYJsFmU5vSE9QpEe0VpUmUXBTmEWB6UnddMkiXNbA2VS2J0UfFlWr6uq1nB/zgJ6W0GQHNnJgzdvhs6jbg0NO4VWlpBhUm3DJCVSde9iSjgyk0d282gPx4ZtsxugG2TwyhJuXWnWFfdnMyG7rcvhFc2cWriyXFpHqK27PxvkSI24ArKCmTyyL4ZlG09sYJrOaHh756QA6yfJkdzq1bKqdt2f6jfKb86Nxe/bySO8F2InS4EiRJzllC6lFVicFWgtSjmIF4f+c7AhqnkIEO8QHXz2JQHf/CtJq7QRFIDNFyLeywmeJM0cMA2noeXuZyTPteDyCR7ZOa4kskkYfBPzmIQ+CNOlvJsCw1dEZL6U6VT9JD/mh1JamuLuW5QGaaIC5KKdN5D1qW71bfYP5IkP+KTnwIssNnV6zNKHo8yRTjIhfFl5HQyZp7vXkIsp3TM1+/tm8pipjOGlSjAfZWvQJi/MTlxX+Ei+zZkXJ5y+UcI/yX+5tVKVLoLmjy4Fzayef9GaH9l5s+b243B8m+pFWmzLmJiWw8wx6xvjoT5i5gARUL83PjqG7bCpPvxqWOlGnOTA3AqqVJZ3X5prqnntLt6IIzOFUs3QdkzriQ3H7Ml8tNiN3rtnT/AlKsxi4a9Zz3wcO31zOqbbCxhT0imcZMKjsbvucweL+r1x5uXK+2byU+jpYzY12MQybMP6agCwOTEs3RliJ0xa8hvMUXfwHdge+z9T+CfZ8u24jY2iuN+6VeEfzJxIXyfBhsrhE1n9u/E4BdPviUf3dVkmOOFxGGzSI3GSVQuu3NE67nMdgES9XuVIFOwU5jEcmQ67H45GRp/pNpuM9LHB+nB6G94F36FAlLpWz9LtO5yZ0cic2swxHMO6MS2TOTgs5mCQ7tBJWn67tCo+UP2w6g4dzBRkYhIjW/Xm7CbYsyGV1BOfyj9pUjug1H3P9AWpROTA/ay82ShL2wgzasdNZtCoO3Fuhfx9O4W9oZzpWch0cBpsQcd3YoMEaRAKgViMVGoJ8JC8H7oKe0hCF+96PJJU1yjLztn43FfUbaNVeR4HO4WS56enT8NPElVZAs6sQQ0q/o9zb7fet1PI6jht/QOSCGYlO7qvj2qsFwrkRlhvnd0lS6yy569XwUbQMR54CziO/iz8JF3ik8T8FoRK1WBR6RgX7BTEeygvgW44ZByg9kP+Mgs9FzPQIeVuNnybCqIsK++FPFrJOZRmZ1VRG2402ylq0H2uMIeinb+pGPQFErsN3fLKmxMx58k8gSbF1FZw9m+0UYwyfaA/ych/M2qAKPFjVx19ZudNAWqkP9lpuO8b+Fu2Dtw9PoDqDMsyUxnRKM3BGKzpRm0B39XOTPneN1OojhsO4nd/eAulI6Gd5Ne360CEMvvRrLyeBztvtNoYfpveoPc9TAnyU5byZLlolN2RNEqzaW6wpf/vAQ07hT6XL8yx9LE9dNiD/gRZRvIL4qxn1tho+K9HuIDeT2V9ozRTqgrSnnDuKWrbu1zWH5l5s9SynMLukSxFTGMPCCaUFlIoDNI8SuaKWVGrWbby21LaShejImS11MblFZUjO0X0EJHmxLShWKBMSDk+9m+NNJX60G2x6dC5Mx8d7MRoBOV8b9g1Rppz2IPqfMzS9eZJvvybdaRT5Vc6lXkz+QkZIaeWi/QKpygz9YipLeU1YafYPqI+B3OxyDamfAW4gSwv7C7xzo8qVbLzdt641UCHwL8GnT6YfcqnhuOeZei2wYY2cisLh8WB8JTQy+a52dJ1EkWJm2dWdN4380bce3TJKS/ur2UnjE8rfiiFyBNBfXgfJDN51HrIBons+ktnU7bs21SayPIkDKV7eSvMsZ3CZQ+wys2Y8vAzu/WSjcd9D8h7fLORdSAOPQzkWcMI5THBMwQFPExKTCoLfxXeBhrCWZFCOoSBslVh8pI6wIUk0ysUVo/sHLcYZpcMhzsH2XGoh8tEyPYleS0ky2DFQ0W9MYcJla0UH4B8UxTuXt4fc2ynoJrBeGnPmJ3MDveIvU1A16Raup9EgQfkJVuv5IiaG7YxYtCqePBzdt7q/eySsFmys+qwFC1PUXaLClRxZKcg+U3rnho6qfZgPxpZA2LPfLQcZj2OstJy8wxmrivu99lCURdJUCUQ5c0UiiRBAle1RCDvZPpiG6ReQGdUj9NG4B41FC6zsNMqfyXbAKHKQTv7bbXVztkpNIyMvzxaCO5p6MmWHoJYlnzG4Gejb1j6CLsBOpDgz+BgWrH6Fqcm9qsu/MHMiduG1klaLZgEobs73lbU7qwKsoKZgjr4r8nItKlKpjuyLdmZDkllQviAZynUU6x79sTLZ3Yz0kG5d0NoHMjSPuQO1t2wauzGfLjBCbAnRg/0+5k1swpQjZFUsvUxROpgZBjMfjDvDfY6QOZk5e9myTl2GsL1gl/eQ3xs59RGnaTdoxXuuME8UbRm9LPSRuXMvFXVnFh0JCJ6pmEQBtvX5kwZySUDZxG8dQaZqnWMuVbUrbi88/fYTiEVH8BNpHdRnfnh2K+mhn5vjKmMLr/MRoZO9cKxaeFHyOVIciPxvTVYX3/Q8U86wzPYVSITirpxL292PLbz9sYouztOL7WgomWH7YRakUj3RMTBUdqVfQa90pCzlqIKz60M/WCnqE5fIGZGQRSlRUBkCnI2fxpc0tU+g3hpqPYK7+bViDdv59QpLc+w8ni1EKH33cqn9GDmBLL2Sf58a7IBk0J8r4rsYOa4UTQVKKb/0aH8gvqHQ9kz3VWoODynGlkK+4wGJtqj+kJRwk6lTqyCneNsgzpMZLMina7XHsyDh2aYz0hV/4xVoTHmyE4Bs5iv6GEBEX0+9JXI5ph+4H+ImY2siL4hhX0G+dFwmq8ofqsa+eXtnHLeM8hPVVx/vlXUeFalhlEwU9B+PjP20gOMeeAHW2+eEl9NesRRAnYD75D6NnjJOiXb53Bh0/VnOD8vUaXsuWCnkFjyrdiIKHusgtkvfLtLG1Vt4Bcb5GAZ6HPojQZTEY4aFQk8Z+dvm3kmhjkZpfK6Xfr2NN3XFj6Ivi2qusfBzP8L7yRfvWeX/3vgFVPvQ5fiHxftUfSCWiB90AuiOH3aqeTTNIeNqnepsUKtvOEHO4XWDMcyDMfu6RMjSwPu0gdpnWQGBSo1wypIIvGZanH8G5Wx5jWmL6kgcuvxA5N0yieQTa3hbmcedWpVqoAU7BS2wRyNPlqm3ke+YlN513aQpbApEhqjxh50OA605bhH8W9imQ+mY7D/hQIdWsycjj+k8ymfU6pY2207wWFau5W8KmemkLn9EPMkBoQ/lxlxgNyAP7+JiCn0cwiy625bLeSxi4pMk7NTKETIGsnj5M4c9emGS+6NnRVN+hb1yKRllTtjNGGDx3GW9iPtHI5TduqUr+/KJWyCp7erRtWdOJg50bzRDwNEdPm0o7xdouZxz8/qglRMpOJopqo75eu6DbXlbhtYzNa6omrJ2SmcjzsqYbE7HUs/0p/YABEgBVm6Uwn/tdzNnCvqvDGrtNQ5MwX2/PTlE9X670zz3uhTm1VveKtb9GT161VYp/yNqBxntlRU3k2qwj2YKXZI091iGHCXPXFo7bT1W6fHCtIrMJKGViIfi7XE1vO9CCp8nimAzhlZoqZKBMpyX0225O0UCCHZhQJJQ1ZCpEfhkt1nquy7yTz+OPL4zNuQ91vJhjhgJBZx2rhO72Z72frpzV/9/oxskiDxZ0WZv1TLMPJ2Cn7PvRec1YlH95FPgS/vwpBb7LiftrXayeyFE2kfcuHOGUytwTW6oaJ0GufeEb9vp3AidMsyp+wrsZWOuOlkHQNg8OF9KoW6Z2Sh5MlNoSjeukKfW9HMmzoJxUDDh1QQtNIUGpF5ckll+lzmpCnscx6jaWI8AGj5lR4/KdgpHORNLEKfslC4PvI6Zgt5pWc8U9KMvCR7EiL01tnh7Z5Bt3LUpqI0fl7+lPGxnROJXfcc6qSNbGAjl8+V/SEzcwrZOSknTNLzHPXnykxzMJNH1qDuONnklF4802PlNfnYfA8ZXMgjdgsX6IXC9TKt2z0jy9Ta7nqGl5tWxR3P2SmesydqtSSVq/e/4t8+XZUYBx3VH9qTRyeLDufQZcdd81BRm2u10prnzBQeVjP0W7pLc4xJ1i76Fu05WSeGaXNFCb+d+dDj+2byaA9UcRsGyY6esXajA5fI354QcpkNmf6BQG68Q2Q7h/2ww3VXUdtxtWu1vJ03zwWD2qRgpYcBX+dlCwFC/Id6+NUD9PsI4pB+IUk6ifNIcK2uQL2zavWJvJ03tAIxaNEDM5Ql9O7Z2JgihbDsQ4+IvF/I+nTSXwNxDi123G+dQNEWs3VFV3o18/t/ahr9ViX6lVdqF3/VW/SK/tLoM02hz7RrraZ26AsKXmnyqzQikNs3GEJVGzSo9vv37/8D0xTaYUlLAAA=" />
<input type="hidden" name="__VIEWSTATE" id="__VIEWSTATE" value="" />
</div>

<script type="text/javascript">
<!--
if((typeof RootNewWindowDirectory == 'undefined') && (!RootNewWindowDirectory)){RootNewWindowDirectory = 'https://global.factiva.com/en';}
isPostProcessing = true;
languageCode = 'en';
logOmniture = true;
// -->
</script><script type="text/javascript">
<!--
function translate(t) {
var r={"yes":"Yes","no":"No","ok":"OK","djheaderaWNPLikeLink":"${djheaderaWNPLikeLink}","djheaderDontShowMe":"Don\'t show me this again","djheaderFirstView":"${djheaderFirstView}","djheaderLoremipsum":"Factiva is powered by the most comprehensive collection of business information in the world.","djheaderGetStarted":"Get Started","djheaderTryitlater":"Try it later","snapShotPrd":"Homepage","snapShotNavBeta":"${snapShotNavBeta}","nameIsRequired":"Please enter a 1-25 character name.","snapShotBeta":"Homepage","tryItNow":"Try It Now!","whatsNewPopUpTitle":"What\'s New","invalidEmail":"You have entered an invalid e-mail address.","profValidEmail":"Please enter a valid e-mail address.","sSun":"Sun","sMon":"Mon","sTue":"Tue","sWed":"Wed","sThu":"Thu","sFri":"Fri","sSat":"Sat","sunday":"Sunday","monday":"Monday","tuesday":"Tuesday","wednesday":"Wednesday","thursday":"Thursday","friday":"Friday","saturday":"Saturday","sJan":"Jan","sFeb":"Feb","sMar":"Mar","sApr":"Apr","sMay":"May","sJun":"Jun","sJul":"Jul","sAug":"Aug","sSep":"Sep","sOct":"Oct","sNov":"Nov","sDec":"Dec","january":"January","february":"February","march":"March","april":"April","may":"May","june":"June","july":"July","august":"August","september":"September","october":"October","november":"November","december":"December","smallAm":"${smallAm}","smallPm":"${smallPm}","capitalAm":"${capitalAm}","capitalPm":"${capitalPm}"};
return (r && t && r[t] || r[t] === '')? r[t] : t || null;
}
// -->
</script>
<script src="/ScriptResource.axd?d=na9ThF35vypYMd7Jk_rxYO38HjXffKotc0wvG_70TpcLVKfZBZPIM8kH05SebEymwP8VMdB5A4PWnvSTWMpmgkzkPMRWw3LUebC7IY1-RzB_CjMRRSx8x83UUIIagDyIsgi9fBsYewhw-mD8wu8dSyqNoNY1&amp;t=7a0cc936" type="text/javascript"></script><link href="/WebResource.axd?d=NhJjFR_XCMs18rNVWoDInluTIB_uBACnN1nBI-8xV6GiPce6R6t2x0Iz5ENz8Px7hAME9tFr3xqyYeeLelCj5XdG2swm5QVvr04cmRtyfAQYQSvyeyzPxiKMkscYlRJ3KlT37g2&amp;t=638484482820000000" type="text/css" rel="stylesheet" /><div id="contentWrapper"><div id="contentLeft" class="carryOverOpen"><span></span><div id="article-NYTF000020240921ek9l0003q" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>National Desk; SECTA</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>Flurry of City Work for Adams Aide's Old Firm</span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By Amy Julia Harris, William K. Rashbaum, Dana Rubinstein and Bianca Pallaro </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>1778 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>21 September 2024</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>1</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 2024 The New York Times Company.  All Rights Reserved. </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >The company received a $154 million contract to provide ''emergency fire watch services'' to the <span class="companylink">New York City Housing Authority</span>. The firm was once owned by the deputy mayor for public safety.</p>
<p class="articleParagraph enarticleParagraph" >Before Philip B. Banks III was named deputy mayor for public safety for New York City, the security company he once owned rarely did business with the city.</p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >But two years after Mayor Eric Adams appointed Mr. Banks to the high-ranking post in his administration, the company Mr. Banks said he had sold years earlier began receiving city business worth millions of dollars, according to records reviewed by <span class="companylink">The New York Times</span>.</p>
<p class="articleParagraph enarticleParagraph" >The firm, City Safe Partners, received a $154 million contract from the <span class="companylink">New York City Housing Authority</span> in January 2024 to provide ''emergency fire watch services'' in Brooklyn, Manhattan and the Bronx, records show. Sheena Wright, the first deputy mayor in the Adams administration and the fiancÃ©e of Mr. Banks's brother, the schools chancellor, sits on the housing authority's board and voted to approve the emergency contract, records show.</p>
<p class="articleParagraph enarticleParagraph" >Mr. Banks's business dealings have been under scrutiny at least since his phones were seized this month by federal agents investigating a possible bribery scheme involving city contracts. The phones of Mr. Banks's brothers -- David Banks, the schools chancellor, and Terence Banks, a consultant with clients who received city contracts -- were also taken as part of the corruption inquiry.</p>
<p class="articleParagraph enarticleParagraph" >The investigation involving Philip Banks and his brothers is one of at least four separate federal inquiries focused on members of the Adams administration -- inquiries that have rocked City Hall and raised questions about Mr. Adams's political future. It was not clear whether City Safe Partners was a focus of any of the investigations.</p>
<p class="articleParagraph enarticleParagraph" >The company's fortunes, however, seemed to have run in parallel with the political fortunes of Mr. Banks and another top Adams aide who was once briefly involved with the firm.</p>
<p class="articleParagraph enarticleParagraph" >A spokesman for the U.S. attorney's office for the Southern District of New York, which is conducting three of the four investigations, including the one that resulted in the phone seizures, declined to comment. So did a spokeswoman for the city's Department of Investigation, which is assisting in the inquiries.</p>
<p class="articleParagraph enarticleParagraph" >Mr. Banks bought an interest in the security firm in 2015, a City Hall spokesman told The Times last year, and he was listed as a member of the firm's management team on an archived version of its website as recently as 2018.</p>
<p class="articleParagraph enarticleParagraph" >Had Mr. Banks retained any financial interest in the company, its city business would likely have raised ethical and legal concerns. But a representative of City Safe Partners said Mr. Banks sold 100 percent of his interest in the firm to its current owner on July 9, 2018.</p>
<p class="articleParagraph enarticleParagraph" >
                     Benjamin Brafman, a lawyer for Mr. Banks, said his client had long had nothing to do with City Safe, which was originally formed as Overwatch Services LLC.</p>
<p class="articleParagraph enarticleParagraph" >''Banks was with Overwatch for less than two years and was long gone before the contracts you reference were ever awarded,'' Mr. Brafman said.</p>
<p class="articleParagraph enarticleParagraph" >On Friday, a spokeswoman for the mayor's office, Liz Garcia, said that Mr. Banks had divested from the company before joining the administration, but she did not say when he did so or provide documents reflecting the divestment. She said he had had no involvement in its operation in years.</p>
<p class="articleParagraph enarticleParagraph" >A lawyer for Sheena Wright, Xavier R. Donaldson, declined to comment.</p>
<p class="articleParagraph enarticleParagraph" >The security company was founded in 2010 by Dwayne Montgomery, a retired police inspector who worked and socialized with Mr. Adams at the Police Department. Mr. Montgomery was charged last year with conspiring to funnel illegal donations to Mr. Adams's 2021 mayoral campaign; Mr. Montgomery pleaded guilty to misdemeanor conspiracy in February and was sentenced to 200 hours of community service.</p>
<p class="articleParagraph enarticleParagraph" >Mr. Banks bought the security firm with a partner, a former police sergeant named Soyini Chan-Shue, from Mr. Montgomery months after Mr. Banks abruptly retired from the Police Department amid a federal corruption probe. Mr. Banks, who had been the chief of department, the highest-ranking uniformed official on the force, was labeled an unindicted co-conspirator in that wide-ranging corruption investigation, which resulted in several convictions, including that of a Police Department chief who served as his top aide, as well as Mr. Banks's close friend, the head of the correction officers' union. Mr. Banks was never charged with any crime.</p>
<p class="articleParagraph enarticleParagraph" >After Mr. Banks joined the security firm, it began using the name City Safe Partners, records show.</p>
<p class="articleParagraph enarticleParagraph" >On its website, the company prominently featured Mr. Banks's credentials with the Police Department.</p>
<p class="articleParagraph enarticleParagraph" >''Philip Banks III is one of the highest-profile security and law enforcement leaders in America,'' the website said. ''His extensive executive-level security and law enforcement experience spans 28 years culminating in his role as the highest-ranking uniformed position in the <span class="companylink">New York City Police Department</span>.''</p>
<p class="articleParagraph enarticleParagraph" >Winnie Greco, who would become Mr. Adams's special adviser and director of Asian affairs, was also prominently displayed on the company's website as a member of the management team in 2018, according to an archived version of the site.</p>
<p class="articleParagraph enarticleParagraph" >Ms. Greco, a major fund-raiser for Mr. Adams during his mayoral campaign, has close ties to New York City's Chinese community. Her homes were searched by agents in February as part of a separate federal investigation.</p>
<p class="articleParagraph enarticleParagraph" >Ms. Greco was engaged by City Safe to generate new business by providing ''training of Chinese security staff,'' but no business ever materialized and she was paid no money by the firm, the company representative said.</p>
<p class="articleParagraph enarticleParagraph" >A lawyer for Ms. Greco, Steven Brill, said his client had no substantive role with the company.</p>
<p class="articleParagraph enarticleParagraph" >''Winnie has no connection to this company,'' he said.</p>
<p class="articleParagraph enarticleParagraph" >City Safe is currently helmed by Ms. Chan-Shue, the former sergeant with the Police Department, who long worked under Mr. Banks during his time there. After more than two decades at the department, Ms. Chan-Shue retired and then became the chief executive of City Safe in 2016, according to her <span class="companylink">LinkedIn</span> profile.</p>
<p class="articleParagraph enarticleParagraph" >Since it was launched, the security firm has advertised its status as a minority-owned business -- a designation that allowed it to qualify for certain government business under programs aimed at giving contracts to firms owned by women and members of minority groups, records show.</p>
<p class="articleParagraph enarticleParagraph" >Before Mr. Adams took office, City Safe did some business with the city, records show. It was paid $4,773 by the Department of Parks & Recreation in 2014 to provide security guards for a summer solstice event at an office building in Harlem, according to a department spokeswoman.</p>
<p class="articleParagraph enarticleParagraph" >It also had some business with the state.</p>
<p class="articleParagraph enarticleParagraph" >In 2017, the company started receiving a portion of a contract with the <span class="companylink">Metropolitan Transportation Authority</span>, a state-run agency, to provide security guards to patrol subway stations as part of a push to improve safety. Another aim of the contract was to combat fare evasion -- which would become a priority for Mr. Adams once he took office and installed Mr. Banks as an influential public safety aide. The company continued as a subcontractor until April 2024, according to the authority. They were paid $7.2 million, the City Safe representative said.</p>
<p class="articleParagraph enarticleParagraph" >After Mr. Adams took office, the company's business with the city and other government entities appeared to take off, records show.</p>
<p class="articleParagraph enarticleParagraph" >In 2022, City Safe Partners was hired as a subcontractor to help provide vetting and identification cards to staff members at Kennedy and Newark Liberty Airports, the company representative said. The subcontract was worth $10 million, according to records the firm provided to the city. Both airports are managed not by city officials but by the state-run <span class="companylink">Port Authority of New York and New Jersey</span>. The work was performed on behalf of the New Jersey-based company Inter-Con Security.</p>
<p class="articleParagraph enarticleParagraph" >In January of that year, Inter-Con announced that it had been selected as the sole awardee of the <span class="companylink">Port Authority</span>'s five-year contract for uniformed security guard services. The company's news release said it would be responsible for providing more than 1,000 security guards for the Kennedy, La Guardia and Newark airports, the World Trade Center campus and the Port Authority Trans-Hudson transit system.</p>
<p class="articleParagraph enarticleParagraph" >Inter-Con's vice president, Matthew Reeser, confirmed to The Times that City Safe had performed subcontracting work under that contract but declined to provide details about the amount and nature of the agreement.</p>
<p class="articleParagraph enarticleParagraph" >In 2023, the <span class="companylink">New York City Housing Authority</span> was unhappy with the performance of a security company that it had hired to provide fire safety services, and the agency began looking for new firms, prioritizing those classified as minority and women-owned businesses, an agency spokeswoman said. The authority launched a competitive bidding process, and four companies submitted quotes, the spokeswoman said. City Safe was one of them.</p>
<p class="articleParagraph enarticleParagraph" >As part of the contract, City Safe would place up to 500 guards in more than 150 public housing developments to monitor them for fires.</p>
<p class="articleParagraph enarticleParagraph" >At a board meeting in January, housing authority officials said that state law had required them to have people watching out for fires around the clock while repairs were being made to fire safety systems. Jamie Rubin, the chair of the authority's board, questioned the need to spend $154 million and said he felt like there ''had to be another path forward.''</p>
<p class="articleParagraph enarticleParagraph" >''It's an eye-watering number for something that seems like there's another solution,'' he said.</p>
<p class="articleParagraph enarticleParagraph" >Nevertheless, the board, which included Ms. Wright, approved the $154 million emergency contract to City Safe. Mr. Banks's former security firm got the job because it was ''the lowest responsive and responsible bidder'' that was not already providing fire and security services to the public housing agency, the authority's spokeswoman said.</p>
<p class="articleParagraph enarticleParagraph" >So far, the company has been paid $13.3 million of the three-year contract and the agency said it did not expect to reach the contract's maximum value.</p>
<p class="articleParagraph enarticleParagraph" >Mihir Zaveri and Jay Root contributed reporting. Susan C. Beachy contributed research.</p>
<p class="articleParagraph enarticleParagraph" >Mihir Zaveri and Jay Root contributed reporting. Susan C. Beachy contributed research.</p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>ART</b>&nbsp;</td><td><br/>Dealings of Philip B. Banks III, the deputy mayor for public safety, have been under scrutiny since his phone was seized this month by federal agents. (PHOTOGRAPH BY ALAMY LIVE NEWS VIA AP/ALAMY, VIA ALAMY LIVE NEWS VIA AP) (A18) This article appeared in print on page A1, A18. </td></tr>
<tr><td align="right" valign="top" class="index"><br/><b>CO</b>&nbsp;</td><td><br/>nychou : New York City Housing Authority</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>c33 : Contracts/Orders | ccat : Corporate/Industrial News | gcat : Political/General News | gcrim : Crime/Legal Action | gpir : Politics/International Relations | gpol : Domestic Politics | greg : Regional Politics | ncat : Content Types | nfact : Factiva Filters | nfcpin : C&E Industry News Filter | npag : Page One Stories</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>namz : North America | nyc : New York City | usa : United States | use : Northeast U.S. | usny : New York State</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>National Desk</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>The New York Times Company</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020240921ek9l0003q</td></tr></table><br/></div></div><br/><span></span><div id="article-REC0000020240616ek6g0000j" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/recLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>News</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>New York is ready for congestion pricing</span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By Sara Pequeño </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>870 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>16 June 2024</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The Bergen Record</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>REC</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>1; HeraldNews</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>O4</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>© Copyright Gannett 2024 All Rights Reserved. </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >One of the many reasons I decided to move to New York City was to avoid driving. Sometimes I miss it; most of the time, I don't. I'm not alone: The majority of people in New York City do not own cars.</p>
<p class="articleParagraph enarticleParagraph" >Nonetheless, 700,000 vehicles enter the city's business district in Manhattan daily.</p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >At the end of June, the city was supposed to implement a congestion pricing plan to charge commuters $15 a day to drive into the area below 60th Street. It was a decision mandated by state law in 2019. Last week, New York Gov. Kathy Hochul put an indefinite pause on that plan, citing the economy.</p>
<p class="articleParagraph enarticleParagraph" >"Circumstances have changed and we must respond to the facts on the ground – not from the rhetoric from five years ago," Hochul said.</p>
<p class="articleParagraph enarticleParagraph" >Unfortunately, the project's pause is a mistake, and will create more headaches the longer it's stalled.</p>
<p class="articleParagraph enarticleParagraph" >Hochul's lack of courage is just another example of the Democratic Party refusing to implement progressive policies. She's pausing the program in favor of appeasing drivers, some of whom don't even live in her state, and it's disappointing for those of us who want to see actual changes before it's too late.</p>
<p class="articleParagraph enarticleParagraph" >Driving in New York City means lots of cars, lots of congestion</p>
<p class="articleParagraph enarticleParagraph" >There are nearly 1 million people who commute to New York City from the Hudson Valley, Long Island and other parts of the state every day. An additional 1.6 million people go to work across boroughs.</p>
<p class="articleParagraph enarticleParagraph" >Congestion pricing would have encouraged more carpooling and public transit usage and cut down on the number of cars in Manhattan on a daily basis.</p>
<p class="articleParagraph enarticleParagraph" >For some people, the switch would have been more difficult: Transit options vary wildly even between boroughs, with fewer subway lines in Queens and the Bronx; Staten Island only has one rail option.</p>
<p class="articleParagraph enarticleParagraph" >On the other hand, the New York subway system is one of the largest in the world with about 470 stations.</p>
<p class="articleParagraph enarticleParagraph" >There are also plenty of transit options for folks coming in from other areas. There's the <span class="companylink">Long Island Rail Road</span>, the Port Authority Trans-Hudson (PATH) train, the Metro-North Railroad, 14 <span class="companylink">Amtrak</span> routes and at least three major commercial bus systems in the surrounding area.</p>
<p class="articleParagraph enarticleParagraph" >I'm aware that many people don't have a choice in whether or not to commute; it's one reason I feel strongly about the need for a robust public transit system. It's also not true that driving is the only way to commute into the city. The problem is that these systems need money to be improved upon – something that, long term, could have been generated by congestion pricing.</p>
<p class="articleParagraph enarticleParagraph" >The cost of lost cash</p>
<p class="articleParagraph enarticleParagraph" >Congestion pricing was also going to benefit the <span class="companylink">Metropolitan Transportation Authority</span> (<span class="companylink">MTA</span>) by bringing in at least $1 billion in revenue annually.</p>
<p class="articleParagraph enarticleParagraph" >That money was supposed to pay for the extension of a subway line, a project that would add subway stations, new electric buses, and renovations to multiple stations to make them compliant with the Americans with Disabilities Act.</p>
<p class="articleParagraph enarticleParagraph" >This is money that the <span class="companylink">MTA</span> desperately needs. After her decision, Hochul initially pushed for a tax hike behind closed doors. State lawmakers rejected the idea but are now looking at providing $1 billion to the <span class="companylink">MTA</span>, without saying where the money would come from.</p>
<p class="articleParagraph enarticleParagraph" >Seems like something that could have been generated with a new toll – oh wait, that was the original plan.</p>
<p class="articleParagraph enarticleParagraph" >Air quality, climate change action take a back seat</p>
<p class="articleParagraph enarticleParagraph" >The financial losses aren't the only way New Yorkers are worse off without congestion pricing. Fewer cars on the road would have had impacts on pollution in the city.</p>
<p class="articleParagraph enarticleParagraph" >The plan was supposed to mean an estimated 120,000 fewer cars on the road, which in turn would improve air quality in three boroughs.</p>
<p class="articleParagraph enarticleParagraph" >New York City planned to reduce greenhouse emissions 85% by 2050. Congestion pricing would have helped us achieve that goal. As a young person, I find it disappointing that people in power are yet again putting climate change on the back burner in the hopes of scoring political points.</p>
<p class="articleParagraph enarticleParagraph" >Congestion pricing isn't perfect; there's a reason multiple lawsuits were filed to stop its implementation. Even so, it would have huge benefits for the city, and would bring us up to speed with cities like London, Stockholm and Singapore.</p>
<p class="articleParagraph enarticleParagraph" >The plan to push it off indefinitely will only delay something that must happen for us to take control of the climate crisis and improve our transit systems. It may not be good news for drivers, but the net benefits for commuters are hard to ignore. Hochul made the wrong choice.</p>
<p class="articleParagraph enarticleParagraph" >Follow <span class="companylink">USA TODAY</span> elections columnist Sara Pequeño on X, formerly <span class="companylink">Twitter</span>, @sara__pequeno and <span class="companylink">Facebook</span> facebook.com/PequenoWrites.</p>
<p class="articleParagraph enarticleParagraph" >Columnist</p>
<p class="articleParagraph enarticleParagraph" >
                     <span class="companylink">USA TODAY NETWORK</span>
                  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>ART</b>&nbsp;</td><td><br/>A congestion pricing plan set to begin June 30 for New York City would have charged $15 for regular passenger cars entering Manhattan below 60th Street during the day. But New York Gov. Kathy Hochul has indefinitely paused the program. | Spencer Platt/Getty Images </td></tr>
<tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>gcat : Political/General News | gtrans : Transport</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>namz : North America | nyc : New York City | usa : United States | use : Northeast U.S. | usny : New York State</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>Gannett Co., Inc.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document REC0000020240616ek6g0000j</td></tr></table><br/></div></div><br/><span></span><div id="article-REC0000020230820ej8k0000s" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/recLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Opinion</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>LETTERS TO THE EDITOR; Bad faith in government</span>
</td></tr><tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>1026 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>20 August 2023</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The Bergen Record</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>REC</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>1; HeraldNews</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>O5</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>© Copyright Gannett 2023 All Rights Reserved. </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >Regarding "State can't scrap OPRA. We have to save it" (Your Turn, Aug. 13):</p>
<p class="articleParagraph enarticleParagraph" >This is the optimally worst time for Democrats in New Jersey to drop the Open Public Records Act. Guest columnist and Citizens Campaign founder Harry Pozycki lays out the reasons we need it. And Democrats: It's not sufficient to defend democracy only on the national level.</p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >The electorate is watching challenges to fair and responsible government on every level. One party was unwilling to do its duty during two impeachments of a former president and shunting its duty to the courts. In the process, the leaders sideline those in their own party whose fealty to the Constitution overshadows their own. One congressman conceives this the right moment and himself the righteous instrument to strip our armed services of their leadership as China and Russia flex their muscles, testing our diplomatic/military balance and prowess even closer to our boundaries. The Supreme Court is outed for neglecting its reputation and responsibility to ensure its ethics. Years of effort working to democratize voting are being challenged.</p>
<p class="articleParagraph enarticleParagraph" >Give the electorate a break. There is an overload of bad-faith governance. We need reasons to have faith. The right to lie is being affirmed in court briefs. What about our right to know? We need some some palatable good news. Don't drop OPRA.</p>
<p class="articleParagraph enarticleParagraph" >Larraine K. Bogert</p>
<p class="articleParagraph enarticleParagraph" >Oradell</p>
<p class="articleParagraph enarticleParagraph" >Teen's work ethic is impressive</p>
<p class="articleParagraph enarticleParagraph" >Regarding "Showroom shine" (Aug. 15):</p>
<p class="articleParagraph enarticleParagraph" >How wonderful to read the story about 13-year-old Tyler Poon, who has his own car detailing business.</p>
<p class="articleParagraph enarticleParagraph" >I am so impressed and happy to read of a mature, industrious young person who works hard to earn his own money. Tyler is more interested in being productive than being addicted to his cellphone and video games.</p>
<p class="articleParagraph enarticleParagraph" >This story gave me renewed hope. Thank you. We need more of these stories.</p>
<p class="articleParagraph enarticleParagraph" >Marilyn Bartholme</p>
<p class="articleParagraph enarticleParagraph" >Mahwah</p>
<p class="articleParagraph enarticleParagraph" >Christie and the GOP</p>
<p class="articleParagraph enarticleParagraph" >Regarding "Christie is the most interesting person in the GOP" (Opinion. Aug. 13):</p>
<p class="articleParagraph enarticleParagraph" >Former Gov. Chris Christie lost my respect when he closed the shore and then sunned himself on those same beaches. Saying that, I must agree that he is a better Republican candidate than former President Donald Trump or Florida Gov. Ron DeSantis.</p>
<p class="articleParagraph enarticleParagraph" >The Republican Party has lost its way and has become a party of negatives, criticizing everything. It seems obsessed with Trump and his problems. In the meantime, the Democrats remain quiet and keep running the country. Democrats are looking to the future, while Republicans sear the past.</p>
<p class="articleParagraph enarticleParagraph" >Christie, a Republican, must look to the future to win. Our leaders are aged. Our future leaders must be found in our youth, Republican or Democratic. Our country is improving economically and politically. If we do not look forward, we will stumble over our past.</p>
<p class="articleParagraph enarticleParagraph" >Peter J. Peirano</p>
<p class="articleParagraph enarticleParagraph" >Ridgewood</p>
<p class="articleParagraph enarticleParagraph" >Put solar panels on warehouse roofs</p>
<p class="articleParagraph enarticleParagraph" >There is much talk about energy independence and how we are going to get there. Unfortunately, our politicians are totally unable to think outside the box and therefore promise a lot but deliver little. There is an easy way for New Jersey to get close to producing all of its energy clean and green.</p>
<p class="articleParagraph enarticleParagraph" >Anybody who has driven through what I call the Grand Canyon of New Jersey, between Exits 11 and 8 on the turnpike, will have seen the endless line of warehouses that have millions upon millions of square feet of flat roofs. These warehouses destroy thousands upon thousands of acres of farmland by covering it with cement, and in return we get jobs that pay mostly minimum wage or just above the minimum.</p>
<p class="articleParagraph enarticleParagraph" >It is time that New Jersey gets something in return from these corporations that turn our state into one giant concrete slab. We need a law that retroactively requires all flat roofs above, say, 1,000 square feet to be 70% covered with solar panels. This would instantaneously make us 90% or more green. All we need are politicians who are willing to take on the warehouse owners and not worry about political contributions to their next campaign.</p>
<p class="articleParagraph enarticleParagraph" >Wilhelm D. Auer</p>
<p class="articleParagraph enarticleParagraph" >Westwood</p>
<p class="articleParagraph enarticleParagraph" >Share toll proceeds with NJ</p>
<p class="articleParagraph enarticleParagraph" >There is a simple solution to resolve Gov. Phil Murphy's lawsuit against the <span class="companylink">New York Metropolitan Transportation Authority</span>'s implementation of congestion price tolling. All the <span class="companylink">MTA</span> has to do is share some of the proceeds. Just like the <span class="companylink">MTA</span>, both <span class="companylink">NJ Transit</span> and the <span class="companylink">Port Authority</span> have multiyear capital plans. New York Gov. Kathy Hochul and <span class="companylink">MTA</span> Chairman Janno Lieber should offer Murphy, NJ Transit, the <span class="companylink">Port Authority</span> and PATH a fair share of revenues generated by these new tolls that Garden State residents will be paying. Why not share 5% each with <span class="companylink">NJ Transit</span>, Port Authority and Port Authority Trans Hudson (PATH) subway? This will still leave the <span class="companylink">MTA</span> with 85% of toll revenues for New York City Transit bus, subway, Staten Island Railway, <span class="companylink">MTA</span> bus, Long Island and Metro-North commuter railroads.</p>
<p class="articleParagraph enarticleParagraph" >Some of these dollars could go toward the New Jersey share of the $18 billion (two new Hudson River tunnels plus rehab of two existing ones) or $39 billion full-scope Gateway project, the $10 billion Port Authority 42nd Street Bus Terminal, and the potential Penn Station improvement projects, along with other <span class="companylink">NJ Transit</span> and PATH capital programs or other transportation improvements beneficial to New Jersey residents. After all, fair is fair. Thousands of New Yorkers are reverse commuters, traveling from Penn Station, Herald Square, Port Authority George Washington and 42nd Street bus terminals via bus, rail and PATH to jobs, schools, sports stadiums, entertainment, Newark Airport and other New Jersey destinations. They benefit from <span class="companylink">NJ Transit</span>, <span class="companylink">Port Authority</span> and PATH capital investments.</p>
<p class="articleParagraph enarticleParagraph" >Larry Penner</p>
<p class="articleParagraph enarticleParagraph" >Great Neck, N.Y.</p>
<p class="articleParagraph enarticleParagraph" >The writer is a transportation advocate, historian and writer who previously served as director of the Federal Transit Administration Region 2 New York Office of Operations and Program Management.</p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>ART</b>&nbsp;</td><td><br/>Tyler Poon details a car at Maserati of Englewood Cliffs. The 13-year-old Edgewater resident owns New Generation Car Cleaning, which offers luxury car detailing services in North Jersey. | Danielle Parhizkaran/NorthJersey.com </td></tr>
<tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>gpol : Domestic Politics | gcat : Political/General News | nlet : Letters | gpir : Politics/International Relations | ncat : Content Types | nfact : Factiva Filters | nfce : C&E Exclusion Filter | niwe : IWE Filter | nrgn : Routine General News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>usnj : New Jersey | usa : United States | namz : North America | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>Gannett Co., Inc.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document REC0000020230820ej8k0000s</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020190125ef1p00037" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>CLM</b>&nbsp;</td><td>LETTER</td></tr>
<tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Editorial Desk; SECT</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>Upgrading PATH System</span>
</td></tr><tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>225 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>25 January 2019</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>The New York Times on the Web</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 2019 The New York Times Company.  All Rights Reserved. </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >To the Editor:</p>
<p class="articleParagraph enarticleParagraph" >Re ''Even With Packed Trains, PATH Will Lose $400 Million This Year'' (news article, Jan. 3):</p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >Commuter railroads keep the region moving in a safe and reliable manner. And it's no secret that, like all major passenger railroads in the United States, they operate at a deficit and require significant subsidies. PATH is no different, and the amount of expenses it recovers through fares is on par with other American transit systems.</p>
<p class="articleParagraph enarticleParagraph" >But the benefit it provides goes well beyond an affordable ride in or out of New York and to Newark, Harrison, Jersey City and Hoboken, 24/7.</p>
<p class="articleParagraph enarticleParagraph" >In 2017, PATH's yearly ridership numbers approached 83 million, compared with 73.7 million in 2014, an 11 percent increase in just three years. That's because major new building developments were built precisely because of their proximity to a PATH station. The residents who have since flocked to them and become riders are a true testament to the system's value.</p>
<p class="articleParagraph enarticleParagraph" >As these communities grow, we must continue to invest in infrastructure built more than 100 years ago -- from station upgrades, to critical repairs, to modern signaling systems and safety technology. The region depends upon it.</p>
<p class="articleParagraph enarticleParagraph" >Michael Marino</p>
<p class="articleParagraph enarticleParagraph" >Jersey City</p>
<p class="articleParagraph enarticleParagraph" >The writer is director and general manager of the Port Authority Trans-Hudson Corporation.</p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>ncolu : Columns | nlet : Letters | ncat : Content Types | nfact : Factiva Filters | nfce : C&E Exclusion Filter | niwe : IWE Filter | nrgn : Routine General News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>usa : United States | namz : North America</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>Editorial Desk | Letter</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>The New York Times Company</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020190125ef1p00037</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020170804ed840003l" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Metropolitan Desk; SECTA</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>PATH, Unprofitable and Underappreciated, Seizes Its Moment</span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By PATRICK McGEEHAN </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>1446 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>4 August 2017</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>17</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 2017 The New York Times Company.  All Rights Reserved. </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >The New York metropolitan area has two subway systems that are busier than Atlanta's. One, despite all of its current problems, is considered a marvel of urban transportation technology. The other once was.</p>
<p class="articleParagraph enarticleParagraph" >New York City's subway system, overcrowded and prone to meltdowns as it may be, is the workhorse of American subways, carrying more than 5.6 million riders a day. Its puny cousin, the PATH train, is more like a Labradoodle, a puzzling crossbreed with its fair share of fans and detractors.</p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >PATH is both subway and commuter train. Its network of tracks is relatively small, but it loses money every year and is so expensive to operate that its owners have considered trying to offload it.</p>
<p class="articleParagraph enarticleParagraph" >Still, PATH plays an integral part in getting workers across the Hudson River every day. And this summer, it has taken on an even larger role, carrying an additional load each weekday of more than 22,000 passengers whose usual trains have been canceled or diverted because of emergency track repairs at Pennsylvania Station in Manhattan.</p>
<p class="articleParagraph enarticleParagraph" >And, with a few exceptions, PATH has managed the challenge of getting all of those displaced commuters across the Hudson on schedule. Michael Marino, who runs the PATH system for the <span class="companylink">Port Authority of New York and New Jersey</span>, said the system had maintained an on-time performance of nearly 98 percent since July 10, when tracks at Penn Station were closed to accommodate the repair work.</p>
<p class="articleParagraph enarticleParagraph" >''This is our time to shine and we're shining,'' Mr. Marino said of the line, which links Newark, Jersey City and Hoboken to Midtown Manhattan and the World Trade Center downtown.</p>
<p class="articleParagraph enarticleParagraph" >With the expectation of an additional 14,000 or 15,000 daily passengers, PATH added four trains per hour between Hoboken Terminal and 33rd Street in Manhattan during the morning and evening rushes, he said. The extra demand has been about 50 percent greater than anticipated, but the system has handled it with few disruptions, Mr. Marino said.</p>
<p class="articleParagraph enarticleParagraph" >''The trains are crowded,'' he said. ''We said that the trains would be crowded. That's what's going to happen.''</p>
<p class="articleParagraph enarticleParagraph" >Mr. Marino said PATH officials had gotten only a small number of complaints from new riders about the trains being crowded or delayed.</p>
<p class="articleParagraph enarticleParagraph" >Corey Kronengold was not one of those satisfied customers. Mr. Kronengold, who lives in Livingston, N.J., and normally rides a <span class="companylink">New Jersey Transit</span> train from South Orange to Penn Station, rode PATH from Hoboken to Midtown for two weeks last month. The train was dependably on time, he said, but he abandoned that route because he found the cars to be ''crowded to the point of being unsafe.'' They were stuffed with so many people that the air-conditioners were overmatched.</p>
<p class="articleParagraph enarticleParagraph" >''It's a lousy way to start your day,'' said Mr. Kronengold, 43, a marketing executive with <span class="companylink">IgnitionOne</span>.</p>
<p class="articleParagraph enarticleParagraph" >The PATH system is allowing diverted <span class="companylink">New Jersey Transit</span> passengers to transfer to and from Manhattan free, as is the New York Waterway ferry that docks at Hoboken Terminal. But Mr. Kronengold said that some evenings, as he rushed to get back to Hoboken, there were no attendants to let him through the turnstiles at the 33rd Street PATH station.</p>
<p class="articleParagraph enarticleParagraph" >After two weeks, he said, he switched to the ferry, which provides shuttle buses in Manhattan that he can ride to within a few blocks of his office, north of Times Square. He is still spending far less time with his 18-month-old daughter, but the commute is less hellish, he said.</p>
<p class="articleParagraph enarticleParagraph" >PATH has been the target of plenty of criticism in its long history. It was the original railroad between New Jersey and Manhattan, its first tube under the Hudson having opened in 1907. That was a few years before the tunnel that connects <span class="companylink">Amtrak</span> and <span class="companylink">New Jersey Transit</span> trains to Penn Station.</p>
<p class="articleParagraph enarticleParagraph" >Back then, the railroad was known as the Hudson & Manhattan and its Hudson Terminal stood where the World Trade Center was later built. After the H.&M. went bankrupt, it was acquired by the <span class="companylink">Port Authority</span> and renamed the <span class="companylink">Port Authority</span> Trans-Hudson in the early 1960s.</p>
<p class="articleParagraph enarticleParagraph" >Since then, as PATH's popularity has rebounded, the agency has spent billions of dollars operating and expanding the system. Still, it may be the least appreciated, most unprofitable subway system in the country.</p>
<p class="articleParagraph enarticleParagraph" >Even before the intensive repairs at Penn Station began, PATH had the seventh-highest ridership of any heavy-rail system in the country. It carried an average of 266,600 weekday passengers in the first three months of this year, a 2.1 percent increase over the first quarter of 2016, according to data compiled by the <span class="companylink">American Public Transportation Association</span>.</p>
<p class="articleParagraph enarticleParagraph" >That total placed PATH behind Philadelphia's Septa trains but well ahead of Atlanta's Marta system, which carried an average of 215,400 passengers each weekday, those statistics showed.</p>
<p class="articleParagraph enarticleParagraph" >When the <span class="companylink">Port Authority</span> convened a special panel on its future three years ago, the group called PATH a vital alternative to crossing the Hudson in cars and buses. But the panel recommended finding a way to reduce the financial drag on the agency, possibly by bringing in a partner to help run it.</p>
<p class="articleParagraph enarticleParagraph" >The panel concluded that the PATH system was far more expensive to operate, relative to its size, than the New York subway or any of the metropolitan area's commuter railroads. They also noted that PATH was much smaller than other heavy-rail systems like the subways in Washington and Chicago and had some characteristics similar to those of commuter railroads like the <span class="companylink">Long Island Rail Road</span> and <span class="companylink">New Jersey Transit</span>.</p>
<p class="articleParagraph enarticleParagraph" >PATH ranked between the <span class="companylink">Long Island Rail Road</span> and the <span class="companylink">Metro-North Railroad</span> in annual passenger trips, but the typical trip on PATH was far shorter. Each of those two commuter railroads carried its passengers for more than two billion total miles annually, compared with 370 million miles for PATH.</p>
<p class="articleParagraph enarticleParagraph" >And unlike those railroads, PATH charges a flat fare for all trips. That fare, now $2.75, has risen to be comparable to the single-ride New York City subway fare. But it does not come close to covering operating costs, which last year amounted to a loss of $415 million. Officials said they hoped the compensation from <span class="companylink">New Jersey Transit</span> would offset their added costs this summer.</p>
<p class="articleParagraph enarticleParagraph" >The panel found that fares covered less than 45 percent of Path's operating costs in 2013, compared with about 60 percent for <span class="companylink">New Jersey Transit</span> and Metro-North and 73 percent for the New York City subway. That ratio left PATH losing an average of $348 million a year for the previous five years, the panel found.</p>
<p class="articleParagraph enarticleParagraph" >It remains the least expensive way of crossing the Hudson, cheaper than a bus from Hoboken, less than half the cost of a ferry and far less costly than the $15 cash toll at the Lincoln and Holland tunnels and the George Washington Bridge. Indeed, the <span class="companylink">Port Authority</span> uses the tolls it collects at the bridge and the Holland and Lincoln tunnels to offset its steep losses on PATH.</p>
<p class="articleParagraph enarticleParagraph" >But, despite those losses and the panel's recommendation to reduce the financial burden on the <span class="companylink">Port Authority</span>, the <span class="companylink">Port Authority</span> plans to spend heavily to expand the PATH system. It intends to extend PATH from downtown Newark to Newark Liberty International Airport at a projected cost of $1.7 billion. The Tri-State Transportation Campaign, a commuter advocacy group, has criticized the plan, arguing that the agency should invest in improving existing stations to accommodate growing ridership.</p>
<p class="articleParagraph enarticleParagraph" >For now, several thousand commuters are just glad that PATH exists. Rosaria Di Grande, who lives in Summit, N.J., and normally rides a <span class="companylink">New Jersey Transit</span> train directly to and from Penn Station on the Morris and Essex line, is one of those.</p>
<p class="articleParagraph enarticleParagraph" >''The PATH is always crowded, but it's been very efficient as the trains are quite frequent and there's never too long of a wait,'' said Ms. Di Grande, an executive assistant at Montblanc in Olympic Tower. ''I do find the PATH trains to be very clean and new.''</p>
<p class="articleParagraph enarticleParagraph" >That has been a revelation to Ms. Di Grande -- before July 10, she had never used it to commute.</p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>ART</b>&nbsp;</td><td><br/>Riders at Hoboken Terminal, above, making their way from New Jersey Transit trains to PATH trains bound for Manhattan. Below, commuters squeezing onto a PATH train. The rail system has lost money for the past several years, even though it is one of the country's busiest. (PHOTOGRAPHS BY VICTOR J. BLUE FOR THE NEW YORK TIMES) </td></tr>
<tr><td align="right" valign="top" class="index"><br/><b>CO</b>&nbsp;</td><td><br/>panynj : Port Authority of New York and New Jersey | mta : Metropolitan Transportation Authority</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i721 : Urban/Commuter Transit | i72101 : Urban Mass Transit Systems | irailtr : Road/Rail Transport | itsp : Transportation/Logistics</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>gcat : Political/General News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>usa : United States | nyc : New York City | usnj : New Jersey | usny : New York State | namz : North America | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>Metropolitan Desk</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>The New York Times Company</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020170804ed840003l</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020150805eb8500069" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Metropolitan Desk; SECTA</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>A 9/11 Artifact Heads to Its New Home at a Trolley Museum in Connecticut</span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By DAVID W. DUNLAP   </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>773 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>5 August 2015</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>16</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 2015 The New York Times Company.  All Rights Reserved.   </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >A perilous, improbable rail journey that began at the World Trade Center on Sept. 11, 2001, will end Thursday when PATH car No. 745 is welcomed with a parade -- bagpipes, dignitaries and all -- through the streets of East Haven, Conn.</p>
<p class="articleParagraph enarticleParagraph" >No. 745 is the astonishingly intact remnant of a seven-car PATH train that was waiting on Track 2 at the trade center terminal for a return trip to Hoboken, N.J., when all hell broke loose. The crew was evacuated safely.</p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >Having survived the collapse of the twin towers, two cars from that train, Nos. 745 and 143, were salvaged by the <span class="companylink">Port Authority of New York and New Jersey</span>. They have been kept ever since in Hangar 17 at Kennedy International Airport, waiting for a new home.</p>
<p class="articleParagraph enarticleParagraph" >When the Shore Line Trolley Museum in East Haven learned this year that one of the 1970s-vintage PATH cars was available for its collection, it did not hesitate. The fact that the car had such a direct link to the 2001 attack made it irresistible.</p>
<p class="articleParagraph enarticleParagraph" >''A number of people involved with the museum are first responders, so it hit a chord with them,'' Bill Wall, the president emeritus, said on Tuesday as he watched No. 745 being loaded on a trailer truck for its journey to Connecticut.</p>
<p class="articleParagraph enarticleParagraph" >The <span class="companylink">Port Authority</span> donated the 51-foot-long, 37-ton rail car to the museum, but the museum is picking up the tab for the transportation, about $20,000.</p>
<p class="articleParagraph enarticleParagraph" >Inside the dusty car -- with its orange walls, brown plastic bucket seats and wood-grain partitions -- time has not simply frozen; it has invested some of the advertising cards with a chilling double significance:</p>
<p class="articleParagraph enarticleParagraph" >''Get there by air.''</p>
<p class="articleParagraph enarticleParagraph" >''Glowing embers waiting to ignite.''</p>
<p class="articleParagraph enarticleParagraph" >''Ordinary people. Extraordinary adventure!''</p>
<p class="articleParagraph enarticleParagraph" >Mr. Wall said the public would be allowed to walk through the car when it is fully installed at the museum next year. Car No. 745 will be kept indoors, he said.</p>
<p class="articleParagraph enarticleParagraph" >The car was hauled out of Hangar 17 on Tuesday to an apron where two cranes waited, side by side. Workers wrapped slings around the front and back of the car. It was lifted off the back of the flatbed trailer on which it had been stored and lowered on to a trailer truck operated by Silk Road Transport of Arkport, N.Y.</p>
<p class="articleParagraph enarticleParagraph" >Jim Keane, the general manager of operations safety at the <span class="companylink">Port Authority</span>, was among those watching the transfer. ''It's nice that it will be seen by the public,'' he said, ''and that people can learn more about what happened that day.''</p>
<p class="articleParagraph enarticleParagraph" >''It has a history much different than so many rail cars,'' Mr. Keane said.</p>
<p class="articleParagraph enarticleParagraph" >That history began in 1972 in Thunder Bay, Ontario, where Hawker Siddeley Canada manufactured the PA-3 series of aluminum-bodied passenger cars for the <span class="companylink">Port Authority</span> Trans-Hudson commuter railroad. On car No. 745, the axle-mounted odometer, or hubometer, registers 201,104 miles of travel.</p>
<p class="articleParagraph enarticleParagraph" >All seven cars of the Hoboken-bound train were removed to Staten Island to be decontaminated and stripped of usable parts, according to the encyclopedic nycsubway.org. Only cars 745 and 143 wound up at Hangar 17.</p>
<p class="articleParagraph enarticleParagraph" >Today, No. 143 looks quite small -- almost toylike -- tucked all by itself in a corner of the 1.8-acre hangar. It is part of an ever-shrinking inventory of artifacts that the <span class="companylink">Port Authority</span> makes available for memorial sites and museums.</p>
<p class="articleParagraph enarticleParagraph" >(For a time, steel artifacts were given away as political favors by Bill Baroni, the deputy executive director of the authority, who was appointed by Gov. Chris Christie of New Jersey. The program has since returned to professional control.)</p>
<p class="articleParagraph enarticleParagraph" >Among the artifacts still available are 88 pieces of steel and 217 non-steel items, including aluminum cladding from the towers, large pieces of the antenna, Port Authority Police Department vehicles, elevator motors and items saved from the concourse-level shopping mall, like clothing, jewelry and toys.</p>
<p class="articleParagraph enarticleParagraph" >''The artifacts program preserves the history and memorializes the lives of those we lost on 9/11,'' said Erica Dumas, a spokeswoman for the authority.</p>
<p class="articleParagraph enarticleParagraph" >It has also transformed workaday objects into resonant symbols.</p>
<p class="articleParagraph enarticleParagraph" >''For most people, this would be a subway car,'' Mr. Wall said. ''Now, it's an icon.''</p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>ART</b>&nbsp;</td><td><br/>PATH car No. 745 was hauled out of its longtime home in Hangar 17 at Kennedy Airport on Tuesday, and lifted by two cranes onto a trailer truck. This week, it moves to East Haven, Conn. (PHOTOGRAPH BY MARK KAUZLARICH/THE NEW YORK TIMES)   </td></tr>
<tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>gcat : Political/General News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>usa : United States | nyc : New York City | usct : Connecticut | usny : New York State | namz : North America | use : Northeast U.S. | usnew : New England</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>Metropolitan Desk</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>The New York Times Company</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020150805eb8500069</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020150105eb1500058" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Metropolitan Desk; SECTA</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>Plan to End Overnight PATH Service Irks Riders</span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By MATT A.V. CHABAN   </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>910 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>5 January 2015</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>14</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 2015 The New York Times Company.  All Rights Reserved.   </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >In the underworld of the subway, it can be hard to tell day from night. The high and low tides of passengers are usually one clue of the time.</p>
<p class="articleParagraph enarticleParagraph" >The PATH platform at Ninth Street in Manhattan at 2:30 in the morning can be a bit disorienting, then, looking more like midday than the middle of the night. As dozens of people waited at that hour on a recent weekday, the only real tipoffs to the time were the drowsy eyes and drooping heads.</p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >As the New Jersey-bound train to Hoboken and Journal Square pulled in, 35 minutes after the previous one, it was standing room only.</p>
<p class="articleParagraph enarticleParagraph" >''It's like Tokyo sometimes: You have to push your way on, and then you're stuck like a sardine,'' Carlos Parada said. He was on his way home to Journal Square after his shift at Strip House, a steakhouse on East 12th Street in Manhattan.</p>
<p class="articleParagraph enarticleParagraph" >As if Mr. Parada's commute were not grueling enough, he and thousands of his fellow riders found out there could be no commute whatsoever in the future. At the end of December, their Legislatures already adjourned, Governors Andrew M. Cuomo of New York and Chris Christie of New Jersey, vetoed a sweeping bill that would have reformed the agency. The governors, who jointly control the <span class="companylink">Port Authority of New York and New Jersey</span>, then unveiled a report of their own calling for changes.</p>
<p class="articleParagraph enarticleParagraph" >Buried among the proposals was one considering the elimination of service between 1 a.m. and 5 a.m. on the <span class="companylink">Port Authority</span> Trans-Hudson, known as PATH, the train linking Manhattan with nearby cities in New Jersey.</p>
<p class="articleParagraph enarticleParagraph" >During this time, ridership plummets, with roughly 2,000 crossing the Hudson on weeknights. They account for just under 1 percent of daily passengers, and eliminating service would save the authority $10 million from its $330 million budget, according to the governors' report.</p>
<p class="articleParagraph enarticleParagraph" >The idea was universally rejected by those lonely souls crisscrossing the Hudson River in the early morning hours, including the bartenders and clubgoers, the swing-shift construction and postal workers, and the foreign-exchange traders heading in on Frankfurt time.</p>
<p class="articleParagraph enarticleParagraph" >To them, the overnight PATH is a lifeline, the only one into or out of New York City when all other trains and buses have stopped and taxis, at $50 to $100, are too costly.</p>
<p class="articleParagraph enarticleParagraph" >''We moved from Clifton to Jersey City just to have the late-night transportation, which is half the reason we're paying what we are for rent,'' said Tricia Torley, who was riding a 1:20 a.m. train home from a Disco Biscuits concert with her boyfriend.</p>
<p class="articleParagraph enarticleParagraph" >Chene Darling, who tended bar at the same concert, caught the 3:40 a.m. train. ''If there was no PATH train, that would change everything,'' she said. ''I guess I'd have to buy a car, or move to the city, neither of which I want to do.''</p>
<p class="articleParagraph enarticleParagraph" >Besides passengers, many musical instruments make the trip home after hours -- guitars, upright basses, keyboards, amps and horns. Nigel Inniss was coming home from a jam session around 3 a.m., his saxophone nestled between his legs.</p>
<p class="articleParagraph enarticleParagraph" >''Especially for musicians and artists, late night is when everything happens,'' he said. ''You'd be cutting off an entire section of the population.''</p>
<p class="articleParagraph enarticleParagraph" >One reason those New Jersey-bound trains were so packed was that there are only four trains running in the entire system between 1 and 5 a.m., two for each of the two PATH lines. Other trains that same early morning were less crowded.</p>
<p class="articleParagraph enarticleParagraph" >The trains also provide a refuge for the homeless. Some cars are almost entirely given over to them; they line the seats largely undisturbed.</p>
<p class="articleParagraph enarticleParagraph" >Talk of eliminating the overnight trains has drawn criticism across North Jersey. ''If either governor actually rode the train during these hours, they never would have considered this,'' Mayor Steven M. Fulop of Jersey City said.</p>
<p class="articleParagraph enarticleParagraph" >The Port Authority chairman, John J. Degnan, who helped write the governors' report, has since emphasized that stopping overnight service is one of at least six proposals for improving the finances of the PATH system.</p>
<p class="articleParagraph enarticleParagraph" >''Curtailment of PATH service would be a last resort,'' he said in a statement, ''and it would be done only after consultation with local officials and public hearings.''</p>
<p class="articleParagraph enarticleParagraph" >Kareem McJagger, a drag performer who moved from Harlem to Hoboken two years ago, said he would be forced to move back if overnight PATH service went away. Dressed in sweatpants and a fur coat, he still had on a puffy wig and fiery makeup aboard the 4:15 a.m. train. For night-life workers, the train can be especially convenient, he said.</p>
<p class="articleParagraph enarticleParagraph" >''Believe it or not, I get fewer funny looks than on the subway'' in New York, he said.</p>
<p class="articleParagraph enarticleParagraph" >Even the finance workers riding the 4:35 a.m. to the World Trade Center were unswayed by the argument for service cuts. ''I don't buy it,'' said Manuel Perez, who commutes from the Ironbound section of Newark to <span class="companylink">JPMorgan Chase</span> offices in Brooklyn each morning. ''You can make the numbers say whatever you want.''</p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>ART</b>&nbsp;</td><td><br/>Riders on a PATH train around 3 a.m. in Manhattan last week.  A proposal would eliminate service between 1 a.m. and 5 a.m. (PHOTOGRAPH BY CHRISTOPHER GREGORY FOR THE NEW YORK TIMES)   </td></tr>
<tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>gcat : Political/General News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>usa : United States | nyc : New York City | usnj : New Jersey | usny : New York State | namz : North America | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>Metropolitan Desk</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>The New York Times Company</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020150105eb1500058</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020140518ea5i0000c" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Classified; SECTA</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>Paid Notice: Deaths; GOLDBERG, DANIEL LOUIS</span>
</td></tr><tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>904 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>18 May 2014</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>22</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 2014 The New York Times Company.  All Rights Reserved.   </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >GOLDBERG--Daniel Louis, my beloved, passed away April 1, 2014 in New York City. He was 91 years old, four days short of his 92d birthday. He was born April 5, 1922 in Philadelphia, PA and later resided in New Jersey and New York, recently wintering in Florida. He was an exceptional person intellectually and a kind, compassionate caring human being. He was a WWII Veteran, a volunteer, in the <span class="companylink">US Army</span> from 1942-1946, first an enlistee then a 90-day-wonder commissioned officer, forever proud of his military service. After Technical High School in Newark, NJ, he attended The <span class="companylink">New Jersey Institute of Technology</span> receiving his BS and MS Degrees in Electrical Engineering, then becoming a registered Professional Engineer (PE) in New York and New Jersey. He also attended <span class="companylink">New York University</span>'s prestigious Courant Institute of Mathematical Sciences. Daniel Goldberg was a brilliant Engineer.</p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" > For 35 years, he was employed by the <span class="companylink">Port Authority of New York and New Jersey</span>, and for 16 of those years, until his retirement in 1985, he was the <span class="companylink">Port Authority</span>'s Chief Electrical Engineer. Much of his electrical engineering design work involved New York City's Holland and Lincoln Tunnels, The George Washington Bridge, The <span class="companylink">Port Authority</span> Bus Terminal, The <span class="companylink">Port Authority</span> Trans Hudson (PATH), The World Trade Center, Kennedy International and LaGuardia Airports, and New Jersey's Newark International Airport, where he played a major role in a 1963 expansive redevelopment, construction and modernization program, and was the consultant for all the lighting and power facilities, receiving high recognition. Through the years he received many <span class="companylink">Port Authority</span> accolades. In 1980, he was awarded the <span class="companylink">Port Authority</span>'s Distinguished Service Medal "For demonstrating an exceptional degree of good judgment, initiative and competence in his field of endeavor and sustained this record of outstanding or distinguished service over a number of years, at least 15 years of service." When retiring, his <span class="companylink">Port Authority</span> plaque was inscribed "A Distinguishing Award To Daniel L. Goldberg, An Engineer A Gentleman A Friend." He was a life member of the <span class="companylink">Institute of Electrical and Electronic Engineers</span> (IEEE) the world-wide largest professional society for the advancement of technology for humanity. In 1984, he was named Fellow, the highest level of membership, his citation "For innovative engineering of large transportation facilities." In 1974, he received an Achievement Award "For Leadership in Preparing and Publishing the 2nd Edition of Electrical Systems for Commercial Buildings." And in 1975, he received the Industrial and Commercial Power Systems Standards Medallion. He also contributed to the 1976, 1986, 1996 editions of the Red Book-Industrial Buildings, and the Bronze Book-Energy Conservation, and was on the Standards Board Review Committee 1982-1992. In 1993, he was the recipient of the Distinguished Service Award "In recognition of his dedication and distinguished service to the IEEE Industry Applications Society." He was involved with the Illuminating Engineering Society and wrote three electrical chapters for the McGraw Hill Handbook on Building Services. He was also an instructor for the IEEE, and held numerous other positions. The immensity of his ongoing contributions to the IEEE culminated in being awarded a special bronze medal inscripted "The IEEE Awards This Third Millennium Medal to Daniel L. Goldberg in 2000 for Outstanding Achievements and Contributions." Daniel was a child of "The Great Depression of the '30's," never forgetting the prevailing poverty, imbuing him with a lasting sense of empathy and philanthropy. For 40 years, he was a member and supporter of The American Technion Society, of The Technion Society of Israel, the leading Engineering and Science Institution, and in 2000 he received their Crystal Triangle, the highest award for being a PATRON. He was also a long time recognized supporter of the National Holocaust Museum in Washington, DC. And he was an Alma Mater supporter of the New Jersey Institute of Technology, Newark College of Engineering, "An Alumni Fellow Supporter For Significant Support to Engineering Education," these among so many others. Archeology was another of Daniel's interests. For many years he was a member of the Biblical Archeology Society, an international organization, taking him on several Middle East excavations, to dig sites of historical biblical importance, mainly to Israel. He was an inveterate traveler and had visited China, India, Egypt, Greece, Morocco, France, England, Germany, Ecuador, Argentina, Mexico, Alaska, and other places. He enjoyed music, and opera, and possessed a delightful sense of humor, wise and witty. His appreciation and knowledge of art and art history were boundless. He was a member of the <span class="companylink">Metropolitan Museum of Art</span> for the past 40 years, and a long time Friend of <span class="companylink">The New York Public Library</span>. And, in his characteristic modest, unassuming manner, he was a member of the American MENSA, the world-wide society for exceptionally high intelligence quotient individuals. After enduring prolonged suffering, David passed away from Parkinson's Disease, that still incurable illness. He is survived by a son, Alan Goldberg, a professor in California, and also a companion of long years, Irene Field, who he deeply loved, and to whom he was so deeply devoted. Daniel L. Goldberg was truly a special person, dedicated to Engineering, from childhood, and blessed with so many other fine attributes. He was loved and respected by all who knew him, and all whose lives he touched, and he will be greatly missed. With everlasting love. Irene</p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>nobt : Obituaries | ncat : Content Types | nfact : Factiva Filters | nfce : FC&E Exclusion Filter | niwe : IWE Filter | nrgn : Routine General News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>usa : United States | nyc : New York City | usnj : New Jersey | usny : New York State | namz : North America | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>Classified | Paid Death Notice | GOLDBERG, DANIEL LOUIS</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>The New York Times Company</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020140518ea5i0000c</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020121220e8ch0001a" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Metropolitan Desk; SECTA</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>Its Restaurants Empty And Its Trains Stalled, Hoboken Encounters Storm's Increasing Toll</span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By RAY RIVERA; Nate Schweber contributed reporting.   </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>1309 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>17 December 2012</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>22</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 2012 The New York Times Company.  All Rights Reserved.   </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >CORRECTION APPENDEDHOBOKEN, N.J. -- It is happy hour at the Hotel Victor Bar and Grill on a Friday and the place is nearly empty. The evening commuters who would normally be emerging from the PATH train station across the street about this time, filling streets with thousands of potential customers, are nowhere in sight.</p>
<p class="articleParagraph enarticleParagraph" >''We should be booming right now,'' David Rousso, the bar's owner, said.</p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >Hurricane Sandy inflicted an estimated $7 billion in damage on transit systems across the region. Harder to calculate is the toll on the economy leveled by commuter disruptions.</p>
<p class="articleParagraph enarticleParagraph" >And perhaps no place has felt that impact more acutely than Hoboken. This city of 50,000 on the banks of the Hudson River thrives on its easy access to trains, buses and ferries into Manhattan. According to census surveys, an estimated 56 percent workers here use public transportation every day, surpassing New York City as the most transit-reliant community in the nation.</p>
<p class="articleParagraph enarticleParagraph" >That convenience has played a crucial role in the city's evolution from a rough-hewed factory town to an enclave of upscale bars and restaurants and pricey condominiums with unfettered views of the Manhattan skyline.</p>
<p class="articleParagraph enarticleParagraph" >But as the hurricane ripped through the region, seawater flooded half the city, and crippled the PATH station at Hoboken Terminal. Now, even as commuter services have returned to normal in much of the region, the station remains closed, severing a critical artery for workers traveling to and from Manhattan.</p>
<p class="articleParagraph enarticleParagraph" >Nearly 30,000 passengers use the station every weekday, according to the <span class="companylink">Port Authority of New York and New Jersey</span>, which operates the PATH system.</p>
<p class="articleParagraph enarticleParagraph" >Many who use the station come from neighboring communities to connect to the PATH -- that is, when it is open.</p>
<p class="articleParagraph enarticleParagraph" >Closed, it has cast a pall over the city, doubling commute times for residents, many of whom are having to pay more for transit alternatives like ferries. It has also slowed the foot traffic around the terminal that is critical to downtown businesses. In interviews, downtown business owners who were lucky enough not to lose their store to flooding reported drops in sales of 25 percent to 70 percent.</p>
<p class="articleParagraph enarticleParagraph" >Adding to the frustration is the uncertainty over when the station will reopen. <span class="companylink">The Port Authority of New York and New Jersey</span> has declined to give estimates, saying only that it could take weeks.</p>
<p class="articleParagraph enarticleParagraph" >''Nobody is telling anybody anything,'' said Mr. Rousso, a former Wall Street trader who opened the bar in April after spending thousands of dollars to renovate it. ''There's been nothing from government. There's been nothing from the <span class="companylink">Port Authority</span>. So we're sitting here in the dark. How long do you stick it out.''</p>
<p class="articleParagraph enarticleParagraph" >Mr. Rousso estimates that business is down by more than half on any given day in what should be one of the busiest times of the year.</p>
<p class="articleParagraph enarticleParagraph" >''Holiday time usually picks up your first three months of the new year, because those are slow,'' he said.</p>
<p class="articleParagraph enarticleParagraph" >By chance, Mr. Rousso produced a commercial for the bar before the storm hit. It began airing on local television last week but has so far ''fallen on deaf ears,'' he said.</p>
<p class="articleParagraph enarticleParagraph" >No one has tallied the economic losses of transit disruptions across the region. But in a 2011 report for New York State, Dr. Klaus H. Jacob, a climate scientist at the Lamont-Doherty Earth Observatory at <span class="companylink">Columbia University</span>, estimated that a storm like Hurricane Sandy could cause $10 billion in physical damage to the region's public transportation systems and another $48 billion in economic damages caused by transportation disruptions.</p>
<p class="articleParagraph enarticleParagraph" >By the time everything is counted, he said in an interview, ''I believe our predictions will be eerily close.''</p>
<p class="articleParagraph enarticleParagraph" >Even businesses blocks away from the Hoboken Terminal have had a significant drop in sales.</p>
<p class="articleParagraph enarticleParagraph" >Hoboken on Rye, a kosher-style deli on First Street, sits about halfway between the PATH terminal and the Second Street station of the Hudson-Bergen Light Rail, which many people take to Hoboken to transfer to the PATH, as the <span class="companylink">Port Authority</span> Trans-Hudson railway is known.</p>
<p class="articleParagraph enarticleParagraph" >''When the PATH is running, they go from there and they have to go past my building,'' said Dennis Blair, 63, who owns the deli with his son. ''It's like cattle.''</p>
<p class="articleParagraph enarticleParagraph" >Without that foot traffic, Mr. Blair said, business is down about 25 percent. A sign in his window reads: ''Shop Local, Help Rebuild Hoboken.''</p>
<p class="articleParagraph enarticleParagraph" >The sign is one of many that have appeared in city storefronts after the storm. Seven weeks later, the physical damage is still apparent. Shops going west along First Street past Garden Street, which became a river during the storm, remain shuttered, their floors and walls stripped out.</p>
<p class="articleParagraph enarticleParagraph" >Signs on the city's flooded Multi-Service Community Center, which houses a number of city services, directs people to makeshift locations. The city estimates it suffered about $10 million dollars of damage to its buildings and infrastructure, not including losses to private businesses and homes.</p>
<p class="articleParagraph enarticleParagraph" >Officials are still trying to assess the total damage the storm wrought on the city. ''We don't have total estimates yet,'' the city's mayor, Dawn Zimmer, said. ''But I know from walking around, from talking to different business owners, that they're definitely hard hit.''</p>
<p class="articleParagraph enarticleParagraph" >The ripples are being felt across the Hudson River in New York.</p>
<p class="articleParagraph enarticleParagraph" >Vibeke Koszeghy, an architect who lives in Hoboken, used to take the 8 a.m. PATH train to arrive at her Union Square office by 8:30. Now, to get to work on time, she said, she takes a 6:20 or 6:30 a.m. bus.</p>
<p class="articleParagraph enarticleParagraph" >''I just changed my life,'' she said waiting for a bus along Washington Street early on a recent morning.</p>
<p class="articleParagraph enarticleParagraph" >It's also changed her evening routine, sharply reducing the amount of time she spends in Manhattan to take advantage of her limited travel options back home. A regular arts patron, she recently gave away tickets to Lincoln Center.</p>
<p class="articleParagraph enarticleParagraph" >''It has changed people's lives to the point they no longer do anything in the city,'' she said. ''That's really hurting both people's social lives and New York business.''</p>
<p class="articleParagraph enarticleParagraph" >At the WindMill, a New Jersey fast-food franchise half a block from the terminal, the owners have sharply cut back the hours of their 20 employees. The restaurant typically stays open until 1 a.m. on weekdays and 4 a.m. on weekends, and depends on the nearby bar crowd for late-night customers.</p>
<p class="articleParagraph enarticleParagraph" >''Hoboken is known as a bar town,'' said Tim Cochrane, another former Wall Street trader and one of the co-owners. ''The price of a drink here is half the price of a drink in Manhattan. So you get a lot of young professionals who come over on the PATH and hit the bars. And what better than chili cheese fries and a hot dog before you go home?''</p>
<p class="articleParagraph enarticleParagraph" >Come 2 a.m. on a typical Friday or Saturday night, ''and you can't get through here,'' he said. ''So we've lost that.''</p>
<p class="articleParagraph enarticleParagraph" >Correction: December 20, 2012, Thursday</p>
<p class="articleParagraph enarticleParagraph" >This article has been revised to reflect the following correction: An article on Monday about Hoboken's struggles after Hurricane Sandy referred incorrectly to the location of Hoboken on Rye, a kosher-style deli that is one of many businesses that have seen a drop in sales since the storm. It is on First Street (not Avenue).</p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>ART</b>&nbsp;</td><td><br/>Hurricane Sandy crippled  the PATH terminal  in Hoboken, severing a  critical artery to Manhattan  across the Hudson  River, at left. ''We  should be booming right  now,'' said the owner of  Hotel Victor Bar and  Grill, below, which is  near the terminal. (PHOTOGRAPHS BY BRIAN HARKIN FOR THE NEW YORK TIMES)   </td></tr>
<tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>gcat : Political/General News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>usa : United States | nyc : New York City | usnj : New Jersey | usny : New York | namz : North America | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>Metropolitan Desk</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>The New York Times Company</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020121220e8ch0001a</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020121210e8c90001i" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>CLM</b>&nbsp;</td><td>LIVING IN</td></tr>
<tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Real Estate Desk; SECTRE</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>Land of Light Taxes and Heavy Shopping</span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By DAVE CALDWELL   </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>1711 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>9 December 2012</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>7</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 2012 The New York Times Company.  All Rights Reserved.   </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >EAST HANOVER wasn't spared much misery when the hurricane struck in late October. Trees were downed and power was out as long as two weeks. Then, once lights and heating came back on, Lisa Beck recalled, an electrical surge almost immediately sparked a fire at her neighbor's house.</p>
<p class="articleParagraph enarticleParagraph" >Mrs. Beck, who graduated from Hanover Park High School and has lived in East Hanover ''my whole adult life,'' said the mayor, Joseph Pannullo, and a priest soon visited the neighborhood, damaged by the storm, to see if everyone was all right.</p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >''It wasn't a political thing, and it wasn't a join-my-church thing,'' Mrs. Beck said. ''It was a caring-community thing.''</p>
<p class="articleParagraph enarticleParagraph" >Many residents of many towns have helped one another in the aftermath of Sandy, but Mrs. Beck believes that being a good neighbor is standard behavior in East Hanover, the Morris County township of 11,000 that is probably best known for State Route 10 -- the strip with, it seems, every big-box store there is.</p>
<p class="articleParagraph enarticleParagraph" >In August Mrs. Beck and her husband, John, put their four-bedroom colonial on the market for $942,500 because their two children are grown and they want to downsize. (They paid $350,000 for the house, new, in 1995.) But they want to stay in the area. To Mrs. Beck, East Hanover feels more like a small town than a suburb.</p>
<p class="articleParagraph enarticleParagraph" >Flavia and John Viana had friends who lived here, and in May they bought a three-bedroom two-bath renovated bilevel on a half-acre lot for $475,000. They moved from Newark with their 4-year-old daughter, who is in prekindergarten, because the price was right and public schools are considered good.</p>
<p class="articleParagraph enarticleParagraph" >Then along came Hurricane Sandy. ''I was really pleased with the way the community took care of everything every day,'' Ms. Viana said. ''That made me feel better about our purchase. For a small town, they really do a lot for their residents.''</p>
<p class="articleParagraph enarticleParagraph" >A shelter was set up at East Hanover Middle School after the hurricane and stayed open until power was restored to every township resident, Mayor Pannullo said. He provided updates to residents who had phone service. Then he drove around in a fire truck or police cruiser, using a bullhorn to deliver updates. ''It's a little town,'' he said, ''so you get to know everybody.''</p>
<p class="articleParagraph enarticleParagraph" >The township, he is pleased to say, has endured the economic downturn without laying off employees. Mr. Pannullo, who moved to East Hanover in 1969 to set up a pest control business from which he is now retired, is one year into his second four-year term as mayor. In two of those years, there has been no tax increase.</p>
<p class="articleParagraph enarticleParagraph" >A lot has changed since he moved into a small Cape (with a septic tank and no gas service) in a development across from a farm. Although the population had tripled from 1950, the 1970 census still counted only 7,734 residents.</p>
<p class="articleParagraph enarticleParagraph" >East Hanover was then a destination, in a manner of speaking: it had three cemeteries on Ridgedale Avenue, the largest two of which (Gate of Heaven and Restland Memorial Park) took over farms during the Depression. An article in The <span class="companylink">New York Times</span> on May 28, 1972, pointed out that the dead outnumbered the living in East Hanover, 9 to 1.</p>
<p class="articleParagraph enarticleParagraph" >But Interstate 280 was being built through the northeast part of town, with two local exits. Mayor Pannullo said of the road, finished in 1973, ''Once that opened up, it just opened up the gates.''</p>
<p class="articleParagraph enarticleParagraph" >WHAT YOU'LL FIND</p>
<p class="articleParagraph enarticleParagraph" >State Route 10 has a <span class="companylink">Costco</span>, a Marshalls, a <span class="companylink">Best Buy</span>, a <span class="companylink">Dick's Sporting Goods</span> outlet, a <span class="companylink">Home Depot</span>, a Party City, a Target, a Michaels, and a <span class="companylink">Bed Bath and Beyond</span>. A bowling alley hunches next to a cineplex. Name a chain restaurant, and you can probably find it along Route 10.</p>
<p class="articleParagraph enarticleParagraph" >But the commercial area accounts for only the southern third of this eight-square-mile township 25 miles west of New York. Its presence -- and that of <span class="companylink">Novartis</span>, the pharmaceutical company, and <span class="companylink">Mondelez International</span>, formerly Kraft -- has helped keep taxes low.</p>
<p class="articleParagraph enarticleParagraph" >''The taxes were a huge drawing point,'' said Bill Cowan, who in April moved with his wife, Alyssa, into a three-bedroom one-and-a-half-bath 1962 split-level on nearly half an acre. They paid $420,000 for the house, about which Mr. Cowan said: ''I always joke with her that I never want to leave.''</p>
<p class="articleParagraph enarticleParagraph" >In 1981, when Terrie K. O. Neuman, the broker-owner of TKO Properties, moved with her family into a four-bedroom two-and-a-half-bath colonial on two-thirds of an acre, Route 10 still was surrounded by farmland. She remembers stopping at the farms to pick up bread, milk and eggs. The house cost $117,000; now, she says, it has a value of about $750,000.</p>
<p class="articleParagraph enarticleParagraph" >These days Route 10 thickens with traffic on weekends and especially around the holidays. But it can be avoided. The township has two east-west arteries, Ridgedale Avenue and River Road; there is a small cluster of local businesses, plus a well-used public park and a library, on Ridgedale.</p>
<p class="articleParagraph enarticleParagraph" >''Route 10 is Route 10 -- you can choose your battles,'' said Joe Califano, a Bergen County police officer who with his wife, Dawn, recently bought a four-bedroom two-and-a-half-bath colonial on half an acre for $460,000. ''You don't go on Route 10 at 5 o'clock in the afternoon. You use the back roads.''</p>
<p class="articleParagraph enarticleParagraph" >Subdivision styles run the gamut, from ranches and split-levels to more recently built homes like the one the Becks are trying to sell. The 2011 tax bill on their property is $10,123. The tax bill on a similar property for sale in neighboring Livingston is $20,566.</p>
<p class="articleParagraph enarticleParagraph" >The draw ''is the low taxes and proximity to all the major highways,'' said Gloria LaForgia, an agent for <span class="companylink">Weichert Realtors</span> who has lived in East Hanover since 1988. ''And you're 20 minutes from all the major malls.''</p>
<p class="articleParagraph enarticleParagraph" >WHAT YOU'LL PAY</p>
<p class="articleParagraph enarticleParagraph" >A recent look at gsmls.com, the Garden State Multiple Listing Service Web site, revealed 36 residential properties for sale. Prices ranged from $249,900, for a two-bedroom one-and-a-half-bath first-floor condominium built in 1993, to $942,500 for a four-bedroom three-and-a-half-bath colonial built in 1995. A four-bedroom two-bath bilevel built in 1979 on a 0.25 acres, listed at $479,000, had an annual tax bill of $6,963.</p>
<p class="articleParagraph enarticleParagraph" >According to the listing service, the average sale price from Jan. 1 to Nov. 27 was $457,186, or about 5 percent less than the corresponding period in 2011. Prices ranged from $255,000, for a 55-plus community condo, to $958,000 for a single-family home.</p>
<p class="articleParagraph enarticleParagraph" >According to <span class="companylink">Trulia</span>, the median sale price in East Hanover from August to October was $510,000, or 27 percent higher than the median a year earlier. The median five years ago was $498,000. The median reached $580,000 late in 2008. (The median represents a midpoint, with half the sales for more and half for less; unlike the average, it tends to be less affected by one or two very high or low sales.)</p>
<p class="articleParagraph enarticleParagraph" >THE SCHOOLS</p>
<p class="articleParagraph enarticleParagraph" >About 350 students from kindergarten through Grade 2 attend Frank J. Smith School. Average class size is 19.6; the state average is 19. Central School has about 350 students in Grades 3 through 5; East Hanover Middle School has about 400 in 6 through 8.</p>
<p class="articleParagraph enarticleParagraph" >About 800 students from East Hanover and neighboring Florham Park attend Hanover Park High School, in East Hanover. SAT averages in 2011 were 534 in math, 503 in reading and 521 in writing, versus 517, 493 and 496 statewide. The graduation rate for the class of 2011 was about 94 percent, versus 83.2 percent statewide.</p>
<p class="articleParagraph enarticleParagraph" >THE COMMUTE</p>
<p class="articleParagraph enarticleParagraph" >Many residents take <span class="companylink">New Jersey Transit</span>'s Midtown Direct from the station in Madison, about six miles from East Hanover. There are eight trains to New York from 5 to 10 a.m. on weekdays, and eight trains to Madison from 4 to 8 p.m. The one-way trip takes about 55 minutes. A monthly pass is $308. Parking is $5 daily.</p>
<p class="articleParagraph enarticleParagraph" >Others drive to the Port Authority Trans-Hudson train station in Harrison, off Interstate 280, about 20 minutes east of East Hanover. The train ride to Lower Manhattan from Harrison takes 20 minutes. An unlimited 30-day PATH pass costs $73. Parking at the Harrison station is $195 a month.</p>
<p class="articleParagraph enarticleParagraph" >WHAT TO DO</p>
<p class="articleParagraph enarticleParagraph" >Dotted with soccer goals and baseball backstops, Joseph G. Lurker Park on Ridgedale Avenue is the epicenter of an extensive recreation program. Signs abounded lately through town beckoning girls to register for basketball.</p>
<p class="articleParagraph enarticleParagraph" >Annual membership to the community pool last year was $60 per resident. The township offers an adult coed volleyball program and a seniors' aerobic program. Funplex, on Route 10, a popular arcade and ''amusement center,'' is a haven for kids' birthday parties. Applegate Farm runs an ice cream stand on Ridgedale Avenue.</p>
<p class="articleParagraph enarticleParagraph" >Livingston Mall is 15 minutes away; the Mall at Short Hills, 20 minutes. Mr. Califano, the police officer, did mention a drawback to East Hanover: ''No supermarkets around,'' he said, ''but that's the only inconvenient part.''</p>
<p class="articleParagraph enarticleParagraph" >THE HISTORY</p>
<p class="articleParagraph enarticleParagraph" >According to the township Web site, European settlers bought several thousand acres from the native Unami in 1708. In addition to cash, the Unami received barter including 15 kettles, 20 axes, 20 hoes, 10 duffel blankets, half a barrel of wine, a barrel of rum, a barrel of cider, three files, a gun bore, an auger, 4 pistols, 4 cutlasses, 10 guns, 100 bars of lead, 20 shirts and 100 knives.</p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>ART</b>&nbsp;</td><td><br/>WREATHED:  Ridgedale  Avenue is one of  two arteries  popular with  East Hanover  residents as an  alternative to the  sometimes  traffic-clogged  big-box corridor  on State Route  10. The upside of  the commercial  bustle is the low  property taxes.  (PHOTOGRAPH BY JUAN ARREDONDO FOR THE NEW YORK TIMES) MAPS   </td></tr>
<tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i85 : Real Estate Transactions | icre : Construction/Real Estate | ireest : Real Estate</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>ccat : Corporate/Industrial News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>usa : United States | namz : North America</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>Real Estate Desk</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>The New York Times Company</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020121210e8c90001i</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020121207e8c70007x" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Metropolitan Desk; SECTA</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>Transportation Officials Ask Congress for Billions in Storm Recovery Effort</span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By RAY RIVERA   </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>821 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>7 December 2012</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>32</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 2012 The New York Times Company.  All Rights Reserved.   </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >WASHINGTON -- It will cost more than $7 billion to repair the damage that Hurricane Sandy did to the New York area's antiquated transit systems and billions more to upgrade the systems to withstand major storms in the future, officials from the region told a <span class="companylink">United States Senate</span> panel on Thursday.</p>
<p class="articleParagraph enarticleParagraph" >The subway stations, tunnels and tracks managed by the <span class="companylink">Metropolitan Transportation Authority</span>, the nation's largest public transportation system, sustained the worst of the destruction, with estimated damage of more than $5 billion, Joseph J. Lhota, the authority's chairman said.</p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >''This figure represents just what we need to get the system back to where it was the day before the storm,'' Mr. Lhota said. Stressing that the region is relying on transportation networks that in some cases are more than a century old, Mr. Lhota and other transportation officials from New York and New Jersey said that it would cost substantially more to upgrade the systems to make them better prepared for storms that could come more frequently, and with greater intensity, with climate change and rising sea levels. ''It's critical that we make the critical investments we need to protect our system from future storms,'' Mr. Lhota said.</p>
<p class="articleParagraph enarticleParagraph" >Securing money beyond the cost of immediate repairs could be difficult at a time when Congress and the Obama administration are jockeying over tax increases and spending cuts, the so-called fiscal cliff.</p>
<p class="articleParagraph enarticleParagraph" >New York and New Jersey lawmakers testifying before a Senate Transportation Committee panel urged Congress to invest in transit infrastructure in the region, which accounts for more than 10 percent of the nation's gross domestic product and depends on mass transit more than any other area of the country.</p>
<p class="articleParagraph enarticleParagraph" >''It's the life blood of New York,'' Senator Charles E. Schumer, a New York Democrat, told the panel.</p>
<p class="articleParagraph enarticleParagraph" >Mr. Lhota did not provide an estimate of how much more would be needed to upgrade the systems, but other speakers did.</p>
<p class="articleParagraph enarticleParagraph" >James Weinstein, executive director of New Jersey Transit, provided the most detailed account yet of damage sustained by that system. The agency, he said, sustained an estimated $400 million in damage and would need $800 million more to make the system more resilient to future storms.</p>
<p class="articleParagraph enarticleParagraph" >The damage included more than $100 million in damaged rail equipment, including rolling stock, and an additional $300 million to fix and replace tracks, wires, signaling, electrical substations and other equipment, Mr. Weinstein said.</p>
<p class="articleParagraph enarticleParagraph" ><span class="companylink">New Jersey Transit</span> has come under intense criticism from riders and officials for parking more than 300 passenger cars and locomotives in rail yards in Hoboken and Kearny that ended up under water during the storm.</p>
<p class="articleParagraph enarticleParagraph" >But the Senate panel largely spared Mr. Weinstein of having to defend the agency's actions on Thursday. Senator Frank R. Lautenberg, a New Jersey Democrat and panel member who called the hearing, briefly raised the issue, but then seemed content to accept Mr. Weinstein's defense that it was the best decision based on forecasts and flood histories of the two locations at the time.</p>
<p class="articleParagraph enarticleParagraph" >''It doesn't sound like there were a lot of choices to be made,'' Mr. Lautenberg said.</p>
<p class="articleParagraph enarticleParagraph" >Mr. Weinstein could face tougher questioning next week in front of a hearing before the transportation committee of the New Jersey Assembly.</p>
<p class="articleParagraph enarticleParagraph" >Joseph Boardman, the president of Amtrak, said the regional carrier was requesting $336 million in emergency federal funding. Of that amount, $60 million would be used to cover estimated operating costs caused by the storm, which flooded four of Amtrak's seven tunnels leading in and out of New York's Pennsylvania Station and damaged a major electrical substation in Kearny.</p>
<p class="articleParagraph enarticleParagraph" >The remaining $276 million would be used to make improvements to strengthen the system. It would also be used to begin design and construction of Amtrak's plan to add two new tunnels under the Hudson River between New York and New Jersey.</p>
<p class="articleParagraph enarticleParagraph" >Patrick J. Foye, executive director of the <span class="companylink">Port Authority of New York and New Jersey</span>, said the authority was still assessing the exact costs of the damage, and what insurance might cover.</p>
<p class="articleParagraph enarticleParagraph" >''But clearly our needs are enormous,'' he said. ''We are facing hundreds of millions of dollars in immediate repair costs, and billions more in mitigation and resiliency measures.''</p>
<p class="articleParagraph enarticleParagraph" >The authority sustained substantial damage in addition to its airports, shipping terminals and tunnels, Mr. Foye said. But the <span class="companylink">Port Authority</span> Trans-Hudson transit system, which carries some 77 million people between New Jersey and Manhattan each year, was hit the worst.</p>
<p class="articleParagraph enarticleParagraph" >The PATH's Hoboken Terminal, which serves about 29,000 people every weekday, remains closed, and officials have not given an estimate of when service will be fully restored.</p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>ART</b>&nbsp;</td><td><br/>Joseph J. Lhota, left, Metropolitan Transportation Authority chairman, in Washington with Senator Charles E. Schumer. (PHOTOGRAPH BY STEPHEN CROWLEY/THE NEW YORK TIMES)   </td></tr>
<tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>gtrans : Transport | gcat : Political/General News | gpol : Domestic Politics | gvcng : Legislative Branch | gpir : Politics/International Relations | gplan : Urban Planning/Development | gvbod : Government Bodies</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>usa : United States | usnj : New Jersey | usny : New York | namz : North America | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>Metropolitan Desk</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>The New York Times Company</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020121207e8c70007x</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020120115e81f0005c" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>CLM</b>&nbsp;</td><td>LIVING IN  ESSEX FELLS, N.J.</td></tr>
<tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Real Estate Desk; SECTRE</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>Holding Fast to Old-Fashioned Ways</span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By DAVE CALDWELL   </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>1663 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>15 January 2012</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>8</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 2012 The New York Times Company.  All Rights Reserved.   </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >EVERY year about this time in Essex Fells, after the first wintry gusts have swept the leaves off the trees and residents have gathered around to watch the annual lighting of the borough's Christmas tree, a large pond off Fells Road turns into the borough's meeting place.</p>
<p class="articleParagraph enarticleParagraph" >This is where residents come, in droves, to ice skate. Some race; others glide in slow circles. And there are those who do not skate at all: someone might build a fire, and someone else might order pizza to be delivered, and they catch up on the latest local news.</p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >''Once the kids get into elementary school, your world opens up, to your kids' friends and their families,'' said Debbie McWilliams, who grew up in Essex Fells and moved back 36 years ago to raise a family with her husband, Tim.</p>
<p class="articleParagraph enarticleParagraph" >With 2,113 residents nestled into 1.3 square miles of hilly, tree-lined terrain, Essex Fells is the smallest community in Essex County and also one of its most desirable. In August, it was the only municipality in the county to make New Jersey Monthly magazine's list of top 10 places to live in the state.</p>
<p class="articleParagraph enarticleParagraph" >Essex Fells has no commercial district; residents shop for food and run errands in the nearby communities of Caldwell, North Caldwell, West Orange and Montclair. Hubs of activity, besides the pond in winter, are the elementary school and the fire hall.</p>
<p class="articleParagraph enarticleParagraph" >''It's so great to be mayor of this town, because everybody volunteers,'' said Ed Abbot, a Manhattan lawyer who has lived in Essex Fells since 1992 and has been its part-time mayor since 2002. ''Our town's so small, but we have a huge fire department. It's a very social town. Everybody knows everybody.''</p>
<p class="articleParagraph enarticleParagraph" >It is not cheap to live in Essex Fells: of the 17 houses listed recently on the Garden State Multiple Listing Service's Web site, seven were priced over $1 million. But the borough sells water to three neighboring communities, lowering residents' tax bills.</p>
<p class="articleParagraph enarticleParagraph" >Essex Fells is also able to maintain its own police force. According to New Jersey State Police statistics, there were only 17 crimes, 15 nonviolent, reported in the borough in 2009 and 2010 combined -- which amounted to the lowest crime rate in Essex County in that period.</p>
<p class="articleParagraph enarticleParagraph" >Kevin Kinney of Rhodes, Van Note Realtors in Upper Montclair, said: ''Every time I drive clients through there, they wonder where they are. They ask, 'Are we still in New Jersey?' ''</p>
<p class="articleParagraph enarticleParagraph" >Debbie and Tim McWilliams grew up in Essex Fells and returned, which does not make them all that unusual. In 1976 they bought a one-bedroom house, expanding it to three and then moving, six years later, to a five-bedroom four-bath colonial that, at $190,000, was ''a little out of our price range,'' she recalled.</p>
<p class="articleParagraph enarticleParagraph" >They have lived there ever since, and even though they could sell the house for perhaps six or seven times what they paid, they are staying put. Their 33-year-old daughter, Carleigh, is expecting a child, and she and her husband are interested in moving from New York.</p>
<p class="articleParagraph enarticleParagraph" >''I talk to them about it every day,'' Debbie McWilliams said, laughing.</p>
<p class="articleParagraph enarticleParagraph" >Susan Carpenter, a bank vice president in Manhattan, grew up in Queens but moved to New Jersey, first to Clifton, where she lived until eight years ago. She and her husband, Rick, then bought a three-bedroom one-and-a-half-bath colonial near Essex Fells School for $560,000.</p>
<p class="articleParagraph enarticleParagraph" >Mr. Carpenter grew up in Essex Fells. He works in sales from home, and the couple's 7-year-old daughter, Caroline, is in first grade at the school, one block away. There is a playground nearby and preserved woodland behind their house.</p>
<p class="articleParagraph enarticleParagraph" >''You can see deer walking through,'' Mr. Carpenter said.</p>
<p class="articleParagraph enarticleParagraph" >Essex Fells was hit hard by Tropical Storm Irene in August and by a Halloween snowstorm that cut power and downed trees, taxing the fire department, which is staffed by about 40 volunteers. Firefighters pumped out basements flooded by the August storm, enabling an eventual clean-up.</p>
<p class="articleParagraph enarticleParagraph" >Then, in December, a fire destroyed a local house. Chris Boeckel, who recently retired as the fire chief, said his wife, Patrice, started a fund-raiser that generated ''tens of thousands of dollars'' for the displaced family. Mr. Boeckel's two sons are also volunteer firefighters.</p>
<p class="articleParagraph enarticleParagraph" >WHAT YOU'LL FIND</p>
<p class="articleParagraph enarticleParagraph" >The Carpenter family often walk to Bloomfield Avenue in much busier Caldwell, about a mile away, to run errands on weekends. Then they retreat to Essex Fells, a town they say seems more like a neighborhood, its streets lined with houses often identified by the names of their previous owners.</p>
<p class="articleParagraph enarticleParagraph" >The borough is in the western part of Essex County, surrounded by the communities of Roseland and West Orange to the south, Verona to the east and West Caldwell and Caldwell to the north. Two exits of Interstate 280 are accessible via a short drive through Roseland, minutes away.</p>
<p class="articleParagraph enarticleParagraph" >Two thoroughfares run parallel through the borough, which has an old-fashioned ambience surprising in a place easily accessible to the city. One, Fells Road, winds from the northeast corner to the southwest; the other, Roseland Road, runs from Roseland to Bloomfield Avenue in Caldwell.</p>
<p class="articleParagraph enarticleParagraph" >House styles are varied: rambling colonials, '60s ranches, new homes faced in stone -- and many have large lawns, some framed by stone walls. The houses sit back from the winding streets and next to tall trees, which add privacy. To a visitor, the atmosphere seems to date to an earlier era.</p>
<p class="articleParagraph enarticleParagraph" >Mayor Abbot put it this way: ''The nicest thing I can hear is from someone who's come back after a number of years and says, 'This town has not changed.' I have done my job.''</p>
<p class="articleParagraph enarticleParagraph" >WHAT YOU'LL PAY</p>
<p class="articleParagraph enarticleParagraph" >Properties for sale, according to the Garden State Multiple Listing Service, range from a two-bedroom 2010 town house, for $412,900, to a 10-bedroom colonial on 1.38 acres, built in 1910 and renovated in 1995, for $1.989 million. In the middle, at $679,900, is a five-bedroom colonial built in 1980, with an annual tax bill of $11,814.</p>
<p class="articleParagraph enarticleParagraph" >Kevin M. Esposito, the borough's tax assessor, said 17 houses sold in Essex Fells in 2010; the average sale price for a condominium was $398,195, and the average for a single-family home was $941,714. He said 21 houses sold in the corresponding period this year; the average sale price for a condo was $421,667, and the average for a single-family house was $866,590.</p>
<p class="articleParagraph enarticleParagraph" >According to Trulia.com, the median sale price in Essex Fells from July to September was $865,000, or 11.2 percent lower than the median in the corresponding period one year earlier. Gail Bradley, an agent for Arcadia Realtors in Roseland, says the market in Essex Fells does not differ all that much from those in surrounding communities; she says that a house will sell if priced right.</p>
<p class="articleParagraph enarticleParagraph" >THE COMMUTE</p>
<p class="articleParagraph enarticleParagraph" >Ms. Carpenter and Mr. Abbot say they are among several residents who commute to New York by driving to the Port Authority Trans-Hudson train station in Harrison, off Interstate 280, about 20 minutes east of Essex Fells. The train ride to Lower Manhattan from Harrison takes 20 minutes. An unlimited 30-day PATH pass costs $65. Additionally, five DeCamp express buses run to the Port Authority on weekday mornings from Roseland and Bloomfield Avenues in Caldwell. The trip takes about 50 minutes; a 40-trip ticket is $261.</p>
<p class="articleParagraph enarticleParagraph" >WHAT TO DO</p>
<p class="articleParagraph enarticleParagraph" >The borough has a recreation program that offers youth programs in soccer, basketball, baseball, softball and girls' lacrosse (founded by the McWilliamses), plus fencing for children over 6 and adults. Residents can also join the North Caldwell Community Pool; yearly family memberships are $455. There is a four-screen movie theater on Bloomfield Avenue in Caldwell. And Grover Cleveland Park, on the northern border, offers jogging, fishing and lighted tennis courts.</p>
<p class="articleParagraph enarticleParagraph" >Mrs. Carpenter, who describes herself as a ''shop-local girl,'' does the family's grocery shopping in West Caldwell. Mrs. McWilliams patronizes a local fishmonger and butcher, and the Whole Foods market in West Orange. Livingston Mall is about a 15-minute drive from the center of town, the Mall at Short Hills about 25 minutes.</p>
<p class="articleParagraph enarticleParagraph" >THE SCHOOLS</p>
<p class="articleParagraph enarticleParagraph" >Enrollment is about 250 at the Essex Fells School, which teaches prekindergarten through Grade 6. Average class size is 16.1; the state average is 18.2. Older students attend public schools operated by the West Essex Regional School District, which also serves Roseland, Fairfield and North Caldwell.</p>
<p class="articleParagraph enarticleParagraph" >West Essex Regional Middle School, in North Caldwell, has 575 students in Grades 7 and 8. West Essex Regional High School, also in North Caldwell, has about 1,000 students in Grades 9 through 12. SAT averages last year were 555 in math, 526 in reading and 537 in writing, versus 520, 496 and 499 statewide. The graduation rate for the class of 2010 was 99.6 percent, versus 94.7 percent statewide.</p>
<p class="articleParagraph enarticleParagraph" >THE HISTORY</p>
<p class="articleParagraph enarticleParagraph" >Essex Fells got its name because it is in Essex County, and a partner in its development was John R. Fell (though ''fells'' can also refer to hilly land). Upon learning that the Pennsylvania Railroad planned to establish service in the area, Mr. Fell's father-in-law, the Philadelphia banker Anthony Drexel, bought 1,000 acres in 1888, intending to build a residential community. The railroad came through in 1891, and Essex Fells became a borough in 1902. Train service was discontinued in 1966.</p>
<p class="articleParagraph enarticleParagraph" >PHOTO: PASSING BY: Fells Road winds through Essex Fells, a small but sought-after borough with a strong sense of community, fairly pricey real estate and reasonably low taxes. (PHOTOGRAPH BY AARON HOUSTON FOR THE NEW YORK TIMES)</p>
<p class="articleParagraph enarticleParagraph" >MAPS</p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i85 : Real Estate Transactions | icre : Construction/Real Estate | ireest : Real Estate</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>ccat : Corporate/Industrial News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>nyc : New York City | usnj : New Jersey | namz : North America | usa : United States | use : Northeast U.S. | usny : New York</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>The New York Times Company</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020120115e81f0005c</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020101103e6b3000ak" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Metropolitan Desk; SECTA</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>A Transit Bargain, Truly Underground</span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By MICHAEL M. GRYNBAUM   </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>1132 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>3 November 2010</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>19</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 2010 The New York Times Company.  All Rights Reserved.   </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >It is New York's other subway, home to oddities like hexagonal trains and sparkling-clean platform floors. Its odor, musty but not unpleasant, is unlike anything else in the city, and it stops at cross streets that can befuddle: Ninth Street in Manhattan? Really?</p>
<p class="articleParagraph enarticleParagraph" >With only six stations in the city, the diminutive PATH network has never quite captured the romance and notoriety of its bigger, better-known brother, the New York City subway.</p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >But though often forgotten, PATH may finally have a reason to brag: it is about to become the last truly underground bargain of New York.</p>
<p class="articleParagraph enarticleParagraph" >With subway fares set to rise sharply next year, a 30-day unlimited pass on PATH will soon cost half as much as a monthly MetroCard -- a difference that could save coupon-clipping New Yorkers about $600 a year.</p>
<p class="articleParagraph enarticleParagraph" >Call it a transit sample sale: even now, at $1.75 a ride, PATH is emerging as an increasingly desirable alternative to the subway, with its $2.25 one-way ticket, for a lucky subset of Manhattanites who happen to live along its small but unique diagonal route.</p>
<p class="articleParagraph enarticleParagraph" >''I like to think of it as a little neighborhood secret,'' said Casey Smith, a former subway rider who abandoned the No. 1 train after discovering the Christopher Street PATH station a few blocks from his West Village apartment.</p>
<p class="articleParagraph enarticleParagraph" >Built as a connection between New Jersey and New York's business districts, PATH makes stops in the West Village, Chelsea and Midtown before terminating a block from Pennsylvania Station. (A separate line runs to the World Trade Center.)</p>
<p class="articleParagraph enarticleParagraph" >To the uninitiated, a ride on the PATH train might seem like stumbling into a Bizarro World subway. The familiar wooden benches of the subway station are replaced by curvy seats made from silver plastic. The ubiquitous security warning -- ''If You See Something, Say Something'' -- is rendered in PATHese: ''You Call It In, We Check It Out.'' The signage has a less graceful version of the subway's Helvetica, and the Corinthian columns are painted an oddly vivid blue.</p>
<p class="articleParagraph enarticleParagraph" >But regular riders say that PATH trains are more frequent, cleaner and quieter than subway trains. ''Significantly less screeching,'' said Mr. Smith, who, like many converts to PATH, stumbled upon the railroad months after moving into his neighborhood. In two years of daily rides on PATH, he said he had never waited longer than five minutes for a train.</p>
<p class="articleParagraph enarticleParagraph" >It is difficult to measure how many PATH passengers ride the train solely within Manhattan, and officials at the <span class="companylink">Port Authority of New York and New Jersey</span>, which operates the railroad, say that those customers have traditionally made up a small percentage of the railroad's New Jersey-centric ridership.</p>
<p class="articleParagraph enarticleParagraph" >Dozens of riders can be spotted boarding Midtown-bound trains on weekday mornings at the PATH stations in Manhattan. With the price of a monthly MetroCard going up to $104 next year, longtime PATH riders say they expect to see more riders coming through the turnstiles of that railroad, where a 30-day pass costs $54.</p>
<p class="articleParagraph enarticleParagraph" >A modern, perk-filled transit system has always seemed out of reach for New York's embittered subway riders, who have resigned themselves to a seemingly endless cycle of service cuts, fare increases and bungled attempts at improved technology.</p>
<p class="articleParagraph enarticleParagraph" >Despite its lower cost, PATH offers high-tech features, like tap-and-go electronic fare cards and onboard television screens that stream news and weather reports -- amenities that the troubled New York City subway is still struggling to install.</p>
<p class="articleParagraph enarticleParagraph" >''It's faster and more pleasant, and it gives you a discount, which I always like,'' said Tina DiFeliciantonio, a television producer, who takes her 7-year-old son, Luca, on PATH each day from their home in Chelsea to his school in the West Village. She said the school's proximity to the PATH train was a factor in deciding to enroll her son there. ''It's cheaper than taking the 1 train,'' she said.</p>
<p class="articleParagraph enarticleParagraph" >The idea for an underground railroad across the Hudson River dates back to the Gilded Age, but PATH, then known as the Hudson & Manhattan Railroad, did not begin operating until 1907, three years after the city's subway opened. It was renamed the Port Authority Trans-Hudson after its current operator took over in 1962.</p>
<p class="articleParagraph enarticleParagraph" >Many of PATH's perks stem from its small size, which makes the railroad easier and cheaper to clean and maintain. The system handles about 245,000 trips on an average weekday, fewer than half of the trips taken on the No. 6 subway line alone, and its 14 stations in New York and New Jersey offer only a small fraction of the reach of the sprawling city subway.</p>
<p class="articleParagraph enarticleParagraph" >PATH's small size has also made the railroad an attractive target for technology companies seeking a test subject for initiatives, like the computer-chip-enabled fare cards now offered on the railroad.</p>
<p class="articleParagraph enarticleParagraph" >PATH is also outside the purview of the Metropolitan Transportation Authority, providing insulation from the authority's financial troubles. (Many New Yorkers' only experience with PATH came when the railroad stayed open during the subway strike of 2005.) The Port Authority has a stronger balance sheet, and the political pressures of a two-state agency have helped keep its fares low. The transportation authority recently approved the third subway fare increase in three years; fares on PATH have gone up twice in the last decade.</p>
<p class="articleParagraph enarticleParagraph" >Still, subway officials insist they are not envious. Asked whether PATH may siphon away riders with its cheaper fares, Charles Seaton, a subway spokesman, said, ''I don't believe it's ever been a concern.'' He noted that the two were too distinct to compare.</p>
<p class="articleParagraph enarticleParagraph" >New York City Transit, the subway's parent agency, includes PATH's ridership figures in its monthly reports to the authority's board. In the 12 months ending in June, PATH's ridership rose at nearly twice the rate of the New York City subway.</p>
<p class="articleParagraph enarticleParagraph" >Georgia Doyle, who for 20 years has used PATH to commute to Midtown from her home on West 10th Street, said she found it more civilized than the subway. Besides ferrying her each day to her job in the garment district, she said the railroad came with other benefits: ''It's great to go to Jersey on the weekends.''</p>
<p class="articleParagraph enarticleParagraph" >PHOTOS: Tina DiFeliciantonio and her son, Luca, 7, in the PATH station in Chelsea. Next year, a monthly PATH pass will cost about half that of a city subway card. (A19); Beth Macri takes PATH from the Christopher Street station to her job in Chelsea. A single-ride ticket on the subway is $1.75. (PHOTOGRAPHS BY MICHAEL APPLETON FOR THE NEW YORK TIMES) (A24)</p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>c314 : Pricing | c3141 : Sales Promotions | gcat : Political/General News | c31 : Marketing | ccat : Corporate/Industrial News | ncat : Content Types | nfact : Factiva Filters | nfcpin : FC&E Industry News Filter</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>usa : United States | nyc : New York City | usny : New York | usnj : New Jersey | namz : North America | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>The New York Times Company</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020101103e6b3000ak</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020080420e44k000bh" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>New Jersey Weekly Desk; SECTNJ</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>For Devils Fans, Trains Seem to Work Fine</span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By DAVE CALDWELL   </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>757 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>20 April 2008</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>2</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 2008 The New York Times Company.  All Rights Reserved.   </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >NEWARK -- A LIFE-SIZE photograph of the Devils goaltender Martin Brodeur is pasted next to a set of doors that leads out of Newark Penn Station. Arrows next to his image point the way to Prudential Center, the team's new arena two blocks away, in the heart of downtown.</p>
<p class="articleParagraph enarticleParagraph" >Not that Devils fans needed help finding the arena. Attendance for Devils games averaged 15,564 in 41 regular-season games at Prudential Center, an increase of nearly 10 percent over last year's average of 14,176 at the former Continental Arena in East Rutherford, officials said.</p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >Jeffrey Vanderbeek, the former Wall Street executive who became the team's owner in 2004, said the Devils are drawing better, in part, because games are easier to get to, in spite of gloomy forecasts that fans would be scared off by the arena's new urban location, and would not respond to entreaties that they take mass transit to get there.</p>
<p class="articleParagraph enarticleParagraph" >Before the arena's opening in late October, Mr. Vanderbeek said team surveys predicted that about 35 percent of fans would use mass transportation to get to the Prudential Center. The actual number, according to surveys done since, is closer to 50 percent.</p>
<p class="articleParagraph enarticleParagraph" >''It's just been fabulous for us,'' Mr. Vanderbeek said in a recent interview. ''And what that's done is open up the state for us.''</p>
<p class="articleParagraph enarticleParagraph" >As the arena was being built, many fans themselves said they would not be following the team to Newark. Some have not. But clearly many others are, and many of those are taking mass transit to get there, and they are finding it quicker, cheaper and simpler.</p>
<p class="articleParagraph enarticleParagraph" >''You don't have to worry about the traffic, and you can have a few beers at the game and enjoy yourself because you already have a designated driver,'' said Jennifer Mitre, a fan from Roebling, not far from Trenton.</p>
<p class="articleParagraph enarticleParagraph" >Ms. Mitre and her friend Kristen Cunningham took the light rail from Roebling to Trenton, then a <span class="companylink">New Jersey Transit</span> train to Newark. They said their round-trip tickets for both trains were $16.10. Parking in the lots around Prudential Center is $20 or $25.</p>
<p class="articleParagraph enarticleParagraph" >Ken and Phyllis Katz, Devils fans from Long Branch, each made the $15 round trip 11 times during the regular season and plan to keep doing it next year. It takes 73 minutes to get to Penn Station, and they hate to think how long it would take by car.</p>
<p class="articleParagraph enarticleParagraph" >''We don't even have to worry about the traffic -- the idiots,'' Mrs. Katz said with a laugh as she stood in the main waiting room at Penn Station.</p>
<p class="articleParagraph enarticleParagraph" >''Everybody needed to get out of Continental Arena right away,'' Mr. Katz said. ''It was aggravating.''</p>
<p class="articleParagraph enarticleParagraph" >When the arena first opened in October with a series of Bon Jovi concerts and the Devils' first games, <span class="companylink">New Jersey Transit</span> stationed several yellow-jacketed ''ambassadors'' at Penn Station to help riders, many of them suburbanites who had never used public transportation.</p>
<p class="articleParagraph enarticleParagraph" >At the end of a Devils game, many of the flat-screen televisions in the arena become tote boards, with information about trains departing from Penn Station and from Broad Street Station six blocks away, from which the Devils run free shuttle buses.</p>
<p class="articleParagraph enarticleParagraph" >The parade of fans coming out of Penn Station before a recent game against the Rangers was a mix of ages -- empty-nesters, young couples, parents with young children, parents with grown children.</p>
<p class="articleParagraph enarticleParagraph" >Bob Swiatek and his son, Robb, a hockey player at Southern Connecticut State (and a Devils fan), had arrived on an Amtrak train from New Haven. They were unsure where they would park, and broadcast advertisements made the train sound simple.</p>
<p class="articleParagraph enarticleParagraph" >''You really can't get lost because of all the signs telling you where to go,'' Bob Swiatek said.</p>
<p class="articleParagraph enarticleParagraph" >Some 3,000 to 4,000 fans take <span class="companylink">New Jersey Transit</span> to a Devils game, with 1,000 to 2,000 using service provided by Port Authority Trans-Hudson and Amtrak. Sixty-five percent of 930 fans surveyed said they would be less likely to go to a game without mass transit.</p>
<p class="articleParagraph enarticleParagraph" >''We've had the opportunity to introduce the system to an entirely new audience,'' said Dan Stessel, a spokesman for <span class="companylink">New Jersey Transit.</span></p>
<p class="articleParagraph enarticleParagraph" >PHOTO: GAME TIME: Hockey fans walking through Newark Penn Station on their way to the Prudential Center, the Devils' new arena. (PHOTOGRAPH BY JASON WOODRUFF FOR THE NEW YORK TIMES)</p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i72101 : Urban Mass Transit Systems | i71 : Railroads | i721 : Urban/Commuter Transit | irailtr : Road/Rail Transport | irpass : Railroad Passenger Transport | itsp : Transportation/Shipping</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>giceh : Ice Hockey | gcat : Political/General News | gspo : Sports/Recreation | gwint : Winter Sports | ncat : Content Types | nfact : Factiva Filters | nfce : FC&E Exclusion Filter | nrgn : Routine General News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>usnj : New Jersey | namz : North America | usa : United States | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>New York Times Digital (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020080420e44k000bh</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020080226e42q0003u" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Metropolitan Desk; SECTB</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>It Went By In a Blur: PATH's 100th   </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By DAVID W. DUNLAP; Ken Belson contributed reporting.   </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>1471 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>26 February 2008</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>1</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 2008 The New York Times Company.  All Rights Reserved.   </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >You couldn't miss the 100th anniversary of the New York City subway in 2004. There were speeches, banners, slogans, displays, events and a shelf full of books to mark the milestone for posterity: ''Subways,'' ''A Century of Subways,'' ''Subwayland,'' ''Subway Memoirs,'' ''The Subway Pictures,'' ''Subway Style,'' ''The Subway and the City.''   </p>
<p class="articleParagraph enarticleParagraph" >But you might easily have missed the centenary of New York's other subway on Monday.   </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >PATH? One hundred years old?   </p>
<p class="articleParagraph enarticleParagraph" >Riders, who were given a free pass from 6 a.m. to 11 p.m. in honor of the anniversary, were sometimes hard-pressed to explain their momentary good fortune. Even long memories do not reach too much farther back than 1962, when the  <span class="companylink">Port Authority of New York and New Jersey</span> took over the ailing railroad.   </p>
<p class="articleParagraph enarticleParagraph" >''I didn't know it was its 100th anniversary until you told me,'' said Oveta Clinton, 34, of West New York, N.J., as she waited in Newark to board a train for Exchange Place in Jersey City.   </p>
<p class="articleParagraph enarticleParagraph" >What Ms. Clinton definitely did notice was that the late-morning train was far more crowded than usual; standing room only, in fact. ''This is never like this,'' she said as she settled in with her coffee and newspaper. ''This is like 8-in-the-morning rush hour.''   </p>
<p class="articleParagraph enarticleParagraph" >Typically, PATH, the Port Authority Trans-Hudson Corporation -- ne Hudson & Manhattan Railroad Company -- carries 240,000 passengers daily under the Hudson River with none of the celebrity or notoriety of its 104-year-old cousin (though New Yorkers have been known to use PATH during subway strikes to get around Manhattan.)   </p>
<p class="articleParagraph enarticleParagraph" >Like the subway, PATH runs around the clock. But it serves only 13 stations, and New York City Transit has 468.   </p>
<p class="articleParagraph enarticleParagraph" >So far as is known, Duke Ellington never performed ''Take the JSQ Train.'' Nor is it known that Mayor Michael R. Bloomberg has been spotted on a PATH train since last April. And all that ''H & M'' conjures in most minds is a Swedish retailer.   </p>
<p class="articleParagraph enarticleParagraph" >But there was a day -- Feb. 25, 1908, to be precise -- when President Theodore Roosevelt could say that the H & M had the ''greatest subaqueous tunnel in the world'' and was a ''bigger undertaking than any Alpine tunnel which has yet been constructed.''   </p>
<p class="articleParagraph enarticleParagraph" >The New York Times went so far on opening day as to call the Hudson & Manhattan Railroad ''one of the greatest engineering feats that has ever been accomplished, greater perhaps than the Panama Canal will be when completed.''   </p>
<p class="articleParagraph enarticleParagraph" >The tunneling had begun a quarter century earlier, under DeWitt Clinton Haskin. The project stopped and started and stopped again, plagued by litigation and bankruptcy, until William G. McAdoo took over.   </p>
<p class="articleParagraph enarticleParagraph" >McAdoo was not just the president of the H & M but a figure so synonymous with the system that passengers often spoke simply of riding the ''McAdoo Tubes.''   </p>
<p class="articleParagraph enarticleParagraph" >President Roosevelt inaugurated the railroad by pressing a button in his White House office that turned on the electric current in the tunnel. At 3:40:30 p.m., a ceremonial train pulled out of the temporary Manhattan terminus at Sixth Avenue and 19th Street, headed for Hoboken.   </p>
<p class="articleParagraph enarticleParagraph" >The governors of New Jersey and New York squeezed aboard with other dignitaries. Cornelius Vanderbilt, the commodore's great-grandson, found himself in the unusual position of being just another crushed commuter, but professed, ''I would rather ride under the Hudson today hanging to a strap than ride to Albany in a private car.''   </p>
<p class="articleParagraph enarticleParagraph" >The train reached Hoboken at 3:51. A crowd of 10,000 gathered outside to cheer the arriving passengers. ''The natural barrier which has separated New York from New Jersey since those States came into existence was, figuratively speaking, wiped away,'' The Times proclaimed.   </p>
<p class="articleParagraph enarticleParagraph" >The palmy days did not last long, however. The railroad, which had put many Hudson River ferryboats out of business, was undermined in turn by the automobile. Its busiest year was 1927, when it carried 113,141,729 passengers. In other news that year, the Holland Tunnel opened.   </p>
<p class="articleParagraph enarticleParagraph" >By the 1950s, the railroad was bankrupt. The Port Authority was persuaded to take it over in 1962 as part of the intricate negotiating in the development of the World Trade Center. In fact, the twin towers rose where the railroad's enormous Hudson Terminal complex once stood, with two massive office buildings that were almost identical.   </p>
<p class="articleParagraph enarticleParagraph" >On this site, PATH commuters are eventually to arrive in an architectural marvel that may go a long way -- at least symbolically -- toward curing any inferiority they may feel about their subway system. The Port Authority has begun building a $2.2 billion World Trade Center transportation hub and PATH terminal, designed by Santiago Calatrava, a celebrated architect and engineer. It is to open in 2011.   </p>
<p class="articleParagraph enarticleParagraph" >Even earlier, new cars will start coming into service, to replace PATH's aging fleet, most of which dates from the mid-60s to the 1970s. Anthony R. Coscia, the chairman of the authority, said a total of $3.3 billion was being invested in the expansion and modernization of PATH.   </p>
<p class="articleParagraph enarticleParagraph" >PATH also has advertising inside its tunnels. Working on the principle of the 19th-century zoetrope toy, which created the illusion of motion, a company called Submedia has installed a series of illuminated panels with static images on tunnel walls. These blur into a continuous image, what the company calls ''a motion picture outside of the train's windows.''   </p>
<p class="articleParagraph enarticleParagraph" >On her way to Exchange Place, Ms. Clinton, who is an editorial assistant, allowed that she liked watching those ads. ''That's very cool,'' she said. ''You don't have that in New York.''   </p>
<p class="articleParagraph enarticleParagraph" >Another thing PATH has that New York City Transit does not is a $1.50 fare. That will rise to $1.75 on March 2, though it will still be cheaper than the basic $2 subway fare, so a frugal commuter traveling between Greenwich Village and Midtown can still save a quarter by using PATH instead of the No. 1 train.   </p>
<p class="articleParagraph enarticleParagraph" >On Monday morning, however, a PATH employee with a megaphone urged hesitant commuters to push through the turnstiles at the World Trade Center station. ''Free rides today, folks,'' he said. ''Free rides. Through the turnstile. Any turnstile.''   </p>
<p class="articleParagraph enarticleParagraph" >Dr. Eric L. Altschuler, 39, slipped through one of the freely spinning turnstiles, unaware of the centenary and actually a bit leery about a free ride. For two and a half years, he has commuted from Manhattan to Newark, where he is an assistant professor of physical medicine and rehabilitation at the University Hospital.   </p>
<p class="articleParagraph enarticleParagraph" >''Frankly, unusual things can happen on the trains,'' he explained. ''I'm a typical New Yorker. I wanted to make sure it wasn't a ruse.''   </p>
<p class="articleParagraph enarticleParagraph" >Once assured that it was purely celebratory, Dr. Altschuler seemed pleased to learn that it was PATH's 100th anniversary and threw in a bit of praise, likening the system to its big-city relation. ''Everybody rides the PATH trains,'' he said. ''It's a whole other world, but it's not a museum. It's alive.''   </p>
<p class="articleParagraph enarticleParagraph" >When the H & M opened, the scheduled time between Hoboken and 19th Street, in the heart of the Ladies' Mile shopping district, was 12 minutes. On Friday, this reporter was spirited from Hoboken to 19th Street, having made three intermediate stops, in only 11 minutes. But he could not get off at 19th. That station was abandoned by the financially ailing company in 1954.   </p>
<p class="articleParagraph enarticleParagraph" >If one looks carefully and does not blink, the stout columns and vaulted ceilings of the 19th Street passenger platform can still be seen as the train passes through. It is a tantalizing reminder of a moment long ago when the PATH system and the Panama Canal could be mentioned in the same breath.   </p>
<p class="articleParagraph enarticleParagraph" >PHOTOS: A PATH train arriving at 33rd Street in Manhattan. (PHOTOGRAPH BY RUTH FREMSON/THE NEW YORK TIMES) (pg.B1); A sign at 14th Street with the short form of the old name, Hudson & Manhattan. (PHOTOGRAPH BY DAVID W. DUNLAP/THE NEW YORK TIMES); Jonathan Feliciano, 18, left, and Robert Quinones, 27, rode PATH to 14th Street on a historic day, the system's 100th anniversary. (PHOTOGRAPH BY RUTH FREMSON/THE NEW YORK TIMES); At left, the Hoboken terminus of the Hudson & Manhattan as it appeared 100 years ago. At right, as it is today, little changed. DAVID W. DUNLAP/THE NEW YORK TIMES (PHOTOGRAPHS BY JOHN TURKELI COLLECTION; DAVID W. DUNLAP/THE NEW YORK TIMES); The H & M station at Sixth Avenue and 19th Street was the original terminus of the system in Manhattan. It was abandoned for passenger use in 1954, though trains still pass through. (PHOTOGRAPH BY JOHN TURKELI COLLECTION) (pg.B2)   </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>CO</b>&nbsp;</td><td><br/>panynj : Port Authority of New York and New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i72101 : Urban Mass Transit Systems | i71 : Railroads | i721 : Urban/Commuter Transit | irailtr : Road/Rail Transport | irpass : Railroad Passenger Transport | itsp : Transportation/Shipping | i763 : Port/Harbor Operations | iship : Water Transport/Shipping</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>gcat : Political/General News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>nyc : New York City | usny : New York | usnj : New Jersey | namz : North America | usa : United States | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>New York Times Digital (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020080226e42q0003u</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020080107e4170003k" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>CLM</b>&nbsp;</td><td>METRO BRIEFING   NEW YORK</td></tr>
<tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Metropolitan Desk; SECTB</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>Manhattan: Unlimited-Ride Cards For Path   </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By KEN BELSON   </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>169 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>7 January 2008</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>7</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 2008 The New York Times Company.  All Rights Reserved.   </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" ><span class="companylink">The Port Authority of New York and New Jersey</span> plans to introduce unlimited-ride cards of varying time periods on PATH trains later this winter. The cards are similar to ones now sold by the Metropolitan Transportation Authority for use on buses and subways. One-day cards will cost $6, 7-day cards will cost $18 and 30-day cards will cost $54. The introduction of the unlimited-ride cards was part of the Port Authority's decision last Friday to raise the one-way fare to $1.75, from $1.50. The price for 10-, 20- and 40-ride fare cards will increase to $1.30 per PATH ride, from $1.20. The new unlimited-ride cards will be introduced in early March, the same time the higher fares will take effect. PATH, for Port Authority Trans-Hudson, is a railroad that connects New York and New Jersey under the Hudson River.   </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td></td></tr><tr><td align="right" valign="top" class="index"><br/><b>CO</b>&nbsp;</td><td><br/>panynj : Port Authority of New York and New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i763 : Port/Harbor Operations | iship : Water Transport/Shipping | itsp : Transportation/Shipping</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>gcat : Political/General News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>nyc : New York City | usny : New York | namz : North America | usa : United States | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>Brief</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>New York Times Digital (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020080107e4170003k</td></tr></table><br/></div></div><br/><span></span><div id="article-REC0000020060208e2280001e" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/recLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>NEWS</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>Rail stop gets taste of airport security ; Test program starts at PATH station  </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>MATTHEW VERRINDER, THE ASSOCIATED PRESS  </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>401 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>8 February 2006</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The Record</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>REC</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>All Editions</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>A03</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>© 2006 North Jersey Media Group Inc. Provided by ProQuest Information and Learning. All rights reserved.
  </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >JERSEY CITY Commuters heading to work in Manhattan walked through metal detectors Tuesday at a busy train station and fed their bags into X-ray machines at the start of a test of an airport- style security screening program.  </p>
<p class="articleParagraph enarticleParagraph" >The $1 million test program by the U.S. Department of Homeland Security is a response to the train bombings in Madrid and London.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >The program at the Exchange Place PATH station is designed to see how well the technology works for large numbers of daily rail travelers. The equipment was desensitized so keys, loose change and cellphones wouldn't set off alarms. To keep passengers moving quickly, commuters aren't required to take off their shoes or empty their pockets.  </p>
<p class="articleParagraph enarticleParagraph" >The scanners are intended to detect large quantities of metal, like the explosives vests used by suicide bombers in the Middle East, said Doug Bauer, an official with the Homeland Security Department.  </p>
<p class="articleParagraph enarticleParagraph" >Some commuters said the screening did not seem thorough enough, while others said any reassurance it offered came with unsettling memories.  </p>
<p class="articleParagraph enarticleParagraph" >"It's also disconcerting, because every time you get screened, you are reminded that an attack like Madrid or London is possible here," said Donal Byard, 38, an accounting professor.  </p>
<p class="articleParagraph enarticleParagraph" >Officials said the screening process takes about one minute; many commuters Tuesday made it through in about 30 seconds.  </p>
<p class="articleParagraph enarticleParagraph" >"It didn't take as long as I thought it would," said Jessica Salles, 31, a lawyer headed to Manhattan who said she was surprised that just her bag was scanned.  </p>
<p class="articleParagraph enarticleParagraph" >"What about my coat?" Salles said. "It seems like a false sense of security."  </p>
<p class="articleParagraph enarticleParagraph" >James Simpson, 53, a messenger from New York who carried a large blue gym bag, agreed.  </p>
<p class="articleParagraph enarticleParagraph" >"I don't think this is going to do anything," Simpson said. "  </p>
<p class="articleParagraph enarticleParagraph" >If the test is considered successful, similar equipment could be used on the rest of the Port Authority Trans-Hudson train system and on other mass transit systems around the country, authorities said.  </p>
<p class="articleParagraph enarticleParagraph" >About 15,000 passengers a day pass through the station.  </p>
<p class="articleParagraph enarticleParagraph" >* * *  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>ART</b>&nbsp;</td><td><br/>Caption: 2 ASSOCIATED PRESS PHOTOS;; 1 - Screener checking a
suitcase at the Exchange Place PATH station in Jersey City.; 2 -
Below, a passenger walks through a metal detector. Tuesday was the
start of a test of an airport-style security screening program.  </td></tr>
<tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i71 : Railroads | issyst : Security Systems | i32 : Machinery/Industrial Goods | i3302 : Computers/Electronics | ielec : Electronics | irailtr : Road/Rail Transport | itsp : Transportation/Shipping</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>gvdhs : Department of Homeland Security | gsec : State Security Measures/Policies | gterr : Acts of Terror | nnam : News Agency Material | gcat : Political/General News | gcns : National Security | gcrim : Crime/Courts | gpir : Politics/International Relations | gpol : Domestic Politics | grisk : Risk News | gvbod : Government Bodies | gvexe : Executive Branch | ncat : Content Types | nfact : Factiva Filters | nfce : FC&E Exclusion Filter | nfcpex : FC&E Executive News Filter | niwe : IWE Filter</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>eland : England | eecz : European Union Countries | eurz : Europe | uk : United Kingdom | weurz : Western Europe</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>TRANSIT  | RAILROAD  | SECURITY </td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>Record / Hackensack, New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document REC0000020060208e2280001e</td></tr></table><br/></div></div><br/><span></span><div id="article-REC0000020060123e21m00030" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/recLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>CLM</b>&nbsp;</td><td>THE ROAD WARRIOR</td></tr>
<tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>NEWS</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>Port Authority passes the buck on consistency   </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>JOHN CICHOWSKI   </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>982 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>22 January 2006</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The Record</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>REC</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>All Editions</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>A03</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>© 2006 North Jersey Media Group Inc. Provided by ProQuest Information and Learning. All rights reserved.
   </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >Hardly anybody likes to work on a holiday, particularly Christmas. But for many working-stiff commuters, there are benefits such as double- or triple-time pay. For Manhattan-bound E-ZPass subscribers, the perks include the rare advantage of breezing over a traffic-free Hudson River crossing at 7 a.m. or so without having to pay the $5 peak-hour fare.   </p>
<p class="articleParagraph enarticleParagraph" >As many commuters know, the E-ZPass toll is only $4 during non- peak hours. Charging more for peak hours and less for non-peak encourages us to use mass transit at rush hour instead of adding to the stop-and-go chaos at Port Authority crossings.   </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >For folks like Mark Ruckhaus, a bargain is a bargain even if it only saves you a buck. But you might need a lawyer, a congressman and a parliamentarian to get back that buck.   </p>
<p class="articleParagraph enarticleParagraph" >"I went across at 6:45 a.m.," said the Glen Rock man, "but the Port charged me $5, not $4."   </p>
<p class="articleParagraph enarticleParagraph" >So, Mark called E-ZPass. At first, a customer service representative "said it was a mistake and I'd be credited," he recalled.   </p>
<p class="articleParagraph enarticleParagraph" >But then she double-checked. When she got back on the line, the dialogue went something like this, according to Mark:   </p>
<p class="articleParagraph enarticleParagraph" >"You were charged peak because you crossed on Monday, Dec. 26," she said.   </p>
<p class="articleParagraph enarticleParagraph" >"Yes, that was the observed holiday," said Mark. "Dec. 25 fell on a Sunday."   </p>
<p class="articleParagraph enarticleParagraph" >"But it was a Monday."   </p>
<p class="articleParagraph enarticleParagraph" >"I know, but hardly anyone was working. Traffic was nothing close to a typical rush hour."   </p>
<p class="articleParagraph enarticleParagraph" >"But it was the Monday after the holiday."   </p>
<p class="articleParagraph enarticleParagraph" >"Yes, but the PATH lines recognized that day as the holiday. The trains ran on a holiday schedule, and PATH stands for Port Authority Trans Hudson, doesn't it?"   </p>
<p class="articleParagraph enarticleParagraph" >Mark had the nerve to raise the consistency argument, which can be effective - unless you're a consumer fighting a bureaucracy or a kid arguing with a mother who catches him eating between meals.   </p>
<p class="articleParagraph enarticleParagraph" >Well, dad always does it!   </p>
<p class="articleParagraph enarticleParagraph" >You know the usual answer to that one:   </p>
<p class="articleParagraph enarticleParagraph" >Well, it doesn't matter.   </p>
<p class="articleParagraph enarticleParagraph" >That's what the E-ZPass customer service rep told Mark.   </p>
<p class="articleParagraph enarticleParagraph" >I bounced her explanation off Port Authority spokeswoman Tiffany Townsend who provided additional logic.   </p>
<p class="articleParagraph enarticleParagraph" >"PATH line rail schedules have nothing to do with toll-crossing charges," said Townsend.   </p>
<p class="articleParagraph enarticleParagraph" >Yes, one is a train system; the other is a fare collection system. But they have the same river in common and an observed federal holiday has a big impact on the way each of them is run. You would think that the rationales for holiday policies governing each would be consistent.   </p>
<p class="articleParagraph enarticleParagraph" >Or at least, you would think the policies might be clear.   </p>
<p class="articleParagraph enarticleParagraph" >According to the Port Authority's Web site, here are the five holidays offering off-peak rates: New Year's Day, Memorial Day, Independence Day, Thanksgiving and Christmas.   </p>
<p class="articleParagraph enarticleParagraph" >Now, here is what Congress has decreed in U.S. Codes, Title 5, Part III, Subpart E, Chapter 61, Subchapter I, Section 6103, about these five and Veterans Day: If any of them falls on a Sunday, then Monday may be observed as a holiday.   </p>
<p class="articleParagraph enarticleParagraph" >Of course, the operative word here is may. Employers and institutions may or may not use the congressional standard. But how did your profit-oriented employer handle the Christmas holiday? Did your boss let you stay home the Monday after Christmas? If so, were you paid? And if you worked that day, were you given extra compensation?   </p>
<p class="articleParagraph enarticleParagraph" >I'll bet you were. So, why can't a public institution that cannot show a profit join in the Christmas spirit and offer its commuters what it gives its employees on a federally observed holiday: A break!   </p>
<p class="articleParagraph enarticleParagraph" >"They already get a break," said Townsend. "Anybody who uses E- ZPass on our six crossings gets a dollar off the toll whenever they travel."   </p>
<p class="articleParagraph enarticleParagraph" >True enough. But Mark was looking at a bigger picture - one that includes cars and trains. He was expecting a consistent public policy that encourages people to use mass transit during peak highway traffic while still encouraging them to use E-ZPass when driving at non-peak hours. Instead, he encountered a double standard and limited incentive to use either mass transit or the electronic toll system.   </p>
<p class="articleParagraph enarticleParagraph" >"I normally take NJ Transit to work from Glen Rock," explained Mark, a broadcast technician. "But on the observed Christmas holiday, the trains were running on a weekend schedule, so if I took mass transit, I would have been late for work."   </p>
<p class="articleParagraph enarticleParagraph" >By car, he did get to work on time. Even the hard-hearted city was more generous than the Port Authority. He easily found a free space because New York suspends metered parking on the observed Christmas holiday.   </p>
<p class="articleParagraph enarticleParagraph" >"Does the PA need that extra dollar from people traveling on a holiday morning?" asked Mark.   </p>
<p class="articleParagraph enarticleParagraph" >Does it need it from the bridge that is, by far, the most lucrative of the agency's six crossings? Some 50,000 vehicles cross each day during morning rush hour, about 80 percent of them E-ZPass customers. The Port Authority could not provide specifics, but if half these folks use the bridge during the same period on the holiday, an extra dollar's revenue for peak fares would amount to $20,000.   </p>
<p class="articleParagraph enarticleParagraph" >That's not much. Maybe the E-ZPass customer service rep was right: It doesn't matter.   </p>
<p class="articleParagraph enarticleParagraph" >"We had only one complaint," said Townsend.   </p>
<p class="articleParagraph enarticleParagraph" >Just one.   </p>
<p class="articleParagraph enarticleParagraph" >Maybe holiday revelers are too busy to notice a small inequity. Maybe we all ought to check our E-ZPass bills more carefully. And maybe, just maybe, the board that runs this $5-billion-a-year agency ought to consider a consumer-friendly policy that recognizes the spirit of a holiday period when most people and most institutions extend themselves.   </p>
<p class="articleParagraph enarticleParagraph" >***   </p>
<p class="articleParagraph enarticleParagraph" >E-mail: cichowski@northjersey.com   </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i763 : Port/Harbor Operations | iship : Water Transport/Shipping | itsp : Transportation/Shipping</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>c131 : Regulatory Bodies | c13 : Regulation/Government Policy | ccat : Corporate/Industrial News | ncat : Content Types | nfact : Factiva Filters | nfcpin : FC&E Industry News Filter</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>usnj : New Jersey | namz : North America | usa : United States | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>COLUMN  | Ruckhaus, Mark  | Townsend, Tiffany </td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>Record / Hackensack, New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document REC0000020060123e21m00030</td></tr></table><br/></div></div><br/><span></span><div id="article-REC0000020051017e1af00010" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/recLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>NEWS</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>9/11 families sue to halt work on PATH station  </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>THE ASSOCIATED PRESS  </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>325 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>15 October 2005</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The Record</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>REC</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>All Editions</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>A03</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>© 2005 North Jersey Media Group Inc. Provided by ProQuest Information and Learning. All rights reserved.
  </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >NEW YORK - Families of victims of the Sept. 11 terrorist attacks want to block construction of a PATH station for commuter trains between New Jersey and Manhattan, saying it would intrude on the footprints of the fallen World Trade Center towers, a coalition member said Friday.  </p>
<p class="articleParagraph enarticleParagraph" >A lawsuit seeking to stop the construction of the permanent station for the Port Authority Trans-Hudson service was filed in U.S. District Court in Manhattan on Thursday.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >The Coalition of 9/11 Families Inc. brought the action against the Port Authority of New York and New Jersey, which owns the 16- acre site where the trade center stood, and federal transit agencies that are funding the building of the PATH station.  </p>
<p class="articleParagraph enarticleParagraph" >Construction on the transportation hub, designed by Spanish architect Santiago Calatrava, began on Sept. 12. Part of the five- level station, which would connect several downtown subway lines with the PATH trains to New Jersey at the trade center site, would build over part of the footprint to the south tower, the lawsuit said.  </p>
<p class="articleParagraph enarticleParagraph" >Although the families always knew that about half of the south tower would be covered by the PATH development, the latest plans propose extra train tracks that were never there before Sept. 11 and an extra platform, said Anthony Gardner, a member of the coalition who filed suit.  </p>
<p class="articleParagraph enarticleParagraph" >The changes would cover a greater part of the south tower and encroach upon the north tower footprint, Gardner said.  </p>
<p class="articleParagraph enarticleParagraph" >"They've never justified the need for this extra platform," Gardner said Friday. "The priority is the commercial, and then the history is secondary."  </p>
<p class="articleParagraph enarticleParagraph" >The lawsuit said the proposed PATH terminal should be rejected because there are feasible alternatives that would not intrude on a historic site.  </p>
<p class="articleParagraph enarticleParagraph" >Port Authority spokesman Steve Coleman said Friday that the agency had not been served with the lawsuit and could not comment.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>CO</b>&nbsp;</td><td><br/>panynj : Port Authority of New York and New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i763 : Port/Harbor Operations | iship : Water Transport/Shipping | itsp : Transportation/Shipping</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>c12 : Corporate Crime/Legal/Judicial | nnam : News Agency Material | ccat : Corporate/Industrial News | ncat : Content Types | nfact : Factiva Filters | nfce : FC&E Exclusion Filter | nfcpex : FC&E Executive News Filter | nfcpin : FC&E Industry News Filter | niwe : IWE Filter</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>nyc : New York City | usnj : New Jersey | usny : New York | namz : North America | usa : United States | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>TRANSIT  | CONSTRUCTION  | LAWSUIT  | 9/11  | FAMILY  | Gardner, Anthony </td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>Record / Hackensack, New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document REC0000020051017e1af00010</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020050725e17p0003z" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>CLM</b>&nbsp;</td><td>Below Ground Zero, Stirrings of Past and Future</td></tr>
<tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Metropolitan Desk; SECTB</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>Turning 9/11 Ruins Into a Transit Hub, With Touches of History   </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By DAVID W. DUNLAP   </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>1570 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>25 July 2005</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>1</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 2005 The New York Times Company.  All Rights Reserved.   </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >Having endured the construction and destruction of the World Trade Center above, the 96-year-old Hudson Terminal -- now a colossal underground ruin at ground zero -- will soon give way to a new transportation hub.   </p>
<p class="articleParagraph enarticleParagraph" >The ground that is to be broken in September for a new trade center terminal on the eastern side of the site includes some astonishing infrastructure: the two-block-long passenger platform level of the Hudson Terminal, later used as loading docks; cast-iron railroad tubes that were turned into truck ramps; a vault where tons of gold and silver were stored; and structural hints -- geometrically patterned flooring here, chocolate-colored brickwork there -- of the once bustling trade center shopping concourse   </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >.In their place will be the lower levels of the PATH terminal and transportation hub designed by Santiago Calatrava, new pedestrian passageways, shops, parking spaces, loading docks and the basement of the third office tower planned for the site.   </p>
<p class="articleParagraph enarticleParagraph" >For now, there is nowhere else at ground zero where time is more palpably suspended than in the tubes and tunnels and truck bays that once served the World Trade Center and, before that, the Hudson & Manhattan Railroad, predecessor to PATH.   </p>
<p class="articleParagraph enarticleParagraph" >Almost four years after the attack, signs still command truck drivers who have long since vanished: ''No Idling.'' ''Not for Service Vehicles.'' ''30 Minute Parking Only for Deliveries. Offenders Will Be Autoclamped and Fined.''   </p>
<p class="articleParagraph enarticleParagraph" >Within the cavernous gloom of the deeply ribbed 15-foot-3-inch-diameter tubes, the quiet is broken every few minutes by the disembodied rumble of a PATH train passing nearby.   </p>
<p class="articleParagraph enarticleParagraph" >''It makes you think we should get out of the way, that a light will come down from the end of the tunnel,'' said Kenneth J. Ringler Jr., executive director of the Port Authority of New York and New Jersey, as he stood last week at the mouth of the tubes.   </p>
<p class="articleParagraph enarticleParagraph" >Mr. Ringler said elements of this subterranean realm -- perhaps some cast-iron tube rings, certainly some ornamental flooring -- would be salvaged.   </p>
<p class="articleParagraph enarticleParagraph" >The authority is committed to preserving the travertine-clad hallway leading to the E train terminus at Chambers Street. It plans to relocate the cruciform steel column that was found at 6 World Trade Center. What the authority does not save, it will document in written descriptions, drawings and large-format photographs.   </p>
<p class="articleParagraph enarticleParagraph" >''It's important that we attempt to preserve some of this for history,'' Mr. Ringler said. ''This is part of the story. It is not necessarily integral to 9/11, but there is history here.''   </p>
<p class="articleParagraph enarticleParagraph" >Tangible history on the eastern third of the trade center site may date to the second half of the 18th century, since the blocks between Greenwich and Church Streets were always on dry land, unlike the western part of the site, which was landfill.   </p>
<p class="articleParagraph enarticleParagraph" >Excavation of areas along Vesey and Liberty Streets could conceivably uncover privies, cisterns, wells or cesspools from the 1750's through the 1850's, according to the environmental impact statement for World Trade Center redevelopment project.   </p>
<p class="articleParagraph enarticleParagraph" >In any case, the milestone year of 1909 is abundantly recalled. That was when William G. McAdoo, founder of the Hudson & Manhattan Railroad Company, opened the Hudson Terminal, an underground complex that stretched from Fulton Street to Cortlandt Street, ushering in three-minute rail service between Jersey City and Lower Manhattan. Above the terminal were two 22-story towers, nearly twins.   </p>
<p class="articleParagraph enarticleParagraph" >Trains approached the Hudson Terminal through the southern tube, which branched into five tracks that looped among six passenger platforms, studded every 20 feet or so with white-tiled columns. They headed back under the river through the northern tube.   </p>
<p class="articleParagraph enarticleParagraph" >In 1962, the bistate Port Authority took over the bankrupt Hudson & Manhattan line -- renaming it Port Authority Trans-Hudson -- in return for the support of New Jersey officials for a huge trade center in New York. The authority had planned to build the center on the East River, but moved it to align with the Hudson Terminal.   </p>
<p class="articleParagraph enarticleParagraph" >During construction of the center, the U-shaped track array was shifted about 450 feet westward to a new PATH terminal, which opened in 1971. The vestigial Hudson Terminal platform area was turned into loading bays for the two low-rise trade center buildings on Church Street, served by Ramp L (the south tube) and Ramp J (the north tube).   </p>
<p class="articleParagraph enarticleParagraph" >These loading bays look today much as they must have on Sept. 10, 2001. There are still rubber bumpers along the edges of the four-foot-high truck docks. Columns and ramp walls are still color coded: green for 4 World Trade Center, purple for 5 World Trade Center. But Ramp L bears deep scars from Sept. 11, 2001. Steel reinforcing bars embedded in the concrete walls have been twisted into Medusa-like tangles. Steel column flanges are bent like wilting leaves. The tunnel ceiling is bowed and braced.   </p>
<p class="articleParagraph enarticleParagraph" >Nearby, a door with a shattered window leads to the vestibule of the Bank of Nova Scotia vault. Inside the vestibule is a massive steel door with six-inch-thick hinges. Behind that door, the bank was storing about $200 million in gold and silver when the World Trade Center came under attack.   </p>
<p class="articleParagraph enarticleParagraph" >''The vault was intact and all the silver and bullion was taken out,'' said Peter L. Rinaldi, general manager of the trade center site in the Port Authority's priority capital programs unit. He witnessed the recovery operation, a month and a half after the attack, and remembers more than 100 armored trucks making their way out of Ramp J.   </p>
<p class="articleParagraph enarticleParagraph" >Directly above the loading bays was the long north-south corridor of the trade center shopping concourse. The most distinctive remnant of the mall is the banded flooring pattern from the crossroads once occupied by a Warner Brothers Studio Store, a Tourneau watch store and Casual Corner and Strawberry clothing stores.   </p>
<p class="articleParagraph enarticleParagraph" >Very little remains of the rest of the concourse except for a small field of 8-by-8-inch floor tiles just south of the crossroads, directly under the steel cruciform.   </p>
<p class="articleParagraph enarticleParagraph" >This was where Benjamin Books once did business, succeeded by Innovation Luggage. Because Innovation's target customer is a 35- to 50-year-old business traveler, the trade center store was in ''the center of our demographic,'' said David R. Petroski, the company's regional director. ''Volume-wise, it was headed toward being one of the top two stores.''   </p>
<p class="articleParagraph enarticleParagraph" >Briefcases and attache cases were big sellers. The store featured a large metal globe and five television monitors tuned to business channels.   </p>
<p class="articleParagraph enarticleParagraph" >Mr. Petroski, 39, who was then district manager for New York City, happened to be in the trade center store that Tuesday morning to catch up on work before the staff arrived to open up. Because he left the door unlocked, a customer had already come in to check out a bug-eye-green Timberland knapsack.   </p>
<p class="articleParagraph enarticleParagraph" >The concourse started to shake. Mr. Petroski figured it was the subway. But then came a roar. And then, visible through the plate-glass storefront, a stampede. ''Businessmen were running for their lives in huge packs,'' he said. ''My first thought was that there must be a shooter in the hall. I couldn't comprehend why hundreds of people were running.''   </p>
<p class="articleParagraph enarticleParagraph" >The customer dropped the knapsack. Without saying a word to each other, the two men joined the flight. After finding sanctuary in the park at 1 Liberty Plaza, Mr. Petroski heard a rumor that the explosion had been caused by a faulty boiler.   </p>
<p class="articleParagraph enarticleParagraph" >''The store's a bank vault -- a quarter million dollars of inventory,'' he said. ''Responsible district manager that I am, I went back into the World Trade Center. Security is saying, 'Get out!' But I'm smarter than that. I know it's just a boiler. I lock everything up.''   </p>
<p class="articleParagraph enarticleParagraph" >After leaving the concourse again, Mr. Petroski heard a noise getting louder and louder and louder as he crossed Church Street. ''In retrospect, it must have been the sound of the engines,'' he said. ''I turned. That's when the second plane crashed.''   </p>
<p class="articleParagraph enarticleParagraph" >Dazed, terrified but uninjured, Mr. Petroski made his way home by subway to Brooklyn. He tried to return to work the next day in the store on the Avenue of the Americas at 21st Street. ''But I just spent the morning crying in the stockroom,'' he said.   </p>
<p class="articleParagraph enarticleParagraph" >At the trade center, of course, one locked door made no difference.   </p>
<p class="articleParagraph enarticleParagraph" >Photos: Cast-iron rings brace a Hudson & Manhattan Railroad tunnel that linked Manhattan and New Jersey. Some tunnels were used as truck ramps for the trade center.; At left, on the east side of the World Trade Center site, the openings of the Manhattan-bound tunnels of the Hudson & Manhattan Railroad, the predecessor to PATH. (Photographs by Andrea Mohin/The New York Times)(pg. B1); The Port Authority says the cruciform steel column found at 6 World Trade Center will be relocated when the new transportation hub is built.; Work will begin in September on a transit hub at the trade center site; a sign in the old Hudson Terminal, which was used for trucks before 9/11. (Photographs by Andrea Mohin/The New York Times)(pg. B4)   </p>
<p class="articleParagraph enarticleParagraph" >Map of The World Trade Center highlighting the former underground truck loading hall. (pg. B4)   </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>gcat : Political/General News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>nyc : New York City | usny : New York | usnj : New Jersey | namz : North America | usa : United States | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>New York Times Digital (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020050725e17p0003z</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020050420e14k00efr" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Metropolitan Desk; SECTB</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>U.S. Bill to Raise Tax Breaks For Commuters Moves Ahead  </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By SEWELL CHAN  </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>650 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>20 April 2005</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>4</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 2005 The New York Times Company.  All Rights Reserved.  </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >A typical commuter who uses the Long Island Rail Road or Metro-North Railroad could save an additional $200 a year in travel expenses under a proposal in Congress that would significantly expand tax-free transit benefits for workers.  </p>
<p class="articleParagraph enarticleParagraph" >Through a pretax salary deduction, employees can have up to $105 a month withheld to pay for subway, bus, train or ferry fares. That amount would increase to $155 a month under an amendment to the major federal transportation bill. The Senate Finance Committee approved the amendment yesterday, and the full Senate and the House are expected to follow suit.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >The New York City area would benefit disproportionately from the increase, said Senator Charles E. Schumer, Democrat of New York, who introduced the amendment.  </p>
<p class="articleParagraph enarticleParagraph" >''New York has the largest number of mass transit users in the country,'' he said in a telephone interview. ''This is one of the few benefits out of Washington where New York gets the lion's share.''  </p>
<p class="articleParagraph enarticleParagraph" >The amendment would cost the Treasury $101 million in tax revenue over 10 years, but Mr. Schumer said that amount could be recouped through savings identified in other areas of the bill. The committee unanimously approved the amendment, with support from the chairman, Senator Charles E. Grassley of Iowa.  </p>
<p class="articleParagraph enarticleParagraph" >The change would have the greatest effect on suburbanites who commute to New York City, including users of New Jersey Transit, Port Authority Trans-Hudson lines and ferries between New York and New Jersey. It would have little effect in New York City, where a 30-day MetroCard, good for unlimited rides on subways and buses, costs $76, less than the current $105 deduction limit.  </p>
<p class="articleParagraph enarticleParagraph" >But the increased benefit would have a significant impact on other parts of the Metropolitan Transportation Authority, which raised commuter rail fares last month, the second time in two years. The most expensive monthly ticket is $394 on Metro-North and $342 on the Long Island Rail Road, although a typical rider pays about $200 a month.  </p>
<p class="articleParagraph enarticleParagraph" >If the increased benefit is approved, typical workers who are in the 35 percent combined federal and state tax brackets and spend $200 a month for transportation will save about $650 a year in taxes, about $200 more than the current savings, said Larry Filler, the president of TransitCenter Inc., a nonprofit group that began administering transit benefits for employers in the New York area in 1987. The monthly limit for pretax spending on parking, currently $200, has always been more generous than the transit benefit, a discrepancy that transit advocates have long viewed as unfair.  </p>
<p class="articleParagraph enarticleParagraph" >The president of the American Public Transportation Association, William W. Millar, said a larger deduction would most likely prompt some drivers to switch to mass transit. ''With the price of gasoline going up and up, this will certainly encourage additional riders,'' he said. ''It's one more step to eliminate some of the discrimination that exists in federal tax law and federal transportation policy against mass transit.''  </p>
<p class="articleParagraph enarticleParagraph" >However, getting employers and workers to participate in the program remains a daunting challenge.  </p>
<p class="articleParagraph enarticleParagraph" >In a study completed last August, TransitCenter found that only 19 percent of commuters in the New York area participated in transit-benefit programs. In many cases their employers did not offer the benefit. Even in companies where such programs were offered, 32 percent of employees did not participate.  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Millar called on transit agencies to increase public awareness of the benefits. He also noted that the first employee transit benefit was introduced in 1984, but only as a tax break for employers that subsidized mass transit commuting costs. ''Free parking was already pretty well established, but free or discounted transit was not well established,'' he said. ''Relatively few employers were looking for ways to add costs.''  </p>
<p class="articleParagraph enarticleParagraph" >The pretax transit benefit was introduced in 1998.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i721 : Urban/Commuter Transit | i72101 : Urban Mass Transit Systems | irailtr : Road/Rail Transport | itsp : Transportation/Shipping</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>gcat : Political/General News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>usa : United States | nyc : New York City | usny : New York | namz : North America | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>New York Times Digital (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020050420e14k00efr</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020050123e11n000ag" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Metropolitan Desk; SECT1</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>Roaring Snowstorm Shoves Northeast Into Winter's Grasp  </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By ROBERT D. McFADDEN; Contributing reporting for this article were Gretchen Ruethling in Chicago, John Holl in New Jersey, David Winzelberg on Long Island, and Jennifer 8. Lee and Winnie Hu in New York.  </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>2292 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>23 January 2005</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>1</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 2005 The New York Times Company.  All Rights Reserved.  </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >A month across the abyss of winter, the season's first major storm buried New York and much of the Northeast yesterday, stifling travel, slowing the pace of life for millions and recasting the landscapes of 12 states.  </p>
<p class="articleParagraph enarticleParagraph" >The storm, touted as a probable blizzard, roared in from the Midwest and turned into a classic northeaster, with 30-to-50-mile-an-hour winds and nebular arms revolving counterclockwise. It moved up the East Coast, gathering ocean moisture and hurling it back at the land as snow that blanketed cities and towns, closed airports, canceled hundreds of flights, choked railways and highways and filled the air with crystalline impressions.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >It began quietly in the New York metropolitan area before noon, a gentle whispering fall in the pale January light. But by evening, it had become a driving force of windblown snow, with gusts that hissed against the windows and mounting accumulations that, meteorologists said, only hinted at the depths to come.  </p>
<p class="articleParagraph enarticleParagraph" >By late this morning, those accumulations were expected to top out at 12 to 18 inches in Central Park and 18 to 24 inches in parts of Northern New Jersey, Eastern Long Island, Southern Connecticut and shore areas of Rhode Island.  </p>
<p class="articleParagraph enarticleParagraph" >Cape Cod and Southeastern Massachusetts were expected to be hit hardest, with more than two feet of snow blown by winds that gusted up to 60 miles an hour. Lesser totals were expected in the Washington area and in Maryland, Delaware, Pennsylvania and southern sections of Vermont, New Hampshire and Maine.  </p>
<p class="articleParagraph enarticleParagraph" >The Northeast was hardly alone in wintry misery. Heavy snows pounded parts of the Midwest, with the Chicago area getting its biggest snowfall of the season: more than 8 inches by yesterday afternoon with more to come. At O'Hare International Airport, flight delays averaged seven and a half hours and hundreds of stranded passengers slept on cots near baggage claim areas.  </p>
<p class="articleParagraph enarticleParagraph" >In Connecticut, Gov. M. Jodi Rell ordered a state emergency operations center to open yesterday as forecasters predicted coastal flooding in parts of New London and Middlesex Counties.  </p>
<p class="articleParagraph enarticleParagraph" >Blizzard warnings were posted for most of New York State. The mid-Hudson Valley was expected to get 20 inches of snow, and up to a foot was forecast for Albany, the Mohawk Valley and parts of western New York.  </p>
<p class="articleParagraph enarticleParagraph" >Three weather-related deaths were reported in Ohio, where a man fell through ice on a pond and two people suffered heart attacks shoveling snow.  </p>
<p class="articleParagraph enarticleParagraph" >In Wisconsin, the Milwaukee suburb of West Allis had 12 inches of snow and up to 5 inches more was expected overnight. Southern Michigan had 6 to 14 inches of snow yesterday, and three-foot drifts were common.  </p>
<p class="articleParagraph enarticleParagraph" >In the pantheon of winter storms in New York, it did not compare with the all-time record blizzard of Dec. 26-27, 1947, which interred the city in 26.4 inches of snow, and was also expected to fall short of the blizzard of Jan. 7-8, 1996, which left 20.2 inches in Central Park. But Todd J. Miner, a meteorologist with Pennsylvania State University, said it could rival the President's Day storm last year, which left 19.8 inches.  </p>
<p class="articleParagraph enarticleParagraph" >''This is a big, nasty snowstorm,'' Mr. Miner said early yesterday afternoon. ''It's possible we will be heading toward well over a foot of snow in Central Park. We're not going to get two feet, but heading toward 18 inches is not a bad signpost, bringing this into the upper echelons of storms. Of the top 12 city snowstorms on record -- 16 inches or higher -- we've probably got a good shot at that.''  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Miner said that in the overnight hours, the storm would almost certainly meet the National Weather Service's criteria for a blizzard -- winds of at least 35 m.p.h., falling or blowing snow and visibility of less than a quarter-mile for three consecutive hours. But whatever the technicalities, you could hardly tell neighbors shoveling huge drifts from their driveway that it was not a blizzard.  </p>
<p class="articleParagraph enarticleParagraph" >And it was cold, bitter cold. It was 29 below zero in Massena, N.Y., 28 below at Saranac Lake, N.Y., 14 below in Syracuse and 10 below in Albany. In New York City it was a relatively balmy; 10 above early yesterday morning and over 20 early last night.  </p>
<p class="articleParagraph enarticleParagraph" >That was frigid enough to keep the customary tourist hordes away from Times Square, which looked more like a ghost town in the swirling snow, and it was obvious that restaurants, theaters and other attractions had a terrible day.  </p>
<p class="articleParagraph enarticleParagraph" >The Port Authority of New York and New Jersey, which operates the three major metropolitan airports -- Kennedy International, La Guardia and Newark Liberty International -- as well as bridges, tunnels and the Port Authority Trans Hudson (PATH) rail system, went into a full mobilization of personnel and equipment, bracing for a rough weekend.  </p>
<p class="articleParagraph enarticleParagraph" >All three airports remained open, but 175 flights were canceled at Kennedy, 120 more were canceled at Newark, and 200 were canceled at La Guardia, where delays ran up to two hours.  </p>
<p class="articleParagraph enarticleParagraph" >A cargo plane overshot a runway at Kennedy at about 3:30 p.m. in an accident that appeared weather-related, but the Port Authority said it remained under investigation last night. Only a three-person crew was on board, and there were no injuries.  </p>
<p class="articleParagraph enarticleParagraph" >Traffic was moving on the George Washington Bridge and the Lincoln and Holland Tunnels, but there were 35 mile-an-hour speed limits on Staten Island bridges.  </p>
<p class="articleParagraph enarticleParagraph" >With whiteout conditions on runways, the intensifying storm closed the Philadelphia airport at 3:30 p.m. and Bradley International Airport near Hartford at 6:30 p.m. and would possibly shut down those in Boston, Albany and other cities, stranding thousands of passengers.  </p>
<p class="articleParagraph enarticleParagraph" >Acting Gov. Richard Codey of New Jersey announced that a state of emergency would be in effect from 8 p.m. to 8 a.m. today, allowing the police to close roads, if necessary, to ensure public safety.  </p>
<p class="articleParagraph enarticleParagraph" >On Long Island, Suffolk County declared a state of emergency as well.  </p>
<p class="articleParagraph enarticleParagraph" >Up and down the Northeast Corridor, driving was treacherous on icy, snow-blown highways, roads and neighborhood streets. There were countless minor accidents, though no deaths or serious injuries were reported, and motorists were advised to stay home or use public transportation. But trains and buses were also delayed by the storm, and getting around, for those who had to, was an ordeal. Many residents heeded the warnings to stay home and many businesses closed for the weekend.  </p>
<p class="articleParagraph enarticleParagraph" >The Long Island Rail Road, which operates 450 trains on 11 branches on a typical weekend, reported only two train cancellations, both on the Greenport line, and only minor delays on the rest of the system.  </p>
<p class="articleParagraph enarticleParagraph" >''But that could change as the storm worsens and the wind increases,'' said Brian P. Dolan, a spokesman. ''They're predicting drifts of snow. If we get 2 to 3 inches an hour, that challenges us to keep pace with the storm.''  </p>
<p class="articleParagraph enarticleParagraph" >In the meantime, he said special trains were spraying antifreeze on power rails and activating electric and gas-powered heaters to keep switches moving.  </p>
<p class="articleParagraph enarticleParagraph" >Even before the snow began falling, the Metro-North Railroad had a signal problem on its upper Hudson line between Croton and Poughkeepsie that delayed about 8 trains for up to 30 minutes.  </p>
<p class="articleParagraph enarticleParagraph" >Marjorie Anders, a spokeswoman, said that railroad officials were meeting in the afternoon to decide whether to cut back service because it appeared that many passengers were staying home.  </p>
<p class="articleParagraph enarticleParagraph" >But Dan Brucker, another spokesman, said late in the afternoon that service remained on or close to schedule on all its lines, although there were plans to sharply cut service today.  </p>
<p class="articleParagraph enarticleParagraph" >New Jersey Transit, which operates 11 rail lines, 3 light rail systems and 240 bus routes around the state and into Manhattan, reduced its service schedule yesterday afternoon until midnight tonight.  </p>
<p class="articleParagraph enarticleParagraph" >Later in the day, New Jersey Transit suspended its South Jersey bus service as of 5 p.m., and its North Jersey buses as of 7:30 p.m. It also reported delays of 15 to 20 minutes on its Northeast Corridor line. Dan Stessel, a spokesman for New Jersey Transit, said that express trains on the Northeast Corridor line had been canceled, although local trains continued to run.  </p>
<p class="articleParagraph enarticleParagraph" >The Midtown Direct line between Penn Station and Dover, N.J., in Morris County was rerouted to Hoboken, where passengers could switch to PATH trains running to 33rd Street in Manhattan.  </p>
<p class="articleParagraph enarticleParagraph" >Forewarned, cities and counties across the region had readied armies of equipment and sent out fleets of salt spreaders and snowplows to counterattack as the snow began falling.  </p>
<p class="articleParagraph enarticleParagraph" >In New Jersey, state transportation officials and the New Jersey Turnpike Authority had more than 2,000 trucks on the roads to plow and spread salt. The Port Authority also had hundreds of pieces of equipment out, and more than 200,000 gallons of liquid de-icing chemicals for use on wings and other surfaces.  </p>
<p class="articleParagraph enarticleParagraph" >In New York, Mayor Michael R. Bloomberg went to the department's Queens repair shop and said 2,500 sanitation workers -- using 1,450 garbage trucks with plows, 82 dump trucks with plows and 350 salt spreaders -- would work around the clock in two shifts to keep major arteries and streets open. He said he expected that all of the city's 6,300 miles of streets would be plowed at least once by the start of the workweek tomorrow morning.  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Bloomberg also issued a few words of caution.  </p>
<p class="articleParagraph enarticleParagraph" >''The streets even after they are plowed will be slippery, so you should take caution,'' he said. ''The streets will be narrower, the snow has to go some place.''  </p>
<p class="articleParagraph enarticleParagraph" >He also urged residents to dress warmly, check on neighbors, take mass transit to work and keep cars off the streets so the plows can get through.  </p>
<p class="articleParagraph enarticleParagraph" >Many schools with Saturday classes closed. Aqueduct and the Meadowlands Racetrack canceled their Saturday racing programs, and many college basketball games were postponed. National Football League conference championship games in Philadelphia (Eagles-Atlanta Falcons) and Pittsburgh (Steelers-New England Patriots) were still on track for today and, with the snow over by game times, only bitter cold and high winds were expected to be factors.  </p>
<p class="articleParagraph enarticleParagraph" >The storm's timing significantly diminished its impact. For millions of suburban commuters and students home for the weekend, the snow was not a great hardship, except for the ordeal of shoveling a driveway or sidewalk, which leads every winter to many heart attacks.  </p>
<p class="articleParagraph enarticleParagraph" >But for many residents of the metropolitan area, the storm provided an opportunity -- one of the few in a relatively mild winter that has recorded a total of only 4.3 inches of snow since autumn -- to get out with sleds, skis or snowshoes and to frolic in the drifts. And for those so inclined, it was a chance to relax indoors, snowed in with Bach, Brubeck or a good book, cozy behind panes embroidered with frost.  </p>
<p class="articleParagraph enarticleParagraph" >For those who ventured out to play -- hooded, booted, muffled to the eyes -- the storm offered glimpses of nature's beauty: empty streets turned into white meadows, black-and-white woodlands painted in moonlight, snowflakes glittering like confections in a bakery -- frosted, glazed, powdered, sugary -- and in the parks children, romping, padded like armadillos.  </p>
<p class="articleParagraph enarticleParagraph" >There were many last-minute shoppers yesterday, even as the snow began falling.  </p>
<p class="articleParagraph enarticleParagraph" >Doreen and Neal Erps, of North Brunswick, N.J., wheeled a cart out of a Home Depot on Route 1 in Edison with cabinet shelves. ''I figure we'll be in the house all weekend long,'' said Mr. Erps. ''We might as well do something productive, and remodeling the bathroom beats shoveling snow.''  </p>
<p class="articleParagraph enarticleParagraph" >Nearby, Howard Myers, of New Brunswick, N.J., was loading up his S.U.V. with groceries and firewood. ''My next stop is the liquor store,'' he said. ''I'm going to get a nice bottle of Scotch, put the logs on the fire and let the storm rage outside while I read my book.''  </p>
<p class="articleParagraph enarticleParagraph" >At a Home Depot on 23rd Street in Manhattan, shovels, salt buckets, windshield scrapers and other storm equipment flew off the shelves. By 12:30 p.m., only four of the 40 snowblowers delivered on Friday -- some fetching $729 -- were left, and the shovels were gone.  </p>
<p class="articleParagraph enarticleParagraph" >''Where are the shovels?'' asked an anxious customer, one of a cluster.  </p>
<p class="articleParagraph enarticleParagraph" >''They are unloading them right now,'' an employee said.  </p>
<p class="articleParagraph enarticleParagraph" >''C'mon, let's go,'' another patron said as the group hurried downstairs to meet the delivery truck.  </p>
<p class="articleParagraph enarticleParagraph" >Photos: Pedestrians around the city, including these in the East Village trudging across St. Mark's Place, experienced the season's first heavy snowfall. (Photo by Chang W. Lee/The New York Times)(pg. 1); Jimmy Tuohy and his daughter, Molly, 10, leaving their mark, briefly, as they pass through the snow-softened landscape at Lincoln Center. (Photo by Ting-Li Wang/The New York Times); As the snow and wind picked up, snow gear was going fast yesterday at the Modell's store at 86th Street and Third Avenue in Manhattan. (Photo by John Marshall Mantel for The New York Times); The New Jersey Turnpike remained passable yesterday afternoon, despite several inches of snow that had fallen. (Photo by Dith Pran/The New York Times); After doing some shopping yesterday, John Campbell of Brooklyn, in a building on 14th Street yesterday, views the scene in Union Square Park. (Photo by Joe Tabacca)(pg. 31)  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>gwea : Weather | gcat : Political/General News | npag : Page-One Story | ncat : Content Types</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>nyc : New York City | usny : New York | namz : North America | usa : United States | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>New York Times Digital (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020050123e11n000ag</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020050115e11f00032" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Metropolitan Desk; SECTB</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>Amtrak and New Jersey Transit Delayed by Two Rail Mishaps  </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By SEWELL CHAN  </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>339 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>15 January 2005</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>4</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 2005 The New York Times Company.  All Rights Reserved.  </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >A downed electrical wire and a broken track switch caused delays yesterday for thousands of New Jersey Transit and Amtrak passengers trying to get to or leave from Pennsylvania Station, officials said. The mishaps were repaired before the start of the evening commuter rush.  </p>
<p class="articleParagraph enarticleParagraph" >At 10:06 a.m., a New Jersey Transit train's pantograph -- the extensible arm that draws electricity into the locomotive -- became tangled in a set of overhead wires as the train moved through a tunnel, according to a spokesman for New Jersey Transit, Dan Stessel.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >No passengers were on board the train, which pulled down parts of the wires. The train was on its way to a rail yard in Sunnyside, Queens, which is used to maintain and turn train cars before they return to Manhattan and then New Jersey.  </p>
<p class="articleParagraph enarticleParagraph" >Only a few minutes later, in a tunnel just west of Pennsylvania Station, another New Jersey Transit train damaged a switch that allows trains to move across tracks, Mr. Stessel said.  </p>
<p class="articleParagraph enarticleParagraph" >He said that both incidents were under investigation, and that a preliminary investigation by Amtrak, which maintains the tunnels, showed that the overhead wires might have been damaged or frayed and that the damage to the switch might have been the result of an error by the train operator.  </p>
<p class="articleParagraph enarticleParagraph" >The wires and the switch were repaired by 2 p.m.  </p>
<p class="articleParagraph enarticleParagraph" >The incidents caused delays of 18 minutes to just more than an hour on 12 New Jersey Transit trains.  </p>
<p class="articleParagraph enarticleParagraph" >Three trains were turned around at Secaucus, where passengers could board other trains to Hoboken and then Manhattan. Two other trains were turned around at Newark, where passengers could use their tickets on Port Authority Trans-Hudson trains to get to the World Trade Center station in Lower Manhattan.  </p>
<p class="articleParagraph enarticleParagraph" >Seven Amtrak trains on the so-called main spine, or Northeast corridor, were delayed by an average of nearly one hour, according to R. Clifford Black, an Amtrak spokesman.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>CO</b>&nbsp;</td><td><br/>njt : New Jersey Transit</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i71 : Railroads | i721 : Urban/Commuter Transit | i72101 : Urban Mass Transit Systems | irailtr : Road/Rail Transport | itsp : Transportation/Shipping</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>gdis : Disasters/Accidents | gcat : Political/General News | grisk : Risk News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>nyc : New York City | usny : New York | namz : North America | usa : United States | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>New York Times Digital (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020050115e11f00032</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020040829e08t0006y" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>CLM</b>&nbsp;</td><td>NEW YORK 2004: NAVIGATING NEW YORK/GROUND ZERO</td></tr>
<tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>National Desk; SECT15</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>Where the World Trade Center Fell, an Aura Still Stands  </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By DAVID W. DUNLAP  </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>1316 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>29 August 2004</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>7</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>(c) 2004 New York Times Company  </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >THE difficulty of visiting ground zero is more than emotional and spiritual, though it is certainly that. After all, this is where 2,749 people died on Sept. 11, 2001.  </p>
<p class="articleParagraph enarticleParagraph" >The difficulty is also practical. There is no one place to take it all in, no public vantage that offers an all-encompassing panorama of the World Trade Center site and neighboring towers, some of which -- it may shock you to see -- are almost as ravaged now as they were that Tuesday morning.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >But if you walk around the 14.6-acre excavation where the twin towers stood, if you filter out the crowds and let your heart grow still, you may begin to see what many New Yorkers see: not an open patch of sky but a fathomless void where something vast and vital ought to be.  </p>
<p class="articleParagraph enarticleParagraph" >Begin in the 18th century, across Church Street, at the exquisite St. Paul's Chapel. President Washington worshiped here when New York was the national capital, and workers from ground zero found solace, comfort, care, hot food and badly needed supplies. The story is told in an exhibition, ''Unwavering Spirit: Hope and Healing at Ground Zero.'' (More information is at  <span class="colorLinks">www.saintpaulschapel.org [http://www.saintpaulschapel.org]</span>.)  </p>
<p class="articleParagraph enarticleParagraph" >The pedestal for the commemorative Bell of Hope outside the chapel is inscribed with the original building plan of the trade center site. Outside the Century 21 store, on Church Street, is a seven-year-old Heritage Trail marker -- no one has presumed to take it down -- that eerily refers to the trade center in the present tense. ''Every weekday, 50,000 people come to work in 12 million square feet of office, hotel and commercial space,'' it says. ''As many as 10,000 visitors in a single day ride the nonstop express elevators -- from the lobby to the 107th floor in 82 seconds -- to take in the spectacular views.''  </p>
<p class="articleParagraph enarticleParagraph" >Across Church Street is an 18-foot-high steel cruciform that was found in the wreckage from 6 World Trade Center, which was the United States Custom House.  </p>
<p class="articleParagraph enarticleParagraph" >Turning right on Liberty Street, you will pass 10 House, the recently rebuilt quarters of Engine Company 10 and Ladder Company 10, which lost three firefighters, two lieutenants and a retired captain. Their rugged faces gaze out across ground zero from a bronze memorial plaque. A sign asks passers-by to ''remember that questions about 9/11 tend to bring back horrible memories for many firefighters.''  </p>
<p class="articleParagraph enarticleParagraph" >Bad memories have physical form on Liberty Street. Through a large opening in the black netting around the 40-story Deutsche Bank building, which is to be dismantled, you can see steel beams and columns installed to repair a 15-story gash created by the falling facade of the south tower. Even more poignant is the grievous injury to the fantastic Gothic facade of 90 West Street, one block closer to the river. But this is now being restored and turned into apartments.  </p>
<p class="articleParagraph enarticleParagraph" >Liberty Street offers the best street-level overlook into ground zero. From here, you can see the remains of the 6 World Trade Center garage, the reconstructed PATH commuter station and the new 7 World Trade Center, under construction, which replaces the third tower that fell on 9/11. Hundreds of steel caps visible in the concrete foundation wall mark the cable anchors that tie the wall to bedrock for reinforcement.  </p>
<p class="articleParagraph enarticleParagraph" >After crossing the Liberty Street footbridge, stay on the second-level passageway in the World Financial Center. What looks from here like a water pipe sticking out of the foundation wall is actually an abandoned train tunnel built in 1909 by the Hudson and Manhattan Railroad. This should give you an idea of the wall's colossal dimension.  </p>
<p class="articleParagraph enarticleParagraph" >In the Winter Garden of the World Financial Center is an exhibition, ''From Recovery to Renewal,'' by the Lower Manhattan Development Corporation. Compare the two models showing the plan envisioned by Daniel Libeskind in 2003 and the design as it evolved, with the twisting Freedom Tower, the birdlike transportation hub and a landscaped memorial that turns the twin tower outlines into water-filled voids. (A virtual version is at:  <span class="colorLinks">www.renewnyc.com/PhotoArchive/ [http://www.renewnyc.com/PhotoArchive/]</span>.)  </p>
<p class="articleParagraph enarticleParagraph" >Then take 15 minutes to walk to the Skyscraper Museum, on the ground floor of the Ritz-Carlton at the southern end of Battery Park City, where you will find a trade center model that was built more than 30 years ago by Yamasaki & Associates, architects of the twin towers.  </p>
<p class="articleParagraph enarticleParagraph" >The towers in this model stand 6.8 feet tall, large enough to convey something of the three-dimensional impact of the trade center, beginning with the fact that you have to crook your neck slightly to see the top of the towers.  </p>
<p class="articleParagraph enarticleParagraph" >As you move around the model, the towers change; at one point divided by a column of light, then fused together, then looking like flat slabs, now like silver-edged pinnacles. You can begin to experience the interplay that gave them an undeniable drama on the skyline. (Information is at  <span class="colorLinks">www.skyscraper.org [http://www.skyscraper.org]</span>, under ''WTC Dossier.'')  </p>
<p class="articleParagraph enarticleParagraph" >A functioning remnant of the World Trade Center is open to the public. It is a 66-foot-long section of the original underground concourse, paved in tawny travertine. It is at the back of the World Trade Center PATH Station, which is entered through a winged canopy on Church Street.  </p>
<p class="articleParagraph enarticleParagraph" >PATH, the Port Authority Trans-Hudson railroad, links New Jersey to Lower Manhattan through the old Hudson and Manhattan tube. This part of the system was closed for more than two years after the attacks.  </p>
<p class="articleParagraph enarticleParagraph" >The core of ground zero is best seen from the PATH station concourse and mezzanine. And it is well worth the $1.50 fare to get beyond the turnstiles. A single-trip QuickCard can be purchased at a vending machine and pay-per-ride MetroCards also work.  </p>
<p class="articleParagraph enarticleParagraph" >To orient yourself in Lower Manhattan, look for the astonishingly detailed, mural-size aerial photograph of the trade center environs at the south end of the mezzanine. Next to it, in blue, is a corresponding street map.  </p>
<p class="articleParagraph enarticleParagraph" >At the north end are two placards, ''Exchange Place Tunnels, 1907'' and ''Early H & M Railroad Cars.'' Standing at the column between them, looking down through a narrow gap in the translucent screen, you can make out a line of rusting steel squares set into the ground at regular intervals. These are the stub ends of the perimeter columns that once supported the north facade of 1 World Trade Center.  </p>
<p class="articleParagraph enarticleParagraph" >If a PATH train happens to have its doors open, jump aboard for the four-minute ride to the next stop, Exchange Place, in Jersey City. All trains stop there. (More information is at  <span class="colorLinks">www.panynj.gov [http://www.panynj.gov]</span>.)  </p>
<p class="articleParagraph enarticleParagraph" >What you will get from this trip is a fleeting but awe-inspiring view of ground zero. You will be able to see what looks like a large wooden crate, enclosing the Freedom Tower cornerstone. The 460-foot-long ramp will swing into view. Then it will seem as if the train is being swallowed into the parking garage, where the remaining columns are still color-coded yellow, red and blue.  </p>
<p class="articleParagraph enarticleParagraph" >Get off at Exchange Place and catch any inbound train, marked ''WTC.'' As you re-enter Manhattan, you will feel the train slow before daylight breaks in. Next comes a screech of wheels as the cars turn into the station.  </p>
<p class="articleParagraph enarticleParagraph" >Jarring as it may be to your ears -- and annoying as it is to commuters -- that familiar screech was the most wonderful sound on Nov. 23, 2003, the day the station reopened. It was the sound of ground zero coming back to life.  </p>
<p class="articleParagraph enarticleParagraph" >Photo (Satellite image courtesy of DigitalGlobe)  </p>
<p class="articleParagraph enarticleParagraph" >Map of Manhattan shows the locations of the Skyscraper Museum and Winter Garden.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>gcat : Political/General News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>nyc : New York City | usa : United States | namz : North America | use : Northeast U.S. | usny : New York</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>New York Times Digital (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020040829e08t0006y</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020040123e01n0006f" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Metropolitan Desk; SECTB</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>A PATH Station That Honors 9/11, and Opens Wide, Too  </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By DAVID W. DUNLAP  </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>1227 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>23 January 2004</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>1</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>(c) 2004 New York Times Company  </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >Where there was darkness on the morning of Sept. 11, 2001, the architect Santiago Calatrava would bring a flood of light in the form of a winged railway station, draped in glass, suffused with natural illumination and, on occasion, open to the clear skies above.  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Calatrava's design for the permanent World Trade Center PATH terminal, which was unveiled yesterday, is a soaring, sculptural, steel-and-glass shell covering a cathedral-like concourse. Through a network of passageways, the terminal would connect the Port Authority Trans-Hudson underground rail line from New Jersey and 14 subway lines.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >'' 'Wow' is the first word that's just got to come to your mind,'' said Mayor Michael R. Bloomberg. He joined in the unveiling with Gov. George E. Pataki and officials of the Port Authority of New York and New Jersey, which will build the terminal. It may cost up to $2 billion. Construction is expected to begin early next year and take four years to complete.  </p>
<p class="articleParagraph enarticleParagraph" >Rather than rely on words alone, Mr. Calatrava took pastels to paper in the Winter Garden of the World Financial Center at Battery Park City. He sketched a child releasing a bird into the air, then superimposed the bird on the outline of the terminal.  </p>
<p class="articleParagraph enarticleParagraph" >About 67,000 commuters boarded PATH trains at the trade center before the attack. Today, a temporary open-air station is used daily by about 30,000 travelers.  </p>
<p class="articleParagraph enarticleParagraph" >In aesthetics and logistics, Mr. Calatrava's design aspires to be far more than a commuter rail station, vital as that is to Lower Manhattan. Its admirers are already mentioning it in the same breath as the old Pennsylvania Station, which was torn down in the early 1960's, and Grand Central Terminal, which Mr. Calatrava claimed as his ''deepest inspirational object.''  </p>
<p class="articleParagraph enarticleParagraph" >Like the original Penn Station, the PATH terminal would bathe travelers in daylight, which would reach all the way to the train platforms 60 feet below ground through the use of glass-block floors above. And like Grand Central, it would be a hub of an underground network linking numerous skyscrapers and subway stations.It would also do something neither of the earlier stations did: move.  </p>
<p class="articleParagraph enarticleParagraph" >Two counterpoised canopies over the main concourse, rising some 150 feet like skeletal birds' wings, could be retracted hydraulically in about two minutes to create a tapered opening almost 50 feet wide at its center. This would ventilate smoke from the building in case of fire and provide natural air-conditioning.  </p>
<p class="articleParagraph enarticleParagraph" >''On a beautiful summer day,'' Mr. Calatrava said, ''the building can work not as a greenhouse but as an open space.'' He also envisioned the symbolic power of opening the roof every year on the morning of Sept. 11, ''giving us the sense of unprotection.''  </p>
<p class="articleParagraph enarticleParagraph" >''The building itself expresses the memory of Sept. 11,'' said Mr. Calatrava, 52, a Spanish architect, engineer and artist who is widely admired for the lyrical quality of his bridges and train stations. Though he has a home on the Upper East Side, the PATH terminal would be Mr. Calatrava's first structure in New York City. (His partners in the design and engineering are DMJM & Harris and the STV Group.)  </p>
<p class="articleParagraph enarticleParagraph" >The angle of the canopies' outer edges would mark the line of the sun at 8:46 on the morning of Sept. 11, when the first plane struck the trade center. The main axis of the 360-foot-long concourse would be aligned with the angle of the sun at 10:28 that morning, when the second tower collapsed.  </p>
<p class="articleParagraph enarticleParagraph" >An architectural expression of these angles was called for in the master site plan by Studio Daniel Libeskind. They formed the Wedge of Light plaza on either side of Fulton Street. In the Libeskind plan, the PATH terminal was to be south of the plaza, in the approximate location of Dey Street, adjoining one of the planned office buildings.  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Calatrava pulled the terminal northward, making it a free-standing structure surrounded by plazas, thereby reopening Dey Street. Because the terminal integrally expresses the Wedge of Light, its design won a warm endorsement from Mr. Libeskind, who has otherwise been struggling to preserve key elements of his year-old site plan.  </p>
<p class="articleParagraph enarticleParagraph" >It was also embraced by Nikki Stern, whose husband, James E. Potorti, died in the north tower. After calling up the images of the design to the screen of her computer, she said her first reaction was -- indeed -- ''Wow.''  </p>
<p class="articleParagraph enarticleParagraph" >''My second reaction was how beautifully it complements Daniel's plan,'' Ms. Stern said, ''and how pleased I am that the Port Authority allowed Calatrava to create something that is respectful, yet so hopeful and functional.''  </p>
<p class="articleParagraph enarticleParagraph" >That is not to say that the difficult issues are all resolved. Among them is the extent to which the platforms and tracks of the new terminal will encroach on the footprints of the north and south towers.  </p>
<p class="articleParagraph enarticleParagraph" >The amount of retail space in the 200,000-square-foot terminal will not only affect its character but could potentially drain life from the surrounding streets.  </p>
<p class="articleParagraph enarticleParagraph" >Construction will also have to be coordinated with that of the memorial, the Fulton Street Transit Center being designed by Nicholas Grimshaw and with the office buildings planned by Silverstein Properties, including 62- and 65-story towers on either side of the terminal.  </p>
<p class="articleParagraph enarticleParagraph" >Only two months ago, the $323 million temporary PATH station opened. A month later, the design of the 1,776-foot Freedom Tower was announced. And last week, the design of the memorial, ''Reflecting Absence,'' was made public.  </p>
<p class="articleParagraph enarticleParagraph" >The permanent PATH terminal and its surrounding network of passageways carries the largest price tag of any of these projects: up to $2 billion, financed with $1.7 billion from the Federal Transit Administration and $300 million in insurance proceeds.  </p>
<p class="articleParagraph enarticleParagraph" >Governor Pataki seemed to anticipate criticism of the project as an extravagance when he summoned the memory of Senator Daniel Patrick Moynihan. ''He would lament the fact that we don't build grand things any more,'' Mr. Pataki said. The governor said he vowed that on the trade center site, ''the buildings themselves would be a lasting tribute to those we lost and to the courage that showed on Sept. 11.''  </p>
<p class="articleParagraph enarticleParagraph" >After the unveiling, Robert B. Tierney, the chairman of the city Landmarks Preservation Commission, asked, ''Should we pre-emptively landmark this?''  </p>
<p class="articleParagraph enarticleParagraph" >The terminal will not in fact be eligible until it turns 30 years old. But glancing at the shimmering models nearby, Mr. Tierney said, ''This will still be flying at that age.''  </p>
<p class="articleParagraph enarticleParagraph" >Photos: The architect Santiago Calatrava and Gov. George E. Pataki at the Winter Garden with a model of the design for the permanent World Trade Center PATH terminal. (Photo by Andrea Mohin/The New York Times)(pg. B1); At the unveiling of the World Trade Center PATH station design yesterday, Santiago Calatrava sketched a bird being released into the air. (Photo by Andrea Mohin/The New York Times); An artist's rendering of the main concourse of the World Trade Center PATH terminal, with the roof open. (Photo by Port Authority of New York and New Jersey)(pg. B7)  </p>
<p class="articleParagraph enarticleParagraph" >Map of New York highlighting new PATH station: The station will be a free-standing structure surrounded by plazas. (pg. B7)  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>CO</b>&nbsp;</td><td><br/>panynj : Port Authority of New York and New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i763 : Port/Harbor Operations | iship : Water Transport/Shipping | itsp : Transportation/Shipping</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>gcat : Political/General News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>usny : New York | usa : United States | namz : North America | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>New York Times Digital (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020040123e01n0006f</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020031124dzbo0002u" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Metropolitan Desk; SECTA</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>Again, Trains Put the World In Trade Center  </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By DAVID W. DUNLAP; Jason George, Jonathan Miller and Colin Moynihan contributed reporting for this article.  </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>1486 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>24 November 2003</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>1</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>(c) 2003 New York Times Company  </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >And yesterday, the people returned to ground zero.  </p>
<p class="articleParagraph enarticleParagraph" >Not those who were impelled to work there or compelled to grieve there, but the many more who have been waiting. People without passes and badges, hard hats and breathing masks; people with no more credentials than curiosity or longing. Or the simple desire to spend a beautiful afternoon in the city on the Sunday before Thanksgiving.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >The World Trade Center PATH Station opened at 2 p.m. after a $323 million, 16-month reconstruction, to applause and tears along the platforms and aboard the trains. On the sides of the cars, ruby-red ''WTC'' destination signs glowed once again.  </p>
<p class="articleParagraph enarticleParagraph" >For the first time since the morning of Sept. 11, 2001, ground zero throbbed with ordinary life and resonated with hundreds of vibrant voices.  </p>
<p class="articleParagraph enarticleParagraph" >''I'm making part of history right now,'' Robert Conard of Silver Spring, Md., said into his cellphone just before 2 p.m., as he was swept with the crowd under the winged entrance canopy on Church Street and into a succession of open-air spaces leading to train platforms 70 feet below ground.  </p>
<p class="articleParagraph enarticleParagraph" >Those pouring in from upstairs met and mingled with passengers getting off the first trains to link Lower Manhattan with New Jersey in more than two years. Before the attack, PATH, the Port Authority Trans-Hudson commuter rail system, carried 67,000 passengers a day home from the World Trade Center.  </p>
<p class="articleParagraph enarticleParagraph" >Since then, commuters have struggled with alternate, round-about routes that have included taking PATH trains that come into Manhattan farther uptown or switching to ferry service. So this morning's rush hour will surely eclipse yesterday's opening.  </p>
<p class="articleParagraph enarticleParagraph" >But that was lively enough. As destinations were announced by a worker with a megaphone -- ''Journal Square and Newark, track No. 4!'' ''Track 3 for Hoboken!'' -- a sea of dark winter coats surged through bright gray and shimmering silver rooms. The spaces are surprisingly luminous and generous in their proportions. But because so much of the station is intended to be temporary, it is deliberately spartan in details, with concrete and exposed steel where there once was travertine.  </p>
<p class="articleParagraph enarticleParagraph" >Wind-breaking screens wrap the main rooms, so the view of ground zero changes under different light conditions from misty to gauzy. It is the first time the public has been able to look around the trade center foundations from within the giant bathtub formed by the rugged slurry walls.  </p>
<p class="articleParagraph enarticleParagraph" >Elsewhere, on interior walls, are giant photographs of Lower Manhattan, accompanied by graphics showing the pattern of streets and skyline. The single amenity is a Hudson News stand on the mezzanine.  </p>
<p class="articleParagraph enarticleParagraph" >A permanent $2 billion PATH station is being designed by Santiago Calatrava, a Spanish architect whose bridges and terminals have been likened to poetry. It is to begin serving passengers in 2006. Like the current station, it will be linked with numerous subway lines.  </p>
<p class="articleParagraph enarticleParagraph" >On ''PATH Hill,'' the broad bank of eight escalators from the mezzanine to the concourse, a defining feature of both the old and new stations, a woman could be overheard explaining to disbelieving companions as they ascended: ''There was absolutely nothing here. Nothing. Everything is absolutely new.''  </p>
<p class="articleParagraph enarticleParagraph" >Not quite everything. About 50 feet of travertine flooring and six shallow travertine steps from the old World Trade Center concourse can still be found in the vestibule between the station and the E train platform.  </p>
<p class="articleParagraph enarticleParagraph" >Keeping them was ''the right thing to do,'' said Robert I. Davidson, chief architect of the Port Authority of New York and New Jersey, who designed the station, working with the engineer Jerrold Dinkels and the Pentagram studio.  </p>
<p class="articleParagraph enarticleParagraph" >There were those in the crowd yesterday, many of them relatives of victims of the Sept. 11 attack, who worried that the Port Authority was not keeping enough, because of its plans to expand the station over more of the twin towers' footprints. Some were also angered that the Port Authority was keeping too much: the World Trade Center name, unmodified, as if the towers themselves were still there.  </p>
<p class="articleParagraph enarticleParagraph" >But Patrick Rodriguez, who was on his way to Newark, said he approved of the name. ''I think it should be the World Trade Center because I grew up in New York City all my life,'' he said, ''and that was part of our history they took away from us.'' Mr. Rodriguez also recalled trips he used to take on PATH to see his father in Jersey City. The screech of the trains as they made the sharp turn into the station brought back memories for many riders, including Agnieszka Warenica, who boarded the train at Journal Square in Jersey City. ''I felt like I was coming back after a long time,'' she said. ''I felt like I was home.''  </p>
<p class="articleParagraph enarticleParagraph" >Amanda Valdes of Bayonne, N.J., got off the train from Exchange Place in Jersey City wiping tears from her eyes. When she arrived at the base of PATH Hill, she gasped at the familiarity of it all. ''Oh, my God,'' she whispered to her friend, Kelly Gallagher. ''They should have done something different.''  </p>
<p class="articleParagraph enarticleParagraph" >''I wasn't prepared,'' she said. ''I thought I would be.''  </p>
<p class="articleParagraph enarticleParagraph" >Senator Jon S. Corzine of New Jersey, who spent 15 years commuting on PATH when he was a Wall Street executive, may have come closest to explaining why the reopening of the World Trade Center station was such a potent event. ''The connection of the ordinary days of our lives with the extraordinary events of that day will never be separated,'' he said.  </p>
<p class="articleParagraph enarticleParagraph" >Though a case of the flu kept Gov. George E. Pataki from attending the opening, he spoke in an interview on Friday about what goes through his mind when he visits the station, looking up into the void where the twin towers stood.  </p>
<p class="articleParagraph enarticleParagraph" >''You just think of the people who were up there on Sept. 11,'' he said. ''Friends. Heroes. So many who did not have the chance to get on that train.''  </p>
<p class="articleParagraph enarticleParagraph" >The train he referred to was the last one out of the World Trade Center on Sept. 11. It was used yesterday for the ceremonial first trip back.  </p>
<p class="articleParagraph enarticleParagraph" >In the second car, No. 801, were Gov. James E. McGreevey, Mayor Michael R. Bloomberg, Mr. Corzine and Senator Frank R. Lautenberg of New Jersey.  </p>
<p class="articleParagraph enarticleParagraph" >They boarded at Exchange Place and heard a two-tone signal at 11:05, accompanied by a voice over the loudspeaker: ''Next stop on this train will be the World Trade Center. Next stop, World Trade Center.''  </p>
<p class="articleParagraph enarticleParagraph" >Halfway through the mile-long journey came an ever-so-slight pause, prompting Mr. McGreevey to remark, ''We wanted a very realistic experience,'' as he recalled being stuck on a malfunctioning PATH train during the Christmas holidays one year.  </p>
<p class="articleParagraph enarticleParagraph" >''But it's not going to happen today,'' he added quickly.  </p>
<p class="articleParagraph enarticleParagraph" >''Well, no technology works all the time,'' Mayor Bloomberg said.  </p>
<p class="articleParagraph enarticleParagraph" >In a moment, the train emerged from the cast-iron tube into daylight. Through windows behind the governor and mayor, the spectacle of ground zero unfolded, as if the train were emerging from a cliff side. As it rounded its way to the platform, the slurry wall of the trade center foundation came clearly into view, as did the long ramp leading into the site from Liberty Street.  </p>
<p class="articleParagraph enarticleParagraph" >(As it happened, Mr. Bloomberg was prescient. Later in the day, a Hoboken-bound car dislodged a communications cable, forcing the temporary suspension of Hoboken service from the trade center station until about 4:15. ''We're working out the bugs in the system,'' said Michael P. DePallo, the director and general manager of PATH.)  </p>
<p class="articleParagraph enarticleParagraph" >One of the first passengers to alight on Track 3 yesterday was Christy Ferer, the mayor's liaison to families of Sept. 11 victims. Her husband, Neil D. Levin, the executive director of the Port Authority, was killed in the attack. His successor at the authority, Joseph J. Seymour, has presided over the reconstruction.  </p>
<p class="articleParagraph enarticleParagraph" >''I'm in awe of the Port Authority because in 16 months they did what they do best: build and engineer,'' Ms. Ferer said as she looked around the new concourse. Then, recalling her husband, she added, ''He'd be very proud to see this organization kick into gear.  </p>
<p class="articleParagraph enarticleParagraph" >''No, let me take that back. I think he'd have expected no less.''  </p>
<p class="articleParagraph enarticleParagraph" >Photos: The canopy and entrance to the redone PATH station at ground zero, looking east across Church Street and down Fulton Street. (Photo by Ozier Muhammad/The New York Times)(pg. A1); Above, Mary Ann Mullady of Hillsborough, N.J., was among those boarding the PATH train at Exchange Place in Jersey City. The ceremonial first ride was aboard the last train to leave the station on 9/11. (Photographs by Ozier Muhammad/The New York Times)(pg. B4)  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i71 : Railroads | irailtr : Road/Rail Transport | itsp : Transportation/Shipping</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>gdis : Disasters/Accidents | gcat : Political/General News | npag : Page-One Story | grisk : Risk News | ncat : Content Types</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>usa : United States | usny : New York | namz : North America | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>New York Times Digital (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020031124dzbo0002u</td></tr></table><br/></div></div><br/><span></span><div id="article-REC0000020031117dzbh0003i" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/recLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>CLM</b>&nbsp;</td><td>OBITUARIES</td></tr>
<tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>LOCAL</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>OBITUARIES </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>1554 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>17 November 2003</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The Record</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>REC</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>© 2003 North Jersey Media Group Inc. Provided by ProQuest Information and Learning. All rights reserved.
 </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >JOHN A. ADAMETZ SR., 84, of Totowa, formerly of Garfield, died Thursday. Before retiring in 1987, he was a custodian for Most Holy Name R.C. Church, Garfield, where he worked for 20 years. Previously, he worked for the U.S. Postal Service in Garfield. He was an Army veteran of World War II and recipient of the Purple Heart. He was a parishioner of Most Holy Name Church and a member of the Knights of Columbus Council in Garfield. He had been a coach for the Babe Ruth League, a member of the Golden Agers, and a social member of Fire Company 3, all in Garfield. Arrangements: Aloia Funeral Home, Garfield. </p>
<p class="articleParagraph enarticleParagraph" >WILLIAM G. BELLENGER, 78, of North Arlington, formerly of Lyndhurst, died Sunday. He was a Navy veteran of World War II. He was a parishioner of Lyndhurst United Methodist Church. Arrangements: Kimak Funeral Home, Carlstadt. </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >ALEXANDER D. BIALY, 82, of Carmel, Ind., formerly of Cliffside Park, died Thursday. Before retiring, he worked for Western Electric Co., Kearny, and was a member of the Telephone Pioneers of America. He had served in the merchant marine. He was a parishioner of Sacred Heart R.C. Church, North Bergen. He was a member of the Polish Democratic-American Club. Arrangements: Konopka Funeral Home, North Bergen. </p>
<p class="articleParagraph enarticleParagraph" >MADELINE E. OEFFLER BILL, 80, of Saddle Brook died Sunday. Before retiring in 1973, she was employed for more than 20 years by Bendix Corp., Teterboro. She was a parishioner of Most Holy Name R.C. Church, Garfield. Arrangements: Vander Plaat Colonial Home, Fair Lawn. </p>
<p class="articleParagraph enarticleParagraph" >MARIE P. DAVIS BUFORD, 76, of Paterson died Tuesday. Arrangements: Bragg Funeral Home, Paterson. </p>
<p class="articleParagraph enarticleParagraph" >JOSEPHINE RUTILIO BUSCIGLIO, 75, of Hackensack died Saturday. She was born in Sicily. Before retiring, she was a seamstress for the Lyndhurst Coat Co., South Hackensack, and a member of the International Ladies' Garment Workers Union. She was a parishioner of St. Francis R.C. Church, Hackensack. Arrangements: G. Thomas Gentile Funeral Home, Hackensack. </p>
<p class="articleParagraph enarticleParagraph" >JUNG K. CHOI, 64, of Wayne died Sunday. He was born in Korea and came to the United States in 1974. He had owned Dana Corp. He served in the Korean Army. He was a member of Bethany Methodist Church, Wayne. Arrangements: Frank A. Patti &Kenneth Mikatarian Funeral Home, Fort Lee. </p>
<p class="articleParagraph enarticleParagraph" >ROSE PATIERNO CONTRINO, 82, of Passaic died Saturday. She had been a seamstress for Clifton Drapery and Metro Mills in Paterson. Arrangements: Intelisano-Scarpa Funeral Home, Clifton. </p>
<p class="articleParagraph enarticleParagraph" >CLARA RONDON CORTADILLO, 95, of Clifton, formerly of Paterson, died Saturday. She was born in Peru and came to the United States in 1972. She was a parishioner of St. George R.C. Church, Paterson. Arrangements: Festa Memorial Funeral Home, Totowa. </p>
<p class="articleParagraph enarticleParagraph" >GUS DI FILIPPI, 86, of Rochelle Park died Friday. Before retiring, he was a supervisor for Merritt Garments, Garfield. He was an Army veteran of World War II and a member of American Legion Post 170, Rochelle Park. He was a parishioner of Sacred Heart R.C. Church, Rochelle Park. Arrangements: Vander Plaat Memorial Home, Paramus. </p>
<p class="articleParagraph enarticleParagraph" >RAYMOND G. EHRMANN, 69, of Bogota died Saturday. Before retiring in 1995, he was a technical representative for DuPont in Clifton, where he worked for 24 years. He was a member of Print New Jersey and its Century Club. He was an Army veteran. He was a parishioner of St. Joseph R.C. Church, Bogota. Arrangements: Petrik Funeral Home, Bogota. </p>
<p class="articleParagraph enarticleParagraph" >DOMENICO GRAZIADIO, 68, of Clifton died Saturday. He was born in Italy and was a member of the Carabinieri police before coming to the United States in 1971. He had been a custom tailor and a partner in European Fashion, West Paterson. Arrangements: Bizub-Quinlan Funeral Home, Van Houten Avenue, Clifton. </p>
<p class="articleParagraph enarticleParagraph" >DANIEL S. GREENE, 22, of Clifton died Sunday. He had worked for A- 1 American Chimney Co., Jersey City. He was a graduate of Clifton High School. Arrangements: Bizub-Quinlan Funeral Home, Van Houten Avenue, Clifton. </p>
<p class="articleParagraph enarticleParagraph" >ANDREW JELTES, 85, of Haledon died Thursday. Before retiring, he was vice president of Ritter Trucking, Rahway. He was a member of Cedar Hill Christian Reformed Church, Wyckoff, and its choir and Senior Citizens, and served as a deacon and elder. He was a former board member of the Christian Healthcare Center and a former member of the Haledon Planning Board. Arrangements: Vander Plaat-Vermeulen Memorial Home, Franklin Lakes. </p>
<p class="articleParagraph enarticleParagraph" >HARRIET H. KERR, 68, of Jupiter, Fla., formerly of Wyckoff, died Friday. Arrangements: Aycock Funeral Home, Jupiter. </p>
<p class="articleParagraph enarticleParagraph" >FRANCES GREENMAN KLEIN, 77, of Glen Rock, formerly of Paramus, died Sunday. Before retiring, she was a bookkeeper. She was a member of Gilda's Club and a former member of the Jewish Community Center, Paramus. Arrangements: Louis Suburban Chapel, Fair Lawn. </p>
<p class="articleParagraph enarticleParagraph" >GEORGE V. MACKLEANIS, 55, of Teaneck died Sunday. He had been a sales representative for A-1 Detailing Supply, Newark. Arrangements: Volk-Leber Funeral Home, Teaneck. </p>
<p class="articleParagraph enarticleParagraph" >SHIRLEY C. MICKELSEN, 81, of Clifton died Saturday. Before retiring, she was a children's librarian at the Newark and Kearny public libraries. She was a member of Third Presbyterian Church, Newark, where she served as a deaconess. Arrangements: Allwood Funeral Home, Clifton. </p>
<p class="articleParagraph enarticleParagraph" >JOSEPH MIZRAHI, 82, of Brooklyn, formerly of Englewood, died Friday. Before retiring, he was a sportswear manufacturer in New York City. He was an Army veteran of World War II. Arrangements: Louis Suburban Chapel, Fair Lawn. </p>
<p class="articleParagraph enarticleParagraph" >MARY E. KRAKOWSKI MONTGOMERY, 92, of Paterson, formerly of Lodi, died Saturday. Before retiring in 1976, she was a packer for Voit Sporting Goods, Maywood, where she worked for 20 years. She was a parishioner of St. Brendan R.C. Church, Clifton, and was a member of the Lodi Senior Citizens Club. Arrangements: Shook Funeral Home, Clifton. </p>
<p class="articleParagraph enarticleParagraph" >VINCENT PIPITONE, 97, of Paramus, formerly of West New York, died Saturday. He was born in Italy. Before retiring, he was a fabric cutter for Argyle Knitting Mills, Park Ridge. He was a parishioner of Annunciation R.C. Church, Paramus. Arrangements: Vander Plaat Memorial Home, Paramus. </p>
<p class="articleParagraph enarticleParagraph" >THE REV. LEON H. PLANTE, 84, of Modesto, Calif., formerly of Ramsey, died Nov. 3. He had been pastor of St. John Memorial Episcopal Church, Ramsey, from 1960 to1977. Subsequently, he was assistant minister of St. Paul Episcopal Church, Modesto, and then helped establish Christ the King Community Episcopal Church, Riverbank, Calif. He had been a member of the Rotary Club and the Ambulance Corps in Ramsey. Arrangements: Van Emburgh-Sneider Funeral Home, Ramsey. </p>
<p class="articleParagraph enarticleParagraph" >MIRAYM VILLA QUINTERO, 56, of Guttenberg died Saturday. She was born in Colombia. Arrangements: Vainieri Funeral Home, North Bergen. </p>
<p class="articleParagraph enarticleParagraph" >BRIDGET KANGLEY "DELIA" REID, 90, of Fair Lawn died Saturday. She was born Ireland and was a registered nurse at Mater Miserecordia Hospital in Dublin before coming to the United States in 1950. Before retiring in 1984, she worked for the former Bergen Pines County Hospital, Paramus, for 15 years. She had served in the British Army Nursing Corps. She was a parishioner of St. Anne R.C. Church, Fair Lawn, and a member of its Leisure and Senior Citizens clubs. She was a member of the Fair Lawn Live Wires. Arrangements: Vander Plaat Colonial Home, Fair Lawn. </p>
<p class="articleParagraph enarticleParagraph" >MILDRED FREUDBERG ROSENBLUH, 99, of Fair Lawn, formerly of Paterson, died Sunday. Arrangements: Louis Suburban Chapel, Fair Lawn. </p>
<p class="articleParagraph enarticleParagraph" >IVY BERESFORD ROUGHGARDEN, 99, of Fair Lawn and Delray Beach, Fla., died Sunday. She was born in England and came to the United States in 1907. She was co-developer of Lyncrest Park in Fair Lawn and Mountain Estates in Wyckoff. She was a member of St. Paul Episcopal Church, Paterson. She was a co-founder of the Volunteer Ambulance Corps and Fire Company 3, both in Fair Lawn. She had been a County Committeewoman and was a past president of the Warren Point Parent-Teacher Association. She was a member of the High Mountain Golf Club and a 50-year member of the Ridgewood Country Club. Arrangements: Vander Plaat Funeral Home, Wyckoff. </p>
<p class="articleParagraph enarticleParagraph" >ROSE J. KRAPP SCOTTI of Ridgewood died Saturday. She had been an accountant for J.H. Monteath. She was a parishioner of St. Raymond R.C. Church in the Bronx. Arrangements: Collins-Calhoun Funeral Home, Rutherford. </p>
<p class="articleParagraph enarticleParagraph" >EDWARD J. STACK JR., 70, of Union City died Sunday. Before retiring 15 years ago, he was a conductor for the Port Authority Trans-Hudson line in Hoboken, where he worked for 28 years. He was a Navy veteran of the Korean War. He was a member of the Brian Stack Civic Association. Arrangements: Leber Funeral Home, Union City. </p>
<p class="articleParagraph enarticleParagraph" >JULIAN S. TONIC of Paterson died Wednesday. Before retiring, he worked for Brogan Cadillac. Arrangements: Bragg Funeral Home, Passaic. </p>
<p class="articleParagraph enarticleParagraph" >GLORIA E. COMBES WARREN, 90, of Wayne died Sunday. Before retiring, she was a teacher for the Wayne school system, where she taught for 39 years. She was a graduate of Newark State College and received a master's degree in reading from William Paterson University. She was a member of the New Jersey Education Association and was a founding member of the Wayne Retired Teachers Association, where she served as sunshine committeewoman. She also was a member of the Wayne Public Library and AARP. Arrangements: Vander May Wayne Colonial Funeral Home, Wayne. </p>
<p class="articleParagraph enarticleParagraph" >RUTH KASMER WILSON, 72, of Vernon Township, formerly of Hackensack, died Saturday. She was a parishioner of Holy Trinity R.C. Church, Hackensack, where she served as a Eucharistic minister. Arrangements: Ferguson-Vernon Funeral Home, Vernon. </p>
<p class="articleParagraph enarticleParagraph" >* * *  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>grel : Religion | nobt : Obituary | gcat : Political/General News | ncat : Content Types | nrgn : Routine General News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>usnj : New Jersey | namz : North America | usa : United States | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>OBIT | COLUMN | Postal Service | Babe Ruth League | Adametz, John A Sr</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>Record / Hackensack, New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document REC0000020031117dzbh0003i</td></tr></table><br/></div></div><br/><span></span><div id="article-REC0000020030530dz5u00036" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/recLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>CLM</b>&nbsp;</td><td>BUSINESS BRIEFS</td></tr>
<tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>BUSINESS</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>BUSINESS BRIEFS </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>Compiled from staff and news service reports. </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>623 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>30 May 2003</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The Record</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>REC</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>© 2003 North Jersey Media Group Inc. Provided by ProQuest Information and Learning. All rights reserved.
 </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >Exchange Place set to open June 29 </p>
<p class="articleParagraph enarticleParagraph" >JERSEY CITY - The Exchange Place commuter train station will open June 29 after being closed since the Sept. 11 terrorist attacks damaged the rail system in lower Manhattan, the Port Authority of New York and New Jersey said. Port Authority Trans-Hudson service to the Wall Street area is to begin in November, a month ahead of schedule, Anthony Cracchiolo, the Port Authority's director of capital programs, said during an agency meeting. The Exchange Place station, at Montgomery Street on the Hudson River, was the last stop in New Jersey before the PATH line entered New York and the World Trade Center area. The restoration was part of a $544 million project to rebuild the line into lower Manhattan. The station, which serves commuter lines extending to Newark and Hoboken, will accommodate 10,000 passengers. </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >* * * </p>
<p class="articleParagraph enarticleParagraph" >Ingersoll-Rand to stay in Bermuda </p>
<p class="articleParagraph enarticleParagraph" >WOODCLIFF LAKE - Defeating the hopes of activists, Ingersoll- Rand Co. shareholders voted Thursday against moving the company's headquarters back to the United States from Bermuda. The industrial equipment and components company said that 58.6 percent of shareholders at its annual meeting voted against the move, while 41.4 percent voted for it. State and union funds had been pushing for the industrial conglomerate to move back from Bermuda, where companies aren't required to pay U.S. taxes on income from foreign subsidiaries. Ingersoll-Rand reincorporated in late 2001 from New Jersey. </p>
<p class="articleParagraph enarticleParagraph" >* * * </p>
<p class="articleParagraph enarticleParagraph" >Lockheed unveils new radar system </p>
<p class="articleParagraph enarticleParagraph" >MOORESTOWN - Lockheed Martin Corp. has introduced a new radar system it hopes to sell to the U.S. Navy for fleet defense. The S- Band Advanced Radar has twice the range of the Navy's current frontline radar, the company said, and is designed to operate effectively close to shore. The company said it has invested $30 million in developing the system since 1999. Lockheed Martin's Moorestown facility is the prime contractor for the Aegis weapons system used for naval fleet defense. </p>
<p class="articleParagraph enarticleParagraph" >* * * </p>
<p class="articleParagraph enarticleParagraph" >Yes, we goofed, GM will say in ads </p>
<p class="articleParagraph enarticleParagraph" >DETROIT - General Motors Corp. is using an unconventional approach to boost North American sales, acknowledging in new advertisements past quality blunders to underscore progress it has made in recent years. The world's largest automaker said Wednesday it will begin a long-term advertising campaign next week aimed at consumers who shy away from GM vehicles because of concerns about quality. The initial installment, called "The Road to Redemption," will run in national and regional newspapers and magazines. GM declined to say how much it was spending on the campaign, which over time will shift from quality to improvements in technology, the environment, and design. In J.-D. Power and Associates' 2003 initial quality study released this month, GM fared best among the domestic automakers, recording an average of 134 problems per 100 vehicles - one more than the industry average. </p>
<p class="articleParagraph enarticleParagraph" >* * * </p>
<p class="articleParagraph enarticleParagraph" >Mortgage rates hit eighth new low </p>
<p class="articleParagraph enarticleParagraph" >WASHINGTON - Rates on 30-year mortgages have dropped to a new low, the eighth time this year that has happened. Average interest on a fixed-rate 30-year mortgage fell to 5.31 percent this week, the third week in a row this benchmark mortgage hit a record weekly low, mortgage company Freddie Mac reported Thursday in its nationwide survey. Last week's 5.34 percent had been the previous low in figures going back to 1971. For 15-year fixed-rate mortgages, a popular option for refinancing, rates remained unchanged at 4.73 percent this week, the lowest since Freddie Mac began tracking them in 1991. Rates for one-year adjustable mortgages, however, nudged up this week to 3.63 percent from 3.61 percent. </p>
<p class="articleParagraph enarticleParagraph" >* * *  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>CO</b>&nbsp;</td><td><br/>ingrac : Ingersoll-Rand Plc | lckhed : Lockheed Martin Corporation</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i32 : Machinery/Industrial Goods | i3254 : Construction Machinery | i364 : Aerospace Products/Parts | iaer : Aerospace/Defense</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>e1122 : Mortgage Applications/Refinancing | mcat : Commodity/Financial Market News | e11 : Economic Performance/Indicators | ecat : Economic News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>berm : Bermuda | usa : United States | namz : North America</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>COLUMN | BUSINESS</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>Record / Hackensack, New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document REC0000020030530dz5u00036</td></tr></table><br/></div></div><br/><span></span><div id="article-REC0000020030423dz4n0004t" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/recLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>CLM</b>&nbsp;</td><td>OBITUARIES</td></tr>
<tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>LOCAL</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>OBITUARIES </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>1108 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>23 April 2003</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The Record</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>REC</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>© 2003 North Jersey Media Group Inc. Provided by ProQuest Information and Learning. All rights reserved.
 </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >CATHIE M. BARNICOAT, 65, of Port Charlotte, Fla., formerly of Wayne and Paterson, died Friday. She was born in Scotland. Before retiring in 2001, she was vice president of client services for Unirisc Insurance Inc., Paramus, where she worked for 15 years. She was past chief daughter of Daughters of Scotia Lodge 72, Hawthorne, and former secretary of the Caledonian Club, Paterson. Arrangements: Allwood Funeral Home, Clifton. </p>
<p class="articleParagraph enarticleParagraph" >JULIA CORSON BLACKWELL, 101, of Tenafly died April 6. Before retiring, she was a nurse at Tenafly Elementary School. She was a member of the Tenafly Board of Health. She was a member of Church of the Atonement, Tenafly, and its Altar Guild, and worked in its Opportunity Shop. Arrangements: Barrett Funeral Home, Tenafly. </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >LACY BURNEY of Passaic died Sunday. He had been a factory worker. Arrangements: Bragg Funeral Home, Passaic. </p>
<p class="articleParagraph enarticleParagraph" >EDNA NELSON BYLANDT, 86, of West Milford, formerly of Pompton Plains, died Monday. Before retiring in 1985, she was an executive assistant for C.-E. Lummus, Bloomfield. Arrangements: Vander Plaat- Vermeulen Memorial Home, Franklin Lakes. </p>
<p class="articleParagraph enarticleParagraph" >TERESA MORRONE DiBENEDETTO, 63, of Garfield died Monday. She was born in Argentina. She was a parishioner of Our Lady of Mount Virgin R.C. Church, Garfield. Arrangements: Aloia Funeral Home, Garfield. </p>
<p class="articleParagraph enarticleParagraph" >HELEN EVANS, 91, of Clifton died Saturday. She was a former membership chairwoman of the Clifton chapter of AARP. Arrangements: Allwood Funeral Home, Clifton. </p>
<p class="articleParagraph enarticleParagraph" >VINCENZA GAGLIARDOTTO, 73, of Hawthorne died Tuesday. Before retiring, she was a bookkeeper for the Bank of Passaic and Clifton, Passaic, where she worked for 15 years. She was a parishioner of St. Anthony R.C. Church, Hawthorne. Arrangements: Browning-Forshay Funeral Home, Hawthorne. </p>
<p class="articleParagraph enarticleParagraph" >LUCILLE E. GUTT, 72, of Bogota died Monday. Before retiring in 1991, she was a credit and collections manager for Prentice-Hall, Englewood Cliffs, where she worked for 40 years. She was a parishioner of St. Joseph R.C. Church, Bogota. Arrangements: Petrik Funeral Home, Bogota. </p>
<p class="articleParagraph enarticleParagraph" >NEIL F. HANSON, 91, of Oradell, formerly of Hasbrouck Heights, died Sunday. He had been a real estate manager for Joseph L. Muscarelle. He was a Marine Corps veteran of the Korean War. Arrangements: Volk-Leber Funeral Home, Oradell. </p>
<p class="articleParagraph enarticleParagraph" >JOAN T. HERBEK, 68, of Lincoln Park died Tuesday. She was a member of American Legion Post 174 auxiliary, Wayne. Arrangements: Keri Memorial Funeral Home, Lincoln Park. </p>
<p class="articleParagraph enarticleParagraph" >HOLGER R. JENSEN, 96, of Clifton died Saturday. He was born in Denmark and came to the United States in 1927. Before retiring in 1968, he was a route salesman for Alderny Dairy, Newark, where he worked for 40 years. He was a founding member and former trustee of Bethany United Presbyterian Church, Bloomfield. He served as a district chairman for the Boy Scouts of America, Bloomfield, where he was a scoutmaster and Cub Scout leader. He was a member of the North Jersey Chapter of the National Woodcarvers Association. Arrangements: Allwood Funeral Home, Clifton. </p>
<p class="articleParagraph enarticleParagraph" >KATHLEEN RAYMOND LILLO, 77, of Hudson, Fla., formerly of Clifton, died April 8. Arrangements: M. John Scanlan Funeral Home, Pompton Plains. </p>
<p class="articleParagraph enarticleParagraph" >PATRICK J. McGAGH SR., 71, of Cresskill died Tuesday. He had been a railroad motorman for Port Authority Trans-Hudson Corp., where he worked for 34 years. Arrangements: Barrett Funeral Home, Tenafly. </p>
<p class="articleParagraph enarticleParagraph" >PASQUAL V. "PATSY" PETRILLO, 86, of Midland Park died Monday. Before retiring, he was a contractor. He was a parishioner of Nativity R.C. Church, Midland Park. He was a member of the Sons of Italy, the Elks Lodge in Westwood, and the Northern Regional Senior Center in Midland Park. Arrangements: Olthuis Funeral Home, Midland Park. </p>
<p class="articleParagraph enarticleParagraph" >ANTHONY J. PICCOLO, 93, of Las Vegas, formerly of Hasbrouck Heights, died Sunday. Before retiring, he was a supervisor for a utility company. He also was a concert violinist and a diamond cutter. He was an Army veteran of World War II. Arrangements: Palm Mortuary, Las Vegas. </p>
<p class="articleParagraph enarticleParagraph" >LORRAINE C. RAINSFORD, 70, of Norwood died Sunday. Arrangements: Cremation Society of New Jersey, Clifton. </p>
<p class="articleParagraph enarticleParagraph" >ALIENE V. McCRACKEN RAZZETTI, 79, of River Edge died Tuesday. Arrangements: Trinka-Faustini Funeral Home, Maywood. </p>
<p class="articleParagraph enarticleParagraph" >GLORIA E. REINHARDT, 80, of Daytona Beach, Fla., formerly of Clifton, died March 29. Arrangements: Cardwell &Maloney Funeral Home, Port Orange, Fla. </p>
<p class="articleParagraph enarticleParagraph" >ANDREA "ANDREW" RIVEZZI, 90, of East Rutherford died Monday. He was born in Italy and came to the United States in 1954. Before retiring in 1969, he worked for Star-Glo Industries, East Rutherford, for 10 years. He was a parishioner of St. Joseph R.C. Church, East Rutherford. Arrangements: Kimak Funeral Home, Carlstadt. </p>
<p class="articleParagraph enarticleParagraph" >EDUARDA C. SAN MIGUEL, 79, of Hackensack died Sunday. She was born in the Philippines. Before retiring in 1988, she was a home economics teacher in the Philippines for 40 years. She was a parishioner of St. Valentine R.C. Church, Bloomfield. Arrangements: Gaita Memorial Home, Little Falls. </p>
<p class="articleParagraph enarticleParagraph" >RADIE MOEZELAAR SCHUIL, 91, of Midland Park, formerly of Hawthorne, died Tuesday. She was a member of Cedar Hill Christian Reformed Church, Wyckoff. Arrangements: Vander Plaat Funeral Home, Wyckoff. </p>
<p class="articleParagraph enarticleParagraph" >ROSETTA SIGNA, 80, of Manahawkin, formerly of Rutherford, died Monday. Arrangements: Collins-Calhoun Funeral Home, Rutherford. </p>
<p class="articleParagraph enarticleParagraph" >MARY R. NORTON SMITH, 74, of Guttenberg died Monday. Before retiring, she was a court clerk in Guttenberg. Arrangements: Vainieri Funeral Home, North Bergen. </p>
<p class="articleParagraph enarticleParagraph" >MICHAEL L. THOMAS of Ellenwood, Ga., formerly of Paterson, died Friday. He had been a corporate counselor for American Express Travel Related Services, where he worked for 14 years. He was a member of First AME Zion Church, Paterson, where he served as a steward. He also was a member of the Voices of Zion Church and Metro Atlanta Decatur AME Church in Georgia. Arrangements: Bragg Funeral Home, Paterson. </p>
<p class="articleParagraph enarticleParagraph" >JOSEPH C. "JAY" WILKERSON, 78, of New Milford died Monday. Before retiring, he worked for Laborers Union Local 472, Newark, for 31 years. He was a member of the New Milford Athletic Club. Arrangements: Boulevard Funeral Home, New Milford. </p>
<p class="articleParagraph enarticleParagraph" >GERALDINE E. BRENT YOUNG, 59, of Teaneck died Sunday. She was a deli clerk at the ShopRite supermarket, Englewood, where she worked for 10 years and was a member of Meat Workers Union Local 464A, Englewood. Arrangements: Nesbitt Funeral Home, Englewood. </p>
<p class="articleParagraph enarticleParagraph" >PASQUALE L. "PAT" ZINZI, 86, of Hackensack died Tuesday. He had owned Luigi Zinzi &Sons Recycling Co., New York City, for 60 years. He was an Army veteran of World War II and a past commander and past district commander of Veterans of Foreign Wars Post 1429 and a member of the American Legion, both in Teaneck. He was a parishioner of Holy Trinity R.C. Church, Hackensack, and was past president of its Leisure Club. He was a member of Knights of Columbus Council 747, Hackensack. Arrangements: G. Thomas Gentile Funeral Home, Hackensack.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>nobt : Obituary | ncat : Content Types | nrgn : Routine General News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>COLUMN | OBIT | Barnicoat, Cathie M</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>Record / Hackensack, New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document REC0000020030423dz4n0004t</td></tr></table><br/></div></div><br/><span></span><div id="article-nytf000020030119dz1j0005g" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>CLM</b>&nbsp;</td><td>
JERSEY FOOTLIGHTS
</td></tr>
<tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>
New Jersey Weekly Desk; Section 14NJ
</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>
JERSEY FOOTLIGHTS
 </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>
By Margo Nash
 </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>578 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>19 January 2003</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td> The New York Times </td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td> Page 11, Column 1 </td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>
c. 2003 New York Times Company
 </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >
Under the River
 </p>
<p class="articleParagraph enarticleParagraph" >
The New Jersey side was all mud. The New York side was all rock.
 </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >
In 1874, 20 men working in candlelight with a horse and a mule inched
their way under the Hudson River from New Jersey toward New York,
starting but not finishing the first Hudson tunnel. But the work they
began would eventually become part of the transportation system
connecting New Jersey and New York, with two sets of tunnels below the
Hudson. After several more efforts, the tunnels were fully operational
by 1909.
 </p>
<p class="articleParagraph enarticleParagraph" >
A new exhibition at the Hoboken Historical Museum, ''The Tubes: Rails
Under the Hudson, 1874 to the Present-day PATH'' uses over 100 items to
chronicle the construction of the tunnels and stations. Among the items
on view are an 1882 drawing showing the progress of the project, and a
1971 poster heralding the opening of the World Trade Center terminal.
 </p>
<p class="articleParagraph enarticleParagraph" >
''It's amazing the amount of work they got done with the labor of
their hands,'' said Terrence Kennedy, a collector of Hudson Tubes
memorabilia, who curated the show.
 </p>
<p class="articleParagraph enarticleParagraph" >
The transportation system, formerly called the Hudson & Manhattan
Railroad became known as the PATH or Port Authority Trans-Hudson when
the Port Authority took it over in 1962.
 </p>
<p class="articleParagraph enarticleParagraph" >
The exhibition, which opens on Tuesday, is on view through April 13.
Information: (201) 656-2240.
 </p>
<p class="articleParagraph enarticleParagraph" >
A Local Boy
 </p>
<p class="articleParagraph enarticleParagraph" >
He was born in Jersey City on Jan. 10, 1944. Because of the war, the
hospital was filled with wounded soldiers, and Navy medical personnel
helped deliver him.
 </p>
<p class="articleParagraph enarticleParagraph" >
A few weeks later, his mother took him and his older sister to Grand
Central Station where they boarded a train for California, to join his
father who was, he said, ''away on business.''
 </p>
<p class="articleParagraph enarticleParagraph" >
His father was Frank Sinatra; his mother and his sister were both
Nancy; and he was Frank Sinatra Jr. The family settled in California,
where the younger Mr. Sinatra grew up. He remembers seeing Hoboken,
where his father was born, for the first time when he went to the
funeral of his grandfather, Martin Anthony Sinatra, a Hoboken fireman.
 </p>
<p class="articleParagraph enarticleParagraph" >
''It was a cold, blustery day,'' he recalled, and seeing the place
where his family came from, he said, made a big impression on him.
 </p>
<p class="articleParagraph enarticleParagraph" >
Mr. Sinatra, left, returns to the state this Friday to appear in New
Brunswick at the New Jersey State Theater with his 36-piece Sinatra
Orchestra, led by Terry Woodson. He will sing the standards his father
recorded as well as some he sang on ''As I Remember It'' (Angel), a CD
he recorded in 1996 as a tribute to his father and the composers and
arrangers associated with him. Information: (877) STATE-11.
 </p>
<p class="articleParagraph enarticleParagraph" >
Smart Audiences
 </p>
<p class="articleParagraph enarticleParagraph" >
Joy Behar, co-host of the ABC television show ''The View,'' is
performing at the Community Theater in Morristown on Saturday at 8 p.m.
 </p>
<p class="articleParagraph enarticleParagraph" >
''Stand-up is still my first love,'' she said. And New Jersey
audiences are ''very bright,'' she added. ''They are good audiences.''
 </p>
<p class="articleParagraph enarticleParagraph" >
Where are audiences tougher?
 </p>
<p class="articleParagraph enarticleParagraph" >
''Washington, D.C.,'' she said. ''They have no sense of humor.''
 </p>
<p class="articleParagraph enarticleParagraph" >
''It's like playing to a bunch of doctors,'' she continued.
''Audiences of doctors are the worst.''
 </p>
<p class="articleParagraph enarticleParagraph" >
Ms. Behar, who grew up in Brooklyn, said she never does New Jersey
jokes. But she likes to do politician jokes.
 </p>
<p class="articleParagraph enarticleParagraph" >
''They are always funny,'' she said.
 </p>
<p class="articleParagraph enarticleParagraph" >
Information on the performance: (973) 539-8008.   Margo Nash
 </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>ART</b>&nbsp;</td><td><br/>
Photos
 </td></tr>
<tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i5020039 : Tunnel Construction | i502 : Heavy Construction | iconst : Construction | icre : Construction/Real Estate</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>gcat : Political/General News | nedc : Commentary/Opinion | ncat : Content Types | nfact : Factiva Filters | nfcpex : FC&E Executive News Filter</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>namz : North America | usa : United States | use : Northeast U.S. | usnj : New Jersey | usny : New York | nyc : New York City</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>NYT Business Information Service (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document nytf000020030119dz1j0005g</td></tr></table><br/></div></div><br/><span></span><div id="article-nytf000020021126dybq0002o" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>
Metropolitan Desk; Section B
</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>
Keeping Peril at Bay for Track Workers Involves More Than Just Questions
of Safety
</span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>
By ROBERT D. McFADDEN
</td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>1028 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>26 November 2002</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td> The New York Times </td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td> Page 3, Column 1 </td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>
c. 2002 New York Times Company
</td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >
They are like ghosts in the dark tunnels, sometimes glimpsed out the
window of a passing subway train, huddled in an alcove or standing
between the girders that divide local and express tracks. They seem to
be doing nothing, but that is just an illusion. Almost any movement --
even a few inches -- could be deadly, and they are pausing only while
the danger rumbles past.
</p>
<p class="articleParagraph enarticleParagraph" >
They used to be known as gandy dancers, after the railway
section-gang tools once made by the Gandy Manufacturing Company of
Chicago. Today, New York City Transit calls them maintenance-of-way
workers: the 7,000 keepers of the tracks, signals, switches, lights and
other equipment along the 722-mile subway system.
</p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >
It has always been a perilous job. ''It's the nature of the beast,''
said John Samuelsen, the official of Local 100 of the Transport Workers
Union who represents the track workers. ''The incidents tend to fall
into two categories -- near misses and fatalities. There's almost
nothing in between.''
</p>
<p class="articleParagraph enarticleParagraph" >
Since World War II, 132 track workers have been electrocuted or
killed by trains in the New York subways, 21 in the last two decades.
Two were struck and killed by trains last week, and those deaths have
produced yet another chapter in the longstanding dispute between union
and management over safety rules.
</p>
<p class="articleParagraph enarticleParagraph" >
Over the weekend, the transit agency, under pressure from the union,
halted all nonemergency track work pending a full review of safety
procedures.
</p>
<p class="articleParagraph enarticleParagraph" >
But the routine work was resumed early yesterday after the agency
agreed temporarily to assign to all crews a flagger whose sole job is to
alert oncoming trains to the workers' presence. In the tunnels,
flaggers, besides keeping a lookout, set up lights that require trains
to slow to 10 miles an hour from 35 or 40 m.p.h.
</p>
<p class="articleParagraph enarticleParagraph" >
Negotiators for both sides met yesterday, but no major progress was
reported on differences over flagging procedures and other safety
issues. On one level, the fundamental question appeared to be simple:
whether to assign a flagger to every work crew, as the union would like,
or only selectively to the larger crews or those with certain jobs to
perform.
</p>
<p class="articleParagraph enarticleParagraph" >
But those questions raise other questions: about productivity and
cost versus safety, about related issues like lighting in the shadowy
tunnels and when to use special equipment that can halt a train in an
emergency -- and especially about who will decide what the operating
rules are.
</p>
<p class="articleParagraph enarticleParagraph" >
The issues are complex, too, because New York's subway system is
unique, vastly larger than any other transit system in the nation, and
operating 24 hours day, while most others shut down at night, when
maintenance can be performed with minimal dangers to track workers.
</p>
<p class="articleParagraph enarticleParagraph" >
And because the New York subway is considered an intracity rail
system, it is not covered by the more stringent safety regulations of
the Federal Railroad Administration, rules that apply to the Port
Authority Trans-Hudson lines, the Long Island Rail Road and the
Metro-North, which traverse municipalities or states.
</p>
<p class="articleParagraph enarticleParagraph" >
As a result, the agency that operates the New York subways has for
decades operated largely under its own rules and those of the state's
Department of Labor, rules that the union says are weak on safety, a
contention disputed by the transit agency.
</p>
<p class="articleParagraph enarticleParagraph" >
Whatever else they are, the rules are extremely complex. There are 30
pages of rules governing flagging procedures, said Dave Katzman, a union
spokesman. Many govern such matters as the placement of emergency lights
at particular distances from workers, or on curves or other partly
obscured trackbeds.
</p>
<p class="articleParagraph enarticleParagraph" >
Some rules call for point-to-point flagging, requiring smaller crews
without flaggers just to post lights for oncoming trains. Others specify
when to ''full-flag'' a job, assigning a flagger to do nothing but alert
trains and work crews to dangers, and to use portable trippers that
automatically apply a train's air brakes if it passes a danger point.
</p>
<p class="articleParagraph enarticleParagraph" >
There are also rules for special flagging procedures, for example, on
more than one track when trains are passing a work site in opposite
directions, or when trains are deliberately routed in the wrong
direction, as in East River tunnels where there are only two sets of
tracks and one must be shut down.
</p>
<p class="articleParagraph enarticleParagraph" >
Beyond the rules, however, is another level of complexity -- their
practical application in the tunnels, a matter that often requires
interpretation and sometimes raises disputes between supervisors under
pressure to get jobs done and work crews concerned about their safety.
</p>
<p class="articleParagraph enarticleParagraph" >
On a given day, hundreds or thousands of workers are down in the
tunnels. Maintenance-of-way involves 2,000 track workers, 2,000 signal
and equipment workers and 3,000 structure and power workers, who
maintain the third rail and perform other work, including carpentry,
plumbing, iron work and other tasks.
</p>
<p class="articleParagraph enarticleParagraph" >
Some work alone or in groups of two or three, examining tracks and
beds, inspecting signals and switches, installing light bulbs or
performing other relatively simple maintenance. Flaggers working
exclusively to protect crews are rarely assigned to small groups, but
are more often assigned to larger groups on major jobs, like replacing
tracks or signals, union officials say.
</p>
<p class="articleParagraph enarticleParagraph" >
''The policy is designed to facilitate production, not to protect
workers,'' said Mr. Samuelsen, the union representative. He also said
that poor lighting in the tunnels endangered the crews.
</p>
<p class="articleParagraph enarticleParagraph" >
''The entire work environment is poorly lit,'' he said. ''It's almost
like a coal mine.''
</p>
<p class="articleParagraph enarticleParagraph" >
Other union officials say that motormen are not given a list of work
sites on their routes each day, and so must remain alert to the lamps
and dangers.
</p>
<p class="articleParagraph enarticleParagraph" >
Paul J. Fleuranges, a spokesman for New York City Transit, denied
that the agency sacrificed safety for productivity, but declined to
comment on the other union assertions, saying that such contentions on
both sides belonged at the bargaining table. Asked about the agency's
past policies on flagging and other issues, he said that the officials
most knowledgeable on those subjects were too occupied with negotiations
to provide information.
</p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i721 : Urban/Commuter Transit | i72101 : Urban Mass Transit Systems | itsp : Transportation/Shipping</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>ccat : Corporate/Industrial News | gcat : Political/General News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>namz : North America | nyc : New York City | usa : United States | use : Northeast U.S. | usny : New York</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>NYT Business Information Service (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document nytf000020021126dybq0002o</td></tr></table><br/></div></div><br/><span></span><div id="article-nytf000020020620dy6k0002e" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>CLM</b>&nbsp;</td><td>
BLOCKS
</td></tr>
<tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>
Metropolitan Desk; Section B
</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>
PATH Project Unloved in the Village
</span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>
By David W. Dunlap
</td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>869 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>20 June 2002</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td> The New York Times </td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td> Page 3, Column 4 </td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>
c. 2002 New York Times Company
</td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >
YOU do not have to stand long on the narrow dead-end PATH train
platforms at Christopher Street or Ninth Street to know that another way
into or out of the stations would be at least convenient. Maybe even a
lifesaver.
</p>
<p class="articleParagraph enarticleParagraph" >
Nor do you have to stand long on the narrow, 19th-century village
streets above the stations to know that the introduction of four new
stairways used by several thousand commuters a day would at least
disrupt, maybe even alter, the neighborhood character.
</p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >
For the moment, there seems to be no middle place to stand.
</p>
<p class="articleParagraph enarticleParagraph" >
PATH, which is proposing to expand both stations, is not bound by the
State Environmental Quality Review Act or the city's Uniform Land-Use
Review Procedure, which commonly govern large construction projects.
While it is easy to criticize these time-consuming regulatory gantlets,
known as Seqra and Ulurp, at least they offer public forums and ground
rules for the debate.
</p>
<p class="articleParagraph enarticleParagraph" >
In their absence, what might have been a tough but straightforward
planning exercise has turned into this summer's Stonewall rebellion,
pitting the immovable Greenwich Village against the irresistible Port
Authority, which runs PATH.
</p>
<p class="articleParagraph enarticleParagraph" >
At issue is a $29.6 million project, scheduled to begin next month,
to add two new stairways at the east end of the Christopher Street
station (emerging at Christopher and Bedford Streets) and two new
stairways at the west end of the Ninth Street station (emerging at
Christopher Street and Waverly Place).
</p>
<p class="articleParagraph enarticleParagraph" >
Unlike Upper West Siders, who will clearly benefit if the new
entrance to the 72nd Street subway station relieves overcrowding,
Greenwich Villagers have less of a stake in the positive outcome of a
commuter rail project that will upend some lives, homes and businesses
for nine months, if not longer.
</p>
<p class="articleParagraph enarticleParagraph" >
There is not much incentive, in other words, for the neighborhood to
say anything but no, which it has said plainly through its elected
officials and the Christopher Street Preservation Alliance.
</p>
<p class="articleParagraph enarticleParagraph" >
Neighbors worry about snarled traffic and fragile building
foundations. They are also concerned that the Waverly Place entrance
would damage the integrity of the district around the former Stonewall
Inn on Christopher Street, listed on both the state and federal
registers of historic places as the setting of the 1969 disturbance
involving gay patrons and the police that helped ignite the gay
civil-rights movement.
</p>
<p class="articleParagraph enarticleParagraph" >
The State Office of Parks, Recreation and Historic Preservation has
determined that the new entrances would be modest and small-scale enough
not to greatly detract from the district, said Wendy Gibson, the press
secretary.
</p>
<p class="articleParagraph enarticleParagraph" >
But David Carter, who is writing a history of Stonewall, said, ''When
you have a very important battlefield -- Gettysburg, Vicksburg or
Custer's last stand -- you don't want to alter any part of it.''
</p>
<p class="articleParagraph enarticleParagraph" >
The Port Authority of New York and New Jersey has wandered into this
battlefield as it tries to cope with the extraordinary increase in
passenger traffic at the two southernmost Manhattan stations in the
trans-Hudson rail system, following the destruction of the PATH hub at
the World Trade Center.
</p>
<p class="articleParagraph enarticleParagraph" >
WE started looking at how we can improve safety and security
throughout the system,'' said Michael P. DePallo, the director and
general manager of the Port Authority Trans-Hudson Corporation.
</p>
<p class="articleParagraph enarticleParagraph" >
Both stations have a central platform at which trains to and from New
Jersey board and discharge, sometimes simultaneously. Each has only a
single twisting staircase at one end. That means passengers at Ninth
Street might be 500 feet from the exit, or more than two city blocks.
Passengers at Christopher Street are 420 feet from fresh air.
</p>
<p class="articleParagraph enarticleParagraph" >
Average daily ridership at Ninth Street has increased to 8,900 to
date this year from 4,100 last year. At Christopher Street, it has grown
to 7,400 from 3,600. So many people come up the Christopher Street
staircase during the morning rush that PATH does not permit riders to go
down the stairs from 7 to 9:45 a.m.
</p>
<p class="articleParagraph enarticleParagraph" >
PATH estimates it would take 18 minutes to empty the Christopher
Street station in the worst case, 14 minutes to empty Ninth Street. The
goal, Mr. DePallo said, is to get closer to the National Fire Protection
Association standard of six minutes.
</p>
<p class="articleParagraph enarticleParagraph" >
At the unused end of each platform, PATH proposes underground
mezzanines with turnstiles, reached through street-level stairways not
unlike a subway station, enclosed by a balustrade. The stairways would
occupy about 18 inches of sidewalk and four feet of street, roughly
three parking spaces' worth.
</p>
<p class="articleParagraph enarticleParagraph" >
PATH officials say they evaluated 16 alternatives and have hired
consultants to re-examine those and an additional 18 possibilities. ''We
have had very significant community involvement,'' Mr. DePallo said.
</p>
<p class="articleParagraph enarticleParagraph" >
Too little, too late, said Ben Green, a co-chairman of the
Christopher Street Preservation Alliance, given that the problem in the
station layout was recognized long before Sept. 11.
</p>
<p class="articleParagraph enarticleParagraph" >
''Had the Port Authority addressed it earlier, there would have been
time to do the obvious studies that are needed here,'' Mr. Green said.
''We all have a problem and we all have to share in the solution.''
</p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>ART</b>&nbsp;</td><td><br/>
Map of Manhattan shows proposed new stairways.
</td></tr>
<tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>gcat : Political/General News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>namz : North America | usa : United States | use : Northeast U.S. | usny : New York</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>New York Times</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document nytf000020020620dy6k0002e</td></tr></table><br/></div></div><br/><span></span><div id="article-rec0000020020201dy210009e" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/recLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>NEWS</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>PATH AWARDS $300M PACT FOR WTC RAILS </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>The Associated Press </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>272 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>1 February 2002</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The Record</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>REC</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>© 2002 North Jersey Media Group Inc. Provided by ProQuest Information and Learning. All rights reserved.
 </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >The Port Authority of New York and New Jersey awarded a $300 million contract Thursday for construction of a new temporary PATH station at the World Trade Center site. </p>
<p class="articleParagraph enarticleParagraph" >The deal includes related work in the Port Authority Trans Hudson tunnels and in Jersey City that is needed to restore access to lower Manhattan from New Jersey and to reopen Jersey City's Exchange Place PATH station. </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >The contract, awarded to a partnership of Tully Construction, Yonkers Contracting Co., and A.J. Pegno Construction, was based on the lowest contractor's fee. The winning bid included a contractor's fee of $16.71 million. The next lowest bid was $23.75 million. </p>
<p class="articleParagraph enarticleParagraph" >Work is expected to begin in three weeks, PA spokesman Alan Morrison said. It should be completed in about two years. </p>
<p class="articleParagraph enarticleParagraph" >"This contract represents the first major step in the reconstruction of lower Manhattan and a milestone in New York City's recovery from Sept. 11," New York Gov. George Pataki said. </p>
<p class="articleParagraph enarticleParagraph" >Before Sept. 11, the World Trade Center station was the busiest in the PATH system, serving approximately 65,000 commuters daily. </p>
<p class="articleParagraph enarticleParagraph" >"The projects approved today will help provide relief to these commuters and restore a vital link between New Jersey and lower Manhattan," Gov. James E. McGreevey said. </p>
<p class="articleParagraph enarticleParagraph" >In addition to constructing a new temporary PATH terminal for lower Manhattan, the contract provides for restoring two PATH tunnels under the Hudson River, expanding the Exchange Place PATH station in Jersey City, and building new crossovers that will allow PATH trains to cross between tunnels at Exchange Place.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>CO</b>&nbsp;</td><td><br/>panynj : Port Authority of New York and New Jersey | yoncon : Yonkers Contracting Co Inc</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i502 : Heavy Construction | i5020039 : Tunnel Construction | i763 : Port/Harbor Operations | icre : Construction/Real Estate | itsp : Transportation/Shipping</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>c33 : Contracts/Orders | c332 : Government Contracts | ccat : Corporate/Industrial News | ncat : Content Types | nnam : News Agency Material</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>namz : North America | nyc : New York City | usa : United States | use : Northeast U.S. | usnj : New Jersey | usny : New York</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>NEW JERSEY | NEW YORK CITY | TRANSIT | RAILROAD</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>Record / Hackensack, New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document rec0000020020201dy210009e</td></tr></table><br/></div></div><br/><span></span><div id="article-nytf000020010924dx9o0000j" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>CLM</b>&nbsp;</td><td>
BOOKS OF THE TIMES
</td></tr>
<tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>
The Arts/Cultural Desk; Section E
</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>
Stories of Trade Center, Now Suddenly Obituaries
</span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>
By RICHARD BERNSTEIN
</td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>1197 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>24 September 2001</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td> The New York Times </td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td> Page 6, Column 3 </td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>(c) 2001 New York Times Company</td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >
TWIN TOWERS
</p>
<p class="articleParagraph enarticleParagraph" >
The Life of New York City's
</p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >
World Trade Center
</p>
<p class="articleParagraph enarticleParagraph" >
By Angus Kress Gillespie
</p>
<p class="articleParagraph enarticleParagraph" >
Illustrated. 263 pages. Rutgers University Press. $26.
</p>
<p class="articleParagraph enarticleParagraph" >
DIVIDED WE STAND
</p>
<p class="articleParagraph enarticleParagraph" >
A Biography of New York's
</p>
<p class="articleParagraph enarticleParagraph" >
World Trade Center
</p>
<p class="articleParagraph enarticleParagraph" >
By Eric Darton
</p>
<p class="articleParagraph enarticleParagraph" >
Illustrated. 241 pages. Basic Books. $15.
</p>
<p class="articleParagraph enarticleParagraph" >
It should come as no surprise that two books on the history of the
World Trade Center have been selling briskly since the terrorist attack.
There is an instant nostalgia in the air, a yearning to hold on to a
landmark suddenly transformed from a symbol of New York's commercial
vibrancy into a mass of rubble and bodies. It happens that the two books
-- ''Twin Towers'' by Angus Kress Gillespie and ''Divided We Stand'' by
Eric Darton -- offer valuable and differing accounts of the trade
center, of how it came into being and what it represented in the larger
narrative of 20th-century history.
</p>
<p class="articleParagraph enarticleParagraph" >
Mr. Gillespie's book, published in 1999, is a straightforward
history, well researched and plainly written. A professor of American
studies at Rutgers University, Mr. Gillespie provides the facts about
the center that many of us, who have taken the twin towers as part of
the landscape without knowing much about their origins or purposes, will
find riveting. His story includes a good deal of politics, especially
the politics that led New York and New Jersey, after years of paralyzing
disagreement, to allow the project to go ahead, with the Ajax Wrecking
and Lumber Corporation demolishing the existing buildings on the site in
1966.
</p>
<p class="articleParagraph enarticleParagraph" >
Mr. Gillespie writes a gripping chapter on the construction of the
towers, showing in detail how the monumental problems of engineering and
design were solved. And he surveys the towers as objects of cultural
history, the way in which they were first loved by critics and then
hated by them, even as the buildings won acceptance by ordinary people.
</p>
<p class="articleParagraph enarticleParagraph" >
Mr. Darton's book, also published in 1999, is a more complex
undertaking, more polemical and, while an interesting contribution, less
accessible. Mr. Darton, a New York journalist and the author of a novel,
''Free City,'' is less interested in the chronological history of the
towers than in what he calls their ''hidden anatomy.'' By this he means
their role in illustrating the unseen forces that mold the history of a
city like New York: commercial and banking interests, behind-the-scenes
maneuverings of captains of industry and politicians and the way in
which publicly stated purposes masked a hidden agenda: real estate
speculation.
</p>
<p class="articleParagraph enarticleParagraph" >
Whereas Mr. Gillespie clearly likes the World Trade Center and has
written a generally happy history of it, Mr. Darton's account has a
sinister, muckraking cast to it. For him the center was the product of a
kind of robber-baron ruthlessness, or what he calls ''the 20th century's
most spectacular act of full-frontal real estate.'' His book is cogently
argued and impressively informed, but it is also needlessly convoluted,
stuffed with pretentious shards of language and therefore annoyingly
difficult to follow.
</p>
<p class="articleParagraph enarticleParagraph" >
Both books tell the essential story, which includes the creation of
the Port Authority of New York and New Jersey in 1921, its history of
building bridges and tunnels and the agreement, reached in the mid-60's,
to build the World Trade Center. It was to be a place where,
essentially, the paperwork of the Port of New York could be
concentrated, but eventually it became more or less ordinary, high-end
office space, available to anybody willing to pay the rent, a
transformation that was a major factor in control of the towers being
placed in private hands a few months ago.
</p>
<p class="articleParagraph enarticleParagraph" >
The compromise that led to the towers' construction included the Port
Authority's agreement to take over an obsolete and money-losing
underground rail connection between Manhattan and New Jersey, now the
Port Authority Trans-Hudson, or PATH, line. It also generated  revenue
for New York City by creating 28 acres of reclaimed land on the Hudson,
where the World Financial Center and Battery Park City now stand.
</p>
<p class="articleParagraph enarticleParagraph" >
Battery Park City is a state project; the city has the option to
regain ownership but so far has not. Mr. Gillespie gives the more
balanced history here, showing ways in which the compromise benefited
New York, while Mr. Darton's story is, as always, darker, focused on the
machinations of figures like David and Nelson Rockefeller in promoting a
public project that, he avers, enhanced their own interests.
</p>
<p class="articleParagraph enarticleParagraph" >
Both writers provide a brief biography of the chief architect of the
towers, Minoru Yamasaki, and they ground him in 20th-century
architectural history. The construction required, in Mr. Gillespie's
version, three major innovations, including a structural system in which
the steel supporting columns were placed on the outer rim of the
buildings rather than in their cores, yielding more usable and
column-free space. To build the towers, cranes were imported from
Australia (and therefore called kangaroo cranes) equipped with hydraulic
systems that enabled them to hoist themselves on the top of the towers
as they rose.
</p>
<p class="articleParagraph enarticleParagraph" >
Mr. Darton provides a deeper account of the architectural background
of the towers, whose design was inspired by the work and theories of the
great Swiss-born architect Le Corbusier (whom, typically, he calls a
''shadowy presence, part Mephisto, part Faust''). Mr. Darton is more
concerned than Mr. Gillespie with the battles fought and lost by the
merchants who occupied several of the streets that would be incorporated
into the giant complex, especially the owners of discount electronics
shops that used to occupy the streets of lower Manhattan.
</p>
<p class="articleParagraph enarticleParagraph" >
Both writers also provide fascinating accounts of the towers' public
reception, particularly how the most influential architectural critics,
Wolf von Eckardt of The Washington Post and Ada Louise Huxtable of The
New York Times, soured on it after first praising it. ''The world's
tallest fiasco,'' was the headline on an article by Mr. von Eckardt in
Harper's. ''This fearful instrument of urbicide,'' he wrote, ''will be
not only the tallest, but unquestionably one of the ugliest buildings in
the world.''
</p>
<p class="articleParagraph enarticleParagraph" >
Mr. Darton, in agreeing with the unfavorable assessments of the trade
center as a piece of architecture, introduces a note of higher silliness
into his analysis, finding common ground between the builders of the
towers and terrorists who might want to destroy them. Both the builders
of something so immense and terrorists, he writes, ''must undertake a
radical distancing of themselves from the flesh and blood experience of
mundane existence 'on the ground.' ''
</p>
<p class="articleParagraph enarticleParagraph" >
That is what might be called a resemblance without a similarity.
Still, it is worth noting that though both Mr. Gillespie and Mr. Darton
understood the World Trade Center's value as a terrorist target, they
assumed in writing their books that it was a permanent part of the life
of New York. And while they were wrong, like everybody else, their books
give us a sense of the historical richness and complexity of what we
have lost.
</p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>ART</b>&nbsp;</td><td><br/>
Photos: Angus Kress Gillespie (Steve Goodman/Rutgers University Press);
Eric Darton (Bill Gubbibs/Basic Books)
</td></tr>
<tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>gbook : Books | gcat : Political/General News | gent : Arts/Entertainment | ncat : Content Types | nobt : Obituary | nrgn : Routine General News | nrvw : Review</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>namz : North America | usa : United States | use : Northeast U.S. | usny : New York</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>Review </td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document nytf000020010924dx9o0000j</td></tr></table><br/></div></div><br/><span></span><div id="article-rec0000020010820dw5k001ap" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/recLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>NEWS</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>PATH SUING U.S. FOR REPORT ON DENIAL OF BRAKE-TEST WAIVER </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>JEFFREY GOLD, The Associated Press </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>524 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>20 May 2000</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The Record</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>REC</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>© 2000 North Jersey Media Group Inc. Provided by ProQuest Information and Learning. All rights reserved.
 </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >The agency that operates the PATH trains says the Federal Railroad Administration is improperly withholding documents on why the FRA denied PATH's request to exempt it from certain brake tests. </p>
<p class="articleParagraph enarticleParagraph" >The Port Authority Trans-Hudson Corp. wants the FRA to reconsider the denial but maintains that it can get no information on how the decision was made. </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >PATH's deputy general counsel, Hugh H. Welsh, said Friday that the dispute over the brake tests is part of a larger conflict: PATH says it is a rapid-transit system whose operations are not compatible with FRA rules, which are designed for commuter lines such as NJ Transit and the Long Island Rail Road. </p>
<p class="articleParagraph enarticleParagraph" >PATH believes it should be regulated by another agency in the U.S. Department of Transportation, the Federal Transit Administration. </p>
<p class="articleParagraph enarticleParagraph" >The FTA oversees the New York City subways and rapid transit systems in Philadelphia, San Francisco, and Washington, D.C., most of which have similar cars and third-rail electrification systems. </p>
<p class="articleParagraph enarticleParagraph" >The question of who regulates PATH affects the cost of modernizing the system, Welsh said. Under FTA rules, PATH could purchase cheaper cars, because the trains would not have to meet the more demanding standards of long-haul railroads. </p>
<p class="articleParagraph enarticleParagraph" >Federal courts have let PATH remain under FRA control, and in 1992 Congress explicitly defined PATH as a commuter railroad, FRA spokeswoman Pamela Barry said. </p>
<p class="articleParagraph enarticleParagraph" >PATH trains use tunnels under the Hudson River to carry people between New York and Hoboken, Jersey City, and Newark, carrying 233,000 passengers daily. The agency is a subsidiary of the Port Authority of New York and New Jersey. </p>
<p class="articleParagraph enarticleParagraph" >The FRA decision came without notice in September, 5 1/2 years after PATH filed its petition, according to a lawsuit PATH filed in U.S. District Court in Newark. </p>
<p class="articleParagraph enarticleParagraph" >PATH was told only that the "field investigation report, and the technical staff's findings" were the basis for the denial. </p>
<p class="articleParagraph enarticleParagraph" >The FRA ignored requests since October by PATH for the report and findings, the lawsuit said. Welsh said he visited the FRA docket clerk in Washington on Nov. 22, only to be told the docket could not be found. </p>
<p class="articleParagraph enarticleParagraph" >PATH says the FRA is violating the Freedom of Information Act by not providing the documents. That law is often used by news reporters and citizens to obtain government documents. </p>
<p class="articleParagraph enarticleParagraph" >The lawsuit asks U.S. District Judge Nicholas H. Politan to order the FRA to surrender the documents and to pay PATH's legal costs. </p>
<p class="articleParagraph enarticleParagraph" >Barry said the FRA could not comment on the lawsuit because it had not yet received a copy of it. The lawsuit was filed May 5. </p>
<p class="articleParagraph enarticleParagraph" >She had no immediate information on why the denial of the waiver came 5 1/2 years after the petition was filed. </p>
<p class="articleParagraph enarticleParagraph" >The FRA official who notified PATH of the denial, Grady C. Cothen Jr., deputy associate administrator for safety standards and program development, was not immediately available for comment, Barry said. </p>
<p class="articleParagraph enarticleParagraph" >The FRA has not penalized PATH, and it hopes to reach a compromise, Barry said.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i3302 : Computers/Electronics | i71 : Railroads | i721 : Urban/Commuter Transit | i72101 : Urban Mass Transit Systems | i8394 : Computer Services | itsp : Transportation/Shipping</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>c12 : Corporate Crime/Legal/Judicial | ccat : Corporate/Industrial News | gcat : Political/General News | gcrim : Crime/Courts</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>nam :  | namz : North America | usa : United States | use : Northeast U.S. | usnj : New Jersey | usny : New York</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>NEW JERSEY | TRANSIT | USA | RAILROAD | GOVERNMENT | Welsh, Hugh H | Barry, Pamela</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>Record / Hackensack, New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document rec0000020010820dw5k001ap</td></tr></table><br/></div></div><br/><span></span><div id="article-rec0000020010829dv8f00q4j" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/recLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>SPECIAL SECTION / NEWS</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>HISTORY IN YOUR BACK YARD Series: COUNTDOWN TO 2000 </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By JOHN CICHOWSKI, Staff Writer </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>2336 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>15 August 1999</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The Record</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>REC</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>© 1999 North Jersey Media Group Inc. Provided by ProQuest Information and Learning. All rights reserved.
 </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >If you've ever been told "important things never happen" in suburbia, come with us for a tour of American history. </p>
<p class="articleParagraph enarticleParagraph" >North Jersey is where almost everything of significance has happened - from war, homicide, and labor unrest, to the birth of baseball and the creation of some of the world's most ingenious inventions. </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >Check the library of the World Wide Web to learn the entire history of the nearly 1,000 square miles that make up Bergen, Passaic, Morris and Hudson counties. But for a start, consult this Millennium Map for some highlights before making your own tour of historic sites. You won't have to go far. </p>
<p class="articleParagraph enarticleParagraph" >1. WOMAN BUYS BERGEN </p>
<p class="articleParagraph enarticleParagraph" >When Dutch Gov. Peter Stuyvesant bought what is now Ridgefield Park, Bogota, and part of Teaneck from the Indians in 1664, he feared that he might not be able to hold the claim because the Dutch had just lost Manhattan to the English. To mask the Dutch purchase, his interpreter - Sara Kiersted - signed the deed, making a woman the owner of Bergen County's first lasting European settlement. </p>
<p class="articleParagraph enarticleParagraph" >2. THE SACKING OF HACKENSACK </p>
<p class="articleParagraph enarticleParagraph" >British-led Hessian soldiers burned the courthouse in Hackensack, near the site of the current Bergen County Courthouse complex, on March 23,1776. As Tories pointed out patriots, Hessians burned houses and captured all male inhabitants. </p>
<p class="articleParagraph enarticleParagraph" >3. REVOLUTIONARY TURNING POINT </p>
<p class="articleParagraph enarticleParagraph" >When 5,000 British troops cornered 1,000 Colonial soldiers at Fort Lee on Nov. 20,1776, George Washington had to lead an escape across New Bridge at the Steuben House in River Edge, prompting Thomas Paine to write, "These are the times that try men's souls." But Washington's army reemerged to defeat the British at Trenton on Christmas Eve. 4. BERGEN MASSACRE </p>
<p class="articleParagraph enarticleParagraph" >One of the Revolution's great massacres was in River Vale on Sept. 27, 1778, when British troops surprised 100 sleeping Virginia cavalrymen and killed 30 to 50 of them with bayonets. Six bodies were not discovered until 1967. A plaque marks their grave in Baylor Massacre Park at Rivervale and Old Tappan roads. </p>
<p class="articleParagraph enarticleParagraph" >5. NEW JERSEYS VALLEY FORGE </p>
<p class="articleParagraph enarticleParagraph" >The coldest winter of the Revolution was at Jockey Hollow in Morristown, where 90 inches of snow fell in 1779-80. Unpaid, starving soldiers stole, ate their pets, and roasted shoes to stay warm. Eighty-nine died, far fewer than the more poorly clothed troops at Valley Forge, Pa., in 1778, but about 1,000 deserted Jockey Hollow. Many were executed. A ball at the nearby Ford Mansion failed to raise money when yet another snowstorm struck, one of 28 that winter. 6. FIRST INDUSTRIAL CITY </p>
<p class="articleParagraph enarticleParagraph" >When Alexander Hamilton saw a waterfall, perhaps at a picnic with Washington on July 19, 1778, he dreamt of a government-run, hydro-powered manufacturing city. Congress rejected the plan for a huge city, so Hamilton wooed private investors, such as Gov. William Paterson, and on May 18, 1792, a much smaller Paterson began - as a corporation. Municipal officials were not elected until 1831. </p>
<p class="articleParagraph enarticleParagraph" >7. FATAL POLITICAL DUEL </p>
<p class="articleParagraph enarticleParagraph" >When Hamilton was fatally wounded in a duel in Weehawken on July 11, 1804, authorities issued murder charges against his killer, Vice President Aaron Burr, who fled to the South. But extradition laws between states were non-existent, and Burr was never tried. No attempt was made to impeach him, although he soon presided at the impeachment trial of a Supreme Court justice. </p>
<p class="articleParagraph enarticleParagraph" >8. FIRST "SUPERHIGHWAY" </p>
<p class="articleParagraph enarticleParagraph" >When fast shipping was needed between the Hudson and Delaware rivers, the Morris Canal was built in 1827 from Jersey City to Phillipsburg, and was not replaced by rails until the early 1900s. The canal, whose path roughly follows Routes 46 and 80, is commemorated in Lincoln Park, Little Falls, and elsewhere. </p>
<p class="articleParagraph enarticleParagraph" >9. THE GUN THAT WON THE WEST </p>
<p class="articleParagraph enarticleParagraph" >Inventor Samuel Colt developed the Colt .45, the first firearm with a revolving chamber and 35 other moving parts, in 1835 in the Gun Mill, parts of which still stand near the Great Falls in Paterson. Colt's business failed, and it took him 20 years to learn how to mass-produce the weapon in Hartford, Conn. </p>
<p class="articleParagraph enarticleParagraph" >10. RAIL, SEA, AND AIR POWER </p>
<p class="articleParagraph enarticleParagraph" >Starting in 1837 with the Rogers Locomotive Works at 2 Market St., Paterson played a huge role in transportation. City locomotive makers produced 80 percent of the nation's 19th century rail engines. John Holland used the Passaic River to test the first practical submarine in 1878. Paterson's Wright Aeronautical Co. made the engine that powered Charles Lindbergh's plane on the first solo Atlantic flight in 1927, and as Curtiss-Wright, the Beckwith Avenue plant became the nation's top builder of World War II engines and propellers. 11. BIRTH OF TELECOMMUNICATIONS </p>
<p class="articleParagraph enarticleParagraph" >Painter Samuel Morse and machinist Alfred Vail were an unlikely pair when they unveiled the first electric telegraph at 33 Speedwell Ave. in Morristown on Jan. 11, 1838. Vail persuaded Morse to abandon a plan for a telegraph language. Vail wanted simpler dot-dash symbols to correspond with letters of the alphabet - Morse Code. Vail's wife, Jane, suggested the tapping device: a piano key. </p>
<p class="articleParagraph enarticleParagraph" >12. BIRTH OF BASEBALL </p>
<p class="articleParagraph enarticleParagraph" >Baseball began on June 19, 1846, at Elysian Fields on a site that became the Maxwell House plant in Hoboken. Historians say Abner Doubleday, once believed to have invented baseball in Cooperstown, N.Y., in 1839, was a West Point plebe then and never visited Cooperstown. Abner Cartwright used the Hoboken site to add symmetry to bat-and-ball games by creating a diamond and foul territory. </p>
<p class="articleParagraph enarticleParagraph" >13. ELLIS ISLAND </p>
<p class="articleParagraph enarticleParagraph" >The arrival point for 12 million immigrants was a tiny spot of land in New York Harbor from 1892 to 1954. Although Ellis Island is federal property, New Jersey battled New York for bragging rights for decades until the U.S. Supreme Court ruled last year that most of the island is in Jersey City. </p>
<p class="articleParagraph enarticleParagraph" >14. NEARLY PRESIDENT </p>
<p class="articleParagraph enarticleParagraph" >When Vice President Garret A. Hobart died on Nov. 21, 1899, it marked Paterson's last flirtation with the White House. Had the city's leading politician lived, he - not Theodore Roosevelt - most likely would have succeeded President William Mckinley, who was killed in 1901. Hobart's home is now the Paterson YWCA, and his summer home is now Camp Veritans in Haledon. </p>
<p class="articleParagraph enarticleParagraph" >15. HOLLYWOOD ON THE PALISADES </p>
<p class="articleParagraph enarticleParagraph" >Thomas Alva Edison did far more for the movies than invent the motion picture camera in 1893. He personally shot scenes for early silent films, such as "The Great Train Robbery," in Little Falls in 1905 and the first cliffhanger, "The Perils of Pauline," which was shot in Fort Lee, the first film capital. 16.THREE ASSASSINS </p>
<p class="articleParagraph enarticleParagraph" >Political assassins Gaetano Bresci, Giuseppe Zangara, and Talmadge Hayer were not contemporaries, but they lived within a few miles of one another. Bresci, a Paterson silk weaver, gunned down Italian King Umberto in Monza, Italy, on July 29, 1900. Zangara, a bricklayer from Elmwood Park (then East Paterson), fired at President-elect Franklin Roosevelt in Miami on Feb. 15, 1933, fatally wounding Chicago Mayor Anton Cermac instead. Hayer, a young Paterson man, and two others shot down Black Muslim leader Malcolm X in Harlem on Feb.21, 1965. 17. ICE CREAM CONE </p>
<p class="articleParagraph enarticleParagraph" >When pushcart vendor Italo Marchiony used baked waffle cups to hold lemon ice and ice cream at the turn of the century, the Hoboken man and his Wall Street customers were delighted. Unlike liquor glasses, cones did not break or need washing, factors which helped make them a national sensation soon after Marchiony introduced them at the Louisiana Purchase Exposition in St Louis in 1904. </p>
<p class="articleParagraph enarticleParagraph" >18. FIRST MANHATTAN CONNECTION </p>
<p class="articleParagraph enarticleParagraph" >Years before cars crossed the Hudson River by bridge and tunnel, trains began carrying commuters between Manhattan and Hoboken in 1908 on the old Hudson and Manhattan Railway, now the Port Authority Trans Hudson (PATH) line. Formerly dependent on ferries, the urbanization of once-rural Hudson and Bergen counties was accelerated by these `tubes," the first permanent link to the big city. </p>
<p class="articleParagraph enarticleParagraph" >19. CIVIL RIGHTS FIRST </p>
<p class="articleParagraph enarticleParagraph" >Forty-four years before Rosa Parks refused to sit in the back of an Alabama bus, Minerva Miller sued the Paterson Show, a theater at 136 Market St., for charging blacks a quarter to see a movie that cost whites only a nickel. In 1911, a Southern-born judge rejected defense claims that New Jersey common law allows proprietors to exclude anyone they wish. He fined the theater $500. </p>
<p class="articleParagraph enarticleParagraph" >20. PATERSON SILK STRIKE </p>
<p class="articleParagraph enarticleParagraph" >One of the nation's landmark labor conflicts began on Feb. 15, 1913, when workers struck the Doherty Mill. Soon, 25,000 were striking, union members were jailed, and police accidentally killed Paterson resident Valentino Modestino. A rally May 19 at the Botto House at 83 Norwood St., Haledon, drew 20,000 supporters. About 15,000 attended a strike pageant at New York's Madison Square Garden on June 13. The Doherty Mill was on Main Avenue, Clifton. </p>
<p class="articleParagraph enarticleParagraph" >21. THE ZIPPER THAT WORKED </p>
<p class="articleParagraph enarticleParagraph" >Hoboken inventor Gideon Sundback pulled his employer, the Hookless Fastener Co., from the brink of bankruptcy in 1913 by perfecting a zipper that would not pop open. The Navy used it first - for flight suits, money belts, and gloves. </p>
<p class="articleParagraph enarticleParagraph" >22. HOT TEXAS WIENERS </p>
<p class="articleParagraph enarticleParagraph" >Hot dog connoisseurs date the creation of the original chili-dog sauce to 1919 at a Paterson eatery called New Systems on Paterson Street. In 1936, Greek immigrant Bill Pappas, who worked there, took his hot Texas wiener version to his own Libby's Lunch at 98 McBride Ave. Several Libby's employees started their own popular wiener restaurants throughout the region, many of which still thrive. </p>
<p class="articleParagraph enarticleParagraph" >23. FLYING FIRSTS </p>
<p class="articleParagraph enarticleParagraph" >Adm. Richard Byrd flew to the North Pole in 1926 in a trimotor monoplane built at Teterboro Airport. A pilot, Jimmy Doolittle, relied on instruments - not sight - for the first time when he took off and landed in test flights at Boonton Township's Rockaway Valley Aerodrome in 1928, a year before his official mission on Long Island. Reaction Motors in Denville built the rocket engine used by Chuck Yeager's plane to break the sound barrier in 1947. </p>
<p class="articleParagraph enarticleParagraph" >24. FM RADIO </p>
<p class="articleParagraph enarticleParagraph" >When it was built in 1936, WFDU's radio tower on Route 9W in Alpine became the first FM radio station - W2XMN. Frequency modulation, invented by Maj. Edwin Armstrong, later was used for television, and today, 70 percent of radio listeners prefer FM to amplitude modulation - AM. But back then, broadcasters refused to abandon AM despite FM's clearer reception. Mired in litigation, the brokenhearted inventor leapt off a Manhattan skyscraper on Jan. 31, 1954. 25. TELEVISION PIONEER </p>
<p class="articleParagraph enarticleParagraph" >Working in a converted pickle factory at 2 Main Ave., Passaic, in the 1930s, Montclair's Allen B. DuMont perfected the cathode ray tube that made commercial television possible. As a leading television manufacturer in the 1950s, DuMont Laboratories employed 5,000 at plants in Elmwood Park and Clifton. </p>
<p class="articleParagraph enarticleParagraph" >26. MUSIC! MUSIC! MUSIC! </p>
<p class="articleParagraph enarticleParagraph" >Hoboken's Frank Sinatra was discovered at the Rustic Cabin on Route 9W in Englewood Cliffs in the 1930s. The Meadowbrook in Cedar Grove was home to Tommy Dorsey, Benny Goodman, and other big bands of the 1930s and 1940s. Passaic's Capitol Theater, a burlesque house in the 1940s and an X-rated movie theater in the 1960s, became New Jersey's hottest rock venue in the 1970s and 1980s. </p>
<p class="articleParagraph enarticleParagraph" >27. CHARITY AND ROBBERY </p>
<p class="articleParagraph enarticleParagraph" >Paterson comedian Lou Costello raised the money to expand St. Anthony's Roman Catholic Church by getting Henny Youngman and Red Skelton to perform at the local premier of the first Abbott and Costello movie on Oct. 30, 1940. The rectory of the Beech Street church was the scene of a landmark armored truck robbery on Dec. 21, 1964, when gunmen surprised guards who routinely stopped there to visit a priest friend. Gunmen forced guards to turn over $511,000 in bank money from their truck. </p>
<p class="articleParagraph enarticleParagraph" >28. PHARMACOLOGICAL MIRACLES </p>
<p class="articleParagraph enarticleParagraph" >When Selman Waxman and researcher Albert Schatz of Passaic developed streptomycin to cure tuberculosis in 1943, it helped start the drug industry's golden age. At its Nutley/Clifton plant, Hoffmann-LaRoche led in tranquilizer development after Montclair's Leo Sternbach developed Librium in 1960 and Valium in 1963. Paterson's Albert Sabin created the oral polio vaccine in the late 1950s. 29. MOB BOSS RUBOUT </p>
<p class="articleParagraph enarticleParagraph" >Four gunmen silenced New Jersey crime boss Willie Moretti for good on Oct. 4, 1951, in a Cliffside Park restaurant then called Joe's Elbow Room at 793 Palisade Ave. Moretti's grave in St. Michael's Cemetery in Lodi remained a meeting place for top North Jersey and New York mobsters for three decades. </p>
<p class="articleParagraph enarticleParagraph" >30. BUBBLE WRAP </p>
<p class="articleParagraph enarticleParagraph" >At a plant in Haledon, Swiss-born Marc Chavannes and Arthur Fielding of Wayne learned how to trap air pockets between two layers of plastic film, and a revolutionary packaging material was born. Bubble Wrap was manufactured in 1957 as AirCap. The pair founded Sealed Air Corp. of Saddle Brook in 1960. </p>
<p class="articleParagraph enarticleParagraph" >31. MALLS! MALLS! MALLS! </p>
<p class="articleParagraph enarticleParagraph" >The nation sported only a few big malls when Paramus became the nation's retail shopping capital with the 1957 openings of the Garden State Plaza on Route 4 on May 1, and the Bergen Mall on Nov. 14. By the 1960s, motorists were flooding the Routes 4 and 17 shopping axis, which boasted six big retail shopping complexes. </p>
<p class="articleParagraph enarticleParagraph" >32. SURROGATE MOM </p>
<p class="articleParagraph enarticleParagraph" >When a family court judge in Hackensack gave custody of "Baby M" to a Tenafly couple instead of her natural mother on March 31, 1987, it marked the first time a surrogacy contract was ruled enforceable in the United States. The mother, Mary Beth Whitehead, agreed to take $10,000 from William Stern to carry his artificially inseminated child, but later, she refused to give up the baby. On Feb. 3, 1988, the state Supreme Court upheld custody for the Sterns, but it banned future childbearing contracts. Whitehead was given visitation rights.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>ART</b>&nbsp;</td><td><br/>MAP - Northern New Jersey </td></tr>
<tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>SERIES | NEW JERSEY | HISTORY</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>Record / Hackensack, New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document rec0000020010829dv8f00q4j</td></tr></table><br/></div></div><br/><span></span><div id="article-rec0000020010829dv4l00dj9" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/recLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>OPINION</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>CAN WE PRESERVE COOPERATION OVER BISTATE PORTS? </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By JAMES AHEARN </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>1035 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>21 April 1999</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The Record</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>REC</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>© 1999 North Jersey Media Group Inc. Provided by ProQuest Information and Learning. All rights reserved.
 </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >A BIT OF POLITICAL jostling has turned serious. The future of the Newark Bay container shipping ports, and of the larger Port of New York, is suddenly open to question. </p>
<p class="articleParagraph enarticleParagraph" >So is the more basic matter of bistate cooperation in the common interests of the metropolitan region. In the 78 years since the Port Authority of New York and New Jersey was created, that goal has been a given. It seems less certain now. </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >The immediate issue is whether the Port Authority will renew the lease of two large container-shipping companies, Maersk and Sea-Land. They employ 1,000 people, with a payroll of $100 million, at the Elizabeth Marine Terminal. Together they move a quarter of the containers that pass through the Port of New York every year. </p>
<p class="articleParagraph enarticleParagraph" >Maersk and Sea-Land want to stay where they are but don't want to pay as much rent as the Port Authority had proposed to charge them. The lines also seek deepening of the channels that lead to their piers, in order to accommodate a new generation of very large vessels. New wharves will be necessary too. </p>
<p class="articleParagraph enarticleParagraph" >The lines are not without alternatives to the New Jersey terminals. The ports of Baltimore and Halifax are naturally deep enough for the new ships and eager to attract them, and Baltimore, in particular, has offered to build a modern cargo-handling facility for Maersk and Sea-Land, with wharves, cranes, and a rail connection. </p>
<p class="articleParagraph enarticleParagraph" >The New York-North Jersey region is the biggest consumer market on the East Coast. A port 150 miles distant would be at a disadvantage in serving it. You would assume that shippers would prefer to deliver directly to the heart of the region, rather than transfer containers from ships to trains, and then to tractor-trailers. </p>
<p class="articleParagraph enarticleParagraph" >Still, Baltimore keeps sweetening its offer, while the situation here has gotten more and more murky. The current problem is New York Gov. George Pataki. He is trying to use the Maersk-Sea-Land negotiations to force a fundamental change in the allocation of Port Authority resources between New York and New Jersey. Basically, he wants more. </p>
<p class="articleParagraph enarticleParagraph" >That was not clear last year. The authority offered a new, 29-year lease to the two lines. The offer was formally presented by the authority's operations committee, headed by Charles Gargano, who is vice chairman of the authority board and Pataki's economic development director. In effect, Gargano was acting as Pataki's agent in the negotiations. </p>
<p class="articleParagraph enarticleParagraph" >In September the lines rejected the offer. The authority went back to the drawing board and came up with a new proposal. But the governor of each state has the power to veto any authority action, and Pataki put the kibosh on this one. Last week Governor Whitman tried to finesse him. She said she would make up the difference between the old offer and the new one, using some of the $300 million in state dredging bonds that New Jersey voters approved in 1996. </p>
<p class="articleParagraph enarticleParagraph" >Nice try. Pataki said no again, and it became clear that he was not so much opposed to the Maersk-Sea-Land deal as to the basic division of Port Authority resources between the two states. He and the ever-combative New York City mayor, Rudy Giuliani, contend that New York is perennially shortchanged, that New Jersey gets the better of the bistate deal. </p>
<p class="articleParagraph enarticleParagraph" >Capital-spending statistics don't support that contention. In the last decade, the authority spent $2.8 billion on projects in New York and $1.7 billion in New Jersey, with an additional $2.3 billion spent on interstate facilities. The authority's master plan calls for spending an additional $5 billion on New York projects, compared to $4.8 billion in New Jersey. </p>
<p class="articleParagraph enarticleParagraph" >"Yes, but," say the New Yorkers. Yes, those numbers look good for New York, but consider what the authority lays out in operating subsidies every year for the Port Authority Trans-Hudson line and for the authority's Manhattan bus terminal. </p>
<p class="articleParagraph enarticleParagraph" >The New Yorkers say that the bus terminal, despite its location, is really a New Jersey facility, and they have a point. Most of the people who use it live on this side of the Hudson and commute to jobs in the city. So too with the PATH line, which links Newark, Jersey City, and Hoboken to lower and midtown Manhattan. </p>
<p class="articleParagraph enarticleParagraph" >The line operates at a deficit, a big one. Most mass-transit lines lose money, but there is a special reason this one is so unprofitable. Its fare is only $1, unchanged since 1987. Tom Kean was governor of New Jersey then. He was popular in his second term and ineligible for a third. He could and did permit the fare to rise, from 75 cents. His successors, first Jim Florio and now Whitman, have balked at any further increase, fearing the political consequences. </p>
<p class="articleParagraph enarticleParagraph" >They have reason for concern. Nevertheless, New York is no less politicized than New Jersey, and the fare on the city's subways and buses is now $1.50, half again as high as the PATH fare. To cover the line's deficits, the authority uses profits from other facilities, such as the airports and the World Trade Center. </p>
<p class="articleParagraph enarticleParagraph" >New Yorkers have a legitimate complaint here. A moderate PATH fare increase is overdue. Pataki seems to have bigger changes in mind, though. He talks vaguely about a restructuring of the authority, with each state assuming responsibility for facilities on its side of the Hudson. </p>
<p class="articleParagraph enarticleParagraph" >New York would direct operations at Kennedy and LaGuardia airports and at the World Trade Center. New Jersey would take over Newark International Airport and the container ports. It is not clear what would happen to the truly interstate facilities _ the George Washington Bridge, the Lincoln and Holland tunnels, and the three bridges between New Jersey and Staten Island. </p>
<p class="articleParagraph enarticleParagraph" >It is hard to tell how serious Pataki is about these notions. They would radically restructure the Port Authority. The region functions economically, culturally, and socially as a unified whole, and the authority has furthered the common interest. It would be a pity to lose that.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>COLUMN | OP-ED | TRANSPORTATION</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>Record / Hackensack, New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document rec0000020010829dv4l00dj9</td></tr></table><br/></div></div><br/><span></span><div id="article-nytf000020010828dv3c00e33" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>
Metropolitan Desk; Section B
</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>
Port Authority Police Press Bias Inquiry on PATH Patrols
</span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>
By MICHAEL COOPER
</td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>829 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>12 March 1999</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td> The New York Times </td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td> Page 4, Column 3 </td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>(c) 1999 New York Times Company</td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >
Port Authority police officials are investigating allegations that a
small group of its officers who patrol the PATH trains in Manhattan have
been stopping and searching young black and Hispanic men without
probable cause, officials acknowledged yesterday.
</p>
<p class="articleParagraph enarticleParagraph" >
The Port Authority Police Department's internal affairs division is
conducting the inquiry, which also is looking into whether the roughly
half-dozen police officers have invented reasons on their arrest reports
to justify stopping those who were found to have drugs on them.
</p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >
It is the latest of several inquiries around the region examining
whether police officers stop members of minority groups because of the
color of their skin, a practice known as racial profiling. The PATH
investigation began several weeks ago after a Port Authority police
officer wrote an anonymous letter to several top officials complaining
of the practice. PATH, the Port Authority Trans-Hudson system, runs
between New York and New Jersey.
</p>
<p class="articleParagraph enarticleParagraph" >
Peter Yerkes, a spokesman for the Port Authority, confirmed that an
investigation was under way. ''We take allegations of misconduct
extremely seriously,'' he said. ''Certain allegations were made recently
and are being investigated thoroughly, as would be the case with any
allegations. We will have no comment until the investigation is
completed.''
</p>
<p class="articleParagraph enarticleParagraph" >
''It has always been Port Authority policy to expect our police to
maintain the very highest standards of professionalism,'' he added.
</p>
<p class="articleParagraph enarticleParagraph" >
The allegations are similar, though on a smaller scale, to those made
elsewhere in the region. The Justice Department is conducting a civil
rights investigation into whether New Jersey state troopers have been
illegally singling out black drivers on state highways. And a civil
rights advocacy group has asked a Federal judge in Manhattan to halt the
operations of the New York City Police Department's elite Street Crimes
Unit, based on accusations that the unit had illegally stopped and
searched tens of thousands of people because of their skin color.
</p>
<p class="articleParagraph enarticleParagraph" >
The officer who came forward with the allegations against the Port
Authority police said in an interview that while he was reluctant to do
something that could damage the reputation of his department and get
fellow officers into trouble, he felt that the civil rights violations
were too serious to ignore.
</p>
<p class="articleParagraph enarticleParagraph" >
''What they are doing is unconstitutional,'' said the officer, who
spoke on the condition of anonymity because he feared retribution from
other officers. ''It has to stop.''
</p>
<p class="articleParagraph enarticleParagraph" >
The officer said police officers who patrol the PATH stations have
got into the habit of frisking young black and Hispanic men, especially
those in hip-hop or sports clothes, solely on the basis of their
appearance. Those who have nothing illegal in their possession are freed
and the encounters are never written up or called in over police radios,
he said. Those caught with drugs are arrested, he said, and the officers
falsify their reports to explain why they stopped them.
</p>
<p class="articleParagraph enarticleParagraph" >
''They will say it was fare evasion, when the subject paid his
fare,'' he said. ''Or write down 'loud radio' when the radio is off, or
the subject looks intoxicated. It's all fabrication.''
</p>
<p class="articleParagraph enarticleParagraph" >
In the letter sent to several chiefs at the authority, he wrote:
''These guys are batting less than .300, but you would never know
because all they ever call in are the catches. So 7 out of 10 illegally
stopped and searched go away unhappy and violated, more poor minority
youth who will never trust a cop again.''
</p>
<p class="articleParagraph enarticleParagraph" >
Gus Danese, president of the Port Authority Patrolmen's Benevolent
Association, the officers' union, said it would be premature to comment
on the allegations because they were still under investigation.
</p>
<p class="articleParagraph enarticleParagraph" >
A detective from the department's internal affairs division has
already interviewed most of the officers who regularly patrol the PATH
system.
</p>
<p class="articleParagraph enarticleParagraph" >
The next step is likely to include scrutinizing the unit's arrest
reports to see if the number of arrests has risen markedly and whether
there are any unusual patterns in the reasons given for stopping people.
Eventually, people who were arrested could be reinterviewed, officials
said.
</p>
<p class="articleParagraph enarticleParagraph" >
The officer who made the allegations said officers have many
incentives to make arrests. For some, he said, it may be a desire to
bring down crime by any means necessary. But he also noted that officers
who make a large number of arrests please supervisors and can get
lucrative overtime pay above their regular salary, which is $62,000 a
year after five years on the job.
</p>
<p class="articleParagraph enarticleParagraph" >
Finally, he said, officers who make arrests near the end of their
shifts are put up overnight in hotels if they need to be on hand within
eight hours of when they go off duty to swear out criminal complaints.
This perk, which officials say is part of the officers' contract, is a
favorite of many Port Authority officers, he said. ''Some officers just
like go to a hotel on the Port Authority and party,'' the officer said.
</p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document nytf000020010828dv3c00e33</td></tr></table><br/></div></div><br/><span></span><div id="article-rec0000020010829dv1c001nr" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/recLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>NEWS</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>PILOT KILLED AS PLANE DIVES INTO RAIL YARD CRASH FOLLOWS TETERBORO TAKEOFF </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>SEAMUS McGRAW, Staff Writer </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>804 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>12 January 1999</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The Record</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>REC</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>© 1999 North Jersey Media Group Inc. Provided by ProQuest Information and Learning. All rights reserved.
 </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >A twin-engine plane plunged "straight down" into a Hudson County freight yard Monday afternoon, killing the pilot and narrowly missing a boxcar loaded with hazardous chemicals, authorities and witnesses said. </p>
<p class="articleParagraph enarticleParagraph" >The Cessna 310R sent a 40-foot pillar of flames into the sky when it crashed less than 150 yards from the busy commuter rail lines of the Port Authority Trans-Hudson railroad. </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >The plane belonged to Primac Air, an Englewood Cliffs courier and charter service. Pilot Ronny Wachter was the only person aboard the four-seat aircraft, which crashed at 3:52 p.m., moments after taking off from Teterboro Airport. The Cessna was bound for Wilmington, Del., according to the Federal Aviation Administration. </p>
<p class="articleParagraph enarticleParagraph" >Investigators told Conrail officials that it looked as if the plane had fallen nose-first from the sky. </p>
<p class="articleParagraph enarticleParagraph" >Al Warren, a Conrail employee, was sitting inside a blockhouse a few hundred yards away when the plane slammed into the rail car at Conrail's Intermodal Terminal. Warren, who guides tractor-trailers around the rail yard, reported the crash to Kearny police, then he and others raced to the the scene, hoping to find someone alive. </p>
<p class="articleParagraph enarticleParagraph" >No PATH passengers and none of Conrail's employees were injured when the plane crashed midway between two house-sized, cargo- carrying metal boxes known as containers, said yard manager Wayne Wagner. </p>
<p class="articleParagraph enarticleParagraph" >Emergency crews from the Kearny Fire Department quickly doused the blaze, then turned to the grim process of recovering the remains of the pilot. </p>
<p class="articleParagraph enarticleParagraph" >Ronald Foster, a Conrail special agent and one of the first officials at the scene, said there was little left of the pilot or the plane. Wachter, 34, of Harrison, was believed to have been killed instantly, police said. </p>
<p class="articleParagraph enarticleParagraph" >Charred and twisted fragments of metal, none larger than a filing cabinet, littered the tracks. </p>
<p class="articleParagraph enarticleParagraph" >But as horrible as the crash was, investigators said it could have been far worse. The plane hit the steel bar that linked two railroad containers loaded with dangerous chemicals, authorities said. </p>
<p class="articleParagraph enarticleParagraph" >One held 80 drums, or 13,892 pounds, of a combustible poison called sodium azide. Another had 101 cartons _ 1,485 pounds _ of a flammable aerosol gas. And a third container had 18 pounds of ethyl methacrylate, a flammable liquid, said Elaine Makatura, spokeswoman for the state Department of Environmental Protection. </p>
<p class="articleParagraph enarticleParagraph" >If the plane had hit the containers, "there would have been absolute explosions, and these were poisonous materials," she said. "The aircraft did not touch the containers, thank God." </p>
<p class="articleParagraph enarticleParagraph" >The crash occurred just before the beginning of the afternoon rush hour, when thousands of commuters board PATH trains there. About 15 trains are scheduled to pass by the Conrail yard between 3 and 4 p.m., a Port Authority spokesman said. </p>
<p class="articleParagraph enarticleParagraph" >"When the plane went down, there was some concern that it might have gone down close to our tracks, so we sent our a response team," said Port Authority spokesman Alan Hicks. "There is no impact on PATH service, and none of our employees or customers were in any danger." </p>
<p class="articleParagraph enarticleParagraph" >Investigators from the National Transportation Safety Board were on the scene late Monday, picking through the wreckage. </p>
<p class="articleParagraph enarticleParagraph" >According to preliminary reports, the 24-year-old plane had left Teterboro and almost immediately developed problems, one investigator said. </p>
<p class="articleParagraph enarticleParagraph" >"The pilot indicated he was having some problems with his controls," the investigator said. The plane disappeared from radar at 3:52 p.m., he said. </p>
<p class="articleParagraph enarticleParagraph" >Soon after, a Conrail employee reported a fire on the tracks. An instant later, he called back to say that it looked as if a plane had crashed. </p>
<p class="articleParagraph enarticleParagraph" >Primac Air offers air-courier services, primarily to Canada. Two years ago, it added charter flights. Company owner Mary Jane McLaughlin-Ely declined to comment on the accident Monday evening. </p>
<p class="articleParagraph enarticleParagraph" >In a 1997 interview, she said the company had six planes, including the Cessna 310R that went down Monday. Federal investigators familiar with the maintenance history of the 24-year- old plane say there had been problems with it in the past. </p>
<p class="articleParagraph enarticleParagraph" >On Jan. 26, 1990, the plane lost "directional control on landing rollout, and veered left into mud." A report said the left wheel on the plane locked up, and the plane was not damaged. </p>
<p class="articleParagraph enarticleParagraph" >The following day, the pilot had to land with the landing gear partly up because it would not deploy properly. </p>
<p class="articleParagraph enarticleParagraph" >Staff Writers Laura Barnhardt, Richard Cowen, Pat Gilbert, and Doug Most contributed to this story.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>ART</b>&nbsp;</td><td><br/>1 - PHOTO - JIRO OSE / STAFF PHOTOGRAPHER - Conrail official Wayne

Wagner speaking to reporters about the plane crash at a rail yard

Monday.; 2 - WABC-TV PHOTO - A televised picture of the scene of the

crash, in which the pilot died.; GRAPHIC - LIZ ROBERTS / STAFF ARTIST

- Plane crash in freight yard. </td></tr>
<tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>KEARNY | TETERBORO | AVIATION | ACCIDENT | DEATH | Warren, Al</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>Record / Hackensack, New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document rec0000020010829dv1c001nr</td></tr></table><br/></div></div><br/><span></span><div id="article-nytf000020010918dube01pea" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>
Metropolitan Desk; Section B
</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>
Autistic Boy Is Reunited With Family After 11 Days
</span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>
By ROBERT D. McFADDEN
</td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>462 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>14 November 1998</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td> The New York Times </td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td> Page 3, Column 6 </td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>(c) 1998 New York Times Company</td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >
A 15-year-old autistic boy who was found wandering in the Erie
Lackawanna Terminal in Hoboken, N.J., on Tuesday but could not identify
himself was reunited with his mother yesterday after she saw a news
report about him on television. He had been missing from his home on
Staten Island for 11 days.
</p>
<p class="articleParagraph enarticleParagraph" >
The boy, Leshawn Ware, was returned to his mother, Stephanie Ware, at
St. Mary Hospital in Hoboken, where he had been cared for since being
found in the waterfront terminal's main waiting room. His mother had
called the New Jersey authorities after seeing his picture Thursday
night on WWOR-TV.
</p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >
Although developmentally disabled, the boy was said by doctors to be
unharmed and in good condition, was clean and did not even seem hungry.
Officials said it was unclear where he had been or what he had done
since he wandered away from his home in the Great Kills section on Nov.
2.
</p>
<p class="articleParagraph enarticleParagraph" >
But his mother told officials he had an affinity for mass transit.
''She said that he has a love for the trains and the buses, and she
surmised that he must have gotten on a train or a bus or the ferry and
had been traveling the system before he wound up in Hoboken,'' said
Penny Bassett-Hackett, a spokeswoman for New Jersey Transit.
</p>
<p class="articleParagraph enarticleParagraph" >
Mrs. Ware, who has four other children, none of whom are autistic,
said that Leshawn, who attends special classes at an annex of Public
School 25 in Staten Island, was wearing a black and white down jacket
when he disappeared. She had called the New York City police to report
him missing.
</p>
<p class="articleParagraph enarticleParagraph" >
When found by Erie Lackawanna station employees, he had no money and
could not identify himself. But he was carrying five Metrocards from the
New York City transit system, and gave his address as 370 Jay Street in
Brooklyn, which turned out to be the headquarters of New York City
Transit.
</p>
<p class="articleParagraph enarticleParagraph" >
The terminal where the boy was found is a hub for New Jersey Transit
trains and buses, the Port Authority Trans-Hudson system and the New
York Waterway Ferry from the World Financial Center in Manhattan. He
might have come from almost anywhere. Law enforcement agencies,
including the New York police, were alerted, but no connection was made
immediately.
</p>
<p class="articleParagraph enarticleParagraph" >
The boy seemed unruffled by his experiences in any case. ''When we
got him, he was clean and not particularly hungry,'' said Joan Quigley,
a hospital spokeswoman. ''We suspect he was using the restrooms in the
train stations to freshen up.'' She also said that workers at a Burger
King may have fed him, but that she  had no details.
</p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document nytf000020010918dube01pea</td></tr></table><br/></div></div><br/><span></span><div id="article-nytf000020010918duan01m46" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>
Metropolitan Desk; Section A
</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>
Persevering Woman Helps Free Stranger In '90 Murder Case
</span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>
By KEVIN FLYNN
</td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>2317 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>23 October 1998</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td> The New York Times </td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td> Page 1, Column 6 </td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>(c) 1998 New York Times Company</td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >
KEARNY, N.J., Oct. 20 -- Do not be deceived by Priscilla Read
Chenoweth's sweet manner, the dust on her resume, the spectacles on her
nose or her matronly appearance. Do not dismiss this 68-year-old widow
as some kind of amateur Angela Lansbury with a law degree.
</p>
<p class="articleParagraph enarticleParagraph" >
That is what prosecutors did seven years ago, Mrs. Chenoweth
believes, when she decided to try to save a stranger who she believed
had been wrongly convicted of murder.
</p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >
''They thought, 'Here's that old grandma and her bleeding heart,' ''
she recalled this week during an interview in her home here.
</p>
<p class="articleParagraph enarticleParagraph" >
But last week, Luis Kevin Rojas, Mrs. Chenoweth's cause and crusade
since 1991, an aspiring engineer who had spent a third of his life in
prison or on trial for a murder he swore he did not commit, was freed
for good, in large part because of her singular persistence.
</p>
<p class="articleParagraph enarticleParagraph" >
After a two-week trial in State Supreme Court in Manhattan, a jury
acquitted Mr. Rojas of charges that, at age 18, he provided the gun used
in the 1990 shooting death of another teen-ager on the streets of
Greenwich Village.
</p>
<p class="articleParagraph enarticleParagraph" >
Beginning in 1991, after reading an account of the case in a New
Jersey newspaper, Mrs. Chenoweth spent much of her savings and many of
her days, nights, weekends and vacations chugging up legal hills for Mr.
Rojas. Mrs. Chenoweth, a lawyer who gave up her practice to become an
editor at The New Jersey Law Journal, smelled injustice in a jury
verdict that year that sent Mr. Rojas, a high school senior with no
prior record, to prison.
</p>
<p class="articleParagraph enarticleParagraph" >
So from the tiny cluttered study of her red-brick home here, aided by
her daughter, her sister and three retired detectives, she set out to
reinvestigate the crime and prove it was a case of mistaken identity,
attributed in part to a reversible jacket of overpowering orange. She
read transcripts, filed motions, interviewed witnesses, enlisted the
help of three dozen lawyers and ultimately spent $50,000 of her own
''old age'' money -- more than a year's salary.
</p>
<p class="articleParagraph enarticleParagraph" >
Before she was through, Mrs. Chenoweth and her team of paid
professionals and conscripted volunteers established an alibi for Mr.
Rojas: he was half a mile away at the time of the shooting. In 1995,
they succeeded in getting the original conviction overturned. And with
the acquittal in the retrial in State Supreme Court, they rolled back
the rest of the stone.
</p>
<p class="articleParagraph enarticleParagraph" >
For this effort, Mrs. Chenoweth has received all the compensation she
ever wanted -- thanks and affection from Mr. Rojas, flowers from his
family, and, perhaps most important to her, the respect of all those
high-priced lawyers she prodded into working with her.
</p>
<p class="articleParagraph enarticleParagraph" >
The lawyers said that, in this case, all those noble adages of
American jurisprudence, like ''justice will prevail,'' proved true only
because of Mrs. Chenoweth. Without her, they said, Luis Rojas would
still be working the sanitation detail at the Coxsackie Correctional
Facility, near Albany, just halfway through his 15-year to life
sentence.
</p>
<p class="articleParagraph enarticleParagraph" >
''Yes, the system worked,'' said Jethro M. Eisenstein, the lawyer who
donated his services and won Mr. Rojas's acquittal. ''But it only worked
because of Priscilla.''
</p>
<p class="articleParagraph enarticleParagraph" >
Even her adversary, Peter Kougasian, an assistant district attorney,
was impressed. ''I have never seen anyone pour their heart and soul into
an effort like this for a stranger,'' Mr. Eisenstein said he told him.
</p>
<p class="articleParagraph enarticleParagraph" >
Mrs. Chenoweth said she became involved because it was obvious to her
that Mr. Rojas, a stable student with no history of violence, was
incapable of playing a role in a murder. Even though the murder victim's
family still believes that Mr. Rojas played a part, Mrs. Chenoweth said
his involvement just never made sense to her.
</p>
<p class="articleParagraph enarticleParagraph" >
''He had no record, no gang, nice friends,'' she said. ''They were
all going to college or were skilled technical workers. He had a
reputation for peaceableness.''
</p>
<p class="articleParagraph enarticleParagraph" >
Reversible Jacket That Led to Arrest
</p>
<p class="articleParagraph enarticleParagraph" >
The shooting occurred just after 2 A.M. on Nov. 18, 1990, when two
groups of teen-agers tussled on lower Broadway, near Waverly Place. One
teen-ager in a Day-Glo orange, reversible jacket pulled out a gun, shot
into the air, then passed the weapon to another youth in a green jacket,
who fired at the other teen-agers as they fled. Javier Bueno, a
19-year-old planning to attend New York University, was shot in the neck
and died three weeks later. Rudy Quesada, 17, was shot in the leg. The
youth in the green jacket who fired the gun has never been found.
</p>
<p class="articleParagraph enarticleParagraph" >
Mr. Rojas, from Union City, N.J., was spotted by the police several
minutes later in the nearby PATH station on West Ninth Street. He was
wearing a similar Day-Glo jacket, but the orange side had been turned
inside out. One of the friends of the youths who were shot was taken to
the station and identified Mr. Rojas, now wearing the jacket orange side
out, as the youth who had supplied the gun. The police took Mr. Rojas to
the crime scene, where other friends of the victims identified him.
</p>
<p class="articleParagraph enarticleParagraph" >
The arrest shocked Mr. Rojas's friends at Emerson High School in
Weehawken, N.J., where 150 students wrote letters to the judge in the
case on his behalf. Though he had once taken a car without permission
and had been caught hopping a turnstile, Mr. Rojas was viewed as a
gentle soul, someone who counseled his peers on the dangers of drug
abuse.
</p>
<p class="articleParagraph enarticleParagraph" >
Mr. Rojas told the police that on the night of the shooting, he had
been at a nearby restaurant with a friend and was returning home to New
Jersey when he was arrested. He said he had reversed his jacket to wear
the burgundy side out because he did not want it to get dirty. That was
the way he usually wore it, he said.
</p>
<p class="articleParagraph enarticleParagraph" >
But the police had seven witnesses who said Mr. Rojas was involved
and they did not believe his account.
</p>
<p class="articleParagraph enarticleParagraph" >
Mrs. Chenoweth and her team would later undermine the reliability of
those identifications. Several witnesses had been drinking or picked out
Mr. Rojas only after seeing him in handcuffs. The defense team contended
that since the police had Mr. Rojas wear the jacket with the orange side
out, the sight of him in custody, in the jacket, within minutes of the
shooting, overpowered the ability of those distraught witnesses to make
a reliable identification.
</p>
<p class="articleParagraph enarticleParagraph" >
Indeed, one of the witnesses identified someone else in the PATH
station that night as also having been involved in the shooting. But
that person turned out to have an ironclad alibi. PATH, the Port
Authority Trans-Hudson, is a rail rapid-transit system between New York
and New Jersey.
</p>
<p class="articleParagraph enarticleParagraph" >
The police lineup the next morning was also stacked against Mr.
Rojas, Mrs. Chenoweth's team asserted. The ''fillers'' in the lineup
were well-rested, clean-shaven police cadets with crew cuts. Mr. Rojas
had medium-length hair and had been up all night.
</p>
<p class="articleParagraph enarticleParagraph" >
Few of these apparent holes in the evidence were brought forward
during Mr. Rojas's first trial, which ended in 1991 with a jury
convicting him of second-degree murder. The defense mounted by his
lawyer for that case, David Fronefield, was later derided by the
Appellate Division.
</p>
<p class="articleParagraph enarticleParagraph" >
''It almost appears, at times, that Mr. Fronefield acted as a second
prosecutor,'' the appellate panel wrote.
</p>
<p class="articleParagraph enarticleParagraph" >
At home in New Jersey, Mrs. Chenoweth read an article about the case
in a local newspaper at her daughter's suggestion. She focused on a
statement from one of Mr. Rojas's teachers, who said the teen-ager's
involvement was simply impossible. Something moved her to speak to the
teacher and members of Mr. Rojas's family. Within weeks, Mrs. Chenoweth
was off to meet Mr. Rojas himself at Rikers Island.
</p>
<p class="articleParagraph enarticleParagraph" >
''It was clear to me that it was a gross injustice,'' she said. ''The
legal system had harmed this man, and the legal system should right the
wrong.''
</p>
<p class="articleParagraph enarticleParagraph" >
A History of Trying To Right Wrongs
</p>
<p class="articleParagraph enarticleParagraph" >
Mrs. Chenoweth had some history at trying to right wrongs. She had
been involved in the civil rights movement for many years and was
arrested in New Jersey in 1963 for sitting down in front of a truck to
protest the lack of construction jobs for blacks.
</p>
<p class="articleParagraph enarticleParagraph" >
But it had been 17 years since the mother of three and grandmother of
five had practiced law, a career she left because ''the money was a
little erratic and I wasn't terribly good at business.''
</p>
<p class="articleParagraph enarticleParagraph" >
While rusty, she did not lack for confidence. After all, she was the
daughter of a Harvard-educated stockbroker so convinced of his abilities
that he used to write corrections in the margins of encyclopedias.
Still, she had practiced for only six years -- 1968 to 1974 -- and in
that time, she had done only a smattering of criminal cases, most of
them misdemeanors.
</p>
<p class="articleParagraph enarticleParagraph" >
''I'd had court experience,'' she recalled, ''but never a jury
trial.''
</p>
<p class="articleParagraph enarticleParagraph" >
Mr. Rojas, nonetheless, liked her from the very start. ''She really
believed in me,'' he said early this week in Mr. Eisenstein's Manhattan
law office.
</p>
<p class="articleParagraph enarticleParagraph" >
Mr. Rojas, whose mother had died while he was in prison, came to rely
on Mrs. Chenoweth for emotional support. He sent her cards on Mother's
Day. She told him to cut his hair. She carried a sport coat in her
briefcase for him to wear in court and worked to persuade experienced
lawyers to get involved in his case.
</p>
<p class="articleParagraph enarticleParagraph" >
''Originally, I only saw myself as a conduit,'' she said, but she
ended up doing a lot of the legal work herself. She also heard about a
selfless retired New York City police detective who tried to crack
unsolved murders. Soon the detective, Denis O'Sullivan, was part of her
team, putting up his own house to secure Mr. Rojas's $50,000 bail.
</p>
<p class="articleParagraph enarticleParagraph" >
Another retired police officer, Mike O'Connor, found a critical
witness -- a PATH police sergeant, John Apel. The sergeant remembered
that on the night of the crime, he had watched Mr. Rojas just miss a
train as it left the station. Research indicated that the train in
question departed that night no later than 2:03 A.M. That meant Mr.
Rojas could not have been involved in the dispute on Broadway, a
half-mile away, at 2:07 A.M., when the shooting started.
</p>
<p class="articleParagraph enarticleParagraph" >
When Mr. Rojas's appeals lawyer, Tina Mazza, introduced this evidence
to the Appellate Division in 1995, it overturned the conviction, also
citing ineffective counsel by his original lawyer and tainted
identifications.
</p>
<p class="articleParagraph enarticleParagraph" >
Released From Prison, Only to Face a Retrial
</p>
<p class="articleParagraph enarticleParagraph" >
After more than four years in prison, Mr. Rojas was out on bail. But
prosecutors decided to retry him. Their reasoning: the witnesses, a
solid, stable group of college students, still believed that Mr. Rojas
was the man in the orange jacket who helped kill their friend.
</p>
<p class="articleParagraph enarticleParagraph" >
''We also had circumstantial evidence,'' said Barbara Thompson, a
spokeswoman for the District Attorney's office in Manhattan, ''so we
decided to let the jury decide.''
</p>
<p class="articleParagraph enarticleParagraph" >
The decision made Mrs. Chenoweth's blood boil. ''They were not
interested in justice. They were interested in securing and keeping a
conviction,'' she said. ''It's like a pride of lions giving up a kill.''
</p>
<p class="articleParagraph enarticleParagraph" >
The families of the victims applauded the move, however. They still
believe that Mr. Rojas was involved. ''I think he got away with a big
lie,'' said Jose Bueno, whose brother was killed. ''Seven people saw him
pull out the gun. Luis Rojas will have to live with that for his whole
life.''
</p>
<p class="articleParagraph enarticleParagraph" >
For the retrial, Mrs. Chenoweth persuaded Mr. Eisenstein, a former
criminal defense lawyer, to try the case for nothing. Their relationship
was close but difficult. Mrs. Chenoweth liked to second-guess. ''She had
some loony ideas about what the rules of evidence allowed,'' he said.
But if he mentioned that, he remembered, ''she would say to me, 'Well,
it's my loony ideas that got Kevin this far.' ''
</p>
<p class="articleParagraph enarticleParagraph" >
At that trial, Sergeant Apel's alibi testimony was critical. Another
witness, William Davis, said he had told the police on the night of the
shooting that Mr. Rojas was the wrong man. The man in the orange jacket
had had a ponytail and was built like a body builder. Mr. Rojas was
5-foot-7 and weighed all of 125 pounds. Jurors noted that there was no
finding of gunpowder on Mr. Rojas's jacket, and the witnesses' accounts
about who had done what during the shooting began to conflict on
cross-examination.
</p>
<p class="articleParagraph enarticleParagraph" >
Still, the jury deliberated for 24 hours before coming back with a
verdict: not guilty on all counts.
</p>
<p class="articleParagraph enarticleParagraph" >
Mrs. Chenoweth and Mr. Eisenstein wept. Mr. Rojas, who is now working
as a construction laborer, smiled and began thinking about how to get
his orange jacket back. He wants to wear it again, a final symbol of
vindication.
</p>
<p class="articleParagraph enarticleParagraph" >
''It's my jacket,'' he said plainly. ''I didn't commit any crime in
it.''
</p>
<p class="articleParagraph enarticleParagraph" >
That night, Mrs. Chenoweth took the train home to New Jersey, by
herself, had dinner with her daughter and started fielding telephone
calls. People have been asking her now to get involved in another case
of someone they say was wrongly accused.
</p>
<p class="articleParagraph enarticleParagraph" >
But Mrs. Chenoweth has a heart condition, asthma and a family that
believes she cannot handle the strain of another crusade.
</p>
<p class="articleParagraph enarticleParagraph" >
''I have taken a blood oath,'' Mrs. Chenoweth said, with obvious
reluctance, ''that I will not get involved in any more.''
</p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>ART</b>&nbsp;</td><td><br/>
Photo: Priscilla Read Chenoweth, center, Luis Rojas, right, and his
lawyer, Jethro M. Eisenstein, with a jacket like the one worn by the
shooter. (Richard L. Harbus for The New York Times)(pg. B10)
</td></tr>
<tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>ncat : Content Types | npag : Page-One Story</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>namz : North America | usa : United States | use : Northeast U.S. | usnj : New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document nytf000020010918duan01m46</td></tr></table><br/></div></div><br/><span></span><div id="article-nytf000020010918du5300nx9" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>CLM</b>&nbsp;</td><td>
F.Y.I.
</td></tr>
<tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>
The City Weekly Desk; Section 14
</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>
F.Y.I.
</span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>
By DANIEL B. SCHNEIDER
</td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>784 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>3 May 1998</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td> The New York Times </td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td> Page 2, Column 5 </td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>(c) 1998 New York Times Company</td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >
The Smelly PATH Tubes
</p>
<p class="articleParagraph enarticleParagraph" >
Q.  I've noticed when I ride the PATH train to New Jersey that the
tunnels have a peculiar, acrid odor, quite unlike anything I've smelled
in the New York City subways. Why?
</p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >
A.  The PATH tubes do tend to smell a little like the Daytona
Speedway, but it has nothing to do with the old tunnels themselves,
buried 60 feet beneath the Hudson. It's the trains. Bill Cahill, a Port
Authority spokesman, identified the source of the odor with two words:
''Brake shoes.''
</p>
<p class="articleParagraph enarticleParagraph" >
PATH trains are not the same as subways. The Port Authority
Trans-Hudson Corporation took control of the venerable Hudson tubes in
1962, eight years after the Hudson and  Manhattan Railroad went
bankrupt, and introduced its new, advanced cars in 1965.
</p>
<p class="articleParagraph enarticleParagraph" >
At speeds over 10 miles an hour, the traction motors on the PATH cars
are used to slow the train ''in a manner similar to downshifting in a
car,'' Mr. Cahill said. Below 10 miles an hour, the train's pneumatic
brakes are applied, pressing the rubber-lined brake shoes against the
cast-steel wheels and ultimately stopping them. The heat created by this
friction, however, burns a tiny bit of brake shoe rubber into the tunnel
air. New cars with similar specifications have been introduced, and
older cars refurbished, but the stinky brakes remain standard equipment
on all 350-odd cars. Subway brake shoes are made not of rubber but of
what a spokesman called ''an organic mixture.''
</p>
<p class="articleParagraph enarticleParagraph" >
The PATH trains, you may have noticed, do a fair amount of stopping
and slowing down below 10 miles an hour. The tunnels were dug piecemeal
by various companies from 1874 to 1908, and some sections meander
strangely. There are even a few S-curves left from a unfinished 1902
trolley line. The scorched-rubber fragrance that lingers in the tunnels,
though not exactly Proust's madeleine, is quite distinctive.
</p>
<p class="articleParagraph enarticleParagraph" >
800 Years of Dissent
</p>
<p class="articleParagraph enarticleParagraph" >
Q.  While walking on the Upper East Side several years ago, I was
surprised to find a Waldensian Church. The Waldensians were 12th-century
dissenters from the Roman Catholic Church who were persecuted for
centuries, and I thought the only surviving parishes were in the Italian
Alps. When I returned recently, the church had become a synagogue. What
became of the Waldensians?
</p>
<p class="articleParagraph enarticleParagraph" >
A.  Four years ago the few remaining Waldensians in the region
decided to dissolve the parish and sell the small, nondescript church on
East 82d Street. By then there were probably only 40 or 50 Waldensians
in the city, said the Rev. Gabriella Lettini, a Waldensian pastor
ordained in Italy and a doctoral candidate at Union Theological
Seminary. Most were elderly, she said, and often no more than 15 went to
Sunday services.
</p>
<p class="articleParagraph enarticleParagraph" >
Waldensians take their name from a 12th-century figure usually called
Valdesius or Valdes or Waldo, a merchant from Lyons, France, who
renounced his worldly goods to pursue a life of evangelical perfection.
In matters of faith and doctrine, the fundamental principle of the
Waldensian Church is the absolute authority of the Bible. While this was
not heretical by itself, the Waldensians incurred the wrath of the Roman
Catholic Church by dismissing ecclesiastical power and translating the
Bible into the vernacular of the day, allowing lay people -- including
women -- to read and preach it, centuries before the Protestant
Reformation.
</p>
<p class="articleParagraph enarticleParagraph" >
A Waldensian congregation was established in the city in the 1890's,
and there were at least two at the turn of the century. One met in a
Presbyterian church in Greenwich Village, the other in a Reformed church
in midtown, Ms. Lettini said, and worshipers numbered in the hundreds.
In 1951, as their numbers dwindled, the two parishes unified and bought
the building at 127 East 82d Street. Members of Congregation Or Zarua,
who bought it in 1994, were astounded to learn that the building was
originally a synagogue, built in 1883 for Kehilath Jeshurun.
</p>
<p class="articleParagraph enarticleParagraph" >
Free From the Circus
</p>
<p class="articleParagraph enarticleParagraph" >
Q.  My gardening friends laughed when I told them that the circus
gives away bags of elephant manure at the end of their spring engagement
in Madison Square Garden. When did this tradition end?
</p>
<p class="articleParagraph enarticleParagraph" >
A.  It didn't. Ringling Brothers and Barnum & Bailey Circus has
offered elephant manure to farmers, gardeners and horticultural
organizations as long as anyone can remember, said Alan Miller, a circus
spokesman. Though the dung has been gratefully received before, it
elicited scant interest in this year's Garden run, Mr. Miller said, and
no giveaway occurred. Interested groups need only contact the circus
before it rolls into town again in March.   DANIEL B. SCHNEIDER
</p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>CO</b>&nbsp;</td><td><br/>fyius : SourceHOV</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>ibcs : Business/Consumer Services</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>m11 : Equity Markets | mcat : Commodity/Financial Market News | nrmf : Routine Market/Financial News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>namz : North America | usa : United States | uss : Southern U.S. | ustx : Texas</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document nytf000020010918du5300nx9</td></tr></table><br/></div></div><br/><span></span><div id="article-nytf000020010918du13000e3" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>
Metropolitan Desk; Section A
</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>
Water Main Ruptures, Creating A Huge Sinkhole on Fifth Avenue
</span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>
By ROBERT D. McFADDEN
</td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>1705 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>3 January 1998</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td> The New York Times </td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td> Page 1, Column 3 </td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>(c) 1998 New York Times Company</td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >
A century-old water main ruptured under lower Fifth Avenue early
yesterday, creating a car-swallowing, curb-to-curb sinkhole and watery
chaos in a bustling neighborhood whose streets resembled Venice for a
few hours. Then, as the rivers receded, a gas main broke and the crater
spewed forth a tower of orange flames.
</p>
<p class="articleParagraph enarticleParagraph" >
No one was injured in the spectacular flood-and-fire day centered at
Fifth Avenue near 19th Street. But water damaged scores of lobbies,
storefronts and basements for blocks around, 40 residents were
evacuated, hundreds of offices and businesses were closed, subways were
halted, traffic was rerouted and gas, water, electric, steam heat and
telephone services were disrupted for many.
</p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >
By last night, the broken water and gas mains had been capped, the
fire had been put out, basements were being pumped out and city
officials were gloomily assessing a tangle of damaged water, gas, sewer
and electrical lines. Most of the utilities were expected to be restored
over the weekend, but officials said Fifth Avenue between 19th and 21st
Streets might be closed for a week.
</p>
<p class="articleParagraph enarticleParagraph" >
''This one's a beaut,'' Commissioner Joel A. Miele of the city's
Department of Environmental Protection said as he surveyed a gigantic
crater that might have been the scene of a bombing or an earthquake.
''This one's as good as I have ever seen, or heard of.''
</p>
<p class="articleParagraph enarticleParagraph" >
As residents, shop owners and office workers stood behind police
barriers on soggy streets and gawked at flames that shot 20 feet in the
air for two hours in the late morning, officials acknowledged that a day
of excitement and some danger had left a residue of frustration for
those who live and work in the area, and that the costs of repairs,
cleanup and lost business might be heavy.
</p>
<p class="articleParagraph enarticleParagraph" >
An owner of Moe Ginsburg, at 162 Fifth Avenue, two blocks from the
break, one of the largest of the many clothing dealers in the area,
estimated lost sales yesterday at $100,000. At Daffy's, a women's
clothing and shoe store at Fifth Avenue and 16th Street that does
$35,000 worth of business a day, water flooded a stock room filled with
spring merchandise.
</p>
<p class="articleParagraph enarticleParagraph" >
Siobhan York, a sales manager at a Nine West store at Fifth Avenue
and 19th Street, said $2 million in women's shoes, leather bags and
other inventory was trapped in her store. ''It's going to kill my
business,'' she said of the prospect of a week's closing.
</p>
<p class="articleParagraph enarticleParagraph" >
While acknowledging financial and other hardships, city officials
noted that it could have been worse. No one was killed or injured, and
officials said the impact of the twin breaks on commuters, traffic and
business generally had been minimized by fortuitous timing, since many
offices and some businesses had closed for a long holiday weekend on the
Friday after New Year's.
</p>
<p class="articleParagraph enarticleParagraph" >
In an age of supersonic speeds and instant global everything, the
events were also a reminder that a basic element of urban life -- a
water system that most New Yorkers take for granted -- is still a
creature of the past, laid down in horse-and-buggy days before anyone
imagined punishing traffic vibrations and 1.5 billion gallons of water
surging daily through 6,000 miles of mains.
</p>
<p class="articleParagraph enarticleParagraph" >
The one that broke yesterday was new in 1897, a 48-inch cast-iron
pipe that carries 40 million gallons of water a day down Fifth Avenue.
Evidently tortured by decades of rumbling traffic and the stress of
temperature extremes in recent days, a chunk about five feet long
ruptured under Fifth Avenue just north of 19th Street around 3 A.M.
</p>
<p class="articleParagraph enarticleParagraph" >
Over the next six hours, millions of gallons of water surged out. It
turned Fifth Avenue and adjacent 18th, 19th and 20th Streets into lakes
a foot or two deep. It flooded scores of lobbies, storefronts and
basements in a predominantly business district dotted with residences.
It rolled east to Broadway and west to Sixth and Seventh Avenues and,
following Manhattan's south-sloping topography, it ran in parallel
rivers a half-mile down into Greenwich Village.
</p>
<p class="articleParagraph enarticleParagraph" >
The waters also flooded nearby subway tunnels and the Port Authority
Trans-Hudson tube that runs from New Jersey to 33d Street, turning the
morning rush into a nightmare for tens of thousands of commuters. The 1,
2, and 3 subway lines in the area were closed from 4:40 A.M. to 9:50
A.M., and F trains were down until 7:35 A.M. PATH service was not
restored until the evening rush.
</p>
<p class="articleParagraph enarticleParagraph" >
Meantime, a swath of Fifth Avenue's asphalt surface had collapsed
into a jagged crater between 19th and 20th Streets. The hole extended 30
to 40 feet up the avenue, from sidewalk to sidewalk, and was 15 feet
deep in spots. At some point, a car from New Jersey that had been parked
on the avenue's east side slid in like a child's toy in a sandbox.
Swallowed with it were parking meters, street signs and chunks of
sidewalk.
</p>
<p class="articleParagraph enarticleParagraph" >
Throughout the morning, as residents were evacuated and barriers were
erected to keep thousands of office and store workers and their
customers out of the area, the police closed off 10 blocks from 17th to
22d Streets between Broadway and Sixth Avenue. Buses and other traffic
were rerouted around the trouble.
</p>
<p class="articleParagraph enarticleParagraph" >
As city environmental and utility officials swarmed over the cratered
scene, crews managed to shut off water leading to the ruptured main
about 8 A.M., but water to many buildings in the area was kept running
over alternative routes on a complex grid that includes three smaller
lines on Fifth Avenue.
</p>
<p class="articleParagraph enarticleParagraph" >
But about 9:45 A.M., as firefighters prepared to raise chunks of
asphalt to let city workers get at the broken water main, the ground
nearby -- much of it sand undermined by the water -- shifted and cracked
open a 3-inch pipe that carries natural gas from a 24-inch main under
the avenue into an office and residential building at 125 Fifth Avenue.
</p>
<p class="articleParagraph enarticleParagraph" >
Gas began flowing upward and some spark -- no one knows exactly --
ignited it into a whooshing tower of flame. Workers in the area, at
first fearing a larger explosion, quickly retreated, and witnesses told
of a sense of peril.
</p>
<p class="articleParagraph enarticleParagraph" >
''We saw these Con Edison workers running away from the flames,''
said Beth Stover-Garcia, who with her husband, Jason, had been evacuated
from their home at 125 Fifth Avenue and were watching from a block away.
She said the flames reached above lampposts as high as the second
stories of nearby buildings.
</p>
<p class="articleParagraph enarticleParagraph" >
About two hours later, firefighters and Con Edison workers drilled
into the gas main, inserted a balloonlike rubber and leather diaphragm
and cut off the gas supply. The flames slowly died, and were out
completely about noon. But gas was shut off to more than 30 buildings
along the avenue and on 19th and 20th Streets.
</p>
<p class="articleParagraph enarticleParagraph" >
Electricity, steam for heating office buildings and telephone service
were also shut off throughout much of the area, officials said. While
the utilities were expected to be restored over the weekend, the Mayor's
Office of Emergency Management said it might take a week to repair
extensive damage to underground water, sewer, gas and electric lines and
to refill the crater and repair and repave Fifth Avenue.
</p>
<p class="articleParagraph enarticleParagraph" >
Late last night, working under the glare of generator-powered
spotlights, crews finished capping off a gas main so they could retrieve
the car that had slid into the hole. One worker, attached to a safety
rope, placed two straps around the bottom of the car, connecting it to
cables from a six-story-high crane. The crane then hoisted the beige
Buick to safer ground.
</p>
<p class="articleParagraph enarticleParagraph" >
While scores of people were evacuated from apartments on Fifth Avenue
and in the adjacent side streets, only 40 residents of Fifth Avenue
buildings were not allowed to return yesterday, the Red Cross said. Some
went to stay with friends or relatives, but 34 registered with the Red
Cross for emergency housing and some were put up in hotels. It was
unclear when all would be home again.
</p>
<p class="articleParagraph enarticleParagraph" >
Some were let back yesterday, but only to get clothing, pets and
other possessions. ''We're trying to accommodate people where we can,''
said Jerome M. Hauer, the director of the Mayor's Office of Emergency
Management. But for those on Fifth Avenue, a quick return was out of the
question. ''Clearly, people won't be allowed into buildings on Fifth
Avenue,'' he said. ''There is no water, no gas and a large amount of
debris in the lobbies.''
</p>
<p class="articleParagraph enarticleParagraph" >
Some basements, including one at 125 Fifth Avenue, the building
closest to the broken mains, had water up to their ceilings. The
building superintendent, Philip Jean-Jacques, said he had looked in some
windows and had seen mud all over the first floor.
</p>
<p class="articleParagraph enarticleParagraph" >
As firefighters pumped out flooded basements in the area, Mr. Hauer
said late yesterday that engineers had inspected the most heavily
flooded buildings and determined that none had sustained any structural
damage.
</p>
<p class="articleParagraph enarticleParagraph" >
Asked about a series of structural problems that had befallen
buildings in midtown Manhattan in recent weeks, including a collapse on
42d Street and bricks tumbling from a facade on Madison Avenue, Mr.
Hauer said he saw no pattern. ''If you look at the individual
situations, there is a fairly good explanation for each one,'' he said.
</p>
<p class="articleParagraph enarticleParagraph" >
As for Fifth Avenue, the traffic barriers were up in the affected
area last night, at least until today. Officials reopened 21st Street to
crosstown traffic late last night.
</p>
<p class="articleParagraph enarticleParagraph" >
''Barring any unforeseen circumstances, it will be six or seven days
before it will be fully reopened,'' Bruce Brodhoff, a spokesman for the
emergency management office, said.
</p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>ART</b>&nbsp;</td><td><br/>
Photo: Fire erupted yesterday on Fifth Avenue, between 19th and 20th
Streets, when a gas line broke, caused by shifting asphalt after a water
main burst. (Philip Greenberg for The New York Times)(pg. A1)
Map of Manhattan showing location of the water main break and fire:
Officials estimate that Fifth Avenue will be closed for a week. (pg. B6)
</td></tr>
<tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>ncat : Content Types | npag : Page-One Story</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document nytf000020010918du13000e3</td></tr></table><br/></div></div><br/><span></span><div id="article-nytf000020011014dsbp01o9v" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>
Section A
</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>
Broken Bars on Drawbridge Are Blamed in Amtrak Crash
</span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>
By ROBERT D. McFADDEN
</td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>2178 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>25 November 1996</td></tr><tr><td align="right" valign="top" class="index"><b>ET</b>&nbsp;</td><td>03:50 AM</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td> The New York Times </td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td> Late Edition - Final </td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td> Page 1, Column 1 </td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>
c. 1996 New York Times Company
</td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >
Federal investigators said last night that an Amtrak train derailed on
a swing bridge in New Jersey on Saturday because a rupture of two small
plates left two connecting rails piggybacked, one atop another, instead
of end to end.
</p>
<p class="articleParagraph enarticleParagraph" >
The preliminary conclusions raised the possibility that across the
nation, more than 100 railroad swing bridges -- those that swivel open
at river crossings to allow boat traffic to pass -- might have problems
similar to those that led to the accident.
</p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >
In a dramatic news conference at the Gateway Hilton in Newark,
officials of the National Transportation Safety Board indicated they had
found crucial evidence on the second day of their investigation that
appeared to explain not only the cause of the derailment, but also why
there had been no red signals that might have halted the train and
prevented the wreck.
</p>
<p class="articleParagraph enarticleParagraph" >
They said that while the displaced rails resembled a small upward
ramp, with the end of one five inches above the other, the rails were
still touching so that electrical contact was not broken and a red
signal that would have halted the oncoming train was never posted.
</p>
<p class="articleParagraph enarticleParagraph" >
The train, rushing toward New York at 60 miles an hour with 108
people aboard, sped onto the Hackensack River bridge and, as its wheels
went up the tiny ramp, flew into the air out of control. It continued
across the 100-foot span, its wheels digging ruts in the railroad ties.
</p>
<p class="articleParagraph enarticleParagraph" >
Beyond the bridge, the train's twin locomotives and four cars
carrying scores of passengers plunged down a 30-foot embankment, coming
to rest in marshland.
</p>
<p class="articleParagraph enarticleParagraph" >
In disclosing the possibility of similar conditions elsewhere,
transportation officials said they would meet in Washington today to
dispatch inspection crews to the more than 100 swing bridges nationwide
in an effort to learn the dimensions of the problem.
</p>
<p class="articleParagraph enarticleParagraph" >
''We want to see if the cracking of sidebars is unique to this type
of bridge, or if it's just normal wear and tear,'' said Patrick
Sullivan, the investigator in charge of the accident for the safety
board. ''We are going to start inspecting all the other bridges around
here and see if we come up with any other problems.''
</p>
<p class="articleParagraph enarticleParagraph" >
In the Northeast, in addition to Amtrak's lines, that type of swing
bridge is found on lines run by New Jersey Transit and the Massachusetts
Bay Transportation Authority, Boston's transit agency.
</p>
<p class="articleParagraph enarticleParagraph" >
Investigators said that an examination of the Portal Bridge, where
the derailment occurred, showed that the two so-called sidebars, which
hold the rails together end to end, were broken. They said that at least
3 of the remaining 14 sidebars on the bridge were cracked in a manner
similar to the broken ones.
</p>
<p class="articleParagraph enarticleParagraph" >
It was unclear last night whether the other cracked plates were
replaced as part of the weekend operations to remove wreckage and repair
hundreds of feet of mangled trackage. But Amtrak and New Jersey Transit
officials said that normal schedules would resume today. On a normal
weekday, about 300 trains cross the bridge, which is the main line of
Amtrak's Northeast Corridor and a main route of New Jersey Transit
commuter trains.
</p>
<p class="articleParagraph enarticleParagraph" >
Mr. Sullivan and Mark Garcia, the safety board's eastern regional
director, said at the news conference that the sequence of events that
led to the wreck apparently began about 4 A.M. Saturday, when the bridge
operator opened the bridge to let a tug and barge pass.
</p>
<p class="articleParagraph enarticleParagraph" >
The operator reported a problem opening the bridge. At either end of
the bridge are miter rails, with tapered ends that mate with other
tapered rails at a point where the bridge and land tracks meet. The
miter rail is 10 feet long, and is fixed atop the bridge to ordinary
rails known as running rails.
</p>
<p class="articleParagraph enarticleParagraph" >
The sidebar plates connect the miter rails to the running rails, and
are bolted to them on either side. It was a pair of these sidebars on
the left rail of the northbound tracks that broke, leading to the
derailment, Mr. Sullivan and Mr. Garcia said.
</p>
<p class="articleParagraph enarticleParagraph" >
To turn the bridge, the entire rotating structure, including its
rails, must be lifted 15 inches so that the miter rails may clear the
landside rails, Mr. Garcia noted. ''It appears now that the sidebars
broke when the bridge operator raised the bridge,'' Mr. Garcia said.
After the tug and barge passed and the operator swung the bridge back
into place, the now misaligned miter rail was lowered onto the running
rail, he said, leaving the jagged piggyback instead of the smooth
end-to-end configuration.
</p>
<p class="articleParagraph enarticleParagraph" >
''The heel of the miter rail, which was supposed to be level with the
running rail, was displaced by approximately five inches,'' Mr. Sullivan
said. The sidebars, he noted, are steel, one and three-quarters of an
inch thick, and held to the rails they connect by bolts.
</p>
<p class="articleParagraph enarticleParagraph" >
But the passage of about 300 trains a day over the bridge, and the
bridge's daily movements, have inevitably placed great stress on the
plates since they were installed with the miter rails in 1992, Mr.
Garcia said. He and Mr. Sullivan said that metallurgists would be called
in to analyze all the sidebars on the bridge.
</p>
<p class="articleParagraph enarticleParagraph" >
Compounding the problem, they said, was the fact that the rails were
indeed in contact with one another, though not properly aligned, so that
sensors that would have detected the problem and put up red signals to
stop the onrushing train were not activated.
</p>
<p class="articleParagraph enarticleParagraph" >
Mr. Garcia said the sidebars on the Portal Bridge were visually
inspected twice a week, ''but you might not be able to see this kind of
crack until you raise the bridge.''
</p>
<p class="articleParagraph enarticleParagraph" >
Mr. Sullivan said that investigators had estimated the cost of the
wreck at about $3 million -- $1.6 million in damage to the twin
locomotives and $1.37 million in damage to the 12 following cars.
</p>
<p class="articleParagraph enarticleParagraph" >
The site of the wreck in Secaucus, a sea of marshes and saw grass
five miles west of Manhattan, was a hive of activity yesterday as the
investigators combed the bridge and tracks for clues, and work crews
with huge cranes and other heavy equipment cleared wreckage, replaced
mangled rails and got Amtrak and New Jersey Transit trains running over
the bridge again.
</p>
<p class="articleParagraph enarticleParagraph" >
While only one of the two sets of tracks over the Portal Bridge were
open, the crews were expected to work all night to clear and repair the
second set, and Amtrak and New Jersey Transit officials predicted near
normal service by this morning's rush hour.
</p>
<p class="articleParagraph enarticleParagraph" >
''We expect tracks to be up in service and everybody to be humming
along Monday morning,'' said Isabel Kaldenback, an Amtrak spokeswoman.
On a normal weekday, Amtrak runs 93 trains through New York in the
Northeast Corridor, and New Jersey Transit runs 203 trains into and out
of New York.
</p>
<p class="articleParagraph enarticleParagraph" >
Only two of the 34 injured people remained hospitalized. Both were
described as Amtrak employees and Philadelphia residents who had
suffered back and neck injuries and were said to be resting comfortably.
They were identified as Ronald Galloway, 42, an electrician, who was at
Christ Hospital in Jersey City, and Robert Cuff, 35, an engineer, at the
Jersey City Medical Center.
</p>
<p class="articleParagraph enarticleParagraph" >
Amtrak's Fast Mail Train No. 12 -- a dozen passenger and mail coaches
pulled by two locomotives on a Washington-to-Boston run with 88
passengers and 20 crew members -- derailed as it reached the bridge at
6:33 A.M. Saturday. Sideswiping an oncoming passenger train, the mail
train continued across the bridge, kept from plunging through the
trestles into the river by guide rails that parallel the main tracks.
Then its twin locomotives, a baggage car and three passenger coaches
plunged over an embankment, coming to rest in mud and marshes.
</p>
<p class="articleParagraph enarticleParagraph" >
Yesterday, investigators examined hundreds of feet of track as well
as the bridge's swing mechanism, signals and sensors. They also
interviewed the crews of both trains, the bridge operator and a signal
maintainer and electrician who were at the scene for a repair job at the
time of the derailment. And they examined Amtrak maintenance records.
</p>
<p class="articleParagraph enarticleParagraph" >
Investigators had the benefit of numerous witnesses and an abundance
of physical evidence in an inquiry that had only just begun.
</p>
<p class="articleParagraph enarticleParagraph" >
Senator Frank R. Lautenberg, a New Jersey Democrat who went to the
scene on Saturday, said in an interview yesterday that he saw what he
took to be a ''broken rail'' on the southwest end of the bridge, where
the train apparently jumped the tracks.
</p>
<p class="articleParagraph enarticleParagraph" >
''What I saw wasn't at the miter joint,'' the Senator said. ''It was
some footage away. It had a crack, a fracture.''
</p>
<p class="articleParagraph enarticleParagraph" >
The Senator, who said he did not want to second-guess the
investigators by trying to interpret what he saw, had said at the scene
on Saturday that miter rails on the bridge might not have been fully
closed after the tugboat and barge passed through two and a half hours
before the derailment.
</p>
<p class="articleParagraph enarticleParagraph" >
Transportation safety board investigators also said on Saturday that
there had been a problem with the bridge mechanism. They quoted the
bridge operator, who was not identified, as saying he had trouble
opening the bridge for the tug and barge, but believed that the
tight-fitting rails had closed properly.
</p>
<p class="articleParagraph enarticleParagraph" >
David A. Bolger, a spokesman for the Federal Railroad Administration,
said that after the tug went through at 4 A.M., the bridge operator
called a signal maintainer at 4:40 A.M. It was not immediately clear
why. The maintainer arrived between 5:15 A.M. and 5:30 A.M. and
inspected the track and signals. He was there, along with an
electrician, when the derailment occurred.
</p>
<p class="articleParagraph enarticleParagraph" >
While drawing preliminary conclusions, the investigators were still
examining a host of possibilities, including the condition of the rails,
the sensors and the signals, and whether the mechanisms and twin engines
that open and close the bridge were working properly.
</p>
<p class="articleParagraph enarticleParagraph" >
The bridge operator, the engineers and other railroad personnel were
interviewed, investigators said, and blood samples were being taken from
the crews and other workers for drug and alcohol tests. The results of
the tests were not expected before the end of the week.
</p>
<p class="articleParagraph enarticleParagraph" >
As the inquiry got under way, 150 Amtrak gandy dancers and crew
members of private contractors specializing in train wrecks were on the
scene yesterday. Using a huge white crane and powerful yellow trucks
with forklifts, they raised from the embankment a 90-foot passenger
coach, its side gashed and its rear end blocking the tracks.
</p>
<p class="articleParagraph enarticleParagraph" >
The crews also removed and replaced hundreds of feet of torn-up
tracks on the northbound line, and worked to repair the overhead
electric lines that supply power to locomotives.
</p>
<p class="articleParagraph enarticleParagraph" >
By 9:35 A.M., the set of tracks usually used for southbound trains
was open again and the first train to cross the bridge since the
accident, an Amtrak Metroliner bound for Washington, slowly passed.
Amtrak then began operating on normal Sunday schedules with 65 trains,
Amtrak officials said.
</p>
<p class="articleParagraph enarticleParagraph" >
New Jersey Transit, which normally runs 116 trains on Sunday, also
resumed a near normal weekend schedule, except for Midtown Direct trains
from Dover to New York, which were traveling only as far as Hoboken,
where passengers could take Port Authority Trans-Hudson trains or
ferries into New York.
</p>
<p class="articleParagraph enarticleParagraph" >
Delays of up to 45 minutes were reported on trains running through
the derailment site yesterday. While that was far better than Saturday's
chaos for travelers, frustrations were apparent again in the crowds at
stations in Newark and New York, and there were no shortages of tales of
confusion and plans gone awry.
</p>
<p class="articleParagraph enarticleParagraph" >
''They keep on saying it will be five minutes, then another five
minutes, then another five minutes,'' Carlo Faria, a pianist trying to
get to a wedding at Lawrenceville, N.J., said at Penn Station in Newark,
where occasional clues blasted from a garbled loudspeaker. His train was
45 minutes late.
</p>
<p class="articleParagraph enarticleParagraph" >
At Penn Station in New York, John Pae waited anxiously for 16 friends
from New Jersey who were coming in for lunch and the Christmas show at
Radio City Music Hall. But their train was only 17 minutes late, with
time to spare for their lunch reservation. ''We saw the wreck,'' Charles
J. Coniglio said. ''We passed by it. It was an awful disaster.''
</p>
<p class="articleParagraph enarticleParagraph" >
Even as Amtrak struggled to recover from the derailment, another
Amtrak train -- a 15-car charter carrying New York Jets fans to Buffalo
-- struck an abandoned car on the tracks near Albany in the predawn
darkness yesterday. No one was injured, but the prow of the locomotive
dented and the fans had to endure a 25-minute delay, in addition to
their team's loss to the Bills.
</p>
<p class="articleParagraph enarticleParagraph" ><pre class="articlePre" >
Diagram: ''THE BRIDGE: A Faulty Connection''
</pre>
</p>
<p class="articleParagraph enarticleParagraph" >
(MORE)
</p>
<p class="articleParagraph enarticleParagraph" >
03:50 EST November 25, 1996
</p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>ART</b>&nbsp;</td><td><br/>
Photo: A car from the Amtrak train that derailed in the marshes of
Secaucus, N.J., on Saturday was lifted by a crane from an embankment
yesterday. (Associated Press) (pg. B2)
</td></tr>
<tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i71 : Railroads | itsp : Transportation/Shipping</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>c13 : Regulation/Government Policy | ccat : Corporate/Industrial News | gcat : Political/General News | ncat : Content Types | npag : Page-One Story</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>namz : North America | usa : United States | usdc : Washington DC | use : Northeast U.S. | usnj : New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document nytf000020011014dsbp01o9v</td></tr></table><br/></div></div><br/><span></span><div id="article-nytf000020011014dsbo01nxp" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>
Section 1; Metropolitan Desk
</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>
Amtrak Derailment in Secaucus Marsh Injures 32
</span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>
By ROBERT D. McFADDEN
</td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>2055 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>24 November 1996</td></tr><tr><td align="right" valign="top" class="index"><b>ET</b>&nbsp;</td><td>05:30 AM</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td> The New York Times </td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td> Late Edition - Final </td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td> Page 1, Column 4 </td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>
c. 1996 New York Times Company
</td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >
An Amtrak fast-mail train approaching New York with 108 passengers and
crew members derailed as it crossed a Hackensack River swing bridge on a
desolate New Jersey meadowland early yesterday. After sideswiping an
oncoming passenger train, six cars of the mail train plunged down a
30-foot embankment.
</p>
<p class="articleParagraph enarticleParagraph" >
No one was killed, but Amtrak said 32 people were injured and dozens
were shaken and terrified as the train -- a dozen passenger and mail
cars behind twin locomotives -- was wrecked at 6:33 A.M. at the Portal
Bridge in the marshes of Secaucus. The site is a mile from the junction
where two New Jersey Transit commuter trains collided last February,
killing 3 people and injuring 162.
</p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >
While the accident was still under investigation, United States
Senator Frank R. Lautenberg, a New Jersey Democrat on the Senate
Transportation subcommittee who spoke to officials at the scene,
indicated that mitered rails on the bridge might not have been closed
completely after a tugboat with a barge passed through two and a half
hours earlier.
</p>
<p class="articleParagraph enarticleParagraph" >
Last night, a spokesman for the National Transportation Safety Board,
Peter Goelz, said that the cause had not yet been determined, but he
agreed with the scenario outlined by the Senator. ''There was apparently
a report of some problem with the bridge about 4 A.M,'' he said. ''A
tugboat radioed the bridge and there were some problems, and the bridge
did not open.''
</p>
<p class="articleParagraph enarticleParagraph" >
Other investigators last night also quoted the bridge operator, who
was not named, as saying he had been unable to fully open the bridge for
the tug and barge at 4 A.M., but that he believed afterward that he had
fully closed the tight-fitting rails.
</p>
<p class="articleParagraph enarticleParagraph" >
If the rails are not closed, a sensor is supposed to automatically
give approaching trains a red light. There were no reports that the
derailed train had passed a red signal, but investigators said they were
examining numerous possibilities, including the conditions of the
bridge's swing mechanism, the sensors and signals, and the accounts of
the bridge operator, the train's engineers and other personnel and of
passengers and other witnesses.
</p>
<p class="articleParagraph enarticleParagraph" >
The train jumped the tracks just as it reached the bridge, but was
prevented from plunging through the trestle into the river by guard
rails installed after a tragedy in 1958 in which a train plunged from a
bridge into Newark Bay, killing 48 people.
</p>
<p class="articleParagraph enarticleParagraph" >
Deeply rutted wooden ties on the bridge yesterday showed where the
wheels had cut in. In any case, safety officials said the guard rails
may have saved many lives by keeping the train upright until it crossed
the bridge, allowing it to roll down an embankment beyond into the
softer ground of the marshes.
</p>
<p class="articleParagraph enarticleParagraph" >
Mr. Goelz said that event recorders had been recovered on one of the
engines as well as on the bridge and its signal. A fourth recorder is
located in an engine still stuck in the mud. Mr. Goelz added that
N.T.S.B. investigators would study the crash scene and meet again today.
</p>
<p class="articleParagraph enarticleParagraph" >
Yesterday's spectacular crash, which left both locomotives nose down
in mud, and a baggage car and three passenger cars derailed, atilt or
crazily zigzagged on torn-up trackage five miles southwest of Manhattan,
blocked Amtrak's main line in the Northeast Corridor, as well as some of
New Jersey Transit's most heavily traveled routes.
</p>
<p class="articleParagraph enarticleParagraph" >
Some Metroliner trains were canceled and thousands of passengers
between Washington and Boston were routed around the wreck throughout
the day by Amtrak-chartered buses that shuttled between the Penn
Stations in New York and Newark. Several New Jersey Transit rail lines
were also shut down. Officials said they hoped to have one of the two
tracks open early today and the other later today, in time for the
Monday morning rush hour.
</p>
<p class="articleParagraph enarticleParagraph" >
Only minutes after a scheduled stop at Newark's Penn Station,
passengers said, Amtrak's Fast Mail, Train No. 12, traveling at about 60
miles an hour, reached the 100-foot trestle that swings open for river
traffic and suddenly began shaking from side to side in a motion some
described as like an earthquake.
</p>
<p class="articleParagraph enarticleParagraph" >
Then, witnesses said, the train lights went out, and there was a
series of bumps and scrapes as the left side of the mail train
sideswiped the rear cars of Amtrak's Carolinian, bound for Raleigh, N.C.
Though off its rails, the mail train proceeded across the bridge, its
wheels cutting into the ties but guided by the guard rails, which are
set on the ties outside the regular rails.
</p>
<p class="articleParagraph enarticleParagraph" >
After the train crossed the swinging-trestle portion of the bridge,
the guard rails kept it upright for another 100 feet as it moved over
the concrete portion of the bridge. The guard rails then gave out, and
the two locomotives, a baggage-mail car and three passenger coaches
plunged to the right down the embankment. In the darkened, hurtling
cars, screams and shouts erupted as passengers, some of them half
asleep, were hurled about in scenes of surrealistic chaos. As the train
halted with both locomotives partially buried in mud and water and a
baggage car and three passenger cars off the tracks, some passengers,
frightened but not badly injured, made their way off the wreck and found
themselves in a lonely marsh.  Though Manhattan's skyline was visible in
the dawning light, they found themselves in a strangely remote place, a
marsh of ponds, streams, saw grass and tufted reeds, a bleak landscape
bleached by autumn to the color of straw. In the distance one could see
truck terminals, warehouses and the elevated New Jersey Turnpike.
</p>
<p class="articleParagraph enarticleParagraph" >
Kenneth Waiters, 43, an Amtrak maintenance foreman on disability
leave, who was on his way to Foxwoods Casino in Connecticut with his
wife, Norma, 39, suffered wrist and ankle sprains and a back injury. But
he and his wife crawled out through an opening in their car where the
front end had been torn open.
</p>
<p class="articleParagraph enarticleParagraph" >
They then waded through marsh without shoes -- they had taken them
off to sleep in the coach -- and waited more than a half hour, they
said, shivering in the early morning cold, before help came. That help
had been summoned by a New Jersey Transit employee who saw the crash
from his signal shack at a nearby trestle bridge. The signalman called
his dispatcher, who notified Amtrak.
</p>
<p class="articleParagraph enarticleParagraph" >
Secaucus firefighters, the first emergency workers on the scene,
found dozens of people standing or sitting on a gravel road nearby, and
a jumble of Amtrak coaches still occupied by dozens of people, whom they
carried or led out to buses and ambulances from nearby communities.
</p>
<p class="articleParagraph enarticleParagraph" >
Thirty-two people -- 16 passengers and 16 Amtrak employees -- were
taken to area hospitals. By day's end, however, only two, both off-duty
Amtrak employees, remained hospitalized. Engineer Robert Cuff, 35, and
Ronald Galloway, 42, both of Philadelphia, were being held for
observation of minor back and neck injuries and were expected to be
released today. Mr. Cuff was being held at Jersey City Medical Center,
where he was listed in stable condition. Mr. Galloway was listed in
stable condition at Christ Hospital in Jersey City.
</p>
<p class="articleParagraph enarticleParagraph" >
The other passengers and crew members were taken to a center for the
elderly at the Municipal Building in Secaucus, where they were given
food and minor medical attention. They were later transferred by bus to
Penn Station in New York.
</p>
<p class="articleParagraph enarticleParagraph" >
Despite Senator Lautenberg's account, Amtrak officials said the cause
of the derailment was unclear and under investigation, according to Rick
Remington, a spokesman. He noted that the train had crossed the bridge
at the bridge's 60-mile-an-hour speed limit and said there had been no
early indication that anything was wrong with the bridge mechanism,
though he noted that a problem had been reported on Friday night.
</p>
<p class="articleParagraph enarticleParagraph" >
''There was an electrical problem with the bridge last night that was
looked at, and we don't see any connection between that and what
happened here,'' Mr. Remington said.
</p>
<p class="articleParagraph enarticleParagraph" >
Another Amtrak spokesman said the last train to use the bridge before
the accident was New Jersey Transit train No. 7805, heading south about
10 minutes before the derailment. It passed without incident. The last
train to cross going north -- the direction of the ill-fated Amtrak --
was a New Jersey Transit train that passed at 1:55 A.M., well before the
4 A.M. malfunction described by investigators and Senator Lautenberg.
</p>
<p class="articleParagraph enarticleParagraph" >
A Secaucus City Councilman, Mike Grecco, said that crews had been
dispatched overnight to make a repair at the bridge, but he said he did
not know what the problem was.
</p>
<p class="articleParagraph enarticleParagraph" >
The bridge, a steel trestle that looks in the distance like an old
Erector set, stands over a bend in the Hackensack River and swings 90
degrees on its axis to allow river traffic, mostly pleasure boats and an
occasional tug from Newark Bay, to pass. The bridge carries two sets of
tracks -- for northbound and southbound traffic -- on rails that are
mitered to fit tightly and smoothly with the regular rails on either
side. A bridge operator in a small shack electrically operates the
bridge and its red and green signal lights.
</p>
<p class="articleParagraph enarticleParagraph" >
The marshy site may have been fortunate, said the Secaucus Mayor,
Anthony Just, who noted that no one had been killed and that there were
few serious injuries. ''The wetlands sponged it; it's like a big
cushion,'' he said of the land.
</p>
<p class="articleParagraph enarticleParagraph" >
David Bolger, a Federal Railroad Administration spokesman, said
officials of his agency would join the National Transportation Safety
Board and state officials in an investigation.
</p>
<p class="articleParagraph enarticleParagraph" >
Mr. Remington, of Amtrak, said the train that derailed had been bound
from Washington to Boston on an early morning run that made a scheduled
stop at Penn Station in Newark shortly after 6 A.M. As it proceeded
toward New York, it was carrying 88 passengers and 20 Amtrak employees,
he said.
</p>
<p class="articleParagraph enarticleParagraph" >
The train apparently had a green light at the bridge, and, though
derailed, was nearly across when it sideswiped the Carolinian. No one
aboard the Carolinian was known to have been injured, and it proceeded
to Penn Station in Newark, where passengers were transferred to another
train and sent on their way.
</p>
<p class="articleParagraph enarticleParagraph" >
At the scene of the crash, four passenger coaches and four
baggage-mail cars were still upright on the rails yesterday afternoon,
but derailed coaches and a tangle of other equipment brought in to raise
and remove them blocked the tracks.
</p>
<p class="articleParagraph enarticleParagraph" >
Northeast Corridor riders and passengers on New Jersey Transit's
North Jersey Coastline trains were able to reach New York only by
traveling to Newark and taking Port-Authority Trans-Hudson trains into
Manhattan. Midtown Direct line trains, which travel from Dover Township
into Penn Station, were ending in Hoboken, where passengers were taking
PATH trains into the city. At least four Metroliner trips were canceled
by Amtrak yesterday.
</p>
<p class="articleParagraph enarticleParagraph" >
Less than a mile from the site of the crash is the junction where two
commuter trains carrying more than 700 passengers collided in a
thunderous roar last Feb. 9. Both engineers and a passenger were killed
and 162 were injured in one of the worst commuter accidents in the
region in decades. An investigation later determined that one of the
engineers had run through a stop signal.
</p>
<p class="articleParagraph enarticleParagraph" >
The worst train accident in the region occurred on Sept. 15, 1958,
when a New York-bound Jersey Central train plunged through an open lift
bridge at Elizabethport, N.J., into Newark Bay, killing 48 people. That
train failed to stop despite three warning signals and an automatic
halting device. The guard rails on bridges in the area were installed as
a result of that accident.
</p>
<p class="articleParagraph enarticleParagraph" ><pre class="articlePre" >
Map of New Jersey showing site of yesterday's Amtrak crash. (pg. 40)
</pre>
</p>
<p class="articleParagraph enarticleParagraph" >
05:30 EST November 24, 1996
</p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RF</b>&nbsp;</td><td><br/>
444790
</td></tr>
<tr><td align="right" valign="top" class="index"><br/><b>ART</b>&nbsp;</td><td><br/>
Photos: A derailment yesterday at the Portal Bridge in the marshes of
Secaucus, N.J., sent two Amtrak engines and several cars down an
embankment. (Associated Press) (pg. 1); An Amtrak train bound for New
York City derailed yesterday after crossing the Hackensack River. The
engines and several cars fell down an embankment by the New Jersey
Turnpike. (Dith Pran/The New York Times) (pg. 40)
</td></tr>
<tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>gcat : Political/General News | ncat : Content Types | npag : Page-One Story</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>namz : North America | usa : United States | use : Northeast U.S. | usnj : New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document nytf000020011014dsbo01nxp</td></tr></table><br/></div></div><br/><span></span><div id="article-rec0000020011015ds7e00n2i" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/recLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>NEWS</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>U.S. LAYS PATH DEATHS TO HUMAN ERROR </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>The Associated Press </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>185 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>14 July 1996</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The Record</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>REC</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>© 1996 North Jersey Media Group Inc. Provided by ProQuest Information and Learning. All rights reserved.
 </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >The accidental deaths of two Port Authority Trans-Hudson railroad workers last year were the result of human error by one of the workers, according to a report by the National Transportation Safety Board. </p>
<p class="articleParagraph enarticleParagraph" >The foreman, who was killed Dec. 2 by a westbound train near the Harrison station, had not obtained permission to work on the track, nor had he posted a watchman, said the NTSB report released Thursday. Two other workers were struck; one was killed, the other suffered a leg injury. </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >NTSB also found that an engineer and operations examiner whose train passed through the area shortly before the accident failed to warn the trainmaster the workers were on the track without a watchman. </p>
<p class="articleParagraph enarticleParagraph" >Soon after the crash, the Port Authority had come to similar conclusions in the deaths of Nunzio Tinnirello, 48, of Bayonne, and Guido Vendittoli, 55, of Secaucus. </p>
<p class="articleParagraph enarticleParagraph" >The engineer of the train that hit the workers said his view was hindered by sunlight, but two passengers told investigators they saw the men working on the tracks.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>NEW YORK STATE | NEW JERSEY | TRANSIT | RAILROAD | ACCIDENT | DEATH | PROBE | RESULT</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>Record / Hackensack, New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document rec0000020011015ds7e00n2i</td></tr></table><br/></div></div><br/><span></span><div id="article-nytf000020011014ds3c00bx5" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>
Section B; Metropolitan Desk
</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>
METRO DIGEST
</span>
</td></tr><tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>859 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>12 March 1996</td></tr><tr><td align="right" valign="top" class="index"><b>ET</b>&nbsp;</td><td>05:00 AM</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td> The New York Times </td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td> Late Edition - Final </td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td> Page 2, Column 4 </td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>
c. 1996 New York Times Company
</td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >
NEW YORK
</p>
<p class="articleParagraph enarticleParagraph" >
Taking Babies Born on Drugs
With New York officials working to toughen child protection laws, there
is widespread sentiment in Albany to give welfare agencies more power to
take babies who are born with drugs in their systems away from their
mothers. Mayor Giuliani has proposed legislation that would begin legal
proceedings against any mother whose baby tested positive for drugs at
birth. The Senate and the Assembly are considering similar measures.
(Page B1.)
</p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >
Cutting Back on Token Clerks
The Transit Authority plans to cut 800 more token booth clerks from New
York City subway stations, slashing their ranks by a third, over the
next two years as it goes ahead with its electronic fare collection
program, officials said. The cuts are part of a five-year plan to trim
$2.4 billion from the Transit Authority's budget. (B1.)
</p>
<p class="articleParagraph enarticleParagraph" >
Charting Response at the U.N.
It is hard not to notice, especially now that Mayor Giuliani has gone to
Israel to show solidarity after the latest Palestinian bombings, that
the United Nations Security Council has yet to take the first step
toward a formal resolution denouncing Islamic terrorists. NYC by Clyde
Haberman. (B1.)
</p>
<p class="articleParagraph enarticleParagraph" >
Officer Blamed for His Injuries
A defense lawyer for a New York City police officer charged with
recklessly shooting a fellow officer working undercover whom he mistook
for a criminal told a jury in the unusual trial that the undercover
officer was to blame for his own injuries because he was not wearing his
badge or ''colors'' that would have identified him. (B3.)
</p>
<p class="articleParagraph enarticleParagraph" >
Giuliani Pays Respects in Israel
When Mayor Giuliani expressed his solidarity with Israel by climbing
aboard a No. 18 bus run trailed by staff members and a gaggle of New
York television crews, the problem was finding space. About 10 regular
riders on the Jerusalem bus found seats in the commotion, only to flee
when it became clear that the driver had no intention of making the
regular stops. The bus line had been recently attacked twice by suicide
bombers, killing 45 people. (B3.)
</p>
<p class="articleParagraph enarticleParagraph" >
Acquittal in Extortion Case
Anthony Aliperti, a former member of the planning board in Brookhaven,
L.I., was acquitted of extortion charges. This makes the third time that
John McNamara, the former Port Jefferson car dealer, has failed to
persuade a Federal jury that he bribed Brookhaven officials to win
zoning approval for real estate developments. (B4.)
</p>
<p class="articleParagraph enarticleParagraph" >
Pataki Asked to Redo Budget
Governor Pataki's most powerful ally in the State Legislature -- the
Senate majority leader, Joseph L. Bruno -- called on the Governor to
rewrite his budget for the 1997 fiscal year, saying it depends too
heavily on Federal actions that seem increasingly unlikely to happen.
Mr. Bruno's comments signaled the nearly complete erosion of support for
Mr. Pataki's $63.8 billion spending plan, even among the Governor's
fellow Republicans. (B4.)
</p>
<p class="articleParagraph enarticleParagraph" >
Conviction in Arranging Murder
Michael Burnett -- a career swindler whose work as a Government informer
played a prime role in exposing major municipal corruption in New York
City and Chicago -- was found guilty in Federal District Court in
Brooklyn of masterminding the murder of a Staten Island woman in what
prosecutors said was his latest swindle. (B5.)
</p>
<p class="articleParagraph enarticleParagraph" >
CONNECTICUT
</p>
<p class="articleParagraph enarticleParagraph" >
Library Fire Brings Outpouring
The public library in Danbury,  Conn., burned, destroying 100,000 books
and as many magazines, videos, compact disks and books on tape. And even
though insurance will cover nearly all the cost of repairing and
restocking the library, clubs, businesses, schools and individuals are
inundating the city with contributions and offers of books. (B1.)
</p>
<p class="articleParagraph enarticleParagraph" >
NEW JERSEY
</p>
<p class="articleParagraph enarticleParagraph" >
A Pitch for the Environment
In a speech in Hackensack, N.J., President Clinton challenged the
Republican-controlled Congress to approve a proposal to use $2 billion
in tax breaks to spur the revitalization of mildly contaminated urban
industrial sites. National Report. (A17.)
</p>
<p class="articleParagraph enarticleParagraph" >
President Clinton mixed peacemaking and politics in New York City,
accepting an award from Irish-American leaders before moving on to a
fund-raiser for Congressional Democrats. (A17.)
</p>
<p class="articleParagraph enarticleParagraph" >
Examining Child Cancer Rates
State Health Department officials in New Jersey said that they would
begin a detailed examination of all childhood cancer cases in Ocean
County in an attempt to learn the causes of an elevated rate of cancer
among children in the county. Although the overall rate of juvenile
brain and central nervous system cancer in the Toms River section of the
county is more than twice what officials expect based on statewide
averages, officials said they currently saw no cause for alarm. (B4.)
</p>
<p class="articleParagraph enarticleParagraph" >
REGION
</p>
<p class="articleParagraph enarticleParagraph" >
New Yorkers Lead Contest
Continuing a 55-year tradition, New York dominated the Westinghouse
Science Talent Search -- and the students each won $10,000 or more in
scholarships -- in the oldest and most prestigious science contest for
high school students around the country. (B4.)
</p>
<p class="articleParagraph enarticleParagraph" >
Director Is Named for PATH
Michael DePallo, a top executive at California's Bay Area Rapid Transit
District, was named director and general manager of the Port Authority
Trans-Hudson rail line for New York and New Jersey. He will take over
the PATH line on April 1. (B5.)
</p>
<p class="articleParagraph enarticleParagraph" >
05:00 EST March 12, 1996
</p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RF</b>&nbsp;</td><td><br/>
359122
</td></tr>
<tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>Summary </td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document nytf000020011014ds3c00bx5</td></tr></table><br/></div></div><br/><span></span><div id="article-nytf000020011014ds3c00bgn" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>
Section B; Metropolitan Desk
</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>
PATH Names New Director For Rail Line
</span>
</td></tr><tr><td align="right" valign="top" class="index"><b>CR</b>&nbsp;</td><td>
By The New York Times
</td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>474 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>12 March 1996</td></tr><tr><td align="right" valign="top" class="index"><b>ET</b>&nbsp;</td><td>05:00 AM</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td> The New York Times </td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td> Late Edition - Final </td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>
c. 1996 New York Times Company
</td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >
SAN FRANCISCO, March 11 -- Michael DePallo, a top executive at
California's Bay Area Rapid Transit District, was named director and
general manager of the Port Authority Trans-Hudson rail line for New
York and New Jersey today.
</p>
<p class="articleParagraph enarticleParagraph" >
Mr. DePallo, 41, who has had many mass transit jobs, will take over
the PATH line on April 1. The line carries 200,000 riders a day between
lower and midtown Manhattan and New Jersey and has 1,244 employees. He
is to receive an annual salary of $129,974, Port Authority officials
said.
</p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >
Mr. DePallo joined the Oakland-based Bay Area Rapid Transit system
in 1992 as the manager of the budget and project management office,
which runs the finances for a rail and subway line spanning four
counties and linking San Francisco with the eastern side of the bay. The
bay area system carries 250,000 commuters every weekday on 77 miles of
track. It employs almost 3,000 people.
</p>
<p class="articleParagraph enarticleParagraph" >
''He's just a first-rate transit professional,'' said Michael
Bernick, on the board of the bay area system. ''They're fortunate to get
him. They have the same challenge we have at BART: how to integrate
their system with other ones? How do you move people from suburb to
suburb?''
</p>
<p class="articleParagraph enarticleParagraph" >
Mr. DePallo oversaw the merging of the district's capital and
operating departments, bringing a sharper focus and greater efficiency
to the agency. He handled many things: negotiating the purchase of new
cars; rehabilitating old ones; building parking garages, and overseeing
research and development into ways to double the district's train
capacity through the bottleneck of the subway tube that runs under San
Francisco Bay.
</p>
<p class="articleParagraph enarticleParagraph" >
''He's creative, a problem solver,'' said Dorothy Dugger, the deputy
general manager of Bay Area transit. ''He knows both the business side
and the operating side.''
</p>
<p class="articleParagraph enarticleParagraph" >
Mr. DePallo said he always wanted to work on a mass transit system.
</p>
<p class="articleParagraph enarticleParagraph" >
''It's always been an interest of mine,'' he said. ''I like the idea
of public transportation. It's great for the environment. I enjoyed
taking it. It's the right way to go.''
</p>
<p class="articleParagraph enarticleParagraph" >
Before coming to the bay area system, where he received $102,648 a
year, Mr. DePallo held management jobs at the Massachusetts Bay
Transportation Authority in Boston and Southeastern Pennsylvania
Transportation Authority in Philadelphia.
</p>
<p class="articleParagraph enarticleParagraph" >
He holds a bachelor of arts degree from Mansfield University in
Pennsylvania and a master's in city planning from the University of
Pennsylvania.
</p>
<p class="articleParagraph enarticleParagraph" >
Mr. DePallo said he did not find the West Coast as car-crazy as it is
reputed to be.
</p>
<p class="articleParagraph enarticleParagraph" >
''Even in California, they're coming to the realization that public
transportation is important,'' he said. ''The mentality of the Wild West
is changing.''
</p>
<p class="articleParagraph enarticleParagraph" >
05:00 EST March 12, 1996
</p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>c41 : Management Issues | c411 : Management Moves | ccat : Corporate/Industrial News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document nytf000020011014ds3c00bgn</td></tr></table><br/></div></div><br/><span></span><div id="article-rec0000020011026drci017ie" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/recLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>NEWS</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>NJ TRANSIT MAY BAN TOBACCO, LIQUOR ADS </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>The Associated Press </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>286 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>18 December 1995</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The Record</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>REC</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>© 1995 North Jersey Media Group Inc. Provided by ProQuest Information and Learning. All rights reserved.
 </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >Joe Camel and the Marlboro Man may be asked to get off at the next stop by NJ Transit officials. </p>
<p class="articleParagraph enarticleParagraph" >The state transit agency is considering banning tobacco and alcohol ads from its publicly owned bus and rail systems. </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >The five-year contract covering more than 8,000 ad poster sites expires in May _ giving the agency an opportunity to set new rules outlining what ads it will accept. </p>
<p class="articleParagraph enarticleParagraph" >NJ Transit's contract with Transportation Display Inc. covers ads on more than 1,800 buses, 700 rail cars, and more than 150 rail and bus stations. </p>
<p class="articleParagraph enarticleParagraph" >The contract with the company nets the transit agency just over $2 million annually in ad revenues, said NJ Transit spokesman Jeff Maclin. </p>
<p class="articleParagraph enarticleParagraph" >Maclin said $213,800 of that came from tobacco ads and $85,800 from alcohol advertisements. During the past four years, that translated into 13 percent of ad revenue from tobacco companies and about 5 percent from alcoholic beverage ads, he said. </p>
<p class="articleParagraph enarticleParagraph" >A move to discontinue the ads would follow a national trend among public transportation agencies. A survey done this fall by the American Public Transit Association found that more than half of 31 transit agencies ban or limit either alcohol or tobacco ads. </p>
<p class="articleParagraph enarticleParagraph" >"It's clear that there is a growing reluctance to allow that kind of advertising to dominate what is carried inside and outside public-transit vehicles," said Chip Bishop, a spokesman for the Washington-based industry trade group. </p>
<p class="articleParagraph enarticleParagraph" >In July 1993, New Jersey's other major transportation agency, the Port Authority Trans-Hudson (PATH),voted to phase out all tobacco ads on its commuter trains and buses and in its terminals.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>NEW JERSEY | TRANSIT | ADVERTISING | ALCOHOL | TOBACCO</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>Record / Hackensack, New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document rec0000020011026drci017ie</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020060709drc300614" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Metropolitan Desk; 1</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>PATH Train Kills 2 Men Working on the Tracks  </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By NEIL MacFARQUHAR  </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>556 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>3 December 1995</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>54</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 1995 The New York Times Company.  All Rights Reserved.  </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >HARRISON, N.J., Dec. 2 -- Two PATH employees were killed and a third was hospitalized today when a train headed from Jersey City to Newark struck them as they worked on the rails near the station here.  </p>
<p class="articleParagraph enarticleParagraph" >The cause of the 2 P.M. accident was not immediately known, said Gwen Williams, a spokeswoman for the Port Authority of New York and New Jersey, which runs the Port Authority Trans-Hudson trains. It occurred in broad daylight about  </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >200 yards east of the station as the train approached it.  </p>
<p class="articleParagraph enarticleParagraph" >An investigator speaking on condition of anonymity said that both eastbound and westbound trains were using the track where the accident occurred because of maintenance work on another track. The workers who were killed were doing minor work on the detour track. They apparently were caught off guard by a train coming from the opposite direction from the one they had been used to. The train was moving slowly as it approached the station, and thus would not have been making much noise, the investigator said.  </p>
<p class="articleParagraph enarticleParagraph" >The hospitalized worker was a flagman, he said.  </p>
<p class="articleParagraph enarticleParagraph" >Ms. Williams said all service on the tracks was suspended for an hour, with some service resuming after 3 P.M. The four sets of tracks there are also used by Amtrak and New Jersey Transit trains as they travel from New York to Newark's Pennsylvania Station, just across the Passaic River from Harrison. A spokesman for Amtrak said its service had temporary delays because trains were slowing down through the area where the accident occurred.  </p>
<p class="articleParagraph enarticleParagraph" >After the accident, the 200 passengers aboard the train were transferred to a second PATH train that had pulled up in front of the first train and taken to Newark, Ms. Williams said. Officials said they knew of no injuries to any of the passengers.  </p>
<p class="articleParagraph enarticleParagraph" >The names of the two men who died on the tracks were not released pending notification of family members.  </p>
<p class="articleParagraph enarticleParagraph" >Ms. Williams said the injured worker, identified by his family as Antonio Griffith, was taken to University Hospital for treatment but was not considered seriously hurt.  </p>
<p class="articleParagraph enarticleParagraph" >Dr. Kristine Thayer, an attending physician at the hospital, said the man was being treated for trauma and tested for internal injuries. But he was in stable condition, and it appeared he suffered only bruises, she said. "He's very, very upset, and we're giving him time to cool out," she said. "The train hit him. He recalled flying through the air. He saw his friends under the train when it went past."  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Griffith's sister, Elvia Texidor, said her brother, who was given sedatives late last night, was too overwrought to explain exactly what happened on the tracks. But she said he told her that "something didn't feel right" seconds before the train hit the men.  </p>
<p class="articleParagraph enarticleParagraph" >"And when he looked up, the train was right on top of them," Ms. Texidor said at the hospital. "He didn't know how the train didn't see them. But he was spared, thank God."  </p>
<p class="articleParagraph enarticleParagraph" >Port Authority officials said that the train's motorman was undergoing drug tests that are standard procedure after any fatal accident.  </p>
<p class="articleParagraph enarticleParagraph" >Map of Northeastern New Jersey detailing Harrison PATH Station.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>CO</b>&nbsp;</td><td><br/>panynj : Port Authority of New York and New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i71 : Railroads | i763 : Port/Harbor Operations | irailtr : Road/Rail Transport | iship : Water Transport/Shipping | itsp : Transportation/Shipping</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>gdis : Disasters/Accidents | gmmdis : Accidents/Man-made Disasters | gtacc : Transport Accidents | gcat : Political/General News | grisk : Risk News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>usny : New York | usnj : New Jersey | namz : North America | usa : United States | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>PORT AUTHORITY TRANS-HUDSON CORP (PATH)  | MACFARQUHAR, NEIL </td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>New York Times Digital (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020060709drc300614</td></tr></table><br/></div></div><br/><span></span><div id="article-rec0000020011026drc1015ni" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/recLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>NEWS</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>PA PROPOSES LOWER BUDGET FOR '96 WITHOUT TOLL HIKES, SERVICE CUTS </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By BOB GROVES, Staff Writer </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>552 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>1 December 1995</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The Record</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>REC</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>© 1995 North Jersey Media Group Inc. Provided by ProQuest Information and Learning. All rights reserved.
 </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >The Port Authority of New York and New Jersey on Thursday proposed a $2.59 billion budget for 1996 that cuts operating expenses for the first time in more than half a century. </p>
<p class="articleParagraph enarticleParagraph" >Port Authority Executive Director George J. Marlin said the proposed budget, which is $105 million lower than the one for 1995, neither raises tolls nor decreases services in 1996. </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >Cuts were made possible, Marlin said, through a 27 percent cut in administrative staff levels, eliminating redundancy and consolidating staff units, and containing the rate of increase in health-care costs. </p>
<p class="articleParagraph enarticleParagraph" >About 600 jobs were eliminated during 1995 through buyouts, attrition, and layoffs. Approximately 300 more jobs will be eliminated in 1996, a Port Authority spokesman said. </p>
<p class="articleParagraph enarticleParagraph" >Other savings came from reducing expenses not related to customer service, such as travel and office supplies, officials said. In November, the authority sold the Vista Hotel, its luxury hotel at the World Trade Center, to Marriott Host Corp. for $141.5 million. </p>
<p class="articleParagraph enarticleParagraph" >"I am proud to reverse a trend of spending increases that stretches back to 1943, when Franklin D. Roosevelt was in the White House," Marlin said Thursday at the Port Authority headquarters at the World Trade Center. </p>
<p class="articleParagraph enarticleParagraph" >The Port Authority board will vote on the proposed budget Dec. 14. </p>
<p class="articleParagraph enarticleParagraph" >Tolls and fares "will definitely not increase in 1996," an authority spokesman said Thursday. </p>
<p class="articleParagraph enarticleParagraph" >"Down the road, we'll be able to give some projection beyond '96, but not today. It's been noted many times that toll increases would be considered only as a last resort," he said. </p>
<p class="articleParagraph enarticleParagraph" >The proposed budget includes $1.5 billion for operating expenses, and $677 milion for capital improvements. </p>
<p class="articleParagraph enarticleParagraph" >It provides $140 million for "service improvements," including replacement of more than 5,000 Port Authority Trans-Hudson train windows to eliminate and prevent "scratchfiti" _ grafitti scratched into glass; and starting up monorail service at Newark International Airport, which is predicted to begin operation in the spring of 1996. </p>
<p class="articleParagraph enarticleParagraph" >A total of $677 million in capital spending will include $296 million for airport improvements, and $155 million for improvements to bridge, tunnels, and PATH, and preliminary work on electronic toll collection, or the E-ZPass program. </p>
<p class="articleParagraph enarticleParagraph" >With E-ZPass, electronic toll machines at toll booths will "read" a credit-card sized electronic tag attached to drivers' windshields, and then charge the toll to the driver's account. The system would allow cars to drive slowly through toll booths, rather than have to stop. It would permit single-payment billing for use of toll facilities in the metropolitian area, including the New Jersey Turnpike and Garden State Parkway, in addition to the Port Authority's bridges and tunnels. </p>
<p class="articleParagraph enarticleParagraph" >"The PA will have a regional back office that allows to use any of the facilities and be billed only to one account," said Roy Pleasant, assistant director of information technology architectures and corporate technology. </p>
<p class="articleParagraph enarticleParagraph" >The authority will use "a competitive process" to choose a toll collection service vendor to install and operate the electronic toll system. It will begin with bridges to Staten Island in the second half of 1997. By the end of 1998, the system would be installed at the George Washington Bridge and the Lincoln and Holland tunnels.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>NEW YORK STATE | NEW JERSEY | TRANSIT | BUDGET | EMPLOYMENT | LAYOFF</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>Record / Hackensack, New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document rec0000020011026drc1015ni</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020050401dr9g002ns" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>National Desk; 1</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>Era Ends at Port Authority As 160 Workers Are Laid Off  </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By CLIFFORD J. LEVY  </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>1214 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>16 September 1995</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>1</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 1995 The New York Times Company.  All Rights Reserved.  </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >Wanda Rivera was sitting at her desk at the headquarters of the Port Authority of New York and New Jersey yesterday morning grimly watching her co-workers being summoned one by one into an office to receive the news that they were being laid off. Then her phone rang. The display said the call was coming from extension 4479. That office. Ms. Rivera knew it was her turn.  </p>
<p class="articleParagraph enarticleParagraph" >"I thought: it is finally here, the dreaded moment," said Ms. Rivera, 31, a secretary. "I was nervous and frustrated and angry and all sorts of feelings combined."  </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >After weeks of bouncing between despair and hope, Ms. Rivera and more than 160 other people lost their jobs yesterday in the first large-scale layoffs at the Port Authority since the Depression. The reductions ushered in a new era for an agency that had long seemed able to dodge the kind of sharp cuts that government and industry have been subjected to in recent years.  </p>
<p class="articleParagraph enarticleParagraph" >But the authority is now being overseen by Gov. George E. Pataki and Gov. Christine Todd Whitman, two Republicans who have vowed to trim its size just as they have scaled back their own states' work forces. Yesterday's layoffs, most of which were at the authority's headquarters at the World Trade Center, were the first phase of reductions that are to total at least 600 positions by early next year, from a staff of about 8,800.  </p>
<p class="articleParagraph enarticleParagraph" >The cuts, intended to avoid an increase in the authority's tolls and commuter railway fare, left a tense scene at the authority's offices in the World Trade Center. In some departments, workers sobbed as they packed up the contents of their desks, saying they were bitter about devoting so many years of their lives to an agency that is now casting them aside.  </p>
<p class="articleParagraph enarticleParagraph" >"I am angry, because when you have been with a company almost nine years, it's almost like it's your second family," said Debra Ashe, 35, a senior office assistant at the office of economic and policy analysis, which is being disbanded, doing away with more than 40 jobs. "It really hurts. You expect to stay here and continue working and retire here one day."  </p>
<p class="articleParagraph enarticleParagraph" >The cuts seemed to sting all the more because the authority was known for nurturing its workers and protecting them from the turmoil that often roils government agencies when new governors or mayors are elected.  </p>
<p class="articleParagraph enarticleParagraph" >"This is the Port Authority," said John Dunham, 31, who was dismissed as a senior economic analyst. "Come on, the Port Authority never laid off anyone."  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Dunham, who arrived at the authority only 15 months ago, added, "It has a reputation and history that is now gone."  </p>
<p class="articleParagraph enarticleParagraph" >The layoffs were such a departure for the authority that it held training sessions for senior officials on Thursday on how to dismiss people. The officials were put through exercises in which they pretended they were telling workers they were being dismissed and then described the benefits the workers were entitled to.  </p>
<p class="articleParagraph enarticleParagraph" >Those being laid off are to remain on the payroll until Oct. 15 and then get a severance package that takes into consideration age and seniority and will average about three months pay per worker, officials said. The authority has also set up a career counseling office that more than 100 workers had visited by last night.  </p>
<p class="articleParagraph enarticleParagraph" >Besides the office of economic and policy analysis, three other areas were hard hit: management and budget; international business, and human resources. George J. Marlin, the authority's executive director, has said that the number of workers in those four areas has grown unwieldy in recent decades.  </p>
<p class="articleParagraph enarticleParagraph" >He has pointed to those areas as examples of what he says is a tendency for middle managers at headquarters to duplicate tasks being done at the authority's many sites, which include the region's three major airports, six bridges and tunnels, the Port Authority Trans Hudson commuter railway, most of the marine terminals at New York Harbor and the World Trade Center itself.  </p>
<p class="articleParagraph enarticleParagraph" >When the authority announced the layoffs last week, it said they were the largest, and virtually the only, such cuts in its 74-year history. But Professor Jameson W. Doig of Princeton University, who has studied the authority, said yesterday that it laid off a similar percentage of its work force in the early 1930's, largely because of the Depression.  </p>
<p class="articleParagraph enarticleParagraph" >Senior authority officials said that by last night they had eliminated about 315 positions in this round of cuts, achieving them from the more than 160 layoffs as well as from voluntary departures and from leaving vacant slots open.  </p>
<p class="articleParagraph enarticleParagraph" >The officials said yesterday that they regretted having to undertake the layoffs, but they also insisted that they had no choice.  </p>
<p class="articleParagraph enarticleParagraph" >"It's always painful even when one person has to lose his job, but we feel that this is a vital and necessary first step in moving the Port Authority in a new direction," said John J. Haley Jr., the deputy executive director of the authority.  </p>
<p class="articleParagraph enarticleParagraph" >But the workers who were dismissed had few nice words for Mr. Pataki, Mrs. Whitman or the authority's senior management. Many of them questioned why the authority made so many cuts in the middle of the work force and left alone high-ranking officials who allowed the middle to grow in the first place.  </p>
<p class="articleParagraph enarticleParagraph" >"They want to be able say, 'We broke the back of the fat, arrogant Port Authority,' " said a worker who was laid off in the office of economic and policy analysis and who spoke on condition of anonymity. "I actually think that there is tremendous fat in this agency. Unfortunately, the real layers of fat are in senior management, but those guys are not going to slit their own throats."  </p>
<p class="articleParagraph enarticleParagraph" >Margie Beck, 28, another employee in the office, agreed, saying that her managers had misled her in recent weeks about how many jobs would be saved. "They didn't have to let all these people go," she said.  </p>
<p class="articleParagraph enarticleParagraph" >Others said their departments had been the focus of so many rumors for so many weeks that they were relieved that it was finally over. Now, they said, they could move on.  </p>
<p class="articleParagraph enarticleParagraph" >"I am not angry, in fact I have come to accept it," said another worker who would speak only on condition of anonymity. "I have come to the realization that it is probably not a good place to be even if you did survive."  </p>
<p class="articleParagraph enarticleParagraph" >Many said they had not yet begun looking for employment because they had thought they might escape the cuts. But now, they said, they would relax for a few days and then plunge into the job market.  </p>
<p class="articleParagraph enarticleParagraph" >"It's over with, I feel much better now," said Ms. Rivera, who had been in the office of economic and policy analysis for 12 years. "Let's see what will happen with the future."  </p>
<p class="articleParagraph enarticleParagraph" >Photo: John Dunham was laid off 15 months after moving to New York. Margie Beck waved goodbye as she and Sandra James left the World Trade Center after being laid off. (Photographs by Marilynn K. Yee/The New York Times) (pg. 25)  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>CO</b>&nbsp;</td><td><br/>panynj : Port Authority of New York and New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i763 : Port/Harbor Operations | iship : Water Transport/Shipping | itsp : Transportation/Shipping</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>c42 : Labor/Personnel Issues | credun : Lay-offs/Redundancies | gjob : Labor Issues | ccat : Corporate/Industrial News | gcat : Political/General News | gcom : Society/Community/Work | ncat : Content Types | nfact : Factiva Filters | nfcpin : FC&E Industry News Filter</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>nyc : New York City | usa : United States | namz : North America | use : Northeast U.S. | usny : New York</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>LAYOFFS AND JOB REDUCTIONS  | LABOR  | FINANCES  | BUDGETS AND BUDGETING  | PORT AUTHORITY OF NY AND NJ  | LEVY, CLIFFORD J  | PATAKI, GEORGE E (GOV)  | WHITMAN, CHRISTINE TODD (GOV) </td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>New York Times Digital (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020050401dr9g002ns</td></tr></table><br/></div></div><br/><span></span><div id="article-rec0000020011026dr8u00t5t" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/recLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>OPINION</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>PATH FARES: A MODEST INCREASE IS WARRANTED </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By JAMES AHEARN </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>978 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>30 August 1995</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The Record</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>REC</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>© 1995 North Jersey Media Group Inc. Provided by ProQuest Information and Learning. All rights reserved.
 </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >RUDY GIULIANI makes two good points in his latest diatribe against the Port Authority and us masses huddled west of the Statue of Liberty. </p>
<p class="articleParagraph enarticleParagraph" >First, the Port Authority Trans Hudson fare of $1 is unjustifiably low compared to the New York City subway fare of $1.25, soon to rise to $1.50. </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >Second, the PATH fare is low not because the line is cheaper to run but because it is more heavily subsidized. One way or another, through a fare increase or a cut in expenditures or both, the PATH deficit should be reduced. </p>
<p class="articleParagraph enarticleParagraph" >No mass transit line in the country makes money. Most are subsidized by taxes. That is not true of PATH. Its deficit is made up by profits on other Port Authority operations, in particular the World Trade Center. </p>
<p class="articleParagraph enarticleParagraph" >Some history is useful here. Prior to 1962 there was no PATH. We had the Hudson Tubes instead. The tubes were a subway line, much like New York's, that ran from Newark, Hoboken, and Jersey City under the Hudson into Manhattan. </p>
<p class="articleParagraph enarticleParagraph" >The formal moniker of the tubes was the Hudson and Manhattan Railroad. It was a private corporation. When it was young, it had made money. Those days were long gone. It was bankrupt. The rolling stock was decrepit, the rails and switches rusty, the work force demoralized. </p>
<p class="articleParagraph enarticleParagraph" >It was run by a court-appointed trustee, Herman Stichman. He had been lobbying for years, tirelessly but without success, to get the Port Authority to take over the line. The authority's board, packed with bankers, lawyers, and moguls like Charles Engelhard, reacted as to an invitation to donate their wealth to Mother Teresa. </p>
<p class="articleParagraph enarticleParagraph" >They were in the business of making money, whether in their private dealings or as custodians of a successful public authority. They did not need to hire a consultant to conclude that the Hudson Tubes would cost the authority a bundle. </p>
<p class="articleParagraph enarticleParagraph" >Stichman was the most press-accessible business leader on the planet. In the late Fifties I was a night rewriteman in the United Press bureau in Newark. Our Rolodex, like that of every newspaper in North Jersey and New York, included Stichman's home and office phone numbers. </p>
<p class="articleParagraph enarticleParagraph" >When some news event occurred that related even remotely to the possibility of a Port Authority takeover of the tubes, you could call Stichman in Manhattan, day or night, and get a response. Once I got him out of bed at 1 o'clock in the morning. He was perfectly affable. He dictated a comment that was pertinent and newsy and closed by saying, "Thank you." </p>
<p class="articleParagraph enarticleParagraph" >Stichman had an ally in the governor of New Jersey, Robert Meyner. But Meyner alone couldn't make the Port Authority do something it didn't want to do. Then something happened that gave him some leverage. It involved his opposite number in New York, Nelson Rockefeller, and Nelson's brother David, head of the Chase Manhattan Bank. </p>
<p class="articleParagraph enarticleParagraph" >The Rockefellers wanted to boost downtown Manhattan. They had an idea for a huge office development, to be called the World Trade Center. The Port Authority would float bonds to build it. The first draft of the plan put the project on the east side of Manhattan. Meyner said no. If the thing was going to be built, he wanted it on the west side, the Hudson River side, where it would be accessible to New Jerseyans. </p>
<p class="articleParagraph enarticleParagraph" >He had a specific site in mind, too, one centered on the Lower Manhattan terminal of the Hudson Tubes. As part of the deal the authority would have to assume responsibility for the tubes. Profits on the World Trade Center would help pay the railroad deficit. </p>
<p class="articleParagraph enarticleParagraph" >The authority board bridled but ultimately assented. Once the authority got into the commuter rail business, it did a thorough job. The tubes were modernized. Today PATH is a clean, efficient operation. But expensive. The $1 fare covers only 36 percent of costs. By comparison, the $1.25 charged passengers on the city subway covers 70 percent of costs. </p>
<p class="articleParagraph enarticleParagraph" >Money that the Port Authority spends on PATH is money it doesn't have for other projects, including projects favored by Mayor Giuliani. The present, Republican governors of the two states are resisting a fare increase. They contend the authority is wasteful. Maybe so. Apparently savings can be realized on PATH operations. Sooner or later, though, the fare will have to be raised. Eighty-five percent of the passengers live in New Jersey. There will be a hullabaloo. </p>
<p class="articleParagraph enarticleParagraph" >The case for a fare increase is set out in a position paper prepared for Giuliani by his chief planner, Joseph Rose, who also calls for a big toll increase on the George Washington Bridge and the Lincoln and Holland tunnels. The present toll is $4. City bridges like the Whitestone, the Triborough, and the Verrazano charge $6 now for a round-trip, with a $1 increase in prospect. Bridge profits help subsidize the city subway. </p>
<p class="articleParagraph enarticleParagraph" >Again, some toll increase would appear warranted, although a $2 raise, much less $3, isn't going to happen soon. This might be an opportune time to experiment with rush-hour surcharges. </p>
<p class="articleParagraph enarticleParagraph" >What isn't acceptable is Rose's argument that because New York City has 47 percent of the population of the metropolitan region, compared to 28 percent for North and Central Jersey, the city deserves more Port Authority money than the New Jersey counties. This misconceives the basic nature of the bistate arrangement. </p>
<p class="articleParagraph enarticleParagraph" >It was, and is, a partnership of equals. New Jersey never would have thrown in with New York in the Port Authority compact of 1921 if this state would be a junior partner. Both states have benefited from the arrangement. We ought to be clear about that, as debate proceeds on details of projects and funding.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>OP-ED | COLUMN | NEW JERSEY | NEW YORK CITY | TRANSIT | CONSUMER | COST</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>Record / Hackensack, New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document rec0000020011026dr8u00t5t</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020050520dr8u00098" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Metropolitan Desk; B</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>Port in a Storm: Planners Question the City's Maritime Future  </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By CLIFFORD J. LEVY  </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>1185 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>30 August 1995</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>1</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 1995 The New York Times Company.  All Rights Reserved.  </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >Government agencies have spent hundreds of millions of dollars in recent years to subsidize the depressed maritime industry in New York City, once one of the premier ports in the world. But now the city's own planning experts are asking for the first time whether the city would be better off if it let some or all of its commercial shipping business just die.  </p>
<p class="articleParagraph enarticleParagraph" >The marine terminals of New York have been eclipsed since the 1960's by those on the New Jersey side of the harbor, which are better equipped for modern shipping because they have more land and easier access to railways and highways. Government-owned terminals in Manhattan, Brooklyn and Staten Island have been unable to keep up, losing $40 million annually in the last two years, with little prospect of breaking even in the near future.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >Given the scarcity of government dollars, the City Planning Department said in a report, New York City should examine whether those subsidies, and the land where the terminals are located, should instead be used for other transportation projects or for other ventures. Those might include factories, office complexes and recreation areas, officials said.  </p>
<p class="articleParagraph enarticleParagraph" >"The current approach of continuing to do what has been done in the past is expensive and, more importantly, not working," the report said. "Serious questions need to be answered about the ability of these facilities to meet the shipping needs of the next century."  </p>
<p class="articleParagraph enarticleParagraph" >That recommendation is the latest attempt to shape the discussion about the future of New York City's once-prosperous waterfront, an expanse of rotting piers, abandoned factories and unrealized potential.  </p>
<p class="articleParagraph enarticleParagraph" >But it appears to put the Planning Department at odds with city policy over the last decade, which has been to insure that New Jersey, with 93 percent of the harbor's trade, does not get all of it.  </p>
<p class="articleParagraph enarticleParagraph" >Though the Planning Department merely called for a debate about the terminals and their costs, and did not say that the city should close any of them, the report has nonetheless angered some Giuliani administration officials, local politicians and terminal operators in the city.  </p>
<p class="articleParagraph enarticleParagraph" >"I am furious," said Sal F. Catucci, chief executive of American Stevedoring, which manages the Red Hook Container Terminal in Brooklyn. "You have a bunch of morons who don't know what they are talking about, who have never had a business of their own. If they don't have a port on this side of the river, this city is going to be a ghost town."  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Catucci and others say they cannot understand how the Planning Department could express such doubts only months after the city and the Port Authority of New York and New Jersey, which operates most of the city's marine terminals, had spent tens of millions of dollars to reopen the Howland Hook Marine Terminal on Staten Island.  </p>
<p class="articleParagraph enarticleParagraph" >Though it was released last week, the recommendation attracted little notice at first because it was buried in a report that was put together to buttress the Giuliani administration's contention that the Port Authority was biased toward New Jersey and must be reorganized.  </p>
<p class="articleParagraph enarticleParagraph" >Mayor Rudolph W. Giuliani cited the report last week when he fired the first shot in a border war with New Jersey, saying the Port Authority should raise the fare on the Port Authority Trans-Hudson, or PATH, commuter railway and the tolls on its bridges and tunnels in order to give more revenue to the city.  </p>
<p class="articleParagraph enarticleParagraph" >Though the Mayor focused on how much the PATH loses annually -- about $200 million -- he failed to note that the report also questions whether the losses on the marine terminals in New York City should be allowed to mount. The major terminals run by the Port Authority in New Jersey turn a collective profit, the Planning Department said. It is not clear where Mr. Giuliani stands on the terminals, but senior administration officials said the report was privately denounced by the city's Economic Development Corporation, which oversees the terminals.  </p>
<p class="articleParagraph enarticleParagraph" >The report said that after some study, the city and the authority might still conclude that the losses were justified or might decide that new investments should be made. But the city and the authority should also determine "whether there are other uses for the marine terminals that would make more economic sense," the report said.  </p>
<p class="articleParagraph enarticleParagraph" >In an interview, Joseph B. Rose, director of the Planning Department and chairman of the City Planning Commission, emphasized that he was not calling for the shutting of the terminals. And he said the recommendations about the terminals had to be seen as part of an overall assessment of the Port Authority, which was the purpose of the report.  </p>
<p class="articleParagraph enarticleParagraph" >All the Port Authority's money-losing ventures, including the PATH line and the Port Authority Bus Terminal, should be scrutinized, he said.  </p>
<p class="articleParagraph enarticleParagraph" >"A lot of the marine investments have to be re-evaluated, in the same way that you re-evaluate the PATH and the industrial parks, and the World Trade Center," Mr. Rose said. "Are these investments and these subsidies operating in such a way that they have the benefit that they were intended to have?"  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Rose also said he thought Howland Hook had more potential than the other terminals, particularly those in Brooklyn, many of which have not had tenants for years.  </p>
<p class="articleParagraph enarticleParagraph" >Peter Yerkes, a spokesman for the Port Authority, declined to comment yesterday on the report, but officials outside the Giuliani administration have not been reticent.  </p>
<p class="articleParagraph enarticleParagraph" >Representative Jerrold L. Nadler, a Democratic whose district includes parts of Manhattan and Brooklyn, sent a letter to Mr. Giuliani in which he praised the Mayor for attacking the Port Authority but also criticized the report's assertions about the marine terminals.  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Nadler, a proponent of building a large containerized shipping port in Brooklyn, told Mr. Giuliani that the city should not give up on its maritime industry.  </p>
<p class="articleParagraph enarticleParagraph" >"Rather than precipitously concluding (as the city planning report seems to do) that these terminals should be closed, there should be a public discussion of whether subsidies to these terminals make sense," Mr. Nadler wrote. "If such a debate were to take place, I am confident that the conclusion would be reached that some of these facilities (particularly those along the Brooklyn waterfront) could be made profitable."  </p>
<p class="articleParagraph enarticleParagraph" >Graph: "THE BOTTOM LINE: Operating at a Loss"  </p>
<p class="articleParagraph enarticleParagraph" >Marine terminals in Manhattan, Brooklyn and Staten Island have been unable to keep up with New Jersey competitors in commercial shipping, losing millions of dollars a year. Here is a breakdown of net operating losses. Brooklyn Marine Terminal  </p>
<p class="articleParagraph enarticleParagraph" >1993 -- $10,013,000  </p>
<p class="articleParagraph enarticleParagraph" >1994 -- $12,327,000 Red Hook Container Terminal  </p>
<p class="articleParagraph enarticleParagraph" >1993 -- $4,123,000  </p>
<p class="articleParagraph enarticleParagraph" >1994 -- $5,040,000 Manhattan Passenger Terminal  </p>
<p class="articleParagraph enarticleParagraph" >1993 -- $15,696,000  </p>
<p class="articleParagraph enarticleParagraph" >1994 -- $9,129,000 Howland Hook Marine Terminal  </p>
<p class="articleParagraph enarticleParagraph" >1993 -- $6,119,000  </p>
<p class="articleParagraph enarticleParagraph" >1994 -- $7,756,000 South Brooklyn Marine Terminal  </p>
<p class="articleParagraph enarticleParagraph" >1993 -- $6 million*  </p>
<p class="articleParagraph enarticleParagraph" >1994 -- $4 million* *estimate Sources: Port Authority of New York and New Jersey; New York City Economic Development Commission.(pg. B5)  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i763 : Port/Harbor Operations | iship : Water Transport/Shipping | itsp : Transportation/Shipping</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>gcat : Political/General News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>nyc : New York City | usny : New York | namz : North America | usa : United States | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>PORTS  | SHIPS AND SHIPPING  | LEVY, CLIFFORD J </td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>New York Times Digital (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020050520dr8u00098</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020050403dr8r001nu" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Metropolitan Desk; 1</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>The PATH War; Giuliani's Call for a Fare Increase Provokes Anger and Veiled Threats  </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By CLIFFORD J. LEVY  </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>969 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>27 August 1995</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>34</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 1995 The New York Times Company.  All Rights Reserved.  </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >When Mayor Rudolph W. Giuliani called for a fare increase on the PATH commuter line last week, opening a new front in the border wars with New Jersey, it almost seemed as if he were complaining about a railway whose route was entirely west of the Hudson and never entered New York City.  </p>
<p class="articleParagraph enarticleParagraph" >The PATH does come into the city, of course, making nearly as many stops in Manhattan as in New Jersey. So if the system provides service in both states, why does Mr. Giuliani keep attacking it and its owner, the Port Authority of New York and New Jersey, declaring that the city is being shortchanged? In other words, who actually benefits from the PATH?  </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >Port Authority officials and transportation experts disagree with Mr. Giuliani's assertion that the Port Authority Trans-Hudson is essentially a New Jersey institution. Although they acknowledge that 85 percent of PATH riders are New Jersey residents, they ask whether Mr. Giuliani would prefer that the 65,000 or so commuters who take PATH daily to Manhattan drive instead.  </p>
<p class="articleParagraph enarticleParagraph" >"If you didn't have PATH, think about the kind of traffic and congestion that would create," said John J. Haley Jr., the deputy executive director of the Port Authority.  </p>
<p class="articleParagraph enarticleParagraph" >New Jersey officials questioned how Mr. Giuliani could say he was determined to revitalize the office market in Lower Manhattan when he wanted to raise the fare on the PATH, which brings tens of thousands of commuters to its World Trade Center station every weekday.  </p>
<p class="articleParagraph enarticleParagraph" >And transportation experts noted that New Jersey commuters are taxed by New York City and State and spend millions of dollars a year on everything from clothes to restaurant meals to Broadway shows. In 1993, the most recent year for which figures are available, New Jersey residents paid $910 million in taxes to New York State and $63 million to New York City, officials said.  </p>
<p class="articleParagraph enarticleParagraph" >There are no statistics on what percentage of those amounts is paid by PATH commuters, but the figures are substantial. One out of every three New Jersey residents who come into jobs in the central business district in Manhattan, from Battery Park to 59th Street, uses PATH.  </p>
<p class="articleParagraph enarticleParagraph" >"We all benefit, both sides of the river, from a good transportation system, and the PATH is an essential part of the region's transportation system," said H. Claude Shostal, president of the Regional Plan Association, which studies land-use issues. "We are one region, one economy. If PATH would disappear, both sides of the river would be deeply hurt."  </p>
<p class="articleParagraph enarticleParagraph" >The issue has long roiled relations between the two states, and is again flaring now that tough times are forcing officials like Mr. Giuliani to focus on patching their budgets.  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Giuliani contends that because so many of PATH's users are New Jersey residents, the $200 million annual subsidy that the railway receives from the Port Authority is a gift to New Jersey, draining money from the authority's finances that could go to projects in New York City.  </p>
<p class="articleParagraph enarticleParagraph" >The size of the subsidy, Mr. Giuliani says, also prevents the authority from paying the city more rent for Kennedy International and La Guardia Airports, which the authority leases from the city. By raising the fare from $1 to at least $1.25, he says, the authority could lower the subsidy on the PATH, which runs through Newark, Harrison, Jersey City, Hoboken and Manhattan.  </p>
<p class="articleParagraph enarticleParagraph" >"If I see New Yorkers getting hosed, I am going to fight for them," Mr. Giuliani said, touching off a backlash from Gov. Christine Todd Whitman and other New Jersey officials, who announced that they were against raising the fare and the $4 toll on the Port Authority's bridges and tunnels. Some New Jersey legislators even threatened to retaliate by increasing commuter taxes for New Yorkers who work in New Jersey.  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Giuliani also drew comparisons between the PATH and the New York City subway system, which is operated by the Metropolitan Transportation Authority, a state agency, and receives a smaller subsidy than the PATH in terms of its costs. He said that city residents should not have to pay $1.25, soon to go up to $1.50, while the PATH remains $1.  </p>
<p class="articleParagraph enarticleParagraph" >Even if the fare were raised by 25 cents, it would yield only another $15 million, hardly significant in relation to the Port Authority's or the city's budget. If the tolls on the agency's six bridges and tunnels were also raised, from $4 to $5, that would bring in $140 million.  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Giuliani doubtlessly knew that by demanding a fare increase, he would stir anger in New Jersey. Yet because so few New Yorkers ride the PATH, he faces little prospect of protests from his constituents, and might instead be able to rally some of them with his complaints that the Port Authority favors New Jersey.  </p>
<p class="articleParagraph enarticleParagraph" >Port Authority officials agree that the PATH deficit is too high, but they say they are working on a plan to reduce it by cutting at least $10 million from the railway's $155 million budget without decreasing service. Eventually, they say, fares should cover about 50 percent of the costs of the system, up from the current 25 to 38 percent, depending on the method of calculation.  </p>
<p class="articleParagraph enarticleParagraph" >No mass-transit system in the country earns a profit, and most transportation experts say that a system is successful if it reaches the 50 percent point.  </p>
<p class="articleParagraph enarticleParagraph" >"I am not saying that we don't have to fix things in PATH, but it is not broken," said Peter S. Kalikow, an authority commissioner from New York. "The last thing that we should do is raise fares and tolls."  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>CO</b>&nbsp;</td><td><br/>panynj : Port Authority of New York and New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i721 : Urban/Commuter Transit | i72101 : Urban Mass Transit Systems | i763 : Port/Harbor Operations | irailtr : Road/Rail Transport | iship : Water Transport/Shipping | itsp : Transportation/Shipping</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>ccat : Corporate/Industrial News | gcat : Political/General News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>nyc : New York City | usny : New York | namz : North America | usa : United States | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>News Analysis  | TRANSIT SYSTEMS  | PRICES (FARES, FEES AND RATES)  | PORT AUTHORITY TRANS-HUDSON CORP (PATH)  | LEVY, CLIFFORD J  | GIULIANI, RUDOLPH W (MAYOR) </td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>New York Times Digital (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020050403dr8r001nu</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020050403dr8j00ada" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Metropolitan Desk; 1</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>New Jersey and New York Argue Over Raising Port Authority Tolls  </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By CLIFFORD J. LEVY  </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>1595 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>19 August 1995</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>1</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 1995 The New York Times Company.  All Rights Reserved.  </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >In a dispute that could cause tensions between their Republican Governors, New York and New Jersey are increasingly at odds over whether to raise the tolls on six bridges and tunnels that connect the two states, as well as the fare on the PATH commuter railway.  </p>
<p class="articleParagraph enarticleParagraph" >Officials in New Jersey are lining up against a higher PATH fare and tolls, fearful of angering their constituents, who use these services more than New Yorkers do. But many of their New York counterparts favor raising at least some of these fees, which, they maintain, are being kept artificially low.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >The issue is flaring now because of a budget deficit at the Port Authority of New York and New Jersey, the bistate agency that runs the PATH and the six crossings: the George Washington Bridge, the Lincoln and Holland tunnels and the three bridges between Staten Island and New Jersey.  </p>
<p class="articleParagraph enarticleParagraph" >The Port Authority tolls have not risen since 1991, and the PATH, or Port Authority Trans-Hudson, fare has been $1 since 1987. And earlier this year, just before he left office, the authority's executive director, Stanley Brezenoff, predicted that there would have to be increases if the agency were to solve its budget problems.  </p>
<p class="articleParagraph enarticleParagraph" >Many New York officials say they are acutely aware of how long the Port Authority tolls and PATH fare have stayed put, even as subway and bus fares and other tolls in New York City keep rising.  </p>
<p class="articleParagraph enarticleParagraph" >"It seems to me only fair that there should be some level of parity between these systems," said Melvin L. Schweitzer, a Port Authority board member from New York who is a strong supporter of raising the tolls and fare.  </p>
<p class="articleParagraph enarticleParagraph" >Another New York board member, who spoke on the condition of anonymity, also endorsed raising the tolls, though he was more hesitant about changing the PATH fare, saying it might reduce ridership. There are six board members from each state, and some have not announced their positions. But people who are familiar with the board's deliberations say the New York side is leaning more toward increases than the New Jersey side.  </p>
<p class="articleParagraph enarticleParagraph" >Many New Jersey officials reject any link between the policies of the Port Authority and those of the Metropolitan Transportation Authority, a New York State agency, which recently moved to raise subway and bus fares in New York City by 25 cents to $1.50, and round-trip tolls on its major bridges and tunnels by $1 to $7. By comparison, the toll at the bridges and tunnels run by the Port Authority is $4, paid by drivers entering New York.  </p>
<p class="articleParagraph enarticleParagraph" >"Whatever we do is independent of any other agency, either in New York or New Jersey," said the Port Authority chairwoman, Kathleen A. Donovan of New Jersey. "Just because fares go up in New York, that doesn't mean we want to raise fares."  </p>
<p class="articleParagraph enarticleParagraph" >The debate over the increases, always politically wrenching, could drag on for months, pitting officials from the two states against each other and drawing in groups representing drivers, mass-transit advocates, environmentalists, industry and others. It could be settled quietly and without rancor, but if the past is any guide, that is unlikely.  </p>
<p class="articleParagraph enarticleParagraph" >Gov. George E. Pataki of New York and Gov. Christine Todd Whitman of New Jersey, either of whom could veto an increase approved by the authority's board, have not announced where they stand yet. But interviews with their senior aides suggest that the Governors' views may be diverging.  </p>
<p class="articleParagraph enarticleParagraph" >The Governors have already had one disagreement over the Port Authority. In January, Mr. Pataki angered Mrs. Whitman when he named George J. Marlin to be Mr. Brezenoff's successor without first consulting with her.  </p>
<p class="articleParagraph enarticleParagraph" >Many New York officials complain that even though the subway and bus fare is to be $1.50 under the M.T.A.'s recent proposal, the PATH fare is still $1 because PATH is far more subsidized than the subways.  </p>
<p class="articleParagraph enarticleParagraph" >Like some other New York officials, Mr. Schweitzer said the authority had to find a way to reduce the deficit on the PATH, which loses about $200 million a year.  </p>
<p class="articleParagraph enarticleParagraph" >"Why is it that we have to continue to subsidize the PATH to such an extent?" asked Mr. Schweitzer, who contends that the rail line drains money that the Port Authority could use for much-needed projects in both states.  </p>
<p class="articleParagraph enarticleParagraph" >His arguments were echoed by Deputy Mayor John S. Dyson of New York City. He said that he had consulted with many of the six New York members of the Port Authority's board and that they were at the very least receptive to an increase in the PATH fare.  </p>
<p class="articleParagraph enarticleParagraph" >"Our view is that the PATH train price ought to be the same as the city subways'," Mr. Dyson said. "The plain strategic fact of the matter is unless this disproportionate PATH subsidy is changed drastically there will never be money to do the things that we need done on this side of the river."  </p>
<p class="articleParagraph enarticleParagraph" >And the senior Pataki official who oversees the Port Authority, Louis R. Tomson, said the administration could not ignore the differences between the M.T.A.'s fees and the Port Authority's.  </p>
<p class="articleParagraph enarticleParagraph" >"It is a large disparity and the question then is whether that disparity is justified," he said. "If the subway fare goes to $1.50, it is going to cause everyone to look harder at the PATH fare."  </p>
<p class="articleParagraph enarticleParagraph" >Depending on how the calculations are done, fares currently cover 47 percent to 59 percent of the cost of the subway and bus system, and 25 percent to 38 percent of the cost of the PATH, officials said. The percentage figure for the New York City system will go up if the increase to $1.50 receives final approval.  </p>
<p class="articleParagraph enarticleParagraph" >New Jersey officials, in addition to leaders of environmental and transportation groups, point out that PATH, which runs from Newark to Jersey City to Manhattan, benefits New York by giving commuters a cheap way of reaching jobs, particularly on Wall Street. It also provides an alternative to driving, reducing congestion and pollution.  </p>
<p class="articleParagraph enarticleParagraph" >And they say raising the PATH fare by 25 cents, to $1.25, would be largely symbolic, generating only about $15 million a year, hardly a dent in the Port Authority's $2.7 billion budget. A PATH increase coupled with a $1 increase on the tolls would yield an extra $140 million a year, officials said.  </p>
<p class="articleParagraph enarticleParagraph" >Ed Lloyd, general counsel for the Tri-State Transportation Campaign, a coalition of advocacy groups, said New York officials who favored a rise in the PATH fare were using skewed logic. The subsidy for PATH is in line with those for similar mass-transit systems around the nation, he said, while the subsidy for New York's subway and bus system is the lowest subsidy for any major system in the country.  </p>
<p class="articleParagraph enarticleParagraph" >"There is a sort of false notion of parity," Mr. Lloyd said. "New York is making a bad policy decision in increasing subway fares, and now they are looking for political cover by saying you should increase PATH as well."  </p>
<p class="articleParagraph enarticleParagraph" >The last time the authority considered an increase, it became entangled in a battle that lasted two years. From 1989 to 1991, two New Jersey Governors, Thomas H. Kean and Jim Florio, repeatedly rejected an increase as politically unpalatable. Finally, Mr. Florio relented halfway, allowing the tolls to rise to $4 from $3 but refusing to budge on the PATH fare.  </p>
<p class="articleParagraph enarticleParagraph" >Governor Whitman, who has made her mark by cutting her state's income taxes by 30 percent, has said she considers raising any kind of toll or fare akin to a tax increase. Mrs. Whitman, as well as Mrs. Donovan and some other Port Authority board members from New Jersey, say they would give their approval only as a last resort after the authority first tried to close its budget gap by cutting spending.  </p>
<p class="articleParagraph enarticleParagraph" >The authority is now seeking to do that by starting a hiring freeze and putting together plans that could end up trimming its staff of 9,000 workers by 5 percent to 10 percent, officials said.  </p>
<p class="articleParagraph enarticleParagraph" >But unless the authority slices off significant parts of its budget, many of its costs, particularly those tied to labor, will keep growing even as revenues from tolls and fares are staying relatively flat. In other words, minor cuts may not solve a structural problem in the budget, so the issue of an increase may continue to dog the agency.  </p>
<p class="articleParagraph enarticleParagraph" >At the same time, though, if Governor Whitman vetoes an increase this year, she is not likely to approve one next year because it is too close to 1997, when she has to run for re-election.  </p>
<p class="articleParagraph enarticleParagraph" >The position of Governor Pataki, who is in his first year, is not known, and if his role in the the debate over the M.T.A. decision is any indication, he may choose to remain on the sidelines. It is a political rule of thumb that elected officials often agree to raising fees in their first year of office because by the time re-election rolls around, voters may have forgotten about it.  </p>
<p class="articleParagraph enarticleParagraph" >Chart: "GETTING AROUND: A Closer Look at the Port Authority" shows revenues from Port Authority bridges, tunnels and Path in 1994, fares for the New York City subway system and round-trip tolls for area crossing from 1970 to 1995. (Source: Port Authority of New York and New Jersey, Metropolitan Transit Authority)(pg. 22)  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>CO</b>&nbsp;</td><td><br/>panynj : Port Authority of New York and New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i721 : Urban/Commuter Transit | i72101 : Urban Mass Transit Systems | i763 : Port/Harbor Operations | irailtr : Road/Rail Transport | iship : Water Transport/Shipping | itsp : Transportation/Shipping</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>ccat : Corporate/Industrial News | gcat : Political/General News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>nyc : New York City | usny : New York | namz : North America | usa : United States | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>BRIDGES AND TUNNELS  | TOLLS  | TRANSIT SYSTEMS  | PORT AUTHORITY TRANS-HUDSON CORP (PATH)  | PORT AUTHORITY OF NY AND NJ  | LEVY, CLIFFORD J </td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>New York Times Digital (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020050403dr8j00ada</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020050402dr7j00o6s" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Metropolitan Desk; B</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>Fire at a Substation Closes 2 PATH Lines  </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By NORIMITSU ONISHI  </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>863 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>19 July 1995</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>2</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 1995 The New York Times Company.  All Rights Reserved.  </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >A fire in a PATH train substation in lower Manhattan yesterday morning knocked out power to two lines, trapped three trains under the Hudson River for at least 90 minutes and disrupted the trips of thousands of commuters who use the service to shuttle between New Jersey and Manhattan.  </p>
<p class="articleParagraph enarticleParagraph" >The fire caused sufficient damage to the substation's power cables that officials could not say if service would resume today on the lines, between Hoboken Station and the 33d Street Station, at Herald Square, and between Journal Square Station in Jersey City and the 33d Street Station. The interruption added to the general confusion of commuters to Manhattan who, in recent days, have also had to be concerned with a potential strike by the Metro-North Commuter Railroad.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >"We're asking riders to check news reports in the morning before leaving home," said Gwen Williams, a spokeswoman for the Port Authority of New York and New Jersey, which owns PATH, short for Port Authority Trans Hudson.  </p>
<p class="articleParagraph enarticleParagraph" >The cause of the fire at the substation, on Christopher Street near Greenwich Street, was not deemed suspicious, said George Semidey, a spokesman for the Fire Department.PATH officials said they knew of nothing unusual going on at the time and were still investigating the cause of the fire, which was extinguished in about an hour by 60 firefighters.  </p>
<p class="articleParagraph enarticleParagraph" >The two disrupted lines carry 38,000 passengers a day, a total of 76,000 rides, Ms. Williams said, adding that the PATH trains' four lines transport 104,500 people a day.  </p>
<p class="articleParagraph enarticleParagraph" >The fire was reported shortly before 9 A.M., after the peak of the rush hour. Port Authority officials estimated that it had disturbed the travel of 3,000 to 5,000 riders; most of them had to take PATH trains to the World Trade Center terminal and take the subway uptown.  </p>
<p class="articleParagraph enarticleParagraph" >But the fire left three trains trapped under the Hudson River for more than 90 minutes. Shortly before 11, they were pulled out by work trains, Ms. Williams said.  </p>
<p class="articleParagraph enarticleParagraph" >Kris Feeny, a senior buyer for Amway in the garment district, said she boarded the Journal Square-Herald Square train at 8:30 and arrived at work after 11 A.M., spending much of that time under the river.  </p>
<p class="articleParagraph enarticleParagraph" >Although the power disabled the train's air-conditioning and the cars grew increasingly hot, she said most of the commuters had demonstrated button-down, starched-shirt stoicism and propriety.  </p>
<p class="articleParagraph enarticleParagraph" >"It was very hot," she said. "It felt like 100 degrees. They told us there was a fire, and we were put on backup electricity. It happens."  </p>
<p class="articleParagraph enarticleParagraph" >She added that being trapped had produced an unexpected benefit: the commuters, who usually maintain a stony silence behind newspapers, began chatting. "One couple talked the whole time," she said, recalling an overheard conversation, "and were going to go out together."  </p>
<p class="articleParagraph enarticleParagraph" >Wuzzy Acosta, who was stuck on the same train, said, "Everybody was very calm, but as soon as they got off the train, they went running to the phones."  </p>
<p class="articleParagraph enarticleParagraph" >Last night's return to New Jersey was also hectic for the thousands who depend on the two lines. Port Authority officials recommended that passengers who usually take the two lines from Herald Square to New Jersey ride the subway instead to the World Trade Center terminal. There, they could catch the PATH's two other lines into New Jersey, which also stop in Hoboken and Jersey City.  </p>
<p class="articleParagraph enarticleParagraph" >At Herald Square, many regular PATH riders complained that they had been left in the dark about the service interruption. Many learned of it after they had gone down to the 33d Street Station's lower level, where PATH officials instructed them to turn around and catch the subway to the World Trade Center.  </p>
<p class="articleParagraph enarticleParagraph" >"Instead of one train, I have to take three," said Barry Mentzer, who works in the garment district and lives in Hoboken. "It's really terrible. Plus, they don't tell you until you get here. I was feeling for the people on Metro-North, and then I come here."  </p>
<p class="articleParagraph enarticleParagraph" >Many were also upset because of the time it took to buy a subway token; they had to wait in a line that snaked around a corner and reached the stairs to the lower level.  </p>
<p class="articleParagraph enarticleParagraph" >"I think PATH should pay for the token," said Marie Cologrande, who lives in Middletown, N.J.  </p>
<p class="articleParagraph enarticleParagraph" >Laura Messersmith, a Rockaway, N.J., resident who has commuted to Manhattan since December, said she was afraid to take the city subway. "I've got to ride the subway, and I've never ridden a subway before," she said. "It's scary. I have to go to a whole new place."  </p>
<p class="articleParagraph enarticleParagraph" >Three people, including a car inspector who first reported the fire, were treated and released at St. Vincent's Hospital and Medical Center for smoke inhalation.  </p>
<p class="articleParagraph enarticleParagraph" >Photo: PATH commuters heading for New Jersey were turned away from the 33d Street Station yesterday afternoon because of damage from a fire, and told to go to the World Trade Center terminal. (Jose R. Lopez/The New York Times) Map of Manhattan  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>CO</b>&nbsp;</td><td><br/>panynj : Port Authority of New York and New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i721 : Urban/Commuter Transit | i72101 : Urban Mass Transit Systems | i763 : Port/Harbor Operations | irailtr : Road/Rail Transport | iship : Water Transport/Shipping | itsp : Transportation/Shipping</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>gdis : Disasters/Accidents | ccat : Corporate/Industrial News | gcat : Political/General News | gcon : Consumer Affairs | glife : Living/Lifestyle | grisk : Risk News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>nyc : New York City | usny : New York | namz : North America | usa : United States | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>FIRES AND FIREMEN  | TRANSIT SYSTEMS  | ACCIDENTS AND SAFETY  | DELAYS (TRANSPORTATION)  | PORT AUTHORITY TRANS-HUDSON CORP (PATH)  | ONISHI, NORIMITSU </td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>New York Times Digital (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020050402dr7j00o6s</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020050403dr6i00pf4" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>The City Weekly Desk; 13</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>F.Y.I.   </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By JESSE McKINLEY   </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>685 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>18 June 1995</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>2</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 1995 The New York Times Company.  All Rights Reserved.   </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >Art for Subway's Sake   </p>
<p class="articleParagraph enarticleParagraph" >Q. In the Christopher Street station on the 1/9 line there are several evocative mosaics. On the uptown side there are two series, and the images they portray are quite odd: one has what resembles a Wayne Newton-as-Lady-Liberty-in-the-3d-ring-of-Hell feel. On what themes were these mosaics hinged, and have I completely missed the point?   </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >A. The four triptychs adorning the walls of the Christopher Street station are the work of Lee Grozgold, an artist who came up with the themes, and the students of Public School 41, who did the sketches on which the mosaics are based. Mr. Grozgold divided the panels into four categories of hero: Founders, Providers (both on the downtown platform), Bohemians and Rebels (uptown).   </p>
<p class="articleParagraph enarticleParagraph" >Each triptych represents a grouping of notables from throughout history that fit into one of these four categories. Thus, your Wayne Newton in Hell is actually a depiction of the Revolutionary rebel Thomas Paine set against the backdrop of the 1969 Stonewall Riots.   </p>
<p class="articleParagraph enarticleParagraph" >"It all makes absolute sense," said Wendy Feuer, director of M.T.A. Arts for Transit, which sponsored the project. But for the less abstract-minded, Ms. Feuer said, explanatory plaques will go up next to each piece. Buses Without Riders   </p>
<p class="articleParagraph enarticleParagraph" >Q. When public buses display the "No passengers" or "Not in service" signs, it is frustrating for would-be passengers not to know the reasons, especially when the buses pass by during rush hour. Why don't they take passengers along? A. The buses are probably "deadheading," traveling empty from their depot to the start of their regular route or back to the depot at the end of their shift, Melissa Farley, a spokeswoman for the Transit Authority, said.   </p>
<p class="articleParagraph enarticleParagraph" >Why is this necessary? The city has 19 bus depots scattered throughout the city where buses are stored and maintained. Keeping enough buses on the street sometimes requires elaborate maneuvering. (One deadheading route involves driving from a depot in Staten Island to West 57th Street in Manhattan.)   </p>
<p class="articleParagraph enarticleParagraph" >They don't carry passengers because it would slow the process and defeat the purpose of the trip. (Besides, imagine how long it would take to get from 57th Street to Staten Island aboard a local.) Four Mystery Letters   </p>
<p class="articleParagraph enarticleParagraph" >Q. How did the PATH system get its name? I assume it's an acronym involving the Port Authority, but I can't figure the last two letters.   </p>
<p class="articleParagraph enarticleParagraph" >A. The PATH system was named in 1962 after the Port Authority took over the ailing Manhattan-Hudson Railroad (once nicknamed "the haul and maul").   </p>
<p class="articleParagraph enarticleParagraph" >Lloyd D. Schwalb, a spokesman for the Port Authority, said planners mulled over several names (the Port Authority Railroad, or PARR, definitely would not do) before one moved them: the Port Authority Trans Hudson. The Levis Guy Leaves   </p>
<p class="articleParagraph enarticleParagraph" >Q. What happened to that huge painting of the guy in Levis on West 34th Street between Fifth Avenue and the Avenue of the Americas? A. The sign was painted over on May 21 after the 34th Street Partnership complained about its size to the Mayor's Office of Midtown Enforcement, which monitors zoning in the area and issues summonses to violators.   </p>
<p class="articleParagraph enarticleParagraph" >Regulations bar advertising signs larger than 200 square feet or more than 25 feet above the curb in the neighborhood, said Lawrence Howard, zoning specialist for the partnership, a business improvement coalition. The Levis mural violated both rules.   </p>
<p class="articleParagraph enarticleParagraph" >The sign was painted in 1992 as an advertisement for County Seat, the store directly below it, at 22 West 34th Street. The Partnership complained as it was being painted, but only recently did the store respond to summonses.   </p>
<p class="articleParagraph enarticleParagraph" >"We always liked it," said Matthew Knopf, general counsel for County Seat, "But we wanted to be good citizens, so we took it down."   </p>
<p class="articleParagraph enarticleParagraph" >Besides the size, the Partnership had other concerns. Mr. Howard said the suggestiveness of the ad -- a view of a man from behind -- "was not keeping with the image we'd like to bring to 34th Street." JESSE McKINLEY   </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i721 : Urban/Commuter Transit | i72102 : Bus/Coach Services | irailtr : Road/Rail Transport | itsp : Transportation/Shipping</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>nsum : News Digest | ncat : Content Types | nfact : Factiva Filters | niwe : IWE Filter</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>nyc : New York City | usa : United States | usny : New York | namz : North America | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>Summary </td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>New York Times Digital (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020050403dr6i00pf4</td></tr></table><br/></div></div><br/><span></span><div id="article-rec0000020011030dqbf01fuo" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/recLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>NEWS</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>WORKERS CAN SUE PATH, COURT RULES </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>The Associated Press </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>336 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>15 November 1994</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The Record</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>REC</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>© 1994 North Jersey Media Group Inc. Provided by ProQuest Information and Learning. All rights reserved.
 </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >Injured workers can sue a subsidiary of the Port Authority of New York and New Jersey in federal court, the Supreme Court ruled Monday. </p>
<p class="articleParagraph enarticleParagraph" >The authority is vulnerable to such lawsuits because the two states that created it are not required to pay its debts, the court said in a 5-4 ruling. </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >The Constitution's 11th Amendment generally shields states from being sued in federal court without their consent. But Justice Ruth Bader Ginsburg wrote for the court that the Port Authority Trans-Hudson Corp., as an agency created by two states, is not entitled to such immunity. </p>
<p class="articleParagraph enarticleParagraph" >"The Port Authority is financially self-sufficient; it generates its own revenues and it pays its own debts," Ginsburg wrote. Allowing it to be sued in federal court does not address the 11th Amendment's aim of protecting "the states' solvency and dignity," she said. </p>
<p class="articleParagraph enarticleParagraph" >Monday's ruling reinstates two workers' lawsuits against PATH, a wholly owned subsidiary of the Port Authority, which was created by a compact between New York and New Jersey. </p>
<p class="articleParagraph enarticleParagraph" >Albert Hess, a train engineer, suffered a hand injury in a workplace accident June 29, 1987. Charles F. Walsh, a conductor, was injured Oct. 6, 1986, when a train door slammed into his wrist and arm. </p>
<p class="articleParagraph enarticleParagraph" >Both sued PATH, contending their injuries resulted from the railroad's negligence. </p>
<p class="articleParagraph enarticleParagraph" >New Jersey law waives the state's immunity against lawsuits filed in federal court within one year after a claim arises. Hess and Walsh missed the one-year deadline but filed their lawsuits within a three-year limit set by federal law. </p>
<p class="articleParagraph enarticleParagraph" >Federal judges in New Jersey dismissed their lawsuits and the 3rd U.S. Circuit Court of Appeals agreed, saying PATH was entitled to immunity under the 11th Amendment. </p>
<p class="articleParagraph enarticleParagraph" >Another appellate court, the 2nd U.S. Circuit Court of Appeals, had ruled in a separate case that PATH was not entitled to such immunity. Monday, the Supreme Court agreed with the 2nd Circuit court.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>NEW JERSEY | NEW YORK CITY | TRANSIT | EMPLOYMENT | LAWSUIT | INJURY | USA | COURT</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>Record / Hackensack, New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document rec0000020011030dqbf01fuo</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020050409dqbf018ym" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Metropolitan Desk; A</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>Court Reopens Demotion Case At City College  </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By LINDA GREENHOUSE  </td></tr>
<tr><td align="right" valign="top" class="index"><b>CR</b>&nbsp;</td><td>Special to The New York Times  </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>1343 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>15 November 1994</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>1</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 1994 The New York Times Company.  All Rights Reserved.  </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >WASHINGTON, Nov. 14 -- The United States Supreme Court today gave City College of New York a second chance to show that its demotion of the outspoken chairman of its black studies department did not violate the Constitution.  </p>
<p class="articleParagraph enarticleParagraph" >In a two-sentence order, issued without an opinion, the Justices vacated a ruling last April by the Federal appeals court in Manhattan that the college had violated Prof. Leonard Jeffries's First Amendment right to free speech when it refused in 1992 to reappoint him to the departmental position he had held for 20 years.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >The Justices ordered the appeals court to reconsider the case in light of a Supreme Court decision, issued in May, that may give public employers more latitude to dismiss employees whose speech, while constitutionally protected, is likely to disrupt the workplace. The court's action today did not necessarily indicate that the Justices had come to any conclusion about the merits of the Jeffries case.  </p>
<p class="articleParagraph enarticleParagraph" >But whether today's action changes the ultimate outcome of the case or simply postpones it, the Court in effect breathed new life into a highly visible dispute that for more than two years has been at the heart of a nationwide debate over the boundaries of academic freedom.  </p>
<p class="articleParagraph enarticleParagraph" >City College removed Dr. Jeffries as chairman of the black studies department after he gave a speech that accused "rich Jews" of having financed the slave trade and, in concert with the Mafia, of having produced Hollywood movies aimed at the "destruction of black people."  </p>
<p class="articleParagraph enarticleParagraph" >Jay Hershenson, a City University vice president, said today that the university was encouraged by the High Court's action. "We welcome further judicial review because we believe the City College administration acted in the best interests of City College, consistent with constitutional requirements," he said.  </p>
<p class="articleParagraph enarticleParagraph" >Dr. Jeffries said he had expected the case to continue. "We're prepared for the legal fight," he said.  </p>
<p class="articleParagraph enarticleParagraph" >After his demotion, Dr. Jeffries remained a tenured professor, with no loss of salary or benefits, until August 1993, when Judge Kenneth Conboy of Federal District Court in Manhattan ordered City College to reinstate him as department chairman. Although the college asserted that it was his poor administrative record and not his speech that led to his demotion, a jury in Federal District Court found otherwise and awarded Dr. Jeffries $360,000 in damages.  </p>
<p class="articleParagraph enarticleParagraph" >The appeals court, while upholding the reinstatement order, set aside the damage award and ordered a new trial on the amount of damages, a question that was not before the Supreme Court today.  </p>
<p class="articleParagraph enarticleParagraph" >As City College framed the question in its Supreme Court appeal, Harleston v. Jeffries, No. 94-112, the issue was "whether the First Amendment compels a university to retain, in a position of leadership, a person who has engaged in hate speech."  </p>
<p class="articleParagraph enarticleParagraph" >The college's Supreme Court brief, prepared by the New York State Attorney General's office, asked the Justices to decide whether "a person who has publicly engaged in speech thought to be detrimental to the purpose and nature of a public university" had a First Amendment right to retain his position. "This case challenges the authority of university administrators to decide who may speak for the university," the brief said.  </p>
<p class="articleParagraph enarticleParagraph" >In urging the Court not to review the case, Dr. Jeffries's lawyer, Joseph Fleming, said that no harm had come to City College as a result of the speech, which Dr. Jeffries made in July 1991 to the Empire State Black Arts and Cultural Festival in Albany.  </p>
<p class="articleParagraph enarticleParagraph" >"What did happen was that there was an increase in the level and intensity of research and debate following the Speech," Mr. Fleming told the Court. "This, one would think, would be welcomed in an academic institution."  </p>
<p class="articleParagraph enarticleParagraph" >The Court's decision today, neither to hear the case itself nor to let the appeals court's ruling stand, was a surprising disposition of a case that the Justices had kept under active consideration since the term began six weeks ago.  </p>
<p class="articleParagraph enarticleParagraph" >It had the earmarks of a compromise between Justices who argued for taking the case and those who wanted to deny it. Given the pace at which the Court reviews the more than 7,000 appeals that come in every year, it is unusual for a case to linger for more than two or three weeks without being acted on.  </p>
<p class="articleParagraph enarticleParagraph" >The United States Court of Appeals for the Second Circuit in Manhattan will not necessarily have an easy time in carrying out the Justices' instruction to reconsider its earlier decision. While the intervening Supreme Court decision, Waters v. Churchill, bears on the question of public employees' First Amendment rights, it arose in a very different context from the Jeffries dispute.  </p>
<p class="articleParagraph enarticleParagraph" >The Court was deeply split in the Waters case, which concerned a public hospital's dismissal of a nurse who had criticized a training program. A Federal appeals court ruled in the nurse's favor in her First Amendment suit against the hospital, ruling that despite the hospital administrators' concern, there was no evidence that the criticism had disrupted the workplace.  </p>
<p class="articleParagraph enarticleParagraph" >The Supreme Court, in a plurality opinion by Justice Sandra Day O'Connor, vacated that decision on the ground that the administrators' "reasonable predictions" of disruption should have received more weight in balancing the employee's rights against the employer's needs.  </p>
<p class="articleParagraph enarticleParagraph" >City College cited the Waters case in its Supreme Court appeal, telling the Justices that the decision's message was that university administrators should prevail in lawsuits like Dr. Jeffries's "where they were motivated by a reasonable expectation that the speaker's words would harm the university."  </p>
<p class="articleParagraph enarticleParagraph" >On campus, several teachers and students interviewed today said that they were not surprised by the court's decision. "The Supreme Court was stacked against Jeffries," said Barry Solomon-Lufti, 49, who teaches science.  </p>
<p class="articleParagraph enarticleParagraph" >"It doesn't take a pound of intelligence to know there is a thing called free speech, but it's not applied to black people," said Mr. Solomon-Lufti, who is black.  </p>
<p class="articleParagraph enarticleParagraph" >Others, though, said they believed the issues involved were not clear-cut. "I don't think Jeffries should have been kicked out for what he said, but I am kind of confused about whether his right to free speech was violated," said Lisa Seay, 27, an education major who is black.  </p>
<p class="articleParagraph enarticleParagraph" >The Court also took these other actions today in a busy day for the New York area: Port Authority Suits  </p>
<p class="articleParagraph enarticleParagraph" >The Court ruled, 5 to 4, that the Port Authority of New York and New Jersey is not constitutionally entitled to immunity from suit in Federal court.  </p>
<p class="articleParagraph enarticleParagraph" >The decision revived negligence suits against the Port Authority Trans-Hudson Corporation by two PATH employees who were injured in separate accidents.  </p>
<p class="articleParagraph enarticleParagraph" >The Justices took this case, Hess v. Port Authority, No. 93-1197, to resolve a conflict between two Federal appeals courts over the immunity issue. The constitutional provision at issue was the 11th Amendment, which bars suits by individuals against states in Federal court.  </p>
<p class="articleParagraph enarticleParagraph" >Writing for the majority today, Justice Ruth Bader Ginsburg said that the Port Authority's financial independence from New York and New Jersey -- which have no legal liability for its debts -- meant that there was no reason for it to share the immunity the states themselves would enjoy. Real Estate Solicitations  </p>
<p class="articleParagraph enarticleParagraph" >Without comment, the Court refused to hear New York State's appeal of a ruling that its regulations aimed at preventing blockbusting in racially mixed neighborhoods were unconstitutional.  </p>
<p class="articleParagraph enarticleParagraph" >The Second Circuit ruled in June that orders issued by Secretary of State Gail S. Shaffer barring real estate agents from soliciting business in specified portions of New York City and Nassau County violated the real estate agents' right to commercial speech. This case was Shaffer v. New York State Association of Realtors, No. 94-530.  </p>
<p class="articleParagraph enarticleParagraph" >Photo: Prof. Leonard Jeffries, who sued over a decision by City College to demote him in 1992. (Chester Higgins Jr./The New York Times)  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>gvsup : Judicial Branch | gcat : Political/General News | gcrim : Crime/Courts | ghome : Law Enforcement | grel : Religion | gcns : National Security | gcom : Society/Community/Work | gpir : Politics/International Relations | gpol : Domestic Politics | gvbod : Government Bodies</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>nyc : New York City | usny : New York | namz : North America | usa : United States | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>BLACKS (IN US)  | ANTI-SEMITISM  | DECISIONS AND VERDICTS  | FREEDOM OF SPEECH  | MOTION PICTURES  | SLAVERY  | JEWS  | CITY COLLEGE OF NEW YORK (CCNY)  | SUPREME COURT (US)  | MAFIA  | JEFFRIES, LEONARD (PROF)  | GREENHOUSE, LINDA </td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>New York Times Digital (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020050409dqbf018ym</td></tr></table><br/></div></div><br/><span></span><div id="article-rec0000020011030dq3m00atn" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/recLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>CLM</b>&nbsp;</td><td>SUPREME COURT</td></tr>
<tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>NEWS</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>INJURED PATH WORKERS TO PRESENT FEDERAL CASE </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>FROM NEWS SERVICE REPORTS </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>181 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>22 March 1994</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The Record</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>REC</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>© 1994 North Jersey Media Group Inc. Provided by ProQuest Information and Learning. All rights reserved.
 </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >Injured railway workers thwarted in their efforts to sue PATH will be able to present arguments to the Supreme Court on whether the Constitution limits their recourse in federal court. </p>
<p class="articleParagraph enarticleParagraph" >The high court said Monday that it will hear arguments by the two employees whose lawsuits against the Port Authority Trans-Hudson Corp. were dismissed by lower courts. </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >Albert Hess of Bayonne, a train engineer, suffered a hand injury in a workplace accident June 29, 1987. Charles F. Walsh of Staten Island, a railroad conductor, was injured Oct. 6, 1986, when a train door slammed into his wrist and arm. Both sued PATH, claiming their injuries resulted from the railroad's negligence. </p>
<p class="articleParagraph enarticleParagraph" >The Constitution's 11th Amendment gives states sovereign immunity against most suits filed in federal court. New Jersey law waives such immunity for lawsuits filed within one year after a claim arises. Hess and Walsh filed their suits more than a year after their accidents, but within a three-year limit set by federal law.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>COLUMN | USA | COURT | RAILROAD | EMPLOYMENT | INJURY | LAWSUIT</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>Record / Hackensack, New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document rec0000020011030dq3m00atn</td></tr></table><br/></div></div><br/><span></span><div id="article-rec0000020011101dpa7019bn" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/recLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>CLM</b>&nbsp;</td><td>OBITUARIES</td></tr>
<tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>NEWS</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>OBITUARIES </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>1354 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>7 October 1993</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The Record</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>REC</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>© 1993 North Jersey Media Group Inc. Provided by ProQuest Information and Learning. All rights reserved.
 </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >JOHN BACHUR SR., 69, of Lodi died Tuesday. Before retiring in 1985, he was a maintenance worker for the New Jersey Department of Transportation. He was an Army veteran of World War II, and a member of the American Legion in Rochelle Park. He was a parishioner of St. Joseph R.C. Church, Lodi. Arrangements: Santangelo Funeral Home, Lodi. </p>
<p class="articleParagraph enarticleParagraph" >CHARLES PAUL BARABAS, 83, of Rutherford, formerly of Jersey City, died Tuesday. Before retiring in 1981, he worked for the Board of Education, Jersey City for 47 years. He was a Navy veteran of World War II. He received a bachelor's degree in 1939 and a master's in 1952, both from Seton Hall University. He was a parishioner of St. Mary R.C. Church, Rutherford. Arrangements: Thomas J. Diffily Funeral Home, Rutherford. </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >GEORGE W. BAUMANN, 88, of West Milford, formerly of Hoboken, died Tuesday. Before retiring 23 years ago, he was a captain for the Hoboken Police Department, where he worked for 39 years. Arrangements: Richards Funeral Home, West Milford. </p>
<p class="articleParagraph enarticleParagraph" >WILLIAM "BUDDY" BECKEL, 69, of East Rutherford, formerly of West New York, died Tuesday. Before retiring in 1985, he was a timekeeper for Bendix Co., Teterboro. He was an Army veteran of World War II. Arrangements: Kohler Funeral Home, Wood-Ridge. </p>
<p class="articleParagraph enarticleParagraph" >ELAINE E. WEISSKIRCHEN BENENATI, 65, of Cliffside Park died Tuesday. Before retiring in August, she was a supervisor for the lockbox division of United Jersey Bank, Hackensack, where she worked for 22 years. She was a member of Trinity Episcopal Church, and Order of the Eastern Star, Chapter 36, both in Cliffside Park. Arrangements: McCorry Bros. Funeral Home, Cliffside Park. </p>
<p class="articleParagraph enarticleParagraph" >LEWIS N. BUCCOLA, 55, of Mendocino, Calif., formerly of Lodi and Saddle Brook, died Tuesday. He was Chief Nurse of the AIDS Ward at San Francisco Medical Center, where he worked for 10 years. He was a graduate of Jersey City College and the Albany Medical Center for Anesthesia. Arrangements: Edward F. Kugler Community Home for Funerals, Saddle Brook. </p>
<p class="articleParagraph enarticleParagraph" >EDWARD BULLENE of Wurtsboro, N.Y., formerly of Northvale and River Vale, died Wednesday. Arrangements: Moritz Funeral Home, Tappan, N.Y. </p>
<p class="articleParagraph enarticleParagraph" >JOHN J. CLIFFORD JR., 46, of New Milford, formerly of Bogota, died Wednesday. He was a car repair man for Port Authority Trans Hudson, where he worked for 19 years. He was a parishioner of St. Joseph R.C. Church, Bogota. Arrangements: Becker Funeral Home, Westwood. </p>
<p class="articleParagraph enarticleParagraph" >FRANKLIN CHASE CONDON, 87, of Tenafly died Sunday. Before retiring, he was a graphic artist. Arrangements: Barrett Funeral Home, Tenafly. </p>
<p class="articleParagraph enarticleParagraph" >CLAIRE DEWEY CONNELLY, 74, of Brick, formerly of Glen Rock and Hoboken, died Tuesday. Before retiring in 1986, she was a registered nurse at Bergen Pines County Hospital, Paramus, where she worked for 10 years, and previously at Christ Hospital, Jersey City. She was a member of the American Nursing Association. She was a parishioner of St. Catharine R.C. Church, Glen Rock, and a member of its Rosary Altar Society. Arrangements: Feeney Funeral Home, Ridgewood. </p>
<p class="articleParagraph enarticleParagraph" >HERMAN DE VOS, 76, of the Whiting section of Manchester Township, formerly of Montville Township and Clifton, died Wednesday. Born in the Netherlands, he came to the United States in 1923. Before retiring in 1986, he co-owned De Vos Nursery, Clifton, where he worked for 40 years. Arrangements: Allwood Funeral Home, Clifton. </p>
<p class="articleParagraph enarticleParagraph" >HAROLD F. FANNING, 82, of Maywood, formerly of Jersey City, died Wednesday. Before retiring in 1971, he was a supervisor for Public Service Electric and Gas Co., Secaucus, where he worked for 35 years. He was a Navy veteran of World War II. He was a member of the Golden Age Club, Maywood. Arrangements: Trinka-Faustini Funeral Home, Maywood. </p>
<p class="articleParagraph enarticleParagraph" >ADELINE BAGLIVI FORTE, 80, of Paterson died Wednesday. She was a parishioner of Blessed Sacrament R.C. Church, Paterson, and a member of its Leisure Club and Rosary Society. Arrangements: Scillieri Funeral Home, Paterson. </p>
<p class="articleParagraph enarticleParagraph" >IRENE I. GARRETT of Hackensack died Monday. She had worked for the U.S. Postal Service, Grand Central Station, New York City. She was a member of Mount Olive Baptist Church, Hackensack. Arrangements: Earl I. Jones Funeral Home, Hackensack. </p>
<p class="articleParagraph enarticleParagraph" >ANDREW GOJDA, 78, of Woodbridge, formerly of Passaic and Clifton, died Tuesday. Born in the Ukraine, he came to the United States in 1947. Before retiring in 1980, he worked for , Hoffmann-La Roche Inc., Nutley, for 20 years. He was a parishioner of St. Nicholas Ukranian Catholic Church, Passaic. Arrangements: Bizub-Quinlan Funeral Home, Lexington Avenue, Clifton. </p>
<p class="articleParagraph enarticleParagraph" >MARION WINIFRED GOTTSCHALK, 78, of Winter Haven, Fla., formerly of Englewood, died Monday. Arrangements: Crisp-Coon Funeral Home, Winter Haven. </p>
<p class="articleParagraph enarticleParagraph" >SANTINA "TINA" FOTI HAM, 62, of Mahwah, formerly of Ramsey, died Wednesday. She had been a teacher at Dater School, Ramsey, where she worked for 24 years. He was a graduate of the University of Vermont, and received a master's degree from William Paterson College. She received the Governor's Teacher Recognition Award in 1989. Arrangements: Van Emburgh-Sneider Funeral Home, Ramsey. </p>
<p class="articleParagraph enarticleParagraph" >RALPH DURIE HERING SR., 81, of Park Ridge died Tuesday. Before retiring in 1963, he was a machine operator for Gould's Woven Label Mills, Park Ridge, where he worked for 35 years. He was a member of Park Ridge Golden Age Club. Arrangements: Becker Funeral Home, Westwood. </p>
<p class="articleParagraph enarticleParagraph" >HELEN SMITH HOFFMAN, 83, of Closter died Saturday. She was a member of Tenafly Methodist Church, and the seniors clubs of Closter, Demarest, and Norwood. Arrangements: Moritz Funeral Home, Closter. </p>
<p class="articleParagraph enarticleParagraph" >BARBARA JACKSON JONES, 80, of Mesa, Ariz., formerly of Haworth, died Sept. 25. Arrangements: Melcher Mortuary, Mesa. </p>
<p class="articleParagraph enarticleParagraph" >ISAAC MELNIKOFF, 75, of Fort Lee died Wednesday. He had a men's wear salesman. He was a member of The New Synagogue, Fort Lee. Arrangements: Gutterman-Musicant, Hackensack. </p>
<p class="articleParagraph enarticleParagraph" >LOUIS MONDELLA, 69, of Palisades Park died Wednesday. Before retiring in 1989, he was an upholsterer for Jalen Co., Paterson, where he worked for 40 years. He was an Army veteran of World War II. He was a parishioner of St. Michael R.C. Church, Palisades Park. Arrangements: Volk-Leber Funeral Homes, Palisades Park. </p>
<p class="articleParagraph enarticleParagraph" >ANGELO F. NAVIELLO, 81, of Lyndhurst died Tuesday. Before retiring in 1973, he worked for Pettit Paint Co., Belleville, for six years. Previously, he worked for Koppers Koke, Kearny, for 30 years. He was a parishioner of Sacred Heart R.C. Church, Lyndhurst. Arrangements: Nazare Memorial Home, Lyndhurst. </p>
<p class="articleParagraph enarticleParagraph" >VITO "BILL" PERAGINE JR., 71, of Ridgewood, formerly of Paramus, died Wednesday. Before retiring in 1989, he was a chauffeur for Compaction Systems, the Bronx. An Army veteran of World War II, he received the Purple Heart. He was a parishioner of St. Luke R.C. Church, Ho-Ho-Kus. Arrangements: Vander Plaat Funeral Home, Wyckoff. </p>
<p class="articleParagraph enarticleParagraph" >DOMINICK POLIFRONIO, 57, of Saddle Brook died Tuesday. He owned Hackensack Dairy. He was a parishioner of St. Francis R.C. Church, Hackensack, and was president of Tristate and Interstate Curb and Walks. Arrangements: J. Rocciola Funeral Home, Hackensack. </p>
<p class="articleParagraph enarticleParagraph" >FLORENCE SBARBORI, 73, of Vero Beach, Fla., formerly of Hasbrouck Heights, died Wednesday. She had been a bookkeeper in the banking business. She was a parishioner of St. John of the Cross R.C. Church, Vero Beach, and a member of its Women's Guild. Arrangements: Cox-Gifford Funeral Home, Vero Beach. </p>
<p class="articleParagraph enarticleParagraph" >BARBARA ANN SMITH, 37, of Paterson died Tuesday. She was a nurse's aide at Barnert Hospital, Paterson. She was a volunteer at the Paterson Task Force and a member of Calvary Baptist Church, Paterson. Arrangements: Bragg Funeral Home, Paterson. </p>
<p class="articleParagraph enarticleParagraph" >LARDIE A. TAMBURRO of Fairview died Wednesday. He had been a pipe fitter with Pipe Fitters Union Local 274, where he worked for 70 years. Arrangements: A. K. Macagna Funeral Home, Cliffside Park. </p>
<p class="articleParagraph enarticleParagraph" >THERESA SLEBODNIK TAUB, 93, of Lyndhurst died Wednesday. She was a member of United Methodist Church and the library board of trustees, both in Lyndhurst. Arrangements: Ippolito Stellato Funeral Home, Lyndhurst. </p>
<p class="articleParagraph enarticleParagraph" >MAY B. HIGGINS TRAUBE, 93, of Park Ridge died Wednesday. She was a parishioner of Our Lady of Mercy R.C. Church, Park Ridge, and a member of Daughters of the American Revolution. Arrangements: Becker Funeral Home, Westwood. </p>
<p class="articleParagraph enarticleParagraph" >LULU KRETSCHMANN VAN ZANDT, 95, of Jersey City died Monday. Arrangements: C. C. Van Emburgh, Ridgewood.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>COLUMN | OBIT</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>Record / Hackensack, New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document rec0000020011101dpa7019bn</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020050409dp2r00fa7" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Metropolitan Desk; 1</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>EXPLOSION AT THE TWIN TOWERS: The Overview; BLAST HITS TRADE CENTER, BOMB SUSPECTED; 5 KILLED, THOUSANDS FLEE SMOKE IN TOWERS  </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By ROBERT D. McFADDEN  </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>2534 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>27 February 1993</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>1</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 1993 The New York Times Company.  All Rights Reserved.  </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >An explosion apparently caused by a car bomb in an underground garage shook the World Trade Center in lower Manhattan with the force of a small earthquake shortly after noon yesterday, collapsing walls and floors, igniting fires and plunging the city's largest building complex into a maelstrom of smoke, darkness and fearful chaos.  </p>
<p class="articleParagraph enarticleParagraph" >The police said the blast killed at least five people and left more than 650 others injured, mostly with smoke inhalation or minor burns, but dozens with cuts, bruises, broken bones or serious burns. The police said 476 were treated at hospitals and the rest by rescue and medical crews at the scene.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >The explosion also trapped hundreds of people in debris or in smoke-filled stairwells and elevators of the towers overhead and forced the evacuation of more than 50,000 workers from a trade center bereft of power for lights and elevators for seven hours. No Bomb Fragments Found  </p>
<p class="articleParagraph enarticleParagraph" >The blast, which was felt throughout the Wall Street area and a mile away on Ellis and Liberty Islands in New York Harbor, also knocked out the police command and operations centers for the towers, which officials said rendered the office complex's evacuation plans useless. [ Page 23. ]  </p>
<p class="articleParagraph enarticleParagraph" >James Fox, an assistant director of the Federal Bureau of Investigation in charge of the agency's New York office, said that no bomb fragments were found but that a joint terrorist task force of Federal agents and city detectives had examined the wreckage and believed that a car bomb had caused the explosion.  </p>
<p class="articleParagraph enarticleParagraph" >There was no warning of an impending explosion, Police Commissioner Raymond W. Kelly said. Jack Killorin, a spokesman in Washington for the Treasury Department's Bureau of Alcohol, Tobacco and Firearms, said that after the blast, authorities received at least nine telephone calls claiming responsibility.  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Killorin said the first call was made 15 minutes after the blast to a non-emergency number of a New York Police Department precinct by an individual who mentioned the conflict in Bosnia. He said other claims were made between an hour and several hours after the event by callers who cited that and a variety of other reasons for the attack. He declined to elaborate.  </p>
<p class="articleParagraph enarticleParagraph" >Some law-enforcement officials said an explosion of such size, without a claim of responsibility in advance, might suggest that it went off accidentally.  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Kelly was more oblique about the cause of the blast, saying only that a car bomb or other type of explosive device was not being ruled out.  </p>
<p class="articleParagraph enarticleParagraph" >Four hours after the explosion, a bomb threat forced the evacuation of the Empire State Building in midtown Manhattan, and there were numerous other bomb threats in the city, the police said. But it was unclear if any were related to the World Trade Center explosion or only the macabre work of pranksters.  </p>
<p class="articleParagraph enarticleParagraph" >As the day ended, a series of investigations began -- into the cause of the explosion and its possible perpetrators, and into what went wrong in what many called a botched evacuation, with no alarms and no instructions for thousands caught in dark, smoky stairwells, in stark contrast to carefully laid plans.  </p>
<p class="articleParagraph enarticleParagraph" >Mayor David N. Dinkins, visiting in Osaka, Japan, was notified by City Hall and, in a telephone news conference, called the Fire Department response the largest for any non-natural disaster in the city's history. He said he had spoken with President Clinton and had thanked him for the cooperation of Federal investigators.  </p>
<p class="articleParagraph enarticleParagraph" >The effects of the blast radiated outward, disrupting most non-cable television transmissions throughout the metropolitan area, halting traffic in most of lower Manhattan and PATH train service under the Hudson River from the trade center to New Jersey, and transforming an ordinary Friday in the financial district into an afternoon of turmoil, death and destruction.  </p>
<p class="articleParagraph enarticleParagraph" >On a day of high drama, tragedy and heroism, there were a thousand stories: rescuers digging frantically for victims in the collapsed PATH station under the towers, soot-streaked evacuees groping for hours in the city's tallest buildings, a woman in a wheelchair carried down 66 stories by two friends, a pregnant woman airlifted by helicopter from a tower roof, and the tales of many others stumbling out, gasping for air, terrified but glad to be alive.  </p>
<p class="articleParagraph enarticleParagraph" >And among the most poignant was that of a class of kindergartners from Public School 95 in Gravesend, Brooklyn. Caught on the 107th floor observatory, they took all day to walk down, singing to keep up their spirits.  </p>
<p class="articleParagraph enarticleParagraph" >Many of those who walked down scores of flights from the upper reaches of the trade center towers said there had been no alarm bells and no instructions from building personnel or emergency workers. While little panic was reported, witnesses said confusion reigned in the darkness of crowded stairwells where smoke billowed and unknown dangers lurked below.  </p>
<p class="articleParagraph enarticleParagraph" >Many put moist towels or handkerchiefs to their faces against the smoke. Others, frightened, remained in their offices, hoping for rescue. As smoke seeped in under the doors, some broke windows to get air. Dozens of people, meantime, were trapped for hours in elevators frozen between floors, among them another class of kindergartners from P.S. 95.  </p>
<p class="articleParagraph enarticleParagraph" >The worst fires were extinguished by midafternoon. By then extensive efforts to assist those caught on the upper floors were already well underway. But the trade center, with 250 elevators and miles of corridors and stairways, posed a major challenge and long after dark last night rescue workers continued to search the labyrinth for stragglers and others still trying to get out.  </p>
<p class="articleParagraph enarticleParagraph" >On a day of confusion, the police and the Emergency Medical Service repeatedly revised the number of people killed by the blast. By early evening the police said five had been killed while the medical service said seven were dead. Several hours later the police increased the number to seven, but shortly after 11 P.M., the police scaled back the figure to five, saying they could not confirm the medical service's report of seven fatalities. There was no clear explanation for the discrepancies.  </p>
<p class="articleParagraph enarticleParagraph" >The blast, which erupted at 12:18 P.M. on the second level of a four-story underground parking garage beneath the trade center's 110-story twin towers and the complex's Vista Hotel, sent cars hurtling like toys, blew out a 100-foot wall and sent the floor collapsing down several stories, creating a crater 60 feet wide that reached deep into the bowels of the parking complex. 'Everything Was on Fire'  </p>
<p class="articleParagraph enarticleParagraph" >It also collapsed the ceiling of a mezzanine in the adjacent Port Authority Trans-Hudson train station, leaving dozens trapped under rubble on a concourse one floor above the platforms where hundreds awaited trains. Witnesses and rescue workers told of a blast of incredible force -- of bodies hurtling through the air, of cars wrapped around pillars, of people burning and scores trapped.  </p>
<p class="articleParagraph enarticleParagraph" >"We crawled under pipes when we arrived and everything was on fire," said Edward Bergen, a 38-year-old firefighter who was one of the first to reach the scene of the blast. "Suddenly, a guy came walking out of the flames, like one of those zombies in the movie, 'The Night of the Living Dead.' His flesh was hanging off. He was a middle-aged man."  </p>
<p class="articleParagraph enarticleParagraph" >Fire Capt. Timothy Dowling, of Engine Company 6, recalled a ghastly scene of fires lapping in the darkness, illuminating a smoking hell of twisted cars and broken concrete. "It looked like a bomb had exploded because of the amount of fire and damage to the floors. All we could do was put water on the flames." 'The Dust Was Blinding'  </p>
<p class="articleParagraph enarticleParagraph" >Ken Olson, 34, a pipefitter, was in the basement when the explosion hit. "All of a sudden all hell broke loose," he said. "All the pipes ruptured, the dust was blinding. Luckily we all stayed together and got out."  </p>
<p class="articleParagraph enarticleParagraph" >Nearby, Vito de Leo, 32, an air-conditioning mechanic, was eating lunch at his desk with other basement trade center workers. Suddenly, the desk rose up, came down and landed on top of him. But its well protected him from a rain of falling debris. "The furniture collapsed, the walls collapsed, the ceiling collapsed," he said. "There was total blackness. I thought I was dead."  </p>
<p class="articleParagraph enarticleParagraph" >Later, wading through knee-deep water amid gas pipelines danging overhead in the garage, a cadre of firefighters, police officers and other rescue workers found two bodies in a lunchroom used by mechanics, another body in the mangled wreckage of a car, and more victims under the debris in the garage.  </p>
<p class="articleParagraph enarticleParagraph" >The five victims -- three men, one woman and one unidentified -- were all believed to heve been killed by the blast. They were not immediately identified but the Port Authority said that they were all believed to be authority workers or people working under contract to the agency. The authorities said that more bodies might be found in the rubble as the search went on.  </p>
<p class="articleParagraph enarticleParagraph" >The police said 420 workers and visitors at the trade center were treated at hospitals, along with 44 firefighters, 11 police officers and one Emergency Medical Service worker.  </p>
<p class="articleParagraph enarticleParagraph" >Meantime, as several fires erupted around the scene of the explosion, heavy smoke billowed up through the corridors, elevator shafts and stairwells of the trade center. Because of the time, shortly after noon, many workers were at lunch at nearby restaurants or at fast-food outlets on the ground floor, from where they easily escaped.  </p>
<p class="articleParagraph enarticleParagraph" >But the police estimated that as many as 50,000 people -- workers, tourists and other visitors were still in the building, many of them trapped on the highest floors -- and it was not merely the blast that shook the entire complex, not merely the growing volumes of smoke pouring upward, that frightened them.  </p>
<p class="articleParagraph enarticleParagraph" >Darkness and the unknown perils that awaited them below added to the fears. Much of the power to the trade center had been knocked out by the blast -- Consolidated Edison said four of its eight feeder cables to the center were shut down.  </p>
<p class="articleParagraph enarticleParagraph" >And within an hour, at the request of the Fire Department, which was trying to protect rescue workers and firefighters in dangerous areas, all power to the trade center was shut off by Con Edison, as well as natural gas and steam to the complex, which houses thousands of offices in six buildings bounded by Church, West, Liberty and Vesey Streets.  </p>
<p class="articleParagraph enarticleParagraph" >Pat Richardi, a Con Edison spokeswoman, said that no hazardous materials, such as polychlorinated biphenals, were in any of the transformers or cables of the trade center, which was completed 22 years ago.  </p>
<p class="articleParagraph enarticleParagraph" >Many of the people climbing down stairs told of having to stop frequently because of panic below; some let pregnant women and old people go through; some nearly passed out with exhaustion; others told of tense minutes in which they sat down on the steps, trying to regain breath in stifling, smoky air.  </p>
<p class="articleParagraph enarticleParagraph" >"It was like sardines, cattle, a herd," said Larry Bianculli, 31, of Hicksville, L.I., who walked down 104 floors with his wool scarf over his sooty face.  </p>
<p class="articleParagraph enarticleParagraph" >Sherri Chambers, 21, a bank employee, said it took her two and a half hours to descend from the 60th floor. "You couldn't even see it was so smoky," she said. "I kept wanting to sit down, but I didn't because if I sat down I thought I wouldn't get up. Firefighter Bill Chupa, 40, of Ladder 20, said many people were trapped in elevators and screaming for help. He said firefighters broke open elevator doors and found people in groups of 8 or 10, lying in darkness on the floor to escape the smoke.  </p>
<p class="articleParagraph enarticleParagraph" >After freeing those in elevators, the rescuers turned to the stairwells and began escorting people down. By midafternoon, there was a steady stream of survivors coming from the towers, many with faces blackened by smoke and gasping for air.  </p>
<p class="articleParagraph enarticleParagraph" >Some of the most spectacular evacuations came when police helicopters landed on the roofs of the trade center towers and carried away 23 people, including a pregnant woman.  </p>
<p class="articleParagraph enarticleParagraph" >Don Burke, who works for the Port Authority on the 66th floor, ran back to his office when he discovered there was a fire and, with a colleague, carried Cathy Collins, a lawyer who uses a wheelchair, to safety in relays.  </p>
<p class="articleParagraph enarticleParagraph" >In a shopping area on the ground floor of the trade center's World Financial Center at 250 Vesey Street, medical and rescue workers set up a triage area of folding chairs, oxygen tanks, blood pressure devices, blankets and other medical aids.  </p>
<p class="articleParagraph enarticleParagraph" >While there was little panic, aides to Gov. Mario M. Cuomo told of a pregnant woman screaming as they descended from the Governor's 57th floor offices in a chaos of darkness and disorder.  </p>
<p class="articleParagraph enarticleParagraph" >The Governor, who was in Albany, said President Clinton had called him to express concern and offer aid from the Federal Emergency Management Agency. Mr. Cuomo also raised some questions: "What emergency devices were available? Did they work? Why were there no lights? Why were there no public announcements. The Port Authority will be called upon to answer, and I'm sure they will."  </p>
<p class="articleParagraph enarticleParagraph" >Power was partly restored to the trade center towers at 7:20 P.M., and by 9:30 rescue workers said everyone had been evacuated.  </p>
<p class="articleParagraph enarticleParagraph" >The ceiling collapse in the PATH station forced a halt to all train service to New Jersey from the trade center, but PATH service from midtown, operating on another line, continued to operate through the day. Subway trains were rerouted and continued to run, but streets throughout the area were closed to clear the way for emergency vehicles.  </p>
<p class="articleParagraph enarticleParagraph" >Photos: Pictures of evacuees (pg. 1) (Associated Press; Carrie Boretz for The New York Times; Marilynn K. Yee/The New York Times); On a street outside the center, an evacuee gasped for breath while being watched by emergency workers and bystanders. (pg. 22) (Marilynn K. Yee/The New York Times) Diagram:  </p>
<p class="articleParagraph enarticleParagraph" >1. The blast occurred in a parking garage on the second level of the multi-level basement of the north World Trade Center tower.  </p>
<p class="articleParagraph enarticleParagraph" >2. It carved a 200- by 100-foot crater on the public parking level, blew through a ceiling and knocked a hole in the wall.  </p>
<p class="articleParagraph enarticleParagraph" >3. Cinder blocks and rubble from the collapsing wall fell to the tracks of the PATH station, and damaged lower levels.  </p>
<p class="articleParagraph enarticleParagraph" >This drawing is schematic and is based on information from reporters and officials at the scene.  </p>
<p class="articleParagraph enarticleParagraph" >(Jean Rutter/The New York Times; illustration by David Montesino and Julie Shaver) (pg. 1)  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RF</b>&nbsp;</td><td><br/>These New York Times reporters and photographers contributed to the coverage of the explosion at the World Trade Center:
&lt;p&gt;
Ralph Blumenthal, Fred R. Conrad, Celia W. Dugger, Seth Faison, Ian Fisher, Lindsey Gruson, Dennis Hevesi, Lynette Holloway, Marvine Howe, Edward Keating, Clifford J. Levy, James C. McKinley Jr., Steven Lee Myers, Alison Mitchell, Maria Newman, Larry Olmstead, Garry Pierre-Pierre, Todd S. Purdum, Dith Pran, Selwyn Raab, Lynda Richardson, Calvin Sims, Ronald Sullivan, Ruby Washington, Craig Wolff.  </td></tr>
<tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i72101 : Urban Mass Transit Systems | i97411 : Broadcasting | imed : Media | i721 : Urban/Commuter Transit | irailtr : Road/Rail Transport | itsp : Transportation/Shipping</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>gdis : Disasters/Accidents | gexplo : Building Fires/Explosions | ccat : Corporate/Industrial News | gcat : Political/General News | gmmdis : Accidents/Man-made Disasters | grisk : Risk News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>nyc : New York City | usny : New York | namz : North America | usa : United States | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>WORLD TRADE CENTER (NYC)  | EXPLOSIONS  | BOMBS AND BOMB PLOTS  | TRANSIT SYSTEMS  | TELEVISION  | PARKING GARAGES AND LOTS  | SMOKE  | FIRES AND FIREMEN  | CHILDREN AND YOUTH  | ELEVATORS AND ESCALATORS  | BLACKOUTS (ELECTRICAL)  | PORT AUTHORITY TRANS-HUDSON CORP (PATH)  | MCFADDEN, ROBERT D  | BURKE, DON  | COLLINS, CATHY </td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>New York Times Digital (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020050409dp2r00fa7</td></tr></table><br/></div></div><br/><span></span><div id="article-rec0000020011108docu025sz" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/recLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>CLM</b>&nbsp;</td><td>OBITUARIES</td></tr>
<tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>NEWS</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>OBITUARIES </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>4238 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>30 December 1992</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The Record</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>REC</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>© 1992 North Jersey Media Group Inc. Provided by ProQuest Information and Learning. All rights reserved.
 </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >BRUNO ACURIO, 100, of Lodi died Monday. Born in Italy, he came to the United States 99 years ago. Before retiring 35 years ago, he worked for Curtiss-Wright Aeronautical Engineering, Wood-Ridge. He was an Army veteran of World War I. He was a parishioner of St. Joseph R.C. Church, Lodi. Surviving are two daughters, Josephine Tamborella of Allendale and Theresa Priazza of Lodi; five grandchildren, and 12 great-grandchildren. His wife, Anna Lo Proto Acurio, died in 1965. Mass will be said Thursday at 11 a.m. St. Joseph Church, with burial in St. Nicholas Cemetery, Lodi. Visiting is today from 2 to 4 and 7 to 9 p.m. at Santangelo Funeral Home, Lodi. </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >WESLEY JOHN ADAMS, 47, of Teaneck died Monday. He worked for Goldberg Slipper Co., South Hackensack. Surviving are two sons, Wesley Jr. and Damian, both of Hackensack; a daughter, Jeania Adams of Englewood; his mother, Marian Adams of Teaneck; a brother, Norman of Las Vegas, Nev.; five sisters, Mary Rollings, Marian Adams, and Brenda Adams, all of Teaneck, Jean Adams of Hackensack, and Doris McElveen of Englewood; and his maternal grandmother, Obena Williams of Englewood. Services will be Thursday at 10 a.m. at Nesbitt Funeral Home, Englewood, with burial in George Washington Memorial Park, Paramus. Visiting is tonight from 7 to 9. </p>
<p class="articleParagraph enarticleParagraph" >HELEN BRIGGS SCHWER BENSEN, 95, of Merritt Island, Fla., formerly of Hackensack, died Sunday. Surviving are two daughters, Anne P. Livingston of Merritt Island and Margaret L. Wilkens of Dumont; two brothers, Murray M. Schwer of Maywood and Harry D. Schwer in North Carolina; four grandchildren, and six great-grandchildren. Her husband, William A., and a brother, Mack M. Schwer, are deceased. Graveside services will be Thursday at 11 a.m. in George Washington Memorial Park, Paramus. Arrangements are by Frech Funeral Home, Dumont. </p>
<p class="articleParagraph enarticleParagraph" >ESTHER D. BLACK, 60, of Garfield, formerly of Passaic, died Monday. She was a licensed practical nurse at Chestnut Hill Convalescent Center, Passaic, for five years. Previously, she worked for Beth Israel Hospital, and the General Hospital Center, both in Passaic. She received nursing training at St. Vincent's Hospital, Montclair. She was a member of the Wallington Presbyterian Church and a former member of the First Presbyterian Church, Passaic. She was a member and past chief daughter of Lady McLean Lodge 34 of the Daughters of Scotia, Passaic. Surviving are her husband, Charles, and a sister, Elizabeth Campbell in Scotland. Services will be Thursday at noon at Allwood Funeral Home, Clifton, with entombment in George Washington Mausoleum, Paramus. Visiting is today from 2 to 4 and 7 to 9 p.m., with Daughters of Scotia services at 8 p.m. </p>
<p class="articleParagraph enarticleParagraph" >JAMES A. BLOHM, 44, of Barnegat, formerly of Hasbrouck Heights, died Monday. He was a waiter for Top of the Mast, Seaside Park, and the Old Mill Inn, Spring Lake Heights. Previously, he was an English teacher for the Bergenfield Board of Education, Newark Academy in Livingston, and Gen. MacArthur Military Academy. He was a 1970 graduate of Fairleigh Dickinson University, Teaneck. Surviving are his parents, George A. and Myrtle Schmidt Blohm. Services will be Thursday at 10 a.m. at Riggs Funeral Home, Forked River, with burial in George Washington Memorial Park, Paramus. Visiting is tonight from 7 to 9. Donations to the American Cancer Society would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >ROBERT BRENT, 73, of Maywood, formerly of Lodi, died Monday. Before retiring 11 years ago, he was an expediter for Presto Lock Co., Garfield. He was an Army veteran of World War II. Surviving are two brothers, Marcel of Wood-Ridge and Albert of Lodi. Mass will be said Thursday at 10 a.m. at St. Joseph R.C. Church, Lodi, with burial in St. Mary Cemetery, Saddle Brook. Visiting is today from 2 to 4 and 7 to 9 p.m. at Alesso Funeral Home, Lodi. </p>
<p class="articleParagraph enarticleParagraph" >SONYA A. BURDEN, 31, of Paterson died Monday. Surviving are her parents, two daughters, three sisters, and two brothers. Services will be Thursday at 1 p.m. at Canaan Baptist Church, Paterson, with burial in Laurel Grove Cemetery, Totowa. Visiting is tonight from 7 to 9 at Bragg Funeral Home, Paterson. </p>
<p class="articleParagraph enarticleParagraph" >GEORGE T. CARVELLAS, 85, of New York City died Monday. He was born in Albania. Before retiring, he was a self-employed real estate agent in New York. Surviving are his wife, Stavroula Stamatis Carvellas; two sons, Thomas in New York City and Alexander of Teaneck; a daughter, Alexandra Molnar of Fair Lawn, and two grandchildren. Services will be Thursday at 10 a.m. at Volk Funeral Home, Teaneck, with burial in George Washington Memorial Park, Paramus. Visiting is today from 2 to 4 and 7 to 9 p.m. </p>
<p class="articleParagraph enarticleParagraph" >ELSIE M. ELSAESSER COLLING, 84, of Vero Beach, Fla., died Sunday. Before retiring, she was a clerk for Lloyd's Register and Shipping, New York City. She was a member of Trinity Episcopal Church, Vero Beach. Surviving are several nieces and nephews, including Robert H. Parker of Rochelle Park. Her husband, Arthur H., died in 1984. Services will be Thursday at 11 a.m. at Vander Plaat Memorial Home, Paramus, with burial in Maple Grove Cemetery, Hackensack. Visiting will be Thursday from 10 to 11 a.m. Donations to the American Heart Association, P.O. Box 1181, Vero Beach, Fla. 32961, would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >ELIZABETH BEAM BRUNDAGE CONRAD, 80, Franklin Lakes died Monday. She had been a secretary for the Franklin Lakes Board of Education. Surviving are her husband, Willard S.; a son, Willard G. of Deep Gap, N.C.; a daughter, Linda C. Baker of Franklin Lakes; a brother, Clifford Brundage of Cochita Lake, N.M.; a sister, Lillian Jiles of Binghamton, N.Y., and four grandchildren. Services are today at 2 p.m. at Oakland Memorial Home. Donations to the Lupus Erythematous Foundation would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >JAMES COONEY, 61, of Leonia died Tuesday. He was a plumber with union Local 2, New York City. He was a Navy veteran of the Korean War. He was a member of the American Legion Plumbers Post, the Bronx. Surviving are four sisters, Ethel O'Mara of Montclair, May Primrose of Garden City, N.Y., Patricia Kent of Montvale, and Gloria Grunn of River Vale; and three brothers, John of Paramus, Joseph of Teaneck, and George of Bronxville, N.Y. Mass will be said Saturday at 10:30 a.m. at St. Joseph R.C. Church, Bogota, with burial in Madonna Cemetery, Fort Lee. Visiting will be Friday from 2 to 4 and 7 to 9 p.m. at Petrik Funeral Home, Bogota. </p>
<p class="articleParagraph enarticleParagraph" >RICHARD T. DREW, 59, of Lyndhurst, formerly of Lodi, died Monday. He was a graphic designer for Minolta Corp., Ramsey. He was an Army veteran of the Korean War. Surviving are his wife, Maureen Casey Drew; a son, Eric of Wallington; a daughter, Patsy Drew of Wallington; his mother, Mary Drew of the Bronx; three sisters, Barbara Axelrod of the Bronx, Patricia Meyering of Yorktown Heights, N.Y., and Ruth Arndt of New Windsor, N.Y., and a grandchild. A brother, Thomas, is deceased. Mass will be said Thursday at 9 a.m. at St. Joseph R.C. Church, Lodi, with burial in St. Mary Cemetery, Saddle Brook. Visiting is today from 2 to 4 and 7 to 9 p.m. at Santangelo Funeral Home, Lodi. </p>
<p class="articleParagraph enarticleParagraph" >PAUL FAILLA, 79, of Passaic died Tuesday. He was a liquor salesman for Federal Wine &Liquor, Kearny. He was a parishioner of St. Anthony R.C. Church, Passaic. Surviving are two sons, Joseph of Wayne and Bart of Denville; a sister, Victoria Riggi of Passaic, and four grandchildren. His wife, Olga Kruliak Failla, died in 1984. Mass will be said Thursday at 9 a.m. at St. Anthony Church, with entombment at Calvary Mausoleum, Paterson. Visiting is today from 2 to 4 and 7 to 9 p.m. at Marrocco Memorial Home, Passaic. </p>
<p class="articleParagraph enarticleParagraph" >NELLIE FISHMAN, 89, of River Edge died Monday. She was born in Russia. She was a member of Hadassah, B'nai B'rith, and Deborah Hospital. Surviving are a son, Eli of New Milford; a sister, Peggy Weene of Boston; five grandchildren, and five great-grandchildren. Her husband, Nathan, and a son, Sumner, are deceased. Arrangements were by Gutterman-Musicant, Hackensack. </p>
<p class="articleParagraph enarticleParagraph" >CARL L. FLOWER, 69, of South Windsor, Conn., died Monday. Before retiring, he worked for Aetna Life &Casualty Co., Hartford, Conn. He was an Army veteran of World War II. Surviving are three sons, Wayne of Jefferson City, Mo., Bruce of Raymond, N.H., and Robert of Hartford; a daughter, Sheri L. Flower of Wethersfield, Conn.; two sisters, Vayle Browne of Tenafly and Fern Miller of New Berne, N.C., and five grandchildren. Also surviving is a companion, Kay Burnham of South Windsor. Graveside services will be Thursday at 1 p.m. at George Washington Memorial Park, Paramus. Arrangements are by Samsel &Carmon Funeral Home, South Windsor. Donations to the American Cancer Society, 20 Mercer St., Hackensack, N.J. 07601, would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >ANTHONY T. FUGNITTI, 69, of North Bergen died Tuesday. Before retiring, he was a pipe fitter for nine years and a member of Local 274, Ridgefield. He was a Marine Corps veteran of World War II. Surviving are a son, Anthony Jr. of North Bergen; a daughter, Trudy Simmons of West Paterson; a brother, Henry of Secaucus; a sister, Marion Cursinella of North Bergen, and two grandchildren. Mass will be said Thursday at 1 p.m. at Our Lady of Libera R.C. Church, West New York, with burial in George Washington Memorial Park, Paramus. Visiting is today from 2 to 4 and 7 to 9 p.m. at Vainieri Funeral Home, North Bergen. </p>
<p class="articleParagraph enarticleParagraph" >WARREN E. JONES, 68, of Englewood died Sunday. Before retiring, he was a freight handler for the Erie-Lackawanna Railroad. He was a Navy veteran of World War II and a lifetime member of Disabled American Veterans Chapter 126, New York City. He was a member of Service Employees Local 32B, New York City. Surviving are two daughters, Monica C. Jones and Phillippa P. McCleod Jones, both of Englewood; his mother, Gladys C. Jones of Winton, N.C.; two sisters, Carrie Jenkins and Bernice Smith; six brothers, Sherman, Roger, Thomas, Ryland, Ashley, and Carlton, and three grandchildren. His wife, Patricia P. McWhorter Jones, is deceased. Services will be Thursday at 10:30 a.m. in the chapel of St. Paul's Episcopal Church, Englewood, with burial in Kensico Cemetery, Valhalla, N.Y. Visiting is today from 3 to 5 and 7 to 9 p.m. at Quirk Funeral Home, Englewood. </p>
<p class="articleParagraph enarticleParagraph" >JOHN KALOS, 35, of Montvale died Tuesday. Surviving are his father, George, a sister, Flora Kalos, and a brother, Nickolas, all of Montvale. His mother, Bessie Malamas Kalos, died in 1986. Services will be Thursday at 10:30 a.m. at G. Thomas Gentile Funeral Service, Hackensack, with cremation at Cedar Lawn Crematory, Paterson. Visiting will be Thursday from 9:30 to 10:30 a.m. </p>
<p class="articleParagraph enarticleParagraph" >DOROTHY H. KLIMKO, 77, of Fair Lawn, formerly of Wallington died Tuesday. Before retiring 12 years ago, she worked for Vornado Corp., Garfield. She was a parishioner of St. Anne R.C. Church, Fair Lawn, and a member of its Rosary Society. Surviving are two brothers, George Klimko of Clifton and Joseph Klimko of Fair Lawn, and a sister, Helen Kennedy of Clifton. Mass will be said Thursday at 9:30 a.m. at St. Anne Church, with burial in St. Mary Cemetery, Saddle Brook. Visiting is today from 2 to 4 and 7 to 9 p.m. at Kamienski Funeral Home, Wallington. </p>
<p class="articleParagraph enarticleParagraph" >THEKLA HOETZL KUEBLER, 81, of Hackensack died Monday. She was born in Germany. She had owned a catering business for 20 years. She was a parishioner of Our Lady Queen of Peace R.C. Church, Maywood, and a member of its Rosary Society. Surviving are her husband, Ernest; two sons, Gordon of Hackensack and Ralph of Silver Springs, Md.; a daughter, Lynda Zazaccone of Mahwah; a sister, Centa Klinger in Germany, and five grandchildren. Mass wil be said Thursday at 10 a.m. at Our Lady Queen of Peace Church. Visiting is today from 3 to 5 and 7 to 9 p.m. at Trinka-Faustini Funeral Home, Maywood. Donations to the Mahwah Ambulance and Rescue Squad, Franklin Turnpike, Mahwah, N.J. 07430, would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >JAMES A. LEAVEY SR., 91, of Northvale died Monday. Before retiring in 1966, he was an accountant for Consolidated Edison, New York City, where he worked for 50 years. Surviving are a son, James A. Jr. of Northvale, and three grandchildren. His wife, Mary Wood Leavey, died in 1968. Mass will be said Thursday at 11:30 a.m. in St. Joseph R.C. Church, Wurtsboro, N.Y., with burial in the church cemetery. Visiting is tonight from 7 to 9 at Van Inwegen-Kenny Funeral Home, Wurtsboro. Arrangements are by Anthony R. Pizzi Funeral Home, Northvale. </p>
<p class="articleParagraph enarticleParagraph" >ISABEL GOLDFARB LEVINE, 89, of Valhalla, N.Y., formerly of Paterson and Fair Lawn, died Tuesday. Before retiring, she was a secretary for Glen Rock Lumber Co., Fair Lawn. She was a member of the Jewish Center, ORT, and Hadassah, all in Fair Lawn. Surviving are a son, Howard S. of Rye Brook, N.Y.; a sister, Pauline Barman of Union, and two grandchildren. Her husband, Isaac, died in 1968. Graveside services are today at 11 a.m. at Silk City Lodge Cemetery, West Paterson. Arrangements are by Louis Suburban Chapels, Fair Lawn. Donations to a favorite charity would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >RITA M. LINDBLOM, 61, of Wayne died Monday. She worked in the data processing department of TRW Inc., Fairfield. Surviving are a daughter, Julie Lindblom of Rutherford, and a son, Kris of Wayne. Her husband, George, died in 1987. Services are today at 8 p.m. at Thomas J. Diffily Funeral Home, Rutherford. Visiting is today from 2 to 4 and 7 to 9 p.m. Donations to Hackensack Medical Center Hospice, 385 Prospect Ave., Hackensack, N.J. 07601, would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >LEON V. MAKSYMOWICZ, 67, of Ridgefield Park died Tuesday. Before retiring in 1983, he was a warehouseman for Ford Motor Co., Teterboro. He was a Navy veteran of World War II and a member of Veterans of Foreign Wars Post 277, Ridgefield Park. Surviving are his wife, Louise Caufield Maksymowicz; two daughters, Susan Maksymowicz and Linda Artuso, both of Ridgefield Park; two sons, Charles Bellmyer of Staten Island and Robert Brushi of Fairview; three sisters, Mary Kocienda and Catherine Olender, both of Port Richey, Fla., and Helen Hellstrom in Maryland, and seven grandchildren. A daughter, Caroline Maksymowicz, is deceased. Mass will be said Thursday at 9:30 a.m. at St. Francis R.C. Church, Ridgefield Park, with burial in Holy Name Cemetery, Jersey City. Visiting is today from 2 to 4 and 7 to 9 p.m. at Vorhees Funeral Home, Ridgefield Park. Donations to the St. Francis Church maintenance fund would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >FARAH JANE MARION, 92, of Bloomingdale died Tuesday. Before retiring, she worked for Amerace Rubber Co., Butler. She was a member of the Bloomingdale Fire Company Ladies Auxiliary. Surviving are two daughters, June Bond of West Milford and Jane Pearson of Butler, and four grandchildren. Her husband, John, died in 1968. Services will be Thursday at 10 a.m. at Richards Funeral Home, West Milford, with burial in Mount Rest Cemetery, Butler. Visiting is today from 2 to 4 and 7 to 9 p.m. </p>
<p class="articleParagraph enarticleParagraph" >KATHLEEN M. McQUEENEY of Leonia died Sunday. Born in Ireland, she came to the United States in 1925. Before retiring in 1986, she was a market researcher for J.P. Stevens Inc., New York City. She was a parishioner of St. John R.C. Church, Leonia. Surviving are two sisters, Mary Wingardner of Lakehurst and Elizabeth Gribben of Old Bridge; and four brothers, James of Queensbury, N.Y., John of the Bronx, Thomas of Princeton Junction, and Joseph of Emerson. Mass will be said Thursday at 10 a.m. at St. John Church, with burial in Maryrest Cemetery, Mahwah. Visiting is today from 2 to 4 and 7 to 9 p.m. at Volk Funeral Home, Palisades Park. </p>
<p class="articleParagraph enarticleParagraph" >ALIDA NYWENING MUCKE, 84, of Middletown, N.Y., formerly of Fort Lee, died Monday. She was a member of Alliance Church, Middletown. Surviving are two daughters, Carole R. Cook of Middletown and Elaine M. Bowling of Norwood; a son, Paul Jr. of Ashboro, N.C.; three brothers, Leonard Nywening of Campbell Hall, N.Y., Albert Nywening of Goshen, N.Y., and Tunis Nywening of Midland Park; three sisters, Grace Nywening of Myerstown, Pa., Gertrude Sotnick of Wyckoff, and Violet Schenzenbach of North Haledon; 15 grandchildren, and 20 great-grandchildren. Services will be Thursday at 3 p.m. at Alliance Church, with burial Saturday in Fair Lawn Memorial Cemetery. Visiting will be Thursday from 1 to 3 p.m. at Alliance Church. Arrangements are by Cornelius, Dodd &Connell Funeral Home, Middletown. Donations to the Alliance Church, 24 Wickham Ave., Middletown, N.Y. 10940, would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >MARY NABSTEDT, 88, of Cresskill died Tuesday. She was the last charter member of the Ladies Auxiliary of the Cresskill Fire Department. Surviving are a son, Robert J. of Kendall Park; a daughter, Virginia Marie Hiller of Montvale, and seven grandchildren. Her husband, Elmer F., and a son, Elmer J., are deceased. Services will be Thursday at 9:30 a.m. at St. Therese R.C. Church, Cresskill, with burial in George Washington Memorial Park, Paramus. Visiting is today from 2 to 4 and 7 to 9 p.m. at Barrett Funeral Home, Tenafly. Donations to the Cresskill Volunteer Fire Department would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >JOHN PLUCHINO, 91, of Wayne, formerly of West New York, died Tuesday. Before retiring 24 years ago, he was a pipe maker for Kaywoodie Pipe Co., West New York, where he worked for 40 years. Surviving are a son, Michael of Teaneck; a daughter, Rosina Agrusti of Wayne, and three grandchilden. Mass will be said Thursday at 11 a.m. at St. Joseph of the Palisades R.C. Church, West New York, with entombement in Hackensack Mausoleum. Visiting is today from 2 to 4 and 7 to 9 p.m. at Vainieri Funeral Home, North Bergen. </p>
<p class="articleParagraph enarticleParagraph" >CASIMIR RAKOWSKI, 51, of Jersey City died Sunday. He had been a sportswriter for the Jersey Journal, Jersey City. He also had been Hudson County director of public relations. In September, he was appointed to the Jersey City Incinerator Authority. He was an organist at St. Paul of the Cross R.C. Church and a member of the YMCA Board of Trustees, both in Jersey City. Surviving are his parents, Casimir and Genevieve Rakowski; a sister, Dorothy; and a brother, Joseph. A sister, Jeanette Rakowski, is deceased. Mass will be said Thursday at 10 a.m. at St. Paul of the Cross Church, with burial in Holy Name Cemetery, Jersey City. Visiting is today from 2 to 4 and 7 to 9 p.m. at McLaughlin Funeral Home, Jersey City. </p>
<p class="articleParagraph enarticleParagraph" >ANNE V. KANE SCHRADER, 83, of Bergenfield, formerly of Lodi, died Sunday. Before retiring in 1974, she was a typist for Mobil Oil Corp., New York City. Surviving are a son, Michael E. of Queens; two sisters, Mary La Pietra of Annapolis, Md., and Elizabeth Rumie of Palisades Park, and two grandchildren. A son, Eugene D., died in 1979. Mass will be said today at 10 a.m. at St. John R.C. Church, Bergenfield, with burial in George Washington Memorial Park, Paramus. Arrangements are by Riewerts Memorial Home, Bergenfield. </p>
<p class="articleParagraph enarticleParagraph" >KATHRYN GELSINON SCHULZ, 88, of Westwood died Sunday. Before retiring in 1969, she was a teacher for the Westwood Board of Education. She was a member of Zion Lutheran Church, Westwood, and its Ladies' Aid Society. She was a member of the Ladies' Auxiliary of the Veterans of Foreign Wars, Westwood, the National Grange, and the National Retired Teachers Association. She was a volunteer at Pascack Valley Hospital, Westwood. Her husband, Fred, died in 1969. Graveside services will be Thursday at 2 p.m. at Westwood Cemetery. Arrangements are by Becker Funeral Home, Westwood. </p>
<p class="articleParagraph enarticleParagraph" >PATRICK J. SHIEL,88, of North Arlington, formerly of Jersey City, died Monday. Born in Ireland, he came to the United States 67 years ago. Before retiring 21 years ago, he was a station agent for Port Authority Trans Hudson Corp., Jersey City. He was a parishioner of Our Lady Queen of Peace R.C. Church, North Arlington. Surviving are three daughters, Eileen M. Reilly of North Arlington, Veronica Madigan of Cedar Grove, and Rosaleen Cross of Randolph Township; a son, Patrick Jr. of Kinnelon; 15 grandchildren, and seven great-grandchildren. His wife, Catherine O'Leary Shiel, died in 1984. Mass will be said Thursday at 11:15 a.m. at Our Lady Queen of Peace Church, with burial in Holy Cross Cemetery, North Arlington. Visiting is today from 2 to 4 and 7 to 9 p.m. at Paro Funeral Home, North Arlington. </p>
<p class="articleParagraph enarticleParagraph" >DOROTHY GOLDBERG SIEGEL, 95, of West New York, formerly of Weehawken, died Sunday. She was born in Austria. Before retiring, she was an attorney. She was a graduate of New York University Law School. She was a member of Temple Beth El, North Bergen, the Brandeis University Women's Auxiliary, the Junior Associates of the Jewish Home and Hospital of New Jersey, and Hadassah, North Hudson Chapter. Surviving are two sons, Daniel of Englewood Cliffs and Mace; eight grandchildren, and four great-grandchildren. Her husband, Murray R., is deceased. Services are today at noon at Wien &Wien, Englewood, with burial in Beth El Cemetery, Washington Township. Donations to a favorite charity would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >MIRIAM BELKIN SMOLEN, 76, of Tamarack, Fla., formerly of Fair Lawn, died Monday. She was a member of Fair Lawn Jewish Center, its sisterhood, Hadassah, and B'nai B'rith Women. Surviving are her husband, Morris; a son, Arthur Belkin of New York City; a stepson, Cary Smolen of Wilton, Conn.; a daughter, Donna Nadler of Teaneck; a stepdaughter, Clare Miller of Edison; a sister, Doris Solomon of Davie, Fla., and eight grandchildren. Her first husband, Abraham, is deceased. Services will be Thursday at 9:30 a.m. at Robert Schoem's Menorah Chapel, Paramus, with burial in King Solomon Cemetery, Clifton. A mourning period will be observed at the Nadler residence, Teaneck. </p>
<p class="articleParagraph enarticleParagraph" >MARIE STEIGER, 82, of Hasbrouck Heights, formerly of Clifton, died Sunday. Surviving are a son, Robert of Brecksville, Ohio; a sister, Helen Woodcock of North Miami Beach, Fla.; a brother, Peter Mattison of Ann Arbor, Mich.; four grandchildren, and two great-grandchildren. Her husband, Arthur, died in 1988. Arrangements are by Costa Memorial Home, Hasbrouck Heights. </p>
<p class="articleParagraph enarticleParagraph" >CHESTER TAYLOR, 94, of Wood-Ridge died Monday. Before retiring 25 years ago, he was a court transcriber for the Southern District Court, New York City. He was a member of the Garden Club, Wood-Ridge, and the Corona Color Slide Club, Hasbrouck Heights. Surviving is a friend, Nancy Cant of Wood-Ridge. His wife, Muriel, died in 1979. Services will be Thursday at 9:30 a.m. at Kohler Funeral Home, Wood-Ridge, with cremation at Cedar Lawn Crematory, Paterson. Visiting is today from 2 to 4 and 7 to 9 p.m. Donations to a favorite charity would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >EDWARD W. TYSON, 81, of Norwood died Tuesday. Before retiring 28 years ago, he was an office clerk for Mobil Oil Co., New York City, where he worked for 21 years. Surviving are his wife, Mary Onufer Tyson; a daughter, Caroline Marsh of Northvale; a brother, Michael Cieplicki, and a sister, Rose Vita, both of West Hempstead, N.Y.; three grandchildren, and a great-grandchild. Mass will be said Thursday at 10 a.m. at Immaculate Conception R.C. Church, Norwood, with cremation at Cedar Lawn Crematory, Paterson. Visiting is today from 2 to 4 and 7 to 9 p.m. at the Anthony R. Pizzi Funeral Home, Northvale. </p>
<p class="articleParagraph enarticleParagraph" >NICHOLAS TZAVARAS, 80, of Fort Lee died Monday. Born in Greece, he came to the United States in 1946. Before retiring in 1980, he owned George Dorros "The Modern Bride", New York City. He was a parishioner of St. Spyridon Greek Orthodox Church, New York City, where he had served on the parish council. He was a Greek Army veteran of World War II. Surviving are his wife, Kiki Dorros Tzavaras; a son, Capt. George Tzavaras CEC-USN in Kensington, Md., and three grandsons. Services will be Thursday at 11 a.m. at St. Spyridon Church, with burial in Brookside Cemetery, Englewood. Visiting is today from 2 to 4 and 7 to 9 p.m. at Frank A. Patti & Kenneth Mikatarian Funeral Home, Fort Lee. Donations to St. Spyridon Greek Orthodox Church, 178th Street and Wadsworth Avenue, New York, N.Y. 10021, would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >WARREN "BIGGUM" WILLIAMS, 39, of Paterson died Sunday. Surviving are his parents, James and Loretta Williams, four brothers, Miles Williams, Roman Williams, Maurice Williams, and Stevie Godbolt, and eight sisters, Delores Williams, Helena Williams, Michelle Foster, Kathy A. Williams, Jacqueline Jackson, Kim Jackson, Melissa Williams, and Jennifer Godbolt, all of Paterson. Four brothers, James Williams Jr., Gregory Williams, Montgomery Williams, and Clifford Williams, are deceased. Services will be Thursday at 10 a.m. at Bragg Funeral Home, Paterson, with burial in Fair Lawn Memorial Cemetery. Visiting is tonight from 7 to 9.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>OBIT | COLUMN</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>Record / Hackensack, New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document rec0000020011108docu025sz</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020050413dock01me9" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Metropolitan Desk; 1</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>PATH Takes Lesson From Storm's Close Calls  </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By IVER PETERSON  </td></tr>
<tr><td align="right" valign="top" class="index"><b>CR</b>&nbsp;</td><td>Special to The New York Times  </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>1446 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>20 December 1992</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>44</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 1992 The New York Times Company.  All Rights Reserved.  </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >HOBOKEN, N.J., Dec. 18 -- In the end, the flooding of the PATH tunnels under the Hudson River in the Dec. 11 storm produced nothing worse than a few close calls and an awful mess.  </p>
<p class="articleParagraph enarticleParagraph" >But if the 7 A.M. train from the World Trade Center had been a few hundred yards farther back in the Hoboken approach tunnel when water flooded the power supply, or if passengers had been waiting where a flooded elevator shaft suddenly disgorged a torrent onto the platform at Exchange Place, the Port Authority might be dealing today with the aftermath of a catastrophe.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >Officials of the Port Authority Trans-Hudson Corporation say their safety procedures enabled them to get everyone out safely, avoiding any deaths or even injuries.  </p>
<p class="articleParagraph enarticleParagraph" >But in reviewing exactly what happened and looking for lessons to avoid future problems, the officials acknowledge that the intensity of the storm took them by surprise, that there were some missteps during the first hours of the emergency, and that there were indeed some close calls.  </p>
<p class="articleParagraph enarticleParagraph" >At one point, for example, time was lost trying to pump water out of the Manhattan end of the World Trade Center tunnel, before engineers concluded that it could only be pumped the other way.  </p>
<p class="articleParagraph enarticleParagraph" >In another case, the Manhattan-bound tunnel of the 33d Street line was inadvertently flooded to a depth of 3 and a half feet when water being pumped out of a network of switches near Hoboken ran unnoticed onto that track. A Train Is Blocked  </p>
<p class="articleParagraph enarticleParagraph" >As the first signs of flooding appeared Friday morning, officials said, dispatchers began holding trains in stations and backing others out of tunnels to high ground. Power was turned off section by section as areas flooded, but not until trains in the area had cleared.  </p>
<p class="articleParagraph enarticleParagraph" >Still, one train did not make it out. The high tides and rain -- which PATH officials had expected to come as heavy snows -- flooded some tunnels in scant minutes, catching the train driven by Carmine Priore in rising water 70 yards from the Hoboken terminal. Ultimately, the 20 passengers waded waist-deep against the oncoming seawater. During the flood, the water overwhelmed some of the railroad's World War I vintage pumps and flooded a section of the outbound Hoboken tunnel, where the track is switched between 33d Street and the Pavonia station, to the top. Working on Restoration  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Priore, who has 10 years experience as an engineer on the PATH trains, realized when the ordeal was over that things could have gone much worse. "I wouldn't say I was scared to the point of being frightened," he said later. "But I wouldn't be human if those thoughts didn't pass through my head."  </p>
<p class="articleParagraph enarticleParagraph" >Now PATH engineers and electricians are struggling in the slime of pumped-out tunnels to restore power to electrical signal relays that flashed into green electrolytic mush when the highly-conductive salt water flooded the high-voltage circuits. They hope to restore service out of Hoboken by Monday.  </p>
<p class="articleParagraph enarticleParagraph" >Path officials say nearly 200,000 trips are taken through the tunnels each weekday.  </p>
<p class="articleParagraph enarticleParagraph" >Meanwhile, the system supervisors are conducting the first rounds of "lessons learned" briefings on what went wrong and what could have been done to avoid the shutdown. The discussion sometimes touches on how much worse it might have turned out. Preparations for Snow  </p>
<p class="articleParagraph enarticleParagraph" >"Sure, we've thought about what could have happened," said Michael A. Scott, general superintendent of PATH. "Fortunately we had some very reasoned minds working in the control towers who assessed the situation very quickly and moved very quickly."  </p>
<p class="articleParagraph enarticleParagraph" >Citing safety concerns, PATH officials would not allow a reporter to visit the tunnels.  </p>
<p class="articleParagraph enarticleParagraph" >In reconstructing the events of Dec. 11, Mr. Scott and others begin with a familiar complaint: early weather forecasts were wrong. Instead of cold and snow, the region got mild temperatures and blinding rain.  </p>
<p class="articleParagraph enarticleParagraph" >"We had people come in prepared for snow removal and to heat up the cars early, make sure the doors weren't iced up," Mr. Scott said.  </p>
<p class="articleParagraph enarticleParagraph" >In any case, it was not the rain that flooded the old railroad tunnels, some of which date back 100 years, but high tides that exceeded the limits of the National Oceanographic and Atmospheric Administration's gauges at the Battery and rolled like small rivers down the stair and elevator shafts at Hoboken and Exchange Place.  </p>
<p class="articleParagraph enarticleParagraph" >The Hoboken PATH station is at sea level at the edge of the Hudson. 10 Minutes to Trouble  </p>
<p class="articleParagraph enarticleParagraph" >The first sign of trouble came about 7 A.M., when the No. 3 track in the Hoboken station went under water. But, Mr. Priore said, the track lay in a low spot and had flooded before. The system's control center at Journal Square switched the Hoboken trains to tracks one and two, and sent out an alert for all trains to be on the lookout for flooding. It did not take long for them to find some, in the Meadowlands area outside Newark.  </p>
<p class="articleParagraph enarticleParagraph" >"We had a 7 A.M. inbound to Newark that reported standing water but no problems getting through the Meadows," said Tony Barber, the Port Authority's head of interstate transportation. "Then the 7:10 out of Newark ran into high water and stalled when the third rail submerged. So within 10 minutes we went from nothing significant to enough water to shut us down."  </p>
<p class="articleParagraph enarticleParagraph" >For the next frenzied hours and in the days since, PATH engineers and outside contractors have been laboring in two 12-hour shifts to pump out the tunnels, repair the system's 70-year-old air-driven sump pumps deep underground and begin the time-consuming task of replacing hundreds of electrical relays and testing miles of power line.  </p>
<p class="articleParagraph enarticleParagraph" >Service between Newark and the World Trade Center was quickly increased to compensate for slowdowns on the Journal Square to 33d Street route, which returned to full service Wednesday. Only service from he Hoboken now remains closed.  </p>
<p class="articleParagraph enarticleParagraph" >A Stroke of Luck  </p>
<p class="articleParagraph enarticleParagraph" >While Mr. Priore's train waited for instructions in the tunnel, television monitors at Journal Square PATH headquarters picked up a puddle of water forming at the base of a temporary construction barrier.  </p>
<p class="articleParagraph enarticleParagraph" >"We could see the puddle of water, and we could see the barrier beginning to bulge out," Mr. Scott recalled. "Then all of a sudden the plywood gave way and the water just shot out across the platform. It came with so much force that it went across the platform and jumped over the tracks and hit the wall on the other side."  </p>
<p class="articleParagraph enarticleParagraph" >It was one of the day's biggest strokes of luck, he said, that no one was standing in the path of the flood.  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Priore first stopped his train outside Hoboken to wait for an outbound train to pass. Then, about 7:30, he was directed to "switch ends" and head for the nearest station at Pavonia. But as he waited for the trains behind him to clear the track, the water flooded the third rail where Mr. Priore's train stood, and power was shut off.  </p>
<p class="articleParagraph enarticleParagraph" >Some passengers grew frightened. One man berated the engineer for making him miss his connection to New Jersey Transit. Some Passengers Read  </p>
<p class="articleParagraph enarticleParagraph" >"It was an uncomfortable position to be in, but it wasn't life-threatening," the engineer said. "The lights and air conditioning went on emergency battery power, and some people just sat there and read their papers."  </p>
<p class="articleParagraph enarticleParagraph" >After more than two hours in the train, Mr. Priore's passengers were lowered one at a time into the waist-deep water. Holding a life-line and aided by two firemen, they made their way back to the platform.  </p>
<p class="articleParagraph enarticleParagraph" >Some of the original, 70-year-old air-driven pumps that clear the tunnels of occasional seepage and condensation water failed. But PATH officials say those pumps were not intended to handle the volume of the Friday flood.  </p>
<p class="articleParagraph enarticleParagraph" >Engineers are examining ways to prevent future flooding, Mr. Scott said. But the most obvious solution -- installing dikes around PATH stations to block water -- would interfere with access by handicapped passengers, he said.  </p>
<p class="articleParagraph enarticleParagraph" >Photo: PATH system supervisors are now conducting the first rounds of "lessons learned" briefings on what went wrong and what could have been done to avoid the shutdown because of the Dec. 11 storm. Firefighters helped the conductor of a train out of the station in Hoboken, N.J., when the storm struck. (David Scull/The New York Times) Map of PATH system between New York City and New Jersey, indicating tubes effected by flooding.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i71 : Railroads | i72101 : Urban Mass Transit Systems | i721 : Urban/Commuter Transit | irailtr : Road/Rail Transport | itsp : Transportation/Shipping</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>gdis : Disasters/Accidents | gntdis : Natural Disasters/Catastrophes | gwea : Weather | gflood : Floods/Tidal Waves | ccat : Corporate/Industrial News | gcat : Political/General News | gwere : Routine Weather Reports | grisk : Risk News | ncat : Content Types | nrgn : Routine General News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>nyc : New York City | usny : New York | namz : North America | usa : United States | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>TRANSIT SYSTEMS  | SUBWAYS  | WEATHER  | RAIN  | FLOODS  | PORT AUTHORITY TRANS-HUDSON CORP (PATH)  | PETERSON, IVER </td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>New York Times Digital (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020050413dock01me9</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020050413doce01ldx" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Metropolitan Desk; A</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>AFTER THE STORM: The Overview; WEAKENING STORM LEAVES NORTHEAST WITH HUGE DAMAGE  </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By ROBERT D. McFADDEN  </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>1682 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>14 December 1992</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>1</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 1992 The New York Times Company.  All Rights Reserved.  </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >Blue skies returned to a storm-ravaged New York metropolitan area yesterday, but at least 112,000 homes had no power, surging tides again flooded coastal communities, and residents and public officials surveyed a monstrous toll of damage from two days of pounding by hurricane-like winds, rain and snow.  </p>
<p class="articleParagraph enarticleParagraph" >The fierce storm, which caused six deaths and scores of injuries in the tri-state region and disrupted travel, business and countless lives, was gone, 500 miles out to sea by midafternoon, its trailing, weakening winds flicking final backhand insults at the twin forks of eastern Long Island.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >As Federal and state disaster officials began damage surveys and thousands of residents returned to flooded homes and began cleaning up the mess, there were no official estimates of the storm's cost. But unofficial estimates, some local damage assessments and comparisons with previous storms suggested that private and public losses were likely to run into hundreds of millions. Lost Beaches and Grim Faces  </p>
<p class="articleParagraph enarticleParagraph" >The storm's legacy was widely apparent along shore areas of Long Island, New Jersey, Connecticut and New York City: in the thousands of battered homes and miles of eroded beaches, in downed trees and power lines, in shattered boats and seawalls and other property -- and especially in the grim faces of the people who had suffered losses and spent sleepless nights in shelters or homes awash.  </p>
<p class="articleParagraph enarticleParagraph" >In many coastal areas yesterday, high tides raked disfigured dunes again and carried water into flooded homes for a third straight day, but there was little left to corrupt: the sea had already done its worst.  </p>
<p class="articleParagraph enarticleParagraph" >And as political leaders toured hard-hit communities, appealed for Federal disaster aid and predicted a return to normal life this week, it was clear that this otherwise nameless storm would join the region's pantheon of blizzards, hurricanes and other natural catastrophes as the Great Nor'easter of '92.  </p>
<p class="articleParagraph enarticleParagraph" >"The ocean still looks savage," Gov. Mario M. Cuomo said at a helicopter pad in lower Manhattan after an aerial survey of collapsed homes on Long Island and in shore areas of Queens and Brooklyn. "I saw huge chunks torn out of seawalls because the ocean was impatient to get to the property."  </p>
<p class="articleParagraph enarticleParagraph" >Among the devastation, the Governor saw scores of coast homes that had been destroyed in Southampton and Quogue and on Fire Island, and the flooded North Shore village of Bayville, where 3,000 residents were unable to return to homes standing in up to eight feet of water.  </p>
<p class="articleParagraph enarticleParagraph" >In New Jersey, Gov. Jim Florio and other officials toured Atlantic City in military vehicles, assessing damage to homes, businesses, the boardwalk and beaches. At a school turned shelter, Sherlyn Gandy and her husband, Claude, who had lost their home, appealed to the Governor.  </p>
<p class="articleParagraph enarticleParagraph" >"I've lost everything and I've got two children," Mrs. Gandy said. "I'm trying to find somewhere to live."  </p>
<p class="articleParagraph enarticleParagraph" >What would happen to them, Mr. Gandy asked, when the school reopened today?  </p>
<p class="articleParagraph enarticleParagraph" >"We can assure you," Mayor James Whelan replied, "you are not going to be put out on the street."  </p>
<p class="articleParagraph enarticleParagraph" >Little Ferry, N.J., with 10,000 people on 1.5 square miles of ground -- much of it below the level of the nearby Hackensack River -- was swamped by floodwaters and virtually shut down. With three feet of water over most of the borough, pumps were working around the clock and hundreds of families took refuge in schools or with relatives. But many refused to leave flooded homes, and were stranded, without power or heat.  </p>
<p class="articleParagraph enarticleParagraph" >"We've got boats, we can get them," said Lynne Spain, a First Aid captain, "but they won't leave. A lot of them are running out of food. They're throwing money out the window and saying, 'Here, buy something.' " Power Failures Continue  </p>
<p class="articleParagraph enarticleParagraph" >Power failures, which hit 988,000 homes and businesses during the storm, continued to plague the region. By late yesterday, utility spokesmen said, more than 112,000 customers were still without service -- 90,000 in New Jersey, 19,000 on Long Island, 3,100 in Connecticut and 277 in New York City and Westchester County. Hundreds of crew members were working around the clock, but it was unclear how long a complete restoration of power would take.  </p>
<p class="articleParagraph enarticleParagraph" >In some flood-stricken Connecticut communities -- Norwalk, Greenwich, Westport and Fairfield -- power had to be shut off to hundreds of customers because of concern over possible electrocutions or fires. Westport's downtown was all but shut down by flooding and power problems.  </p>
<p class="articleParagraph enarticleParagraph" >In and around New York City, most transporation systems crippled by the storm -- subways, rail and bus lines, airports, ferries, highways, bridges and tunnels -- were back to normal yesterday, and officials predicted few problems for this morning's rush. But Port Authority Trans-Hudson trains between New York and New Jersey were still out and service was doubtful for today.  </p>
<p class="articleParagraph enarticleParagraph" >One of the weekend's worst trouble spots, the flooded Franklin D. Roosevelt Drive, reopened at 7:30 A.M. yesterday. Officials said 55 cars had been partly submerged and abandoned on the drive.  </p>
<p class="articleParagraph enarticleParagraph" >In the storm's wake, the National Weather Service forecast a garland of sumptuous December days, with brilliant sunshine and temperatures in the 40's today and the 50's tomorrow. Returning to Damaged Homes  </p>
<p class="articleParagraph enarticleParagraph" >Businesses and schools that closed early for the storm Friday were expected to be open today. And while the storm for most New Yorkers was just a vivid memory, hundreds faced more lasting effects as they returned to heavily damaged homes in four areas -- Staten Island, Broad Channel in Queens, Sea Gate in Brooklyn and Edgewater Park in the Bronx.  </p>
<p class="articleParagraph enarticleParagraph" >Mayor David N. Dinkins toured the stricken areas yesterday and tried to assure residents that the city would do all it could to help them. "The big thing now, besides cleaning up all of the debris, is disaster relief," he said at a shelter in Far Rockaway.  </p>
<p class="articleParagraph enarticleParagraph" >On southeastern Staten Island, the Mayor saw hundreds of flooded homes in Oakwood and Foxbeach Avenues. Greg Nagy, 36, stood outside his home, its yellow siding ringed by a waist-high line of dirt where the water had risen, piles of sopping furniture and clothing at his feet. "It's gone," he told the Mayor. "There's nothing left except what I'm wearing."  </p>
<p class="articleParagraph enarticleParagraph" >Nearby, 40 residents gathered sullenly and Mr. Dinkins noticed a placard. "This child has a sign that says 'The Forgotten Borough'," he said. "It's not forgotten at all. That's why we're here."  </p>
<p class="articleParagraph enarticleParagraph" >While there were no official damage estimates, some Federal, state and local officials offered assessments, and it was clear that the toll on private homes, businesses, vast stretches of eroded beaches, shattered seawalls and other public property would be extremely high.  </p>
<p class="articleParagraph enarticleParagraph" >Senator Alfonse M. D'Amato, who toured Long Beach, L.I. -- where as much as 100 feet of oceanfront sand 5 feet deep was washed away and hundreds of homes were damaged -- said the region's losses might run "in the billions." He added: "When it is all totaled up, this will be the costliest storm to have hit our region."  </p>
<p class="articleParagraph enarticleParagraph" >Representative Charles E. Schumer, a Democrat of Brooklyn, quoted Army Corps of Engineers experts as saying it would cost $40 million to $50 million to restore eroded beaches in the Rockaways alone.  </p>
<p class="articleParagraph enarticleParagraph" >New York's Congressional delegation urged President Bush to declare the state a disaster area, entitling homeowners and businesses to grants, low-interest loans and other Federal assistance programs.  </p>
<p class="articleParagraph enarticleParagraph" >The extent of the region's losses could also be glimpsed in local damage estimates by New Jersey officials: in Atlantic City, $10.7 million to the boardwalk and beaches; in Dover Township, $20 million to the Ortley Beach boardwalk, $5 million to flooded homes and $3 million to docks; in Berkeley Township, $2.5 million to public property. The Worst, Suffolk Head Says  </p>
<p class="articleParagraph enarticleParagraph" >On Long Island, the Suffolk County Executive, Robert J. Gaffney, estimated that private and public damage in his county alone would run to hundreds of millions of dollars. He said that Fire Island had lost 16 homes and 7 more were in danger, and that 25 more were destroyed in the Hamptons and Quogue. "It's worse than any other storm I've ever seen," he said.  </p>
<p class="articleParagraph enarticleParagraph" >Other officials said the Gateway National Recreation Area, a network of parks in New Jersey, Staten Island, Brooklyn and Queens, had received $15 million in beach damage, and an $8 million road at Sandy Hook, north of Sea Bright, was lost.  </p>
<p class="articleParagraph enarticleParagraph" >In Connecticut, where one to two feet of snow fell in the north and more than 200 motor-vehicle accidents were reported, three people were killed. The heaviest damage hit homes near the coast, where high winds and huge tides drove flood waters ashore.  </p>
<p class="articleParagraph enarticleParagraph" >In East Hartford, a 911 dispatcher, Wilma McKelvey, told yesterday how she helped deliver a baby by telephone during the height of the storm Saturday, when an ambulance could not reach the home of Mr. and Mrs. Michael Amanek in time. In a six-minute call, she talked the father through the delivery as his two other children watched. She had him tie the umbilical cord with a shoelace.  </p>
<p class="articleParagraph enarticleParagraph" >"It seemed like forever, but it just happened so quick," the dispatcher-cum-obstetrician said. "The kids were there and he was screaming out to them, 'You have a baby boy!' Everyone was laughing and crying at the same time." The mother and 7 1/2-pound baby, John, were hospitalized and doing fine yesterday.  </p>
<p class="articleParagraph enarticleParagraph" >Photos: Sea Gate, Brooklyn: Rochelle Cohen looked at the remains of her house, which crumbled on Saturday. (Michelle V. Agins/The New York Times); Sea Bright, N.J.: National Guard officers give residents a ride to check the condition of their homes. (Jim Estrin/The New York Times); Litchfield, Conn.: Residents pulled out the shovels to make paths in the more than 24 inches of snow. (Suzanne DeChillo/The New York Times) (pg. B5)  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>gdis : Disasters/Accidents | gwea : Weather | ghurr : Storms | gcat : Political/General News | gflood : Floods/Tidal Waves | gwere : Routine Weather Reports | gntdis : Natural Disasters/Catastrophes | grisk : Risk News | ncat : Content Types | nrgn : Routine General News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>nyc : New York City | usny : New York | namz : North America | usa : United States | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>WEATHER  | RAIN  | FLOODS  | BLACKOUTS (ELECTRICAL)  | MCFADDEN, ROBERT D </td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>New York Times Digital (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020050413doce01ldx</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020050413dob601eid" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Editorial Desk; A</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>PATH Put the Safety Of Its Passengers First</span>
</td></tr><tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>186 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>6 November 1992</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>28</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 1992 The New York Times Company.  All Rights Reserved.  </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >To the Editor:</p>
<p class="articleParagraph enarticleParagraph" >"Why You Can't Trust the Port Authority" (letter, Oct. 21) by Bill Hine, on the Port Authority Trans-Hudson emergency ventilation facility on Morton Street, neglects the reason the facility was built: the safety of the thousands of commuters who travel between New York and New Jersey daily. Yes, the facility could have been constructed with most of the fan equipment underground.</p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >However, the need for emergency exits, as well as the prohibitive expense and extended time to construct an underground facility, led to the conclusion that the course we took would best serve the public interest.</p>
<p class="articleParagraph enarticleParagraph" >As recently as today, the Morton Street ventilation equipment was invaluable in alleviating a smoke condition at the 14th Street Station.</p>
<p class="articleParagraph enarticleParagraph" >As for the impact of the facility on the planned Hudson River park, PATH worked closely with the Governor's and the Mayor's West Side panel to modify, at great expense, the design and improve the surrounding landscape. RICHARD R. KELLY V.P. & General Manager, PATH New York, Oct. 23, 1992 </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i72101 : Urban Mass Transit Systems | i721 : Urban/Commuter Transit | irailtr : Road/Rail Transport | itsp : Transportation/Logistics</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>ccat : Corporate/Industrial News | nlte : Letters to the Editor (Discontinued from 15th March 2016) | ncat : Content Types | nfact : Factiva Filters | niwe : IWE Filter | nlet : Letters | nrgn : Routine General News | nfce : C&E Exclusion Filter</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>usa : United States | namz : North America</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>Letter  | TRANSIT SYSTEMS  | PORT AUTHORITY TRANS-HUDSON CORP (PATH) </td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>The New York Times Company</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020050413dob601eid</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020050413do7h002c4" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Metropolitan Desk; B</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>Power Loss Strands PATH Commuters  </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>CR</b>&nbsp;</td><td>AP  </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>156 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>17 July 1992</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>4</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 1992 The New York Times Company.  All Rights Reserved.  </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >HOBOKEN, N.J., July 16 -- Up to 100,000 commuters were stranded at the height of the morning rush yesterday after two 4-inch electrical cables feeding the PATH train system lost power. Half of the stranded commuters were stuck in tunnels.  </p>
<p class="articleParagraph enarticleParagraph" >The failure of the 26,000-volt lines shut down the entire system linking New Jersey and New York at 7:30 A.M., the peak of rush hour. Service was restored 90 minutes later.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >"That's about the worst time it could happen," said D. Joy Faber, a spokeswoman for the Port Authority Trans Hudson train system.  </p>
<p class="articleParagraph enarticleParagraph" >The cable failures also knocked out electricity to 4,200 homes and businesses in Jersey City, according to Neil Brown, a spokesman for the Public Service Electric and Gas Company, which owns the cables. He said the utility company could not pinpoint the cause of the failure.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>CO</b>&nbsp;</td><td><br/>pseg : Public Service Electric and Gas Co | pubsel : Public Service Enterprise Group Inc</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i71 : Railroads | i72101 : Urban Mass Transit Systems | i721 : Urban/Commuter Transit | irailtr : Road/Rail Transport | itsp : Transportation/Shipping</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>ccat : Corporate/Industrial News | gcat : Political/General News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>usny : New York | namz : North America | usa : United States | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>TRANSIT SYSTEMS  | DELAYS (TRANSPORTATION)  | PORT AUTHORITY TRANS-HUDSON CORP (PATH) </td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>New York Times Digital (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020050413do7h002c4</td></tr></table><br/></div></div><br/><span></span><div id="article-rec0000020011108do6m00y9z" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/recLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>CLM</b>&nbsp;</td><td>OBITUARIES</td></tr>
<tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>NEWS</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>OBITUARIES </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>3759 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>22 June 1992</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The Record</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>REC</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>© 1992 North Jersey Media Group Inc. Provided by ProQuest Information and Learning. All rights reserved.
 </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >PHILIP E. AFFUSO, 91, of Fort Lee, formerly of West New York, died Saturday. Before retiring in 1977, he owned Willie's Tavern in West New York for 40 years, and also owned a business in Hoboken. Surviving are his wife, Jean Saracino Affuso; a son, Dr. Philip S. Affuso of Ridgewood; a daughter, Therese Valacer of Fort Lee; a sister, Sister Therese, O.P., of Livingston, and eight grandchildren. Mass will be said Tuesday at 10:30 a.m. in St. Nicholas R.C. Church, Palisades Park, with burial in Grove Church Cemetery, North Bergen. Visiting is today from 2 to 4 and 7 to 9 p.m. at the A. K. Macagna Funeral Home, Cliffside Park. Donations to Holy Name Hospital, 718 Teaneck Road, Teaneck, N.J. 07666, would be appreciated. </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >ALFRED D. ALBERTSEN, 84, of Saddle Brook, formerly of Hackensack, died Saturday. Before retiring in 1976, he was manager of office personnel for the Port Authority Trans-Hudson rail line in Hoboken, where he worked for 40 years. He was a parishioner of St. Anne R.C. Church, Fair Lawn, and a member of its Seniors. He also was a member of the Knights of Columbus and the Saddle Brook Seniors. Surviving are a daughter, Virginia E. Albertsen-D'Alessandro of Midland Park, and a granddaughter. His wife, Eleanor J. Widmaier Albertsen, and a sister, Ruth Gebhart, are deceased. Mass will be said Tuesday at 9:30 a.m. in St. Anne Church, with burial in Gate of Heaven Cemetery, Valhalla, N.Y. Visiting is today from 2 to 4 and 7 to 9 p.m. at the Vander Plaat Colonial Home, Fair Lawn. Donations to the American Diabetes Association, P.O. Box 6423, Bridgewater, N.J. 08807, would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >LEE R. BRADLEY, 46, of Lodi, formerly of Hasbrouck Heights, died Sunday. He was a shift supervisor at Hoffmann-La Roche, Nutley, where he worked for 22 years. He was an Army veteran and a member of the Veterans of Foreign Wars, Hasbrouck Heights. He was a girls softball coach for the Lodi Little League and a member of the Hoffmann-La Roche Rifle and Pistol Club. Surviving are his wife, Frances-Ann Arra Bradley; two sons, Mark and Lee Jr., and two daughters, Michele Rice and Luanne Bradley, all of Lodi; his mother, Caryll Nauert Bradley of Hasbrouck Heights; a brother, Kenneth of Napanoch, N.Y., and two grandchildren. Mass will be said Tuesday at 9:30 a.m. in St. Francis de Sales R.C. Church, with burial in St. Nicholas Cemetery, both in Lodi. Visiting is today from 2 to 4 and 7 to 9 p.m. at the Alesso Funeral Home, Lodi. </p>
<p class="articleParagraph enarticleParagraph" >NICHOLAS J. BROPHY, 84, of Ridgefield, formerly of Weehawken, died Saturday. Before retiring in 1980, he was a welding supervisor at the Brooklyn Navy Yard, where he worked for 30 years. He was a parishioner of St. Matthew R.C. Church, Ridgefield. Surviving are his wife, Helen Horton Brophy; a son, James of Kingston, N.Y.; two daughters, Barbara Leddy of Norwood and Maureen Di Staulo of Ridgefield; 10 grandchildren, and three great-grandchildren. Mass will be said Tuesday at 10:30 a.m. in St. Matthew Church, with burial in Madonna Cemetery, Fort Lee. Visiting is today from 2 to 4 and 7 to 9 p.m. at the Blackley Funeral Home, Ridgefield. Donations to the Holy Name Hospital Hospice, 718 Teaneck Road, Teaneck, N.J. 07666, would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >MARY E. LAZUR COLEMAN, 85, of Toms River, formerly of Little Ferry, died Friday. Before retiring, she worked in the housekeeping department of St. Margaret of Cortona R.C. Church, Little Ferry, for 27 years. She was also a parishioner of the church. Surviving are a son, John F. Tomasura of Toms River; a sister, Katherine Andersen of Dumont; three grandchildren, and six great-grandchildren. Her first husband, John Tomasura, and her second husband, Charles Coleman, are deceased. Mass will be said Tuesday at 9:30 a.m. at St. Margaret of Cortona Church, with burial in St. Joseph Cemetery, Hackensack. Visiting is today from 3 to 5 and 7 to 9 p.m. at Trinka-Faustini Funeral Home, Little Ferry. </p>
<p class="articleParagraph enarticleParagraph" >GERTRUDE BINGHAM CONVERY, 76, of Saddle Brook, formerly of Paterson, died Saturday. She was a parishioner of Sacred Heart R.C. Church, Rochelle Park. Surviving is her husband, John; three sons, John of Cincinnati, Joseph of Milford, Mich., and Thomas of Chatham; a daughter, Marian Convery of Lyndhurst; a sister, Margaret Dotzauer of Fair Lawn, and nine grandchildren. Mass will be said Wednesday at 10 a.m. at Sacred Heart Church, with burial in George Washington Memorial Park, Paramus. Visiting will be Tuesday from 2 to 4 and 7 to 9 p.m. at Vander Plaat Memorial Home, Paramus. </p>
<p class="articleParagraph enarticleParagraph" >(2sp) </p>
<p class="articleParagraph enarticleParagraph" >NIELS DAHL, 85, of Clifton, formerly of Passaic, died Saturday. Born in Sweden, he came to the United States in 1927. He was a driver for North Jersey Livery Co., Clifton, and Scarpa Livery Service, Passaic. Previously, he was a machinist for ITT, Nutley, where he worked for 19 years. He was a member of the First Lutheran Church, Masonic Lodge 203, and Svea Lodge 92 of the Vasa Order in America, all in Clifton. He was a member of the Scottish Rite, Lincoln Park, and Salaam Shrine Temple, Livingston. Surviving are his wife, Lily Lindstamer Dahl; a daughter, Lily-Ann Kranking of Southbury, Conn., and two grandchildren. Seven sisters and two brothers are deceased. Services will be Tuesday at 10 a.m. at the First Lutheran Church, with burial in East Ridgelawn Cemetery, Clifton. Visiting is today from 2 to 4 and 7 to 9 p.m., with Masonic services at 8 p.m., at Allwood Funeral Home, Clifton. Donations to the First Lutheran Church Memorial Fund, 1337 Van Houten Ave., Clifton, N.J. 07013, would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >JENNIE CHIODO DiANUZZO, 96, of Lodi died Saturday. Surviving are five sons, Frank of Haledon, Mike of New City, N.Y., Joseph of Pequannock, Nicholas of Parsippany-Troy Hills, and Robert of Wood-Ridge; three daughters, Marie Morris of Westbrook, Conn., Palma Mathe of Apopka, Fla., and Geraldine Di Anuzzo of Lodi; two brothers, Robert Chiodo and Charles Chiodo, both of Lodi; 15 grandchildren, and five great-grandchildren. Her husband, Nicola, and a son, John, are deceased. Mass will be said Wednesday at 9 a.m. at St. Joseph R.C. Church, Lodi, with burial in George Washington Memorial Park, Paramus. Visiting will be Tuesday from 2 to 4 and 7 to 9 p.m. at Alesso Funeral Home, Lodi. Donations to the Lodi Volunteer Ambulance Corps would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >GEORGE T. DiFERDINANDO SR., 68, of Glen Rock, formerly of Paterson and Hawthorne, died Friday. He was co-owner and president of Multiline Insurance Agency of New Jersey, Fair Lawn. He was a graduate of Seton Hall University and Valley Forge Military Academy. An Army veteran of World War II, he was a member of the Riverside Veterans, Paterson. He was a parishioner of St. Catharine R.C. Church, Glen Rock. Surviving are his wife, Sarina Caporlingua DiFerdinando; three sons, Robert of Burlington, Vt., George Jr. of Albany, N.Y., and James of Glen Rock; a daughter, Debora McCoy of Wayne; a brother, Albert of Hawthorne, and seven grandchildren. Services will be Tuesday at 9:30 a.m. at Scillieri Funeral Home, with burial in Calvary Cemetery, both in Paterson. Visiting is today from 2 to 4 and 7 to 9 p.m. Donations to the Northwest Bergen Hospice, 505 Goffle Road, Ridgewood, N.J. 07450, would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >(3s. 2sb. 1sl. 1se) </p>
<p class="articleParagraph enarticleParagraph" >ARMANDO DISCINI, 87, of Suffern, N.Y., died Saturday. He was born in Italy. Before retiring in 1968, he was a molder for American Brake Shoe Co., Mahwah, where he worked for 38 years. He was an Army veteran of World War II. Surviving are two sons, Bernard A. of Suffern and Armand of Mahwah; a brother, Etolo of Suffern, and five grandchildren. His wife, Philomena Napolitano Discini, died last year. Mass will be said Wednesday at 10 a.m. at Sacred Heart R.C. Church, Suffern, with burial in St. Luke Cemetery, Ho-Ho-Kus. Visiting will be Tuesday from 2 to 4 and 7 to 9 p.m. at Scarr Funeral Home, Suffern. </p>
<p class="articleParagraph enarticleParagraph" >HARRY JOHN EGGLINGER, 85, of Ridgefield Park died Sunday. Before retiring in 1968, he was a traffic manager for Woolworth's in the Bronx, where he worked for 41 years. He was a parishioner of St. Francis R.C. Church, Ridgefield Park. Surviving are his wife, Isabel Miller Egglinger; and a brother, Edward of Nokomis, Fla. Mass will be Wednesday at 9:30 a.m. at St. Francis Church, with burial in Holy Name Cemetery, Jersey City. Visiting will be Tuesday from 2 to 4 and 7 to 9 p.m. at Vorhees Funeral Home, Ridgefield Park. </p>
<p class="articleParagraph enarticleParagraph" >(2sp) </p>
<p class="articleParagraph enarticleParagraph" >ANNA JORDAN "MAMMA DEAR" FARRIS, 103, of Paterson died Friday. She was a deaconess and mother of Mission Church of God, Paterson. Surviving are many children, grandchildren, great-grandchildren, and great-great-grandchildren. Also surviving is a dear friend, Leola Davenport of Paterson. Her husband, Leon, a daughter, Nellie Westbrook, and a son, Charles, are deceased. Services will be Tuesday at 10 a.m. at Prophetic Church of God, Paterson, with burial in Mount Holiness Memorial Park, Butler. Visiting is tonight from 7 to 9 at Bragg Funeral Home, Paterson. </p>
<p class="articleParagraph enarticleParagraph" >JOHN ROBERT GREENWOOD, 74, of Bergenfield died Sunday. Before retiring in 1987, he owned Greenwood's Grocery, Bergenfield, for 32 years. He also was an engineer and a pilot. A graduate of the New Jersey Institute of Technology he was a member of the Professional Engineers Association and the National Society of Professional Engineers. Surviving are his wife, Jeanette Fisherman Greenwood; a daughter, Kathryn Steppacher of Philadelphia; three brothers, Thomas of Homestead, Fla., James of Arcadia, Fla., and Alfred of Venice, Fla.; a sister, Maude Greenwood of La Grange, Calif., and two grandsons. Arrangements are by Gutterman-Musicant, Hackensack. Donations to the Alzheimer's Disease and Related Disorders Association, 299 Cherry Hill Road, Suite 305, Parsippany, N.J. 07054, would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >ELIZABETH SLEZAK HANIK, 75, of Ridgefield died Saturday. Before retiring in 1978, she was a bookkeeper for the Social Science Research Council, New York City. She was a member of the Slovak Lutheran Church, New York City. Surviving are five nephews, Kenneth Slezak of Oradell, Dennis Slezak of Norwood, and Douglas Bothwell, Glen Bothwell, and Peter Bothwell, all in California. Cremation will be at Cedar Lawn Crematory, Paterson. Arrangements are by Blackley Funeral Home, Ridgefield. Donations to the Slovak Lutheran Church, 332 E. 20th St., New York, N.Y. 10003, would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >MABEL A. O'BRIEN JONES, 87, of Edgewater died Saturday. Before retiring 12 years ago, she worked for Lever Bros., Edgewater, for 30 years. She was a parishioner of Holy Rosary R.C. Church, Edgewater. Surviving are a son, Robert of Edgewater; 12 grandchildren, and 12 great-grandchildren. Her husband, William R. Sr., and a son, William R. Jr., are deceased. Mass will be said Tuesday at 10:30 a.m. at Holy Rosary Church, with burial in Madonna Cemetery, Fort Lee. Visiting will be today from 2 to 4 and 7 to 9 p.m. and Tuesday from 9 to 10 a.m. at Hunt Funeral Home, Fort Lee. </p>
<p class="articleParagraph enarticleParagraph" >SOPHIE DUNIKOWSKI KANIEWSKI, 81, of Ridgefield, formerly of West New York, died Friday. Before retiring in 1951, she was an operator for New York Telephone Co. Surviving are a son, Raymond of Ridgefield; a daughter, Doris Levonchuk of Garfield, and two grandchildren. Her husband, Julius, died last year. Mass will be said Tuesday at 9:30 a.m. at St. Matthew R.C. Church, Ridgefield, with burial in Holy Name Cemetery, Jersey City. Visiting is today from 2 to 4 and 7 to 9 p.m. at Joseph Z. Konopka Funeral Home, North Bergen. (3s. 2sb. 1sl. 1se) </p>
<p class="articleParagraph enarticleParagraph" >VINCENT THOMAS KELLEHER, 78, of Clifton died Saturday. He was born in Jersey City. Before retiring 13 years ago, he worked for Continental Can Co., Paterson. He was an Army veteran of World War II and a member of Veterans of Foreign Wars Post 7165, Clifton. A former Democratic county committeeman, he was a member of the Clifton Democratic Club. He was a former member of the Volunteer First Aid Squad and the Civil Defense League and a parishioner of St. Philip the Apostle R.C. Church, all in Clifton. Surviving are his wife, Mary Frances Boylan Kelleher; two sons, Michael O'Brien of Peoria, Ill., and George O'Brien of Clifton; a daughter, Barbara Schmitt of Kinnelon; 11 grandchildren, and six great-grandchildren. Mass will be said Wednesday at 10 a.m. at St. Philip the Apostle Church, with entombment at Garden of Memories, Washington Township. Visiting is Tuesday from 2 to 4 and 7 to 9 p.m. at Bizub-Quinlan Funeral Home, Van Houten Avenue, Clifton. </p>
<p class="articleParagraph enarticleParagraph" >ISABEL FANCHER LOWE, 92, of Wantage, formerly of Ridgewood, died Saturday. She was a graduate of Columbia University. She was a member of West Side Presbyterian Church, its Women's Guild, and the College Club, all in Ridgewood. Surviving are two sons, Robert B. of Wantage and Ralph F. of Leesburg, Va., and four grandchildren. Her husband, Ralph, is deceased. Services will be Tuesday at 11:30 a.m. at C. C. Van Emburgh, Ridgewood, with burial in Warwick (N.Y.) Cemetery. Visiting will be Tuesday from 10:30 to 11:30 a.m. Donations to West Side Presbyterian Church, 6 Monroe St., Ridgewood, N.J. 07450, would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >BARBARA McGEE MALONEY, 59, of Wood-Ridge, formerly of Hasbrouck Heights, died Saturday. Before retiring last year, she was a teachers aide and secretary at the Paul Davis School, Wood-Ridge, where she worked for 15 years. Previously, she was a research chemist for Hoffmann-La Roche Inc., Nutley. She was a 1954 graduate of Caldwell College and a parishioner of Assumption R.C. Church, Wood-Ridge. Surviving are her husband, Edward; three daughters, Bonnie Maloney of Hasbrouck Heights and Trish Maloney and Teri Maloney, both of Wood-Ridge; a brother, William McGee in Vermont, and two grandchildren. Mass will be said Tuesday at 10:30 a.m. at Assumption Church, with burial in Immaculate Conception Cemetery, Montclair. Visiting is today from 2 to 4 and 7 to 9 p.m. at Kohler Funeral Home, Wood-Ridge. </p>
<p class="articleParagraph enarticleParagraph" >JAMES F. McKEOWN, 88, of Wayne died Saturday. Before retiring in 1966, he worked for Linen Thread Co., Paterson. He was a parishioner of St. Bonaventure R.C. Church, Paterson. Surviving are two daughters, Ruth Banks and Doris Martin, both of Wayne; a sister, Frances Cavaliere of Branchville, and five grandchildren. His wife, Ella Zerull McKeown, died in 1976. Mass will be said Tuesday at 9:30 a.m. at St. Bonaventure Church, with burial in Holy Sepulchre Cemetery, Totowa. Visiting is today from 2 to 4 and 7 to 9 p.m. at Minchin Funeral Home, Paterson. </p>
<p class="articleParagraph enarticleParagraph" >ANDREW MIHAL, 61, of Upper Saddle River, formerly of Garfield, died Saturday. He owned D&M Sheet Metal Co. for 30 years and Drew-Marco Co. for 21 years, both in East Rutherford. An Army veteran of the Korean War, he was a member of Benda-Roehrich Veterans of Foreign Wars Post 2867 and American Legion Post 170, both in Garfield. He was a parishioner of Saints Peter and Paul Russian Orthodox Cathedral, Passaic, and a member of the Sheet Metal Contractors of Northern New Jersey, the Black Meadow Club, and the Rhododendron Society. He was a member of Masonic Lodge 176 and Elks Lodge 2267, and a former member of the Jaycees, all in Garfield. Surviving are his wife, Irene Mocilenko Mihal; two sons, Mark of Ramsey and Drew of Saddle Brook; two daughters, Melanie A. Mihal of Los Angeles and Debra M. Mihal of Boulder, Colo.; his mother, Anna Masha Mihal of Garfield; a brother, John in New Jersey; a sister, Ann Elsner of Cherry Hill, and three grandchildren. Services will be Wednesday at 10:30 a.m. at Kimak Funeral Home, Carlstadt. Visiting will be tonight from 7 to 9 and Tuesday from 2 to 4 and 7 to 9 p.m. Donations to the American Cancer Society, 20 Mercer St., Hackensack, N.J. 07601, would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >JAMES G. PERA, 82, of Point Pleasant, formerly of Hoboken, died Saturday. Before retiring in 1972, he was a stevedore for Holland America Line, New York City, where he worked for 35 years. Surviving are three sons, Peter of Cliffside Park, James Jr. of Hackensack, and Joseph, and eight grandchildren. His wife, Louise Ruocco Pera, died in 1988. Mass will be said Wednesday at 10 a.m. at St. Ann R.C. Church, Hoboken, with burial in Holy Name Cemetery, Jersey City. Visiting will be tonight from 7 to 9 and Tuesday from 2 to 4 and 7 to 9 p.m. at Lawton-Turso Funeral Home, Hoboken. </p>
<p class="articleParagraph enarticleParagraph" >ANNA G. EMMERTH SCHULIAR, 86, of Dumont, formerly of Bergenfield, died Sunday. Surviving are a son, Edward P. of Fort Lee; two grandchildren, and two great-grandchildren. Her husband, Peter M., died in 1970. Mass will be said Wednesday at 9:30 a.m. at St. Mary R.C. Church, Dumont, with burial in Mount Carmel Cemetery, Tenafly. Visiting will be Tuesday from 2 to 4 and 7 to 9 p.m. at Riewerts Memorial Home, Bergenfield. </p>
<p class="articleParagraph enarticleParagraph" >M. MILDRED CLOWER SOMERVILLE, 90, of Rutherford died Saturday. She was a graduate of Bucknell University. She was a member of the First Presbyterian Church and the Woman's Club and was a volunteer for the American Red Cross, all in Rutherford. She was a member of the Town and Gown Society and the Daughters of the American Revolution. There are no immediate survivors. Her husband, Oliver T., died in 1969. A memorial service will be July 1 at 10:30 a.m. at the First Presbyterian Church. Visiting is today from 2 to 4 and 7 to 9 p.m. at Collins-Calhoun Funeral Home, Rutherford. </p>
<p class="articleParagraph enarticleParagraph" >JOSEPH SONDEJ, 87, of Wallington, formerly of Passaic, died Friday. Before retiring 20 years ago, he worked for Interstate Dyeing and Finishing Co., Passaic, for 30 years. He was a parishioner of Most Sacred Heart of Jesus R.C. Church, Wallington, and a member of the Polish National Union. Surviving are a son, John Sondey of Hatfield, Pa.; a daughter, Helen Jarotski of Wallington; four sisters in Poland; five grandchildren, and four great-grandchildren. His wife, Frances Wojciechowska Sondej, died in 1990. Mass will be said Tuesday at 9 a.m. at Most Sacred Heart of Jesus Church, with burial in St. Mary Cemetery, Saddle Brook. Visiting is today from 2 to 4 and 7 to 9 p.m. at Kamienski Funeral Home, Wallington. </p>
<p class="articleParagraph enarticleParagraph" >SALVATORE SPALLITTA, 71, of Lodi died Saturday. Born in Italy, he came to the United States in 1932. Before retiring in 1977, he was a laminator for D. S. Processing Co., Clifton. He was an Army veteran of World War II. Surviving are his wife, Elaine Friedman Spallitta; a son, Peter of Saddle Brook; two daughters, Jo-Ann Graziano of Rochelle Park and Barbara Spallitta of Lodi; a sister, Jennie Gianni of Lodi, and three grandchildren. Mass will be said Wednesday at 11 a.m. at St. Joseph R.C. Church, Lodi, with entombment at Garden of Memories, Washington Township. Visiting will be tonight from 7 to 9 and Tuesday from 2 to 4 and 7 to 9 p.m. at Alesso Funeral Home, Lodi. Donations to the Lodi Volunteer Ambulance Corps would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >MARGARET "GRAMA" SOLLEY TELLONE, 81, of Carlstadt, formerly of East Rutherford and Jersey City, died Sunday. Before retiring in 1971, she was a medical records supervisor for Martland Medical Center, Newark, where she worked for two years, and previously for the Jersey City Medical Center, where she worked for many years. She was a member of the Carlstadt Seniors. Surviving are three daughters, Dolores Barnes and Barbara Ross Hocking, both of Carlstadt, and Janet Balletto of Jersey City; seven grandchildren, and 12 great-grandchildren. Her husband, Frank, and two sisters, Mary Solley and Gertrude Cavaciuti, are deceased. Services will be Wednesday at 11 a.m. at the First Presbyterian Church, Carlstadt. Visiting will be Tuesday from 2 to 4 and 7 to 9 p.m. at Kimak Funeral Home, Carlstadt. </p>
<p class="articleParagraph enarticleParagraph" >JOSEPH H. WARD, 67, of Bergenfield died Saturday. Before retiring in 1981, he was a repairman for New York Telephone Co., where he worked for 34 years. A Navy veteran of World War II, he was a member of Veterans of Foreign Wars Post 6467, Bergenfield. He was a member of the Telephone Pioneers of America. Surviving are a son, Kenneth of Mahwah; a daughter, Grace DeRubeis of Bergenfield, and a granddaughter. His wife, Loretta, died last year. Mass will be said Tuesday at 10 a.m. at St. John R.C. Church, Bergenfield, with entombment at Garden of Memories, Washington Township. Visiting is today from 2 to 4 and 7 to 9 p.m. at Riewerts Memorial Home, Bergenfield, with a VFW service at 8. Donations to the Bergenfield VFW would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >BLANCHE COE WEYNAND, 84, of Bergenfield died Saturday. Before retiring, she was a saleswoman for Mandee Shops in Bergenfield. She was a member of Clinton Avenue Reformed Church and the Sunshine Club Seniors, both in Bergenfield. Surviving is a son, Guy C. of West Park, N.Y. Her husband, Henry F., died in 1958. Services will be Tuesday at noon at Riewerts Memorial Home, Bergenfield, with burial in George Washington Memorial Park, Paramus. Visiting is today from 2 to 4 and 7 to 9 p.m. Donations to the Clinton Avenue Reformed Church Endowment Fund would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >GENEVIEVE PETERSON WORNER, 68, of Fort Lee, formerly of Leonia, died Sunday. Before retiring in 1984, she was a bookkeeper for Patent Scaffold Co., Fort Lee. Surviving are three daughters, Linda Pacailler of Cliffside Park and Gayle Klausner and Karen Cleary, both of Fort Lee; a son, Thomas of Fort Lee; her mother, Margaret Casper Peterson of Leonia; a sister, Audrey Bronk in North Carolina, and six grandchildren. Her husband, Gilbert, died in 1978. Services will be Tuesday at 9:30 a.m. at Frank A. Patti &Kenneth Mikatarian Funeral Home, Fort Lee, with burial in Brookside Cemetery, Englewood. Visiting is tonight from 7 to 9.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>COLUMN | OBIT</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>Record / Hackensack, New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document rec0000020011108do6m00y9z</td></tr></table><br/></div></div><br/><span></span><div id="article-rec0000020011108do4m00eeq" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/recLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>CLM</b>&nbsp;</td><td>OBITUARIES</td></tr>
<tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>NEWS</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>OBITUARIES </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>3442 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>22 April 1992</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The Record</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>REC</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>© 1992 North Jersey Media Group Inc. Provided by ProQuest Information and Learning. All rights reserved.
 </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >MILES "TOBY" ALEXANDER III, 48, of Paterson died Friday. Surviving are his wife, Wilhelmina Brown Alexander; two sons, Miles and Terrance, his mother Alverta Alexander, and two sisters, Delores Judson and Teresa Alexander, all of Paterson, and a granddaughter. Services are pending. Visiting will Friday from 7 to 9 p.m. at Bragg Funeral Home, Paterson. </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >EDWARD T. BANTA, 68, of New Milford died Sunday. Before retiring in 1989, he was office manager for Blossom Farm Products Co., Fair Lawn, where he worked for 10 years. Previously, he worked for Roman Products Co., South Hackensack. An Army veteran of World War II, he was a member of Veterans of Foreign Wars Post 4290. Surviving are his wife, Louise Challener Banta, and a brother, Joseph of Englewood. A son, Edward T. Jr., died in 1977. Mass will be said Thursday at 11 a.m. at St. Theresa R.C. Church, Cresskill, with burial in Mount Carmel Cemetery, Tenafly. Visiting is today from 2 to 4 and 7 to 9 p.m. at Boulevard Funeral Home, New Milford. </p>
<p class="articleParagraph enarticleParagraph" >BERTHA J. DODDERER BISCHOFF, 94, of Cliffside Park died Sunday. She was a member of the Cliffside Park Seniors. Surviving are a son, Arthur P. Glesmer of Lehigh Acres, Fla.; a granddaughter, and a great-granddaughter. Her husband, Otto G., died last year. Services are tonight at 8 at McCorry Bros. Funeral Home, Cliffside Park. Visiting is today from 2 to 4 and 7 to 9 p.m. </p>
<p class="articleParagraph enarticleParagraph" >PATRICIA G. CONNOLLY, 51, of the Turnersville section of Washington Township, Gloucester County, formerly of Glen Rock, died Monday. She was an operator for New Jersey Exchange Co., Ridgewood. Surviving are two daughters, Beth Connolly of Washington Township and Mary Gravelle of Clarion, Pa., and two grandchildren. Mass will be said Friday at noon at St. Catherine R.C. Church, Glen Rock, with entombment at Madonna Mausoleum, Fort Lee. Visiting will be Thursday at 6:30 p.m. at McGuinness Funeral Home, Washington Township. Donations to Kennedy Memorial Hospital Intensive Care Unit, Hurffville-Cross Keys Road, Turnersville, N.J. 08012, would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >ESTELLA R. DEYO of Hackensack died Sunday. Before retiring in 1989, she was a saleswoman for Jenny Banta Inc. of Ridgewood, where she worked for 40 years. She was a member of Varick Memorial A.M.E. Zion Church, Hackensack. Surviving are two daughters, Arlene Howard of Paterson and Rita Trott of Hackensack; a son, Crawford of Ridgewood, and seven grandchildren. Services will be Thursday at 1 p.m. at Varick Church, with burial in Cedar Lawn Cemetery, Paterson. Visiting is tonight from 7 to 9 at the church. Arrangements are by Earl I. Jones Funeral Home, Hackensack. </p>
<p class="articleParagraph enarticleParagraph" >CAROL ANN MEIXELL DODD, 51, of Largo, Fla., formerly of Teaneck, died April 10. She was hostess at Village Inn, Largo. She was a Navy veteran. Surviving are her husband, Stephen; two sons, Kevin of Seminole, Fla., and Stephen of Largo; two daughters, Stacy Furey of River Edge and Suzanne Dodd of Ridgefield Park; her mother, Tessi Meixell, and a sister, Pat Scully, both of Warrensburg, N.Y., and two grandchildren. Arrangements were by National Cremation Society of Clearwater, Fla. </p>
<p class="articleParagraph enarticleParagraph" >MYRTLE FORBELL JAEGER of North Bergen died Monday. She was a parishioner of Our Lady of Fatima R.C. Church, North Bergen, a charter member of its Rosary Society, a founder of its Altar Guild, director of its Bingo Kitchen, and past president of the Knights of Columbus Ladies Auxiliary in North Bergen. Surviving are a sister, Virginia Goger of Port Richey, Fla., and three grandchildren. Her husband, Robert, and a son, Robert J., are deceased. Mass will be said Friday at 11 a.m. at Our Lady of Fatima Church, with burial in Fairview Cemetery. Visiting will be tonight from 7 to 9 and Thursday from 2 to 4 and 7 to 9 p.m. at Leber Funeral Home, Union City. </p>
<p class="articleParagraph enarticleParagraph" >WILLIAM T. JONES of Paterson died Tuesday. He had been a construction worker for Public Service Electric and Gas Co. A Navy veteran of the Korean War, he was a member of American Legion Post 187, Paterson. Surviving are two sisters, Eileen Caffrey of Paterson and Florence Grery of Clifton; and two brothers, Harold of Paterson and Donald of Oakland. Services will be Friday at 9:15 a.m. at Scillieri Funeral Home, Paterson, with burial in Laurel Grove Cemetery, Totowa. Visiting will be Thursday from 2 to 4 and 7 to 9 p.m. </p>
<p class="articleParagraph enarticleParagraph" >FRANCIS E. JONES, 68, of Moonachie died Sunday. Before retiring in 1985, he was a motorman for the Port Authority Trans-Hudson rail line, Jersey City, where he worked for 35 years. A Marine Corps veteran of World War II, he was a recipient of the Purple Heart and a member of Veterans of Foreign Wars Post 809, Little Ferry. Surviving are his wife, Florence Cuozzy Jones; a son, John of Ridgewood; a daughter, Kathy Gravina of Hillsdale; two brothers, Richard in Florida and Robert of Manchester Township, and four grandchildren. A brother, William, died in January. Arrangements are by Minchin Funeral Home, Paterson. </p>
<p class="articleParagraph enarticleParagraph" >ALBERT E. KUEPPERS, 94, of Wayne, formerly of Brick and Beachwood, died Tuesday. He had been a home builder, an architect, and an engineer. An inventor and a design engineer, several of his creations were patented when he worked for Curtiss-Wright Aeronautical Corp. during World War II. He designed and raced cars at local tracks, and coached and played baseball for area teams. An Army veteran of World War I, he was a member of American Legion Post 240, Prospect Park. He was a member of St. Paul Lutheran Church, Beachwood, and Masonic Lodge 287, Wyckoff. Surviving are a son, Robert C.; two grandchildren, and two great-grandchildren. His first wife, Irene Satton Kueppers, and his second wife, Margaret Weiss Kueppers, are deceased. Services will be Friday at 12:30 p.m. at Vander May Wayne Colonial Funeral Home, Wayne, with burial in Calvary Cemetery, Paterson. Visiting will be Thursday from 3 to 5 and 7 to 9 p.m. </p>
<p class="articleParagraph enarticleParagraph" >ELLEN La FORGE, 34, of Garfield, formerly of Rutherford, died Monday. Survivoring are her parents, Edwin and Phyllis Tabian La Forge Jr., and four brothers, Michael, Edwin, Thomas, and Kevin, all of Garfield; and a sister, Margarite Flemming of Medford. Mass will be said Thursday at 10:00 a.m. at St. Joseph R.C. Church, East Rutherford, with burial in St. Michael Cemetery, South Hackensack. Visiting is today from 2 to 4 and 7 to 9 p.m. at Thomas J. Diffily Funeral Home, Rutherford. Donations to Hackensack Medical Center Hospice, 385 Prospect Ave., Hackensack 07601, would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >EDNA H. GRUENWALD LUCENT, 88, of Hightstown, formerly of Leonia, died Monday. Surviving are three daughters, Gladys Lucent and Jean Lander, both of Hightstown, and Edna Raucci of Glen Rock; five grandchildren, and eight great-grandchildren. Her husband, Joseph C., died in 1961. Services will be Thursday at 10 a.m. at Vander Plaat Colonial Home, Fair Lawn, with burial in George Washington Memorial Park, Paramus. Visiting is today from 2 to 4 and 7 to 9 p.m. </p>
<p class="articleParagraph enarticleParagraph" >JAMES J. McCLAIN, 88, of Alpine died Monday. Before retiring, he was a chauffeur for General Motors Corp., Tarrytown, N.Y. He was a parishioner of Our Lady of Mount Carmel R.C. Church, Tenafly. Surviving are his wife, Dorothy Odell McClain; a son, George A. of Delray Beach, Fla.; three grandchildren, and three great-grandchildren. Graveside services will be Thursday at 1 p.m. at George Washington Memorial Park, Paramus. Arrangements are by Moritz Funeral Home, Closter. </p>
<p class="articleParagraph enarticleParagraph" >WANDA BIELECKI McGRATH, 83, of Ridgefield Park, formerly of Jersey City, died Tuesday. Before retiring in 1974, she was a seamstress. She was a parishioner of St. Francis R.C. Church, Ridgefield Park. Surviving are a daughter, Dolores McGrath-Picone of Palisades Park; a brother, Anthony Bielecki of Paramus; a sister, Irene Andryszewski in Florida; six grandchildren, and 10 great-grandchildren. Her husband, John, and a son, John Jr., are deceased. Mass will be said Thursday at 9 a.m. at St. Francis Church, with burial in Holy Cross Cemetery, North Arlington. Visiting is tonight from 7 to 9 at Vorhees Funeral Home, Ridgefield Park. </p>
<p class="articleParagraph enarticleParagraph" >EVELYN BARNES McNAMEE, 80, of North Bergen died Tuesday. Surviving are a son, Robert J. of Teaneck; a sister, Lorraine Konrad, and two grandsons. Her husband, Robert, is deceased. Services will be Thursday at 11 a.m. at Leber Funeral Home, Union City, with burial in Hackensack Cemetery. Visiting is today from 2 to 4 and 7 to 9 p.m. </p>
<p class="articleParagraph enarticleParagraph" >MORRIS MILLER, 78, of Delray Beach, Fla., formerly of Paterson, died Monday. Before retiring in 1977, he owned Jacobs Pharmacy, Paterson. Previously, he co-owned Miller Pharmacy, Teaneck. He was a member of Temple Emmith, Delray Beach, and a past member of United Brotherhood Congregation, Paterson. Surviving are his wife, Mildred Sussman Miller; two sons, Dr. Edward Miller and David, both in Massachusetts; two brothers, Phillip and Nathan, both of Delray Beach, and two grandchildren. Services are today at 2 p.m. at Robert Schoem's Menorah Chapel, Paramus, with burial in United Brotherhood Cemetery, Saddle Brook. </p>
<p class="articleParagraph enarticleParagraph" >REGINA DAUM MILLER, 94, of Hasbrouck Heights died Tuesday. She was a parishioner of Corpus Christi R.C. Church, Hasbrouck Heights. Surviving are two sons, Eugene of Hasbrouck Heights, and the Rev. Robert Miller of Dunellen; a granddaughter, and a great-granddaughter. Her husband, Richard, and a son, Edward, are deceased. Mass will be said Friday at 10 a.m. at Corpus Christi Church, with burial in Holy Cross Cemetery, North Arlington. Visiting will be Thursday from 2 to 4 and 7 to 9 p.m. at Costa Memorial Home, Hasbrouck Heights. Donations to Tomorrows Children's Fund, in care of Hackensack Medical Center, 30 Prospect Ave., Hackensack, N.J. 07601, would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >MARY G. LoCASTRO PALAZZO, 85, of North Bergen, formerly of West New York, died Tuesday. She was born in Italy. Before retiring in 1970, she was a seamstress. Surviving are five daughters, Faye M. Kronewitter of Bergenfield, Jean M. Compesi of Wood-Ridge, Nickie Rufo of North Bergen, Rose T. Lewis of Old Tappan, and Marie M. Fontana of Union City; a sister, Vincenza Distafano of Cliffside Park; 15 grandchildren, and 13 great-grandchildren. Her husband, Frank, died in 1969. Mass will be said Friday at 10 a.m. at St. John R.C. Church, Bergenfield, with burial in Flower Hill Cemetery, North Bergen. Visiting will be Thursday from 2 to 4 and 7 to 9 p.m. at Riewerts Memorial Home, Bergenfield. Donations to the American Diabetes Association, Bergen County Chapter, 404 State St., Suite 205A, Hackensack, N.J. 07601, would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >LORRAINE G. PAPPALARDO, 39, of New Milford died Monday. She was an office manager for Giorgio Sant Angelo Collectibles, Carlstadt. Surviving are her mother, Rose Corpina Pappalardo of Whiting; and a brother, Lewis A. Pappalardo of East Rutherford. Mass will be said Friday at 10 a.m. at Church of the Assumption, Emerson, with burial in Maryrest Cemetery, Mahwah. Visiting will be today and Thursday from 2 to 4 and 7 to 9 p.m. at Kimak Funeral Home, Carlstadt. </p>
<p class="articleParagraph enarticleParagraph" >JANET M. RYAN PHILLIPS of Englewood died Monday. Before retiring in 1979, she was a school nurse for the Englewood Board of Education, where she worked for 25 years. She was a parishioner of St. Cecilia R.C. Church, Englewood, and a member of its Women's Club. She was a member of the Woman's and Evening Woman's clubs, both in Tenafly, and was a volunteer for the American Heart Association, American Cancer Society, Englewood Hospital, and Southeast Seniors, Englewood. Surviving are two daughters, Carol Phillips of Belmont, Mass., and Sandra Anglund of Glastonbury, Conn.; a sister, Catherine Cunningham of Saybrook, Conn., and a granddaughter. Her husband, John A., died in 1990. A memorial Mass will be said Saturday at 10:30 a.m. at St. Cecilia Church. Arrangements are by Barrett Funeral Home, Tenafly. Donations to the Janet Phillips Memorial Nursing Scholarship Fund, in care of Dwight Morrow High School, Englewood, N.J. 07631, would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >STEPHEN E. POLIGNONE, 38, of North Arlington, formerly of East Rutherford, died Monday. He was a truck driver for Fred Heinzelman & Sons, Carlstadt. Surviving are a daughter, Melanie Polignone in California; his mother, Edna Capozzi Polignone of Rutherford; a brother, Martin of Monroe, N.Y.; and two sisters, Yvonne Butterfield of Monroe and Leslie Polignone of Middletown, N.Y. Services will be Friday at 10 a.m. at Ippolito Stellato Funeral Home, Lyndhurst, with burial in Holy Cross Cemetery, North Arlington. Visiting will be Thursday from 2 to 4 and 7 to 9 p.m. </p>
<p class="articleParagraph enarticleParagraph" >EVELYN EDNA HELMERS RAINSLEY, 79, of Sebring, Ohio, formerly of Wyckoff, died April 13. She was a poet. She was a member of the Wyckoff Reformed Church. She received a master's degree in theater arts from Montclair State College. Surviving are a daughter, Lynn E. Rainsley of Bangor, Maine; a son, Glen E. of Kent, Conn.; a brother, Arthur Helmers of Aurora, Colo., and three grandchildren. Her husband, William E., died last year. Arrangements were by Gednetz-Ruzek Funeral Home, Sebring. </p>
<p class="articleParagraph enarticleParagraph" >ELAINE GINSBURG REITER, 62, of Fair Lawn died Tuesday. She was a self-employed licensed public accountant. She was a member of Deborah Heart and Lung Foundation and B'nai B'rith. Surviving are her husband, Sidney; two daughters, Joanne Reiter of Fort Lee and Diane Fressola of St. Louis; a sister, Lillian Handelman of Queens, and two grandchildren. Services will be Thursday at 11:30 a.m. at Robert Schoem's Menorah Chapel, Paramus, with burial in Beth Moses Cemetery, Pine Lawn, N.Y. A mourning period will be observed at the family residence. Donations to Deborah Heart and Lung Foundation, Browns Mills, N.J. 08015, would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >KATHRYN E. KNOTT SCHAFFER, 91, of Hasbrouck Heights, formerly of Jersey City, died Tuesday. She was a parishioner of Corpus Christi R.C. Church, Hasbrouck Heights, and a life member of its Rosary Society. She was a charter member of Bergen County AARP, a member of Holy Name Hospital Auxiliary, Catholic Daughters of America, Jersey City, and a former member of Hasbrouck Heights Leisure Club. Surviving are two daughters, Audrey Ertle of Hasbrouck Heights and Edna Schappert of Montville; four grandchildren, and eight great-grandchildren. Her husband, George, and two sisters, Mary Hankins and Margaret Wessler, are deceased. Mass will be said Thursday at 9 a.m. at Corpus Christi Church with burial in George Washington Memorial Park, Paramus. Visiting is today from 2 to 4 and 7 to 9 p.m. at Costa Memorial Home, Hasbrouck Heights. </p>
<p class="articleParagraph enarticleParagraph" >DORIS S. MANAHAN SCRIBNER, 69, of Palisades Park died Monday. Surviving are two daughters, Maren Piatt of Chester, N.Y., and Dawn Eby of San Francisco; a son, Foster of Muskego, Wis.; two sisters, Dolly Luff of Oakland and Rose McCord in Arizona; six grandchildren, and a great-grandchild. Her husband, Ralph, is deceased. Arrangements are by Lippincott Funeral Home, Chester. Donations to Second Reformed Church, Hackensack, would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >GEORGE J. STICKEL, 61, of Washington Township died Monday. Before retiring this year, he was a photography media adviser at Bergen Community College, Paramus, where he worked for 23 years. He was an Air Force veteran of the Korean War. Surviving are his wife, Constance M.; and a daughter, Lori Ann Stickel of Saddle Brook. Services will be Friday at 12:15 p.m. at Becker Funeral Home, Westwood, with entombment at Garden of Memories, Washington Township. Visiting will be tonight from 7 to 9 and Thursday from 2 to 4 and 7 to 9 p.m. Donations to Bergen Community College Foundation, 400 Paramus Road, Paramus, N.J. 07652, or Holy Name Hospital Hospice, 718 Teaneck Road, Teaneck, N.J. 07666, would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >JULIA DUDAS SWED, 80, of Clifton died Tuesday. She was a parishioner of St. Paul R.C. Church, Clifton, and a member of Saints Peter and Paul Polish National Catholic Church Leisure Club, Passaic. Surviving are her husband, John; a daughter, Linda Swed of Clifton; three brothers, John Dudas of Forest Lake, Dr. George Dudas of Convent Station, and Albert Dudas of Wayne; and a sister, Anna Schmidt of Clifton. Mass will be said Friday at 10:30 a.m. at St. Paul Church, with entombment at Calvary Mausoleum, Paterson. Visiting will be Thursday from 2 to 4 and 7 to 9 p.m. at Bizub-Quinlan Funeral Home, Lexington Avenue, Clifton. Donations to National Society to Prevent Blindness, 200 Centennial Ave., Suite 203, Piscataway, N.J. 08854-3910, would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >LINA A. THIES, 92, of Newark died Tuesday. Born in Germany, she came to the United States in 1931. She was a member of Third Presbyterian Church, Newark. Surviving is a daughter, Hannah Hamilton of Clifton; two sisters, Frieda Volkmann and Irma Heinsohn, both in Germany; two granddaughters, and a great-grandson. Services will be Friday at 11 a.m. at Allwood Funeral Home, Clifton, with burial in Glendale Cemetery, Bloomfield. Visiting will be Thursday from 2 to 4 and 7 to 9 p.m. </p>
<p class="articleParagraph enarticleParagraph" >JAMES R. TUCKER, 89, of Ridgewood died Friday. Before retiring in 1966, he was executive secretary of the Paramus Chamber of Commerce for three years. Previously, he was an executive in charge of nationwide real estate for Sinclair Oil Corp., New York City, where he worked for 46 years. He was a 33rd-degree Mason. Surviving are his wife, Mary E. Campbell Tucker; a son, Jack E. of Clifton Park, N.Y.; three grandchildren, and two great-grandchildren. Arrangements were by Direct Cremations, Paterson. </p>
<p class="articleParagraph enarticleParagraph" >ANNA FILAKOWSKI VASEL, 78, of Clifton died Monday. Before retiring in 1965, she was a seamstress for Dubanowich Sportswear Co., Passaic, where she worked for 15 years. Previously, she worked for Jay Bee Manufacturing Co., Passaic, for many years. She was a parishioner of St. Michael Byzantine Catholic Cathedral, Passaic. Surviving are her husband, Michael; a daughter, Bernardine A. Stalbaum of Clifton; two sons, Gregory M. of Clifton and Matthew M. of Toms River, and six grandchildren. Mass will be said Thursday at 10:30 a.m. at St. Michael Church, with burial in St. Michael Cemetery, South Hackensack. Visiting is today from 2 to 4 and 7 to 9 p.m., with a parastas at 7:30 p.m. at Shook Funeral Home, Clifton. </p>
<p class="articleParagraph enarticleParagraph" >MICHAEL S. VENTURA, 79, of Fort Lee, formerly of North Bergen, died Sunday. Before retiring in 1975, he was supply manager for Brunswick Laundry Co., Jersey City. Recently, he was a crossing guard in Fort Lee for four years. Surviving are his wife, Theresa Marrone Ventura; two daughters, Patricia Bove of Fort Lee and Carol Ventura of Hackensack; two brothers, Anthony of Dumont and Frank of Ridgefield; a sister, Anna Borchert of Lodi, and a grandson. Mass will be said Thursday at 10 a.m. at Madonna R.C. Chapel, Fort Lee, with burial in Holy Cross Cemetery, North Arlington. Visiting is today from 2 to 4 and 7 to 9 p.m. at A.K. Macagna Funeral Home, Cliffside Park. </p>
<p class="articleParagraph enarticleParagraph" >ROSE F. BRINDISI WITKOWSKI, 77, of Lodi died Tuesday. She was a parishioner of St. Joseph R.C. Church, Lodi. Surviving are two sons, Alexander of Fair Lawn and David of Lodi; a brother Daniel Brindisi of Lodi; six grandchildren, and six great-grandchildren. Her husband, Alexander, died in 1980. Mass will be said Thursday at 10 a.m. at St. Joseph Church, with burial in St. Nicholas Cemetery, Lodi. Visiting is today from 2 to 4 and 7 to 9 p.m. at Alesso Funeral Home, Lodi. </p>
<p class="articleParagraph enarticleParagraph" >JOHN ZMUDSKY, 72, of the Forked River section of Lacey Township, formerly of Garfield and Clifton, died Tuesday. Before retiring in 1980, he owned Van Houten Wine and Liquor Co., Passaic, for 18 years. He was an Army veteran of World War II. He was a parishioner of St. John Kanty R.C. Church, Clifton. Surviving are his wife, Nancy Spinella Zmudsky; two daughters, Rosalind Harrington of Little Falls and Patricia Mascuch of Clifton; a son, John D. of Clifton; three sisters, Antoinette Basch in California, Anne Daly of Garfield, and Helen Mascinka of Oakland, and six grandchildren. A brother, Michael, and a sister, Mary Szep, are deceased. Mass will be said Thursday at 9 a.m. at St. John Kanty Church. Visiting is today from 2 to 4 and 7 to 9 p.m. at Shook Funeral Home, Clifton. Donations to St. Jude Children's Research Hospital, P.O. Box 50, Memphis, Tenn. 38101, would be appreciated.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>OBIT | COLUMN</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>Record / Hackensack, New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document rec0000020011108do4m00eeq</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020050413do3f00r8z" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>New Jersey Weekly Desk; 12NJ</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>For PATH, On-Time Record of 90 Percent  </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By JAY ROMANO  </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>1010 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>15 March 1992</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>1</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 1992 The New York Times Company.  All Rights Reserved.  </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >AT 5:48 A.M. on Jan. 14, a PATH track maintenance crew, riding a flatcar in a tunnel under the Hudson River, spotted smoke coming from a section of the electrified third rail. The crew immediately contacted the system control center in Jersey City by radio, asking that the power to the rail be shut down.  </p>
<p class="articleParagraph enarticleParagraph" >In the control center, a dimly lighted, cavernous room beneath PATH headquarters in Journal Square, blood pressures mounted and pulses began to race. In less than an hour, tens of thousands of commuters would begin boarding trains for their daily trek into Manhattan.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >Each working day nearly 100,000 commuters use the PATH system to travel from stations at Harrison, Newark, Hoboken and Jersey City into Manhattan. Nearly 70 percent of all passengers entering New York City by rail from New Jersey are carried on PATH trains.  </p>
<p class="articleParagraph enarticleParagraph" >The system, which took nearly 35 years to build, was opened in 1908 by the Hudson and Manhattan Railroad, a privately owned corporation, and taken over by the Port Authority of New York and New Jersey in 1962; hence its new name, the Port Authority Trans Hudson system.  </p>
<p class="articleParagraph enarticleParagraph" >At peak rush hours, trains charge through the system's two underground tunnels at 55 miles an hour every three minutes. One tunnel takes commuters to the downtown financial district and the other to stations in lower and midtown Manhattan. In recent months, said Mike Scott, PATH's general superintendent, his system has achieved an on-time performance of better than 90.4 percent in 1991, up from 88.5 the previous year.  </p>
<p class="articleParagraph enarticleParagraph" >"A good day for me is running without delays," he said. "I'd say I have a lot of good days." Nerve Center of System  </p>
<p class="articleParagraph enarticleParagraph" >The nerve center of the PATH system is the John F. Hoban Operations Control Center in Jersey City, where supervisors keep constant watch over a 43-mile network of tracks, 343 passenger cars and 1,200 employees. The officials also monitor the system's 13 passenger stations over 120 closed-circuit video cameras. And when something goes wrong -- like a smoking third rail an hour before the morning rush -- the control center vibrates with life.  </p>
<p class="articleParagraph enarticleParagraph" >As soon as the work crew called in its report on Jan. 14, the PATH official in charge of regulating power to the system studied the electronic maplike representation of the network displayed on the wall in front of him. In each of the two tunnels under the riverbed there are two tracks, eastbound and westbound. With the flick of a switch, the control center official cut power to the section of rail near the track maintenance crew and its flatcar.  </p>
<p class="articleParagraph enarticleParagraph" >Across the room, the trainmaster on duty gazed at a similar electronic board, which looked something like a lighted Christmas tree lying on its side; it showed every inch of track and the location of every piece of rolling stock in the system. A single block of red light represented the now stationary flatcar. In less than an hour, that single red light could trigger an underground gridlock that would paralyze the PATH system.  </p>
<p class="articleParagraph enarticleParagraph" >In another section of the control room, technicians began setting track signals so that trains could operate on the adjacent unaffected track. With one-way traffic, though, only one train every half-hour could make the underwater crossing through that tunnel.  </p>
<p class="articleParagraph enarticleParagraph" >The trainmaster then contacted the PATH car maintenance center in Harrison. Within minutes, another block of light -- a car maintenance crew -- could be seen on the board coursing toward the flatcar in the tunnel.  </p>
<p class="articleParagraph enarticleParagraph" >Upon its arrival, the maintenance crew discovered that a short circuit had literally welded the third rail to one of its supports. The crew worked feverishly in the cramped, dark tunnel to repair the damage. By 6:11 A.M., just 23 minutes after the short was first discovered, the repair was completed.  </p>
<p class="articleParagraph enarticleParagraph" >The crews were cleared from the tracks, power was restored and PATH officials could get back to their routine.  </p>
<p class="articleParagraph enarticleParagraph" >"So many things can go wrong," said Richard C. Leone, chairman of the Port Authority, speaking of the authority's transportation network as a whole. "For commuters, it's the whole experience that affects the way they feel. All the pieces matter." In the case of PATH, he said, "the system, which has been good for years, has gotten even better since we invested in new equipment."  </p>
<p class="articleParagraph enarticleParagraph" >Over the last several years, Mr. Leone said, more than $900 million has been put into equipment, track repair and maintenance; modernization of communications and signaling using fiber-optic cable, and installation of new ventilation equipment. In addition, elevators for disabled passengers are now being installed at most stations to meet new Federal requirements.  </p>
<p class="articleParagraph enarticleParagraph" >One of the most significant additions to the PATH system is the $225 million car maintenance center built in Harrison two years ago. Before the Harrison center was constructed, railroad cars were maintained in a Jersey City location that had only one track in and out.  </p>
<p class="articleParagraph enarticleParagraph" >"We were really stuck," said Mr. Scott, the PATH superintendent. "It was like trying to get the cars in the shop with a funnel."  </p>
<p class="articleParagraph enarticleParagraph" >It was impossible, he said, to repair a car in the middle of a train without tying up the entire train. With the new multitrack shop in Harrison, he said, cars can be separated from trains and worked on individually.  </p>
<p class="articleParagraph enarticleParagraph" >Most PATH trains on the Newark-Jersey City-World Trade Center line are eight-car trains, Mr. Scott said. Others operating in and out of Hoboken and the midtown Manhattan stops are seven-car trains. Each car is designed to hold 129 passengers.  </p>
<p class="articleParagraph enarticleParagraph" >"If we're running late, we get car loadings well in excess of that," Mr. Scott said. "We call it a crush-load."  </p>
<p class="articleParagraph enarticleParagraph" >Photo: Passengers waiting for the PATH train doors to open at the Hoboken station. About 100,000 people commute on the line. (pg. 18) (Keith Meyers/The New York Times)  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>CO</b>&nbsp;</td><td><br/>panynj : Port Authority of New York and New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i71 : Railroads | i72101 : Urban Mass Transit Systems | i721 : Urban/Commuter Transit | i763 : Port/Harbor Operations | irailtr : Road/Rail Transport | iship : Water Transport/Shipping | itsp : Transportation/Shipping</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>gdis : Disasters/Accidents | ccat : Corporate/Industrial News | gcat : Political/General News | grisk : Risk News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>nyc : New York City | usnj : New Jersey | namz : North America | usa : United States | use : Northeast U.S. | usny : New York</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>TRANSIT SYSTEMS  | PORT AUTHORITY TRANS-HUDSON CORP (PATH)  | ROMANO, JAY  | SCOTT, MIKE </td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>New York Times Digital (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020050413do3f00r8z</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020050413do3f00r8y" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>New Jersey Weekly Desk; 12NJ</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>The State of the Manhattan Commute  </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By JAY ROMANO  </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>2038 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>15 March 1992</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>1</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 1992 The New York Times Company.  All Rights Reserved.  </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >A SALESMAN who is a half-hour late for a power breakfast with his biggest customer, or a lawyer who misses most of her daughter's first piano recital, might find it hard to believe. But the daily journey of New Jersey's half-million commuters to and from Manhattan, transportation experts say, has been improving in terms of congestion and delays.  </p>
<p class="articleParagraph enarticleParagraph" >In part, that reflects some substantial investments by the Port Authority of New York and New Jersey in ramp repair and track maintenance. More significant, though, is the fact that the ranks of New Jersey commuters have been dwindling of late. Last year, the total number of individual trips under and over the river, most of them by commuters, was 406.6 million, down about 8 million from 1990.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >"There's a little less congestion on everything," said Richard C. Leone, the Port Authority's chairman, "and that is very noticeable during rush hours. It's probably a byproduct of the recession." Loss of 300,000 Jobs  </p>
<p class="articleParagraph enarticleParagraph" >Fewer New Jerseyans have jobs to commute to, he suggested. Over the last four years, New York City has lost about 300,000 jobs.  </p>
<p class="articleParagraph enarticleParagraph" >The Manhattan-bound commuting network includes PATH and N.J. Transit trains, the Holland and Lincoln Tunnels, the Weehawken and Hoboken ferries and the George Washington Bridge.  </p>
<p class="articleParagraph enarticleParagraph" >Because the network is so interconnected, a problem in one area often spills over into others. Word of a stalled train sends riders scurrying to their cars; a delay at one tunnel evokes a backup at the other. But a little competition within the network can sometimes help relieve congestion. The success of the two small ferry lines has siphoned off about 13,500 commuters a day who would otherwise be using the tunnels, rails, bridge or PATH.  </p>
<p class="articleParagraph enarticleParagraph" >An experimental electronic toll-collection program for specially equipped buses using the Lincoln Tunnel is proceeding well, the Port Authority says, and is helping the traffic flow. And the $900 million spent to improve PATH's efficiency and the $100 million spent on improving approaches to the George Washington Bridge have helped substantially, Mr. Leone said. Improved Performance  </p>
<p class="articleParagraph enarticleParagraph" >Even N.J. Transit trains, sometimes criticized for failing to meet their schedules, have achieved a better on-time performance.  </p>
<p class="articleParagraph enarticleParagraph" >The overall satisfaction with the commuting network expressed by transportation officials, however, is not always shared by individual commuters. Random interviews found some people with new complaints -- a Maplewood resident, for example, who observed an increase in PATH delays in recent months, or a Manalapan resident who complained that the N.J. Transit buses had been more crowded lately.  </p>
<p class="articleParagraph enarticleParagraph" >But in general, those interviewed said either that they had seen some slight improvement in their commute or, more often, that they had seen no particular change in the last year.  </p>
<p class="articleParagraph enarticleParagraph" >One common refrain, however, was unhappiness with the increases in fares paid by many New Jersey commuters. The cost of driving a car back and forth across the bridge or through the tunnels, for example, rose by one-third last April. As of July 1990, fares rose on N.J. Transit trains; the cost of a one-month commuter ticket from Edison, for example, jumped from $139 to $171.  </p>
<p class="articleParagraph enarticleParagraph" >For those Manhattan-bound commuters who use the Port Authority bridge and tunnels to get to work, the future is likely to look a great deal like the present. Mr. Leone says he expects no fare increases in the near future.  </p>
<p class="articleParagraph enarticleParagraph" >An N.J. Transit spokesman, Jeff Lamm, would make no such predictions because of uncertainty over the amount of Federal mass-transit funds that will be available in fiscal 1993.  </p>
<p class="articleParagraph enarticleParagraph" >The Port Authority estimates that by 2015, the commuting population will rise by only 15 percent. Long before then, a number of developments should have improved the transportation network's efficiency.  </p>
<p class="articleParagraph enarticleParagraph" >In light of the success of the ferries, the Port Authority is studying preliminary proposals for additional ferries at other points along the waterfront.  </p>
<p class="articleParagraph enarticleParagraph" >The expansion of the electronic toll-taking program at the Lincoln Tunnel to a greater number of buses and possibly to vans and automobiles should help speed the flow of traffic within the next five years, the Port Authority says.  </p>
<p class="articleParagraph enarticleParagraph" >The major change that is apt to occur first, however, involves N.J. Transit, though its impact will also be felt elsewhere in the commuter network. A recently enacted Federal transportation bill provides funds that will allow nearly all N.J. Transit train passengers to take trains heading directly into New York City. Many of these passengers now use PATH.  </p>
<p class="articleParagraph enarticleParagraph" >PATH SYSTEM  </p>
<p class="articleParagraph enarticleParagraph" >The Port Authority Trans Hudson system carries nearly 100,000 passengers a day into the World Trade Center and to stations from Christopher Street to 33d Street.  </p>
<p class="articleParagraph enarticleParagraph" >A network of buses and trains takes commuters to the main PATH stations in Hoboken, Newark and Jersey City. For example, independently operated vans and buses along with state-owned N.J. Transit buses deliver commuters to Newark's Pennsylvania Station and Journal Square in Jersey City, and passengers switch to PATH trains for the last leg of their journey into Manhattan.  </p>
<p class="articleParagraph enarticleParagraph" >Over the last few years PATH ridership has remained relatively constant at about 52 million passenger-trips each year. But in 1991 it fell by about 600,000.  </p>
<p class="articleParagraph enarticleParagraph" >The PATH fare has remained at $1 each way since 1987, and the Port Authority expects no change in the near future.  </p>
<p class="articleParagraph enarticleParagraph" >N.J. TRANSIT  </p>
<p class="articleParagraph enarticleParagraph" >A public transportation service, N.J. Transit is operated by the New Jersey Transit Corporation, a state agency created in 1979. Its 1,800 buses serve 154 routes, and its 642 trains operate on 12 rail lines.  </p>
<p class="articleParagraph enarticleParagraph" >N.J. Transit buses, along with dozens of independent bus operators, use the George Washington Bridge in Fort Lee, the Lincoln Tunnel in Weehawken and the Holland Tunnel in Jersey City to transport nearly 140,000 commuters into Manhattan on an average weekday.  </p>
<p class="articleParagraph enarticleParagraph" >N.J. Transit trains from the south connect with Amtrak's Northeast Corridor and take some commuters directly into Penn Station in Manhattan. N.J. Transit trains from northern and western New Jersey end their runs at the Hoboken Terminal, where passengers change to PATH or commuter ferries.  </p>
<p class="articleParagraph enarticleParagraph" >Between 1990 and 1991, the total number of passenger trips on direct-to-New York trains dropped from about 23,000 to 21,850.  </p>
<p class="articleParagraph enarticleParagraph" >Over the years, fare increases may have played a role. For example, the monthly commuter fare from Metro Park in Edison to Penn Station in Manhattan was $139 in 1988. In 1991 the same monthly ticket cost $171.  </p>
<p class="articleParagraph enarticleParagraph" >Officials of N.J. Transit are concerned that a rail fare increase may be inevitable if Federal transportation operating subsidies -- the staple of most urban mass-transit systems -- are cut, as anticipated, in President Bush's fiscal 1993 budget.  </p>
<p class="articleParagraph enarticleParagraph" >Within the next few years, N.J. Transit officials say, two major developments are in store for their rail commuters. The projects will be paid for with hundreds of millions of dollars in Federal funds earmarked for capital expenditures. One project will connect the Morris and Essex lines with Amtrak's Northeast Corridor at Kearny. That will make it possible for passengers on those lines to commute directly into Manhattan, bypassing the Hoboken Terminal and the PATH system.  </p>
<p class="articleParagraph enarticleParagraph" >In addition, the Secaucus transfer will allow N.J. Transit passengers from the north to switch at a new station in to trains running directly into New York on Amtrak's Northeast Corridor.  </p>
<p class="articleParagraph enarticleParagraph" >GEORGE WASHINGTON BRIDGE  </p>
<p class="articleParagraph enarticleParagraph" >Owned and operated by the Port Authority, the bridge was opened to traffic in 1931. It is the only 14-lane suspension bridge in the world.  </p>
<p class="articleParagraph enarticleParagraph" >On an average weekday nearly 200,000 auto and bus passengers pass through the 31 tollbooths on the New Jersey side. The number of passenger trips in 1991 was 151.2 million, down 4.2 million from 1990.  </p>
<p class="articleParagraph enarticleParagraph" >The fare, which includes travel in both directions, was raised from $3 to $4 in April 1991. A 20-ticket book for the bridge can be purchased for $60, a daily saving of $1.  </p>
<p class="articleParagraph enarticleParagraph" >Over the last year the Port Authority has been making extensive repairs on the ramps and approaches leading to both the New Jersey and New York sides of the bridge. Those repairs have taken place largely at night and on weekends, causing few delays for commuters. As much as $100 million has been spent in the last few years on maintenance and upgrading of the bridge.  </p>
<p class="articleParagraph enarticleParagraph" >LINCOLN TUNNEL  </p>
<p class="articleParagraph enarticleParagraph" >The tunnel, which is also owned by the Port Authority, connects Weehawken on the Hudson River waterfront with midtown Manhattan. It consists of three tubes and was completed in May 1957.  </p>
<p class="articleParagraph enarticleParagraph" >More than 72,000 automobile passengers and 114,000 bus passengers use the tunnel each weekday. The total number of trips in 1991 was 133.6 million, down 1.9 million from the previous year.  </p>
<p class="articleParagraph enarticleParagraph" >Automobile tolls were raised last year to $4 for a round trip and $72 for a book of 20 tickets.  </p>
<p class="articleParagraph enarticleParagraph" >The Port Authority has completely refurbished the north tube -- the oldest of the three tubes -- and plans to start work on the south tube sometime this year. The repairs have had little impact on commuter traffic flow, since they have been done almost entirely at night.  </p>
<p class="articleParagraph enarticleParagraph" >Recently, electronic toll collection devices were installed, enabling specially equipped buses to move through the toll plaza without stopping. The program is still experimental, but officials say some form of the technology will probably be in place for cars, trucks and buses within the next five years.  </p>
<p class="articleParagraph enarticleParagraph" >HOLLAND TUNNEL Opened to traffic in 1927, the Holland was the first Hudson River vehicular tunnel. More than a mile long, it serves about 56,000 commuters a day traveling between Jersey City and downtown Manhattan. Fares for the Holland Tunnel are the same as for the Lincoln Tunnel.  </p>
<p class="articleParagraph enarticleParagraph" >In 1991, the total number of trips through the Holland Tunnel was 50.2 million, down 300,000 from 1990.  </p>
<p class="articleParagraph enarticleParagraph" >WEEHAWKEN FERRY  </p>
<p class="articleParagraph enarticleParagraph" >The Port Imperial Ferrybus System, owned by Arcorp Properties in Weehawken, has grown rapidly since it began operations in 1987. During the first year of operation the ferry had about 370,000 passenger trips with two Weehawken dockings and two New York dockings, one at 38th Street and the other at the Whitehall Terminal downtown.  </p>
<p class="articleParagraph enarticleParagraph" >By 1991, with four ferries making a combined total of 100 round trips each day, the total number of passengers had grown to more than two million. That also represented an increase of 360,000 over 1990.  </p>
<p class="articleParagraph enarticleParagraph" >Officials of Arcorp Properties say they are now contemplating some additional ferry service as well as improvements to the existing terminals.  </p>
<p class="articleParagraph enarticleParagraph" >The fare for a one-way crossing to 38th Street is $4; for downtown, $5. Midtown fares include free bus service on the New York side to several areas of Manhattan. Monthly tickets can be purchased for $125 to midtown and $140 to downtown.  </p>
<p class="articleParagraph enarticleParagraph" >HOBOKEN FERRY  </p>
<p class="articleParagraph enarticleParagraph" >A joint venture of the Port Authority, Hartz Mountain Industries and Arcorp Properties, the ferry service started in October 1989. The ferry makes about 60 round trips a day between the Hoboken Terminal and Battery Park City near the World Trade Center. Last year there were more than 1.5 million passenger trips, an increase of 400,000 over 1990. The $2 price of a one-way ticket has not changed since the service began.  </p>
<p class="articleParagraph enarticleParagraph" >Another ferry operator, TNT Hydrolines in Atlantic Highlands, carries an average of about 950 passengers a day from two dockings in Monmouth County to the financial district in lower Manhattan.  </p>
<p class="articleParagraph enarticleParagraph" >Fares for the 20-mile, 45-minute trip average $20 for a round trip. Monthly 40-trip ticket books cost $420. By way of comparison, a similar monthly ticket for N.J. Transit into Penn Station would cost about $250, not including the cost of parking at the train station, and the trip would take about an hour and 15 minutes.  </p>
<p class="articleParagraph enarticleParagraph" >Photos: PATH commuters riding escalators as they entered the World Trade Center in Manhattan. Right, the PATH command center at Journal Square in Jersey City. (pg. 1); (Photographs by Keith Meyers/The New York Times)  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>CO</b>&nbsp;</td><td><br/>panynj : Port Authority of New York and New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i721 : Urban/Commuter Transit | i72101 : Urban Mass Transit Systems | i763 : Port/Harbor Operations | irailtr : Road/Rail Transport | iship : Water Transport/Shipping | itsp : Transportation/Shipping</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>gcat : Political/General News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>nyc : New York City | usnj : New Jersey | namz : North America | usa : United States | use : Northeast U.S. | usny : New York</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>TRANSPORTATION  | COMMUTING  | PORT AUTHORITY OF NY AND NJ  | ROMANO, JAY </td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>New York Times Digital (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020050413do3f00r8y</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020050413do2n00n67" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Financial Desk; 3</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>Tech Notes; Trains That Don't Screech  </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By JOHN HOLUSHA  </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>432 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>23 February 1992</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>7</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 1992 The New York Times Company.  All Rights Reserved.  </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >One unpleasant part of commuting by rail is the shriek of steel wheels on steel rails, particularly when trains go around sharp corners. The sounds are produced because the wheels on the outside of a curve have a longer distance to travel than those on the inside.  </p>
<p class="articleParagraph enarticleParagraph" >In a car, differential gears compensate for such unequal travel. But each pair of steel train wheels is solid metal, with no compensation mechanism. As a result, one wheel is dragged along the track. The metal-on-metal scraping produces the high-pitched sound, which can be grating for people who live near tracks or who are waiting on platforms.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >Standard lubricants could reduce the noise by allowing the wheels to slip more easily, but it would also reduce stopping power. In general, railway operators prefer loud trains that can halt when needed to ones that slide quietly through stations unable to stop for passengers.  </p>
<p class="articleParagraph enarticleParagraph" >Now Century Lubricating Oils, a Kansas City-based subsidiary of a British company, says it has developed a product that can cut down on the noise but still allow trains to stop normally. Instead of a traditional grease or oil, the product, called Centrac, is a polymer resin that contains friction-modifying materials.  </p>
<p class="articleParagraph enarticleParagraph" >The resin is formed into the shape of a stick that is four or more inches long and has a cross-section dimension of about a square inch. The sticks are mounted in spring-loaded brackets attached to the underside of the railway cars. One pushes against the face of the wheel that meets the track and the other against the flange, and they work like stick deodorants in applying the product.  </p>
<p class="articleParagraph enarticleParagraph" >Two different sticks are used. The one rubbing against the flange acts like a traditional lubricant and reduces friction. But the one that treats the wheel face modifies the friction properties, the manufacturer says, without reducing stopping power. Since transit systems use different equipment, different brackets must be designed for each application.  </p>
<p class="articleParagraph enarticleParagraph" >Jeff Meloy, an official of Century, said one early test produced a 27.5 decibel reduction in peak noise. The overall reduction in sound was less, but because some of the shrillest frequencies were reduced sharply, the offensiveness of the noise appeared to drop more than the total sound level.  </p>
<p class="articleParagraph enarticleParagraph" >The system has had preliminary tests by the Port Authority Trans-Hudson system, with good initial results, officials said. But the cost of modifying the cars may delay widespread installation.  </p>
<p class="articleParagraph enarticleParagraph" >Diagram of new brake system on trains to eliminate screech  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i71 : Railroads | irailtr : Road/Rail Transport | itsp : Transportation/Shipping</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>gdis : Disasters/Accidents | ccat : Corporate/Industrial News | gcat : Political/General News | grisk : Risk News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>usa : United States | namz : North America</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>RAILROADS  | NEW MODELS, DESIGN AND PRODUCTS  | NOISE  | CENTURY LUBRICATING OILS (CO)  | HOLUSHA, JOHN </td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>New York Times Digital (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020050413do2n00n67</td></tr></table><br/></div></div><br/><span></span><div id="article-rec0000020011110dncm01tbi" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/recLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>CLM</b>&nbsp;</td><td>OBITUARIES</td></tr>
<tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>NEWS</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>OBITUARIES </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>3657 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>22 December 1991</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The Record</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>REC</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>© 1991 North Jersey Media Group Inc. Provided by ProQuest Information and Learning. All rights reserved.
 </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >ELEANOR CORISSA SOKOL ABATE, 52, of Elmwood Park died Friday. She was a machine operator for the Continental Extrusion Co., Cedar Grove. Surviving are a son, James in Virginia; four daughters, Eleanor Mc Laughlin of Elmwood Park, Lisa Dorans of Branchville, Deanna Aste of Paramus, and Lara Abate of Elmwood Park; her mother, Helen Muransky Sokol; two brothers, John Sokol of Clifton and Steven Sokol of Lodi, and seven grandchildren. Services will be Monday at noon at Patrick J. Conte Funeral Home, Elmwood Park, with burial in George Washington Memorial Park, Paramus. Visiting is today from 2 to 4 and 7 to 9 p.m. Donations to the American Cancer Society or a local hospice program would be appreciated. </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >JOSEPH ALESSI SR., 83, of Garfield, formerly of Lodi, died Friday. Before retiring, he worked for the former Raybestos-Manhattan Rubber Co., Passaic. He was a member of the North Jersey Pigeon Club, Garfield. Surviving are a son, Joseph Jr. of Paterson; three daughters, Joann De Francesco of Saddle Brook, Rosemary Stackhouse of Wallington, and Barbara Alessi Burson of Lodi; three brothers, James of Garfield, Michael of Clifton, and Charles of Lodi; two sisters, Rose Veronelli of Bergenfield and Loretta Woodring of Clifton; seven grandchildren, and five great-grandchildren. Services will be Monday at 11 a.m. at Santangelo Funeral Home, Lodi, with burial in St. Mary Cemetery, Saddle Brook. Visiting is today from 2 to 4 and 7 to 9 p.m. </p>
<p class="articleParagraph enarticleParagraph" >ROSE ZIEGLER APPERT, 89, of Oakland, formerly of Bogota and Ridgefield Park, died Saturday. She was born in Switzerland. She was a Gold Star Mother. Surviving are a daughter, Sophie Brown of Lake Ariel, Pa., and seven grandchildren. Her husband, Meinrad, died in 1963, and a son, Joseph, died in 1944 in the Battle of the Bulge. Mass will be said Monday at 10 a.m. at Our Lady of Perpetual Help R.C. Church, Oakland, with burial in St. Joseph Cemetery, Hackensack. Visiting will be Monday from 8:30 to 9:30 a.m. at Oakland Memorial Home. </p>
<p class="articleParagraph enarticleParagraph" >ALIDA GERRITSEN AUPPERLEE, 69, of Hawthorne died Saturday. She was a member of the Covenant Christian Reformed Church, North Haledon, and its Mary and Martha Society. She was a volunteer for the Eastern Christian Children's Retreat. Surviving are her husband, Andrew; a son, David in Michigan; a daughter, Susan Dyer of Hawthorne; a brother, Adrian Gerritsen of Wayne, and six grandchildren. Services will be Tuesday at 11 a.m. at Browning-Forshay Funeral Home, Hawthorne, with burial in Fair Lawn Memorial Cemetery. Visiting will be Monday from 3 to 5 and 7 to 9 p.m. Donations to the Eastern Christian Retreat or the Eastern Christian School Association would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >LIBBY La GRECA BERTOLOTTI, 67, of Garfield died Friday. She was a parishioner of Our Lady of Sorrows R.C. Church, Garfield. Surviving are a son, Leonard of Joliet, Ill.; a daughter, Darla Siniawa, three brothers, Samuel La Greca, James La Greca, and Frank La Greca, and a sister, Marie Astorino, all of Garfield; three grandchildren, and a great-grandchild. Her husband, John, died in 1980. Mass will be said Monday at 9 a.m. in Our Lady of Sorrows Church, with burial in Calvary Cemetery, Paterson. Visiting is today from 2 to 4 and 7 to 9 p.m. at Marrocco Memorial Chapel, Clifton. </p>
<p class="articleParagraph enarticleParagraph" >MARY M. BRASIER, 92, of Lavallette, formerly of Dumont, died Friday. She was a member of Union Church, Lavallette, and its Women's Aid Society. She was also an associate member of the North Reformed Church, Dumont. Surviving are a daughter, Carolyn Stone, and a sister, Louise Clausen, both of Lavallette; five grandchildren, and eight great-grandchildren. Her husband, Edward, died in 1981. Arrangements are by Timothy E. Ryan Home for Funerals, Lavallette. </p>
<p class="articleParagraph enarticleParagraph" >PATRICIA G. BURGESAN, 65, of Naples, Fla., formerly of Cresskill, died Wednesday. Surviving are a sister, Sunday Boyajian of Naples, and a brother, Anthony Mancuso of Cresskill. Services will be Monday at 10:30 a.m. at Fairview Mausoleum. Visiting will be Monday from 9 to 10:30 a.m. at A.K. Macagna Funeral Home, Cliffside Park. </p>
<p class="articleParagraph enarticleParagraph" >LUCY BIVIANO CARBONE, 80, of Jersey City died Friday. She was a parishioner of St. Anne R.C. Church and a member of the Nelson Avenue Block Association, both in Jersey City. Surviving are her husband, Jack; a son, Jack of Jersey City; a daughter, Ann Barthel of Park Ridge; four grandchildren, and two great-grandchildren. Mass will be said Monday at 11 a.m. in St. Anne Church, with burial in George Washington Memorial Park, Paramus. Visiting is today from 2 to 4 and 7 to 9 p.m. at Thomas J. Diffily Funeral Home, Rutherford. Donations to the Tomorrows Children's Fund, in care of Hackensack Medical Center, 30 Prospect Ave., Hackensack, N.J. 07601, would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >PAUL G. ENGOLD, 94, of Haledon died Saturday. Before retiring in 1966, he was chief of the Haledon Police Department, where he worked for 37 years. He was a member of Local 1, Local 80, and Local 114 of the Policemen's Benevolent Association and a life member of the New Jersey State Association of Chiefs of Police, the Passaic County Police Chiefs Association, and the International Association of Chiefs of Police. He had been a member of Haledon Fire Company 1 and was a life member of the Haledon Exempt Firemen's Association. He also was a life member of Pompton Hamilton Masonic Lodge 246 and the Haledon Masonic Club. Surviving are his wife, Mary Carbone Engold; two sons, Harold and Paul, both of Haledon; seven grandchildren, and nine great-grandchildren. Services will be Tuesday at 10 a.m. at Rinaldi-De Luccia-Vander Plaat Funeral Home, Haledon, and 10:30 a.m. at Laurel Grove Memorial Park Chapel, Totowa. Visiting will be Monday from 2 to 4 and 7 to 9 p.m. Donations to the Haledon Fire Department would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >WILLIAM J. FLEMING, 75, of Little Ferry died Friday. Before retiring 13 years ago, he was a conductor on the Port Authority Trans-Hudson rail system for 41 years. He was a member of the PATH union. Surviving are his wife, Elizabeth Bradley Fleming; a son, Donald of Little Ferry; two daughters, Patricia Clough of West Milford and Joyce Cecere of Little Ferry; a brother, John of New York City; six grandchildren, and a great-grandchild. Mass will be said Tuesday at 9:30 a.m. at St. Margaret of Cortona R.C. Church, Little Ferry, with burial in Gate of Heaven Cemetery, Valhalla, N.Y. Visiting will be Monday from 3 to 5 and 7 to 9 p.m. at Trinka-Faustini Funeral Home, Little Ferry. </p>
<p class="articleParagraph enarticleParagraph" >CATHERINE DIEKHAUS GAGGERO, 72, of Lodi died Saturday. Before retiring in 1981, she was an office clerk for Medical Economics in Westwood. She was a member of the Rosary Society of Most Sacred Heart R.C. Church, Rochelle Park, and the AARP. Surviving are her husband, George; a son, Richard of Lodi; three brothers, George of Secaucus, Victor of Union City, and Steve of Jersey City, and two grandchildren. Mass will be said Monday at 9:30 a.m. at Most Sacred Heart Church, with entombment in Holy Cross Mausoleum, North Arlington. Visiting is today from 2 to 4 and 7 to 9 p.m. at Alesso Funeral Home, Lodi. </p>
<p class="articleParagraph enarticleParagraph" >GIOVANNINA GIUDICE, 96, of Wayne, formerly of Paterson, died Saturday. She was born in Italy. Before retiring, she was a seamstress for B &B Coats, Paterson. She was a parishioner of St. Michael R.C. Church, Paterson, and a member of Local 134 of the International Ladies' Garment Workers Union. Surviving are two nieces, Clara Giudice and Theresa Giudice, both of Wayne. Mass will be said Tuesday at 9:30 a.m. in St. Michael Church, with entombment in the Garden of Memories Mausoleum, Washington Township. Visiting will be Monday from 2 to 4 and 7 to 9 p.m. at Rinaldi-De Luccia-Vander Plaat Funeral Home, Haledon. </p>
<p class="articleParagraph enarticleParagraph" >RONALD GULLI, 43, of Clifton died Friday. He was a hairstylist at Gunther's Salon, Maywood, where he worked for 25 years. He was an Army veteran of the Vietnam War and a parishioner of St. Brendan R.C. Church, Clifton. Surviving are his mother, Laura Marrone Gulli, and three brothers, Richard, Patrick, and Frank, all of Clifton. His father, Pasquale, died in 1984. Mass will be said Monday at 10 a.m. at St. Brendan Church, with entombment in Calvary Mausoleum, Paterson. Visiting is today from 2 to 4 and 7 to 9 p.m. at Santangelo Funeral Home, Lodi. </p>
<p class="articleParagraph enarticleParagraph" >FLORENCE COOK HAVILL, 90, of Wayne died Saturday. She was a member of the Wayne Presbyterian Church, the Wayne Golden Agers, and the Packanack Leisure Club. Surviving are a son, Gerard of Wayne; two sisters, Mildred Loock of Rochester, N.Y., Lillian Coapman of San Bernardino, Calif., four grandchildren, and six great-grandchildren. Her husband, Gerard, died in 1977. Services will be Monday at 8 p.m. at Vander May Wayne Colonial Funeral Home, Wayne. Entombment will be in White Haven Memorial Park Mausoleum, Rochester. Visiting will be Monday from 3 to 5 and 7 to 9 p.m. Donations to the Wayne Presbyterian Church, 1200 Alps Road, Wayne, N.J. 07470, would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >SONJA HUSBY, 46, of Southwick, Mass., formerly of Randolph, died Dec. 15. She was a financial services specialist for the American Telephone &Telegraph Co. She was a graduate of Montclair State College and Fairleigh Dickinson University. She was a member of Trinity Lutheran Church, Dover. Surviving are two sons, Richard Forest of Southwick and David Forest of San Francisco; a daughter, Elizabeth Forest of San Francisco; her mother, Aase Rise Husby of Wyckoff; and a brother, Erik Husby of Lexington, Mass. Arrangements were by Pease Funeral Service, Westfield, Mass. Donations to the Palliative Care Center at Western Massachusetts Hospital, 91 E. Mountain Road, Westfield, Mass. 01085, would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >DOROTHY PHILPIT JOHNSON, 78, of Washington Township, formerly of Wood-Ridge, died Saturday. She was a parishioner of Our Lady of Good Counsel R.C. Church, Washington Township. Surviving are a son, John of Cape Coral, Fla.; a daughter, Dorothea Johnson of Washington Township; a sister, Helen Comment of Wood-Ridge, and two grandchildren. Her husband, Hugh, died in 1987. Mass will be said Tuesday at 9:30 a.m. at Our Lady of Good Counsel Church, with burial in George Washington Memorial Park, Paramus. Visiting will be tonight from 7 to 9 and Monday from 2 to 4 and 7 to 9 p.m. at Vander Plaat Memorial Home, Paramus. </p>
<p class="articleParagraph enarticleParagraph" >OTTO "DUTCH" KAELIN, 77, of West New York died Friday. Before retiring, he owned the VanBuren Tavern, West New York. He was an Army veteran of World War II. Surviving is a dear friend, Estelle Baumgarth. Mass will be said Tuesday at 9 a.m. at St. Joseph of the Palisades R.C. Church, West New York, with burial in Madonna Cemetery, Fort Lee. Visiting will be Monday from 2 to 4 and 7 to 9 p.m. at Vainieri Funeral Home, North Bergen. </p>
<p class="articleParagraph enarticleParagraph" >HAZEL M. KESSLER, 81, of Norwood, formerly of Teaneck, died Friday. Before retiring in 1975, she was a secretary for the Royal Insurance Co., New York City, for 49 years. She was a member of the Church of the Holy Communion of Norwood and the Pascack Valley Hospital Auxiliary. Surviving is a sister, Marie Lafrenz of Cresskill. Her husband, Harry, died in 1973. Services will be Monday at 11 a.m. at the Church of the Holy Communion, with burial in George Washington Memorial Park, Paramus. Arrangements are by Anthony R. Pizzi Funeral Home, Northvale. Donations to the Church of the Holy Communion or to the Norwood Volunteer Ambulance Corps would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >ELIZABETH MAKOVICZKI LAFKO, 63, of Saddle Brook died Friday. Before retiring last year, she was an accounts payable clerk in the credit and collections department of Stern's department store, Paramus. She was a parishioner of St. Leo R.C. Church, Elmwood Park. Surviving are her husband, Edward; two daughters, Carol Stever and Dorothy Lafko, both of Saddle Brook; and two brothers, Joseph Makoviczki and George Makoviczki, both in Canada. Mass will be said Monday at 10:30 a.m. at St. Leo Church, with burial in St. Mary Cemetery, Saddle Brook. Visiting is today from 2 to 4 and 7 to 9 p.m. at Kugler Community Home for Funerals, Saddle Brook. </p>
<p class="articleParagraph enarticleParagraph" >JOHN "JACK" OATES, 71, of Martin, Ga., a native of Passaic, died Saturday. He was manager of the Dogwood Motel, Dorville, Ga. He was an Army veteran of World War II. Surviving are a brother, Francis of Lakewood, Ohio; and three sisters, Mary Mahoney of Cranford, Patricia Frantz of Plantation, Calif., and Ruth Calabrese of Rochelle Park. Mass will be said Tuesday at 9:30 a.m. in Most Sacred Heart R.C. Church, Rochelle Park, with burial in Maryrest Cemetery, Mahwah. Visiting will be Monday from 2 to 4 and 7 to 9 p.m. at Vander Plaat Memorial Home, Paramus. </p>
<p class="articleParagraph enarticleParagraph" >LENA DAVENHAUER PERULLI, 86, of Rochelle Park, formerly of Hoboken and Jersey City, died Saturday. She was born in Germany. Before retiring, she was a maintenance worker for the Keuffel &Esser Co. in Morristown. She was a parishioner of Sacred Heart R.C. Church, Rochelle Park. Surviving is a niece, Mary Schnakenburg of Rochelle Park. Mass will be said Tuesday at 10:30 a.m. at Sacred Heart Church, with burial in George Washington Memorial Park, Paramus. Visiting will be Monday from 2 to 4 and 7 to 9 p.m. at Vander Plaat Memorial Home, Paramus. </p>
<p class="articleParagraph enarticleParagraph" >THOMAS P. RAPP, 56, of Clifton died Saturday. He was a tractor-trailer driver for Edison Foam Co. Previously, he worked for Crest Foam Corp., Moonachie. He was a member of Teamsters Local 875, Elmhurst, N.Y. He was an Army veteran of the Korean War and a member of the Clifton American Legion and the Fire Reserve. He was a parishioner of St. Clare R.C. Church, Clifton. Surviving are his wife, Edna B. Van Wagenen Rapp; three sons, Thomas P. Jr. of Grey Bull, Wyo., William of Schenectady, N.Y., and Christian of Garfield; three daughters, Dolores Irwin of Palm Beach, Fla., Judith Post of Hazlet, Mich., and Theresa Femiano of West Paterson; his stepmother, Mildred Rapp of Daytona Beach, Fla.; a brother, Albert of Greenwood Lake, N.Y.; three sisters, Jane Dericks of Andover, Mary Gomes of Hawthorne, and Hon Bradley of Kinnelon, and 11 grandchildren. Mass will be said Tuesday at 9:30 a.m. at St. Clare Church, with burial in East Ridgelawn Cemetery, Clifton. Visiting will be Monday from 2 to 4 and 7 to 9 p.m. at Allwood Funeral Home, Clifton. </p>
<p class="articleParagraph enarticleParagraph" >KEVIN S. RODRIGUEZ, 30, of Ridgefield Park died Thursday. He was a bricklayer and a member of Local 44 of the Bricklayers, Masons, and Plasterers International Union of America. Surviving are his father, John of Ridgefield Park; three sisters, Linda Graner and Barbara Krauss, both of Rockaway, and Theresa Masie of Seaford, N.Y.; and two brothers, John of Bergenfield and James of Rockaway. His mother, Harriet Logan Rodriguez, died in 1986. Mass will be said Monday at 10 a.m. at St. Francis R.C. Church, Ridgefield Park, with burial in George Washington Memorial Park, Paramus. Visiting is today from 2 to 4 and 7 to 9 p.m. at Vorhees Funeral Home, Ridgefield Park. </p>
<p class="articleParagraph enarticleParagraph" >LEONARD SANFORD, 63, of Hawthorne, formerly of Paterson, died Saturday. Before retiring in 1981, he was a driver for Trip Distributors, Paterson. He was an Army veteran of the Korean War. Surviving are his wife, Shirley Potter Sanford; two sons, Stephen of Hackensack and David of Hawthorne; a brother, Donald in Florida; three sisters, Eva Spicer of Wayne, Margaret Walsh of Pequannock, and Annie Pace of Haledon, and a grandchild. Services will be Tuesday at 10 a.m. at Browning-Forshay Funeral Home, Hawthorne, with burial in Laurel Grove Memorial Park, Totowa. Visiting will be Monday from 3 to 5 and 7 to 9 p.m. </p>
<p class="articleParagraph enarticleParagraph" >MARY MAYER SANKOVICH, 86, of Apopka, Fla., formerly of Clifton, died Friday. She was a parishioner of Holy Trinity R.C. Church, Passaic. Surviving are four daughters, Joan Hauser of Manassas, Va., Eileen Miceli of Rockaway, Nancy Eaton of Apopka, and Maryann Caroll of New Providence; two brothers, Fred Mayer of Wauwatosa, Wis., and William Mayer of Elmwood Park; nine grandchildren, and nine great-grandchildren. Her husband, Michael, died in 1987. Mass will be said Tuesday at 10 a.m. at Holy Trinity Church, with entombment in Calvary Mausoleum, Paterson. Visiting will be Monday from 2 to 4 and 7 to 9 p.m. at Bizub-Quinlan Funeral Home, Van Houten Avenue, Clifton. </p>
<p class="articleParagraph enarticleParagraph" >MARGARET BARCLAY SHUART, 93, of Little Falls, formerly of Ramsey, died Friday. Before retiring, she owned Margaret B. Shuart Realty, Ramsey. She was a parishioner of Holy Angels R.C. Church, Little Falls, and St. Paul R.C. Church, Ramsey. She was a member of the Riding Club of New York and had served as a volunteer for the American Red Cross during World War II. Surviving is a niece, Martha Schmitt of Totowa. Her husband, Edmond, died in 1976, and two sisters, Martha Stinson and Hazel Whitehead, also are deceased. Graveside services will be held Tuesday at noon at Maryrest Cemetery, Mahwah. Arrangements are by Norman A. Parker Funeral Home, Little Falls. </p>
<p class="articleParagraph enarticleParagraph" >ANTHONY SIDOTI, 84, of Lincoln Park, formerly of North Bergen, died Friday. Before retiring, he was a plasterer and a member of Plasterers union Local 30, New York City. Surviving are a son, Anthony of North Bergen; a daughter, Margaret Wisniewski of Bayonne; a sister, Angelina Lauria of Fairview; four grandchildren, and a great-grandchild. His wife, Mary Dispenza Sidoti, and a daughter, Frances Webb, are deceased. Services will be Monday at noon at Brennan Funeral Home, West New York, with burial in Flower Hill Cemetery, North Bergen. Visiting is today from 2 to 4 and 7 to 9 p.m. Donations to a favorite charity would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >THERESA EOVINO SIRMARCO, 72, of Fairview, formerly of Cliffside Park, died Friday. Surviving are her husband, Emil; a daughter, Joyce DiBlasio of New Milford; two sisters, Louise Prisco of Saddle Brook and Bridgette Palazzo of Cliffside Park, and two grandchildren. Mass will be said Monday at 9:30 a.m. at Our Lady of Grace R.C. Church, Fairview, with burial in George Washington Memorial Park, Paramus. Visiting is today from 2 to 4 and 7 to 9 p.m. at A.K. Macagna Funeral Home, Cliffside Park. </p>
<p class="articleParagraph enarticleParagraph" >HELEN HASTINGS SLANE, 81, of Cliffside Park died Thursday. Surviving are a son, Michael of Greensboro, N.C.; three brothers, George Hastings of Upper Greenwood Lake, John Hastings of Washingtonville, N.Y., and Thomas Hastings in Virginia; two sisters, Cecelia Cardarelli of Forked River and Katherine Yeaton of Kingston, N.Y.; four grandchildren, and two great-grandchildren. Her husband, John, died in 1981. Mass will be said Monday at 9:45 a.m. in Epiphany R.C. Church, Cliffside Park, with burial in Fairview Cemetery. Visiting is today from 2 to 4 and 7 to 9 p.m. at McCorry Bros. Funeral Home, Cliffside Park. </p>
<p class="articleParagraph enarticleParagraph" >VITO D. SPINELLA, 67, of Little Falls, formerly of Clifton, died Saturday. He was born in Italy. He owned Spinella &Spinella, an architectural firm in Clifton, for 21 years. He was a member of UNICO and a director of the Girls' Club, both in Clifton. He was also a member of the Architect League, Paramus, the New Jersey Society of Architects, Woodbridge, and the American Institute of Architects, Washington, D.C. He was a parishioner of Holy Angels R.C. Church, Little Falls. Surviving are a son, Peter of Verona; a daughter, Diana Nielsen of Little Falls; three brothers, Enzo in Italy, Joseph of Cedar Grove, and Philip of Little Falls, and five grandchildren. His wife, Catherine Falcone Spinella, died last year. Mass will be said Monday at 9:30 a.m. at Holy Angels Church, with burial in Immaculate Conception Cemetery, Montclair. Visiting is today from 2 to 4 and 7 to 9 p.m. at Gaita Memorial Home, Little Falls. </p>
<p class="articleParagraph enarticleParagraph" >REBECCA VROGENDEWEY SWEETMAN, 87, of Clifton died Saturday. She was a member of the Northside Christian Reformed Church, Clifton. Surviving are three daughters, Alice Rogers of Clifton, Ruth Pomykala of Schaumburg, Ill., and Evelyn Verbeck of Merrimack, N.H.; nine grandchildren, and 12 great-grandchildren. Her husband, Gilbert, died in 1959. Services will be Tuesday at 11 a.m. at Vander Plaat Memorial Home, with burial in George Washington Memorial Park, both in Paramus. Visiting will be Monday from 2 to 4 and 7 to 9 p.m. </p>
<p class="articleParagraph enarticleParagraph" >GEORGE WASHINGTON THOMAS, 75, of Paterson died Friday. Before retiring, he worked for the state Department of Transportation. He was a Navy veteran of World War II. Surviving are a son, John of Paterson; four daughters, Mary Grier, Karen Carter, and Elaine Williams, all of Paterson, and Marian Brown of Irvington; two sisters, Marie Cox and Ruth Alexander, both of Paterson; 16 grandchildren, and six great-grandchildren. His wife, Carrie, died in 1975. Services will be Monday at 11 a.m. at Bragg Funeral Home, Paterson, with burial in Fair Lawn Memorial Cemetery. Visiting is tonight from 7 to 9. </p>
<p class="articleParagraph enarticleParagraph" >FLORENCE O'KEEFE WANSER, 90, of Ramsey died Thursday. Before retiring in 1965, she was a secretary for the American Tobacco Co., New York City. She was a parishioner of St. Paul R.C. Church, Ramsey. Surviving are several nephews and nieces, including Margaret Terwilliger of Ramsey. Her husband, Clarence, died in 1970. Graveside services will be Monday at 11 a.m. at George Washington Memorial Park, Paramus. Arrangements are by Van Emburgh-Sneider Funeral Home, Ramsey.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>COLUMN | OBIT</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>Record / Hackensack, New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document rec0000020011110dncm01tbi</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020050415dnbm00ws8" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Metropolitan Desk; B</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>In Eye of Economic Storm, the Port Authority Battens Down  </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By ALAN FINDER with JACQUES STEINBERG  </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>2166 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>22 November 1991</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>1</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 1991 The New York Times Company.  All Rights Reserved.  </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >Buffeted by the recession and humbled somewhat by the mixed results of its ambitious innovations of the last decade, the Port Authority of New York and New Jersey is pulling back. It is no longer planning the incinerators, industrial parks and other economic development projects that occupied it in the 80's.  </p>
<p class="articleParagraph enarticleParagraph" >The huge agency -- the largest, most varied, most powerful port authority in the country -- is returning to basics, concentrating its money, expertise and attention on renewing its prime assets: the region's airports, trans-Hudson bridges and tunnels, the port and the PATH rail line that connects Newark, Jersey City, Hoboken and Manhattan. And even with $1.7 billion in annual revenue, its ability to finance everything it would like to do is uncertain. Following a Mandate  </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >"We're in a period, and I think it will be a sustained period, of limited resources," said Stanley Brezenoff, who became the Port Authority's executive director last year and who once served as Edward I. Koch's first deputy mayor. "Like everyone else in the region, we're experiencing the effects of the recession."  </p>
<p class="articleParagraph enarticleParagraph" >In the flush 1980's the authority experimented with developing everything from real-estate development to a fish market. It spent $25 million on the fish market, called Fishport, in south Brooklyn, which proved to be a dismal failure because the fishing boats and wholesalers never came. Other enterprises have proved more successful. A Japanese company assembles subway cars in an industrial park in Yonkers, and tenants are filling a legal center in downtown Newark (dubbed "tort port" by the agency's staff).  </p>
<p class="articleParagraph enarticleParagraph" >For 70 years, the bi-state agency has continually re-invented itself, largely because its mandate to help develop the local economy is broad. It was created to make order among the 14 different railroads that served the port in 1921. By the next decade, its responsibilities had broadened to include the Holland Tunnel and the George Washington Bridge.  </p>
<p class="articleParagraph enarticleParagraph" >Quasi-autonomous, the Port Authority is influenced primarily by three masters: the Governors of New York and New Jersey, and the economy. Each Governor appoints six of the agency's 12 board members, and each can veto any of the authority's initiatives. One of the prime tasks of the executive director and board chairman, who reign from offices high in the World Trade Center, is to maintain a delicate balance between the states, with each reaping equal dividends from the agency's activities.  </p>
<p class="articleParagraph enarticleParagraph" >Every new director and board chairman has reshaped the Port Authority's role, often with considerable prodding from the Governors. Mr. Brezenoff and Richard C. Leone, who took over as board chairman last year, have made clear that their mandate is to scale back the agency's costs and ambitions.  </p>
<p class="articleParagraph enarticleParagraph" >The top priority now is rebuilding the region's three major airports -- Kennedy International, La Guardia and Newark -- to make them more competitive with the country's newest and best regional airports, Mr. Brezenoff said. The renovations, he said, "are the single most important things that the Port Authority can do."  </p>
<p class="articleParagraph enarticleParagraph" >The authority is proposing one significant and large-scale innovation, to be coupled with the airport rehabilitations: a $1.6 billion light rail line connecting Kennedy and La Guardia Airports with the subway and Long Island Rail Road in Queens. A similar rail line would connect Newark Airport with mass transit in Newark and would cost an estimated $400 million to $500 million.  </p>
<p class="articleParagraph enarticleParagraph" >The rail links, long sought by planners who considered the Port Authority unfriendly to mass transit, will be financed by a new $3 tax on departing airport passengers that was approved last year by Congress.  </p>
<p class="articleParagraph enarticleParagraph" >Other bold initiatives are unlikely, Mr. Brezenoff said, because of the economic climate -- the agency's revenues are essentially flat -- and because it will cost at least $3 billion to rebuild the airports.  </p>
<p class="articleParagraph enarticleParagraph" >"I wish we were here when it looked like the sky's the limit," Mr. Leone said. "The major thing we are doing is exercising a certain amount of prudence." Still a Leader, But Quite Vulnerable  </p>
<p class="articleParagraph enarticleParagraph" >The retooling of the Port Authority comes at a particularly vulnerable time. Besides the ordinary cycles of the economy, the agency confronts broad economic and geopolitical forces that have enabled other regions to compete more effectively with the New York area than at any time this century.  </p>
<p class="articleParagraph enarticleParagraph" >Just last year, New York and New Jersey were eclipsed by Los Angeles as the nation's largest maritime port. The port, which consists primarily of huge container shipping terminals on the waterfronts of Newark and Elizabeth, N.J., has grown far more slowly over the last decade than a half-dozen other ports on the East and West Coasts. But it is still ranked second in the value of cargo that it handles.  </p>
<p class="articleParagraph enarticleParagraph" >Kennedy Airport is still the country's leading center for air cargo. But the deregulation of the airline industry and the growth of other regional airports as trans-Atlantic gateways, including those in Washington, Atlanta, Chicago and Boston, has cut into the proportion of domestic travelers who come to Kennedy to board flights to Europe and beyond. This year Kennedy accounted for 40 percent of the trans-Atlantic passengers departing from the United States, down from 50 percent in 1985, according to the Port Authority.  </p>
<p class="articleParagraph enarticleParagraph" >The difficulties of getting into and out of Kennedy, which is now 30 years old, and is somewhat run-down in appearance, do not enhance its image or that of the region, many critics say.  </p>
<p class="articleParagraph enarticleParagraph" >"It's embarrassing," said George Sternlieb, a professor emeritus at Rutgers University and former director of the university's Center for Urban Research. "You go down to Florida and you look at Tampa-St. Pete, for example, and we look like an old-age home."  </p>
<p class="articleParagraph enarticleParagraph" >Plans to rebuild Kennedy, drawn up four years ago, centered on the construction of a $1 billion central terminal that would connect passengers to individual airline terminals by a monorail. The new terminal, along with the monorail, was canceled last year because the financially pressed airlines, which would have borne most of the cost, could not afford it. Facing Wider Needs With Limited Ability  </p>
<p class="articleParagraph enarticleParagraph" >Some critics say these problems indicate that the Port Authority's difficulties are far deeper than the recession. They contend that the agency no longer has the financial wherewithal, the vision or the political muscle to represent the long-term interests of the New York region.  </p>
<p class="articleParagraph enarticleParagraph" >"It has gone from being an autonomous agency to being a collection of what used to be competence," Dr. Sternlieb said. "Still very heavy in terms of overhead, it's an old-fashioned steam engine that for the moment is just sort of lying in the scrap heap, with people occasionally ripping off the pieces.  </p>
<p class="articleParagraph enarticleParagraph" >"Conceptually, it's never been more needed than now," Dr. Sternlieb said. "It should be the spearhead to provide some regional leadership. What the region is missing is leadership, someone to sell the region."  </p>
<p class="articleParagraph enarticleParagraph" >But others maintain that the outlook is less bleak than it appears. They say that it is vitally important for the agency to focus its efforts on the airports, and that Kennedy could be made much better even without a new main terminal.  </p>
<p class="articleParagraph enarticleParagraph" >Many academics and planners who monitor the Port Authority also said the new emphasis on providing mass transit lines to the airports represented a major, and welcome, departure.  </p>
<p class="articleParagraph enarticleParagraph" >"The airports are now far more important to the region than ever," said Mitchell L. Moss, a professor of planning at New York University and director of the university's Urban Research Center. "The airports are more important to the economy today than the seaports were 100 years ago. They are the modern day equivalent of the steamships and railroads tied into one."  </p>
<p class="articleParagraph enarticleParagraph" >Stephen Berger, who ran the Port Authority between 1985 and 1990, is credited with shifting the agency's attention back to the airports.  </p>
<p class="articleParagraph enarticleParagraph" >"Steve Berger was important," Professor Moss said, "because he got the Port Authority back to basics, back to infrastructure, back to the airports and transportation." Mr. Brezenoff and Mr. Leone "are going to build on Berger," he said.  </p>
<p class="articleParagraph enarticleParagraph" >Historically, the Port Authority grew into a $1.7 billion-a-year enterprise because of the rise of the automobile and the airplane. Revenue from tolls at the George Washington Bridge, the Lincoln and Holland Tunnels and the three bridges connecting New Jersey and Staten Island -- along with money from the airports and from the World Trade Center, which is also owned by the authority -- have underwritten many other agency activities, including the Port Authority Trans-Hudson line and the economic development projects.  </p>
<p class="articleParagraph enarticleParagraph" >The Port Authority was often more sympathetic to the demands of automobile drivers than to planners' longing for more mass transit to ease the area's clogged highways, several experts said. Many people see a new receptivity to rail transit under Mr. Leone and Mr. Brezenoff.  </p>
<p class="articleParagraph enarticleParagraph" >"I sense a new openness and a willingness to consider more than the bottom line," said Richard T. Anderson, president of the Regional Plan Association, a research and advocacy group.  </p>
<p class="articleParagraph enarticleParagraph" >Several Port Authority officials said that even with the rise in international travel from other American airports, the region's airports were holding their own. While losing its proportion of international passenger traffic, Kennedy is still very much No. 1, said David Z. Plavin, the Port Authority's director of aviation.  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Plavin also said Kennedy would be rebuilt, even without the central terminal, so that people could move into and around the airport more easily. For example, the airport's road system is being reconfigured, dividing access to terminals into quadrants so cars and buses no longer have to circle the entire airport, contributing to massive traffic jams, just to get to their destinations.  </p>
<p class="articleParagraph enarticleParagraph" >And earlier this month, three major airlines that have used Kennedy's International Arrivals terminal for decades -- Japan Airlines, Lufthansa and Air France -- said they wanted to build a new terminal at the site of a building abandoned last year by Eastern Airlines.  </p>
<p class="articleParagraph enarticleParagraph" >At La Guardia, the main terminal is being rebuilt, and a new terminal under construction at the airport's eastern end, next to the Trump Shuttle, should be open next spring, Mr. Plavin said. Branching Out With Mixed Results  </p>
<p class="articleParagraph enarticleParagraph" >While the Port Authority has spent significant amounts of money in the 1980's on the airports, particularly Newark, much of the agency's intellectual and political capital was expended on new economic development projects, like industrial parks and incinerators.  </p>
<p class="articleParagraph enarticleParagraph" >The agency's top executives, along with the Governors of New York and New Jersey, wanted the agency to branch out. Industrial parks were created in the South Bronx, Yonkers and Elizabeth, N.J., in response to the Governors' desire for the authority to help more with regional economic development.  </p>
<p class="articleParagraph enarticleParagraph" >The legal center in downtown Newark was built only after Thomas H. Kean, then the New Jersey Governor, said the agency should commit itself to the project if it wanted his support for a 1983 toll increase, a former Port Authority official said.  </p>
<p class="articleParagraph enarticleParagraph" >The overall results were mixed. An incinerator in Newark is performing two technological miracles: burning garbage to create electricity, and making money for the Port Authority in the process. But the agency has worked for a decade to create huge office and residential developments on the waterfronts of Hoboken, Brooklyn Heights and Hunters Point, Queens. So far, not a brick has been laid.  </p>
<p class="articleParagraph enarticleParagraph" >Peter C. Goldmark Jr., who as executive director between 1977 and 1985 presided over the Port Authority's era of expansion, said it was not surprising that the track record was uneven.  </p>
<p class="articleParagraph enarticleParagraph" >"If you try different things, some of them aren't going to work," he said. "If you're only going to play if you bat a thousand, don't even come to the ballpark."  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Goldmark, who now heads the Rockefeller Foundation, said the Port Authority should pursue more of the kinds of difficult, capital-intensive, long-term projects that it does best. He said the agency should build more industrial parks, get involved in hazardous waste disposal and consider helping to repair the city's public works, particularly the East River bridges.  </p>
<p class="articleParagraph enarticleParagraph" >"I think our fundamental problem is that the political leadership doesn't know what to ask of the Port Authority," he said.  </p>
<p class="articleParagraph enarticleParagraph" >But others think that less should be asked, so the agency can concentrate on its primary responsibilities. Mr. Berger, who succeeded Mr. Goldmark, said he thought that real estate and industrial parks and incinerators could be left to other government agencies and the private sector.  </p>
<p class="articleParagraph enarticleParagraph" >"By putting a lot of intellectual energy into these areas, you really didn't do what was crucial -- putting money into the basic infrastructure," he said.  </p>
<p class="articleParagraph enarticleParagraph" >Map "The Port Authority's Reach" shows locations of bridges, airports, tunnels, terminals and other projects. (pg.B2)  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>CO</b>&nbsp;</td><td><br/>panynj : Port Authority of New York and New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i764 : Airports | i763 : Port/Harbor Operations | iairtr : Air Transport | iship : Water Transport/Shipping | itsp : Transportation/Shipping</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>gcat : Political/General News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>nyc : New York City | usny : New York | namz : North America | usa : United States | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>ECONOMIC CONDITIONS AND TRENDS  | BRIDGES AND TUNNELS  | AIRPORTS  | PORT AUTHORITY OF NY AND NJ  | FINDER, ALAN  | STEINBERG, JACQUES  | BREZENOFF, STANLEY </td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>New York Times Digital (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020050415dnbm00ws8</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020050415dnb100y49" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Metropolitan Desk; B</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>Nameless Storm Swamps the Shoreline  </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By ROBERT D. McFADDEN  </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>1574 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>1 November 1991</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>1</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 1991 The New York Times Company.  All Rights Reserved.  </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >A ferocious nameless Atlantic storm masquerading as a Halloween hurricane battered coastal areas of the Northeast and mid-Atlantic states again yesterday and left widespread flooding, smashed homes, boats and sea walls, and heavy property damage. A Long Island fisherman was drowned and at least two people were missing.  </p>
<p class="articleParagraph enarticleParagraph" >The storm -- a monster hundreds of miles wide that lashed like a hurricane but was born in Canadian waters, not the tropics, and fell just short of a hurricane's 74-mile-an-hour winds -- struck late Tuesday, bludgeoned the coast Wednesday and yesterday with seas as high as a two-story house and was wobbling back out to sea last night, centered 300 miles off Virginia.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >"This makes Hurricane Bob look like nothing," Donna Nelson, a spokeswoman for the Red Cross in Boston, told The Associated Press, in a comparison frequently echoed yesterday. Hurricane Bob caused an estimated $780 million in damage when it swept through the region Aug. 19.  </p>
<p class="articleParagraph enarticleParagraph" >The maverick Northeaster was a strange storm with a deceptively dangerous wallop. On the satellite pictures, it did not look like one of those killers with great spiral arms and impenetrably dark clouds charging swiftly toward a hapless coast. It was huge and it packed high winds, but it brought little rain and moved with menacing slowness. And while it devastated the coast, just a short way inland it left many people unfazed and much property untouched. Worst Probably Over  </p>
<p class="articleParagraph enarticleParagraph" >"This is a very dangerous coastal storm -- one of the worst of this type to hit the tri-state area in the last 20 years," said Gene Salerno, a meteorologist with the National Weather Service in New York. The service posted more flood warnings last night, but said the worst was probably over.  </p>
<p class="articleParagraph enarticleParagraph" >From Maine to the Carolinas the storm ravaged thousands of seaside homes and summer cottages, including President Bush's vacation retreat at Kennebunkport, Me., sent thousands of people fleeing to inland shelters, and knocked out power in many areas.  </p>
<p class="articleParagraph enarticleParagraph" >The body of a Long Island man who went fishing late Wednesday afternoon was found early yesterday morning beneath a drawbridge just north of Jones Beach. The state police identified the victim as Arnold Rauch, 40 years old, of East Meadow. The police said he was either blown into the water or slipped off the bridge in the storm.  </p>
<p class="articleParagraph enarticleParagraph" >Four of five Air National Guardsmen whose helicopter went down in the storm 60 miles southeast of Long Island were found riding gargantuan seas in a life raft and plucked to safety early yesterday by Coast Guard vessels that caught a glimpse of their strobe light in the pitch darkness.  </p>
<p class="articleParagraph enarticleParagraph" >The helicopter had crashed Wednesday night on its way back to Long Island from an unsuccessful mission to rescue a sailor adrift 210 miles south of Nantucket. The sailor was rescued late last night by a merchant vessel, the Coast Guard said.  </p>
<p class="articleParagraph enarticleParagraph" >The search continued for the fifth guardsman, and also for a fisherman swept from a rocky point at Narragansett, R.I., on Tuesday.  </p>
<p class="articleParagraph enarticleParagraph" >In Massachusetts, which took a particularly heavy hit from the storm, people reached back for comparisons beyond the August hurricane to the great blizzard of 1978.  </p>
<p class="articleParagraph enarticleParagraph" >In the New York metropolitan area, damage in many sections was heavier than that left by Hurricane Bob. Roads and hundreds of homes were flooded on the shores of Long Island, New Jersey and Connecticut, thousands of people were evacuated inland and there was massive beach erosion throughout the region.  </p>
<p class="articleParagraph enarticleParagraph" >Long Island's eastern and southern shores were hit hardest. In Westhampton Beach, an affluent Suffolk County community where many celebrities have summer homes, the storm destroyed 18 houses and severely damaged 10 others. Some were swept out to sea, while others were whipped into scattered piles of lumber with furnishings half buried in the sand. Tides eight feet above normal deposited several feet of sand in 50 homes, and 100 others were left precariously tottering on wave-battered stilts.  </p>
<p class="articleParagraph enarticleParagraph" >Near record tides pushed by 60-mile-an-hour winds sent the ocean surging into New Jersey's shore communities, forcing evacuations in Sea Bright, Egg Harbor and Monmouth Beach and flooding low-lying areas of Atlantic City and along the Hackensack and Passaic Rivers.  </p>
<p class="articleParagraph enarticleParagraph" >Gov. Jim Florio of New Jersey declared a limited state of emergency last night in Monmouth, Ocean, Atlantic and Cape May Counties and ordered the National Guard "to take the necessary steps to protect the lives, property and safety of the residents of our shore communities." 'Sucked Them Off the Shore'  </p>
<p class="articleParagraph enarticleParagraph" >In Connecticut, tides five feet above normal and winds that gusted over 60 miles an hour lashed and flooded coastal areas from Stamford to Groton, with Milford among the hardest hit communities. Most shore streets in Milford were flooded as waters swelled 10 to 12 feet above normal, the police reported.  </p>
<p class="articleParagraph enarticleParagraph" >Eight men were pulled from the waters of Long Island Sound in three boating accidents on Wednesday near Fairfield, Bridgeport and Westbrook, and two teen-agers in a paddle boat were rescued three miles off Fairfield after "the wind just sucked them off shore," said Petty Officer Joseph Coughlin of the Coast Guard at New Haven.  </p>
<p class="articleParagraph enarticleParagraph" >Most of New York City was spared the brunt of the storm, but flooding and some property damage was reported in Sea Gate, Brooklyn, where expensive homes were evacuated as frantic residents tried unsuccessfully to sandbag a sea wall, and in adjacent Coney Island, where cars were two to three feet under water.  </p>
<p class="articleParagraph enarticleParagraph" >Service on the Port Authority Trans-Hudson rail lines between Hoboken, N.J., and the World Trade Center in lower Manhattan was shut down for about an hour in the afternoon because of flooding caused by rising tides on the Hudson River. The water rose high enough on the tracks to affect the power rail and interfere with the signal system, a spokesman said.  </p>
<p class="articleParagraph enarticleParagraph" >The National Weather Service said the storm, technically called an extratropical storm because it did not originate in the tropics, developed off Nova Scotia early in the week and moved westward -- opposite to the prevailing eastward weather movements -- picking up power from the sea like a hurricane and swirling with counterclockwise winds typical of a hurricane.  </p>
<p class="articleParagraph enarticleParagraph" >The storm ravaged more than 100 seaside homes in Maine, including the President's at Kennebunkport, which reportedly sustained heavy damage, including some flooding and broken windows caused by waves 20 to 30 feet high. Mr. Bush planned to interrupt a political trip to the West to inspect the waterlogged and debris-strewn property tomorrow, White House aides said.  </p>
<p class="articleParagraph enarticleParagraph" >The storm cut off several island and peninsular communities, including Chappaquiddick, which lost access to Martha's Vineyard when four feet of water covered a causeway.  </p>
<p class="articleParagraph enarticleParagraph" >There were a dozen rescues at sea. A Coast Guard helicopter plucked three people from a disabled sailboat in 30-foot seas 70 miles south of Nantucket, then saved three coastguardsmen who had been lowered into a small boat earlier in an attempted rescue. And four New England sailors were saved from a sinking sailboat off the Virginia shore on Wednesday and taken aboard a Coast Guard helicopter. Fairways Under Water  </p>
<p class="articleParagraph enarticleParagraph" >Some of the storm's worst damage was inflicted on eastern Long Island, where eight-foot waves pounded Westhampton Beach, 71 miles east of Manhattan. Roads and bridges throughout the village were under as much as four feet of water, as were the fairways of the country club golf course. Some houses had water halfway up their garage doors. Some areas of Fire Island were evacuated as the storm approached.  </p>
<p class="articleParagraph enarticleParagraph" >The Long Island Lighting Company said 22,000 customers lost power during the storm, but all power was restored by noon yesterday. There were short disruptions on the Long Island Rail Road's Long Beach and Port Washington branches overnight as a result of flooding. Some roads were closed, including the fragile Dune Road in the Hamptons, where 1,000 feet of roadway was washed out when water rushed across the barrier beaches.  </p>
<p class="articleParagraph enarticleParagraph" >In New Jersey, the second highest tide ever recorded at Atlantic City crested at 1:44 A.M. at nine feet above mean high water, three inches short of the record. Normal high tide is about four feet above mean low water. Heavy surf battered the city, but the casinos remained busy, with only a few gamblers taking a break to take in the pounding sea and the crisp night air.  </p>
<p class="articleParagraph enarticleParagraph" >But in a number of shore communities, the damage was worse and people had to be evacuated. Governor Florio said he would tour the area today, and had already ordered state officials to coordinate relief efforts with Federal agencies. 'Have Been Washed Away'  </p>
<p class="articleParagraph enarticleParagraph" >"The devastation throughout these shore counties is serious," Mr. Florio said in declaring a limited emergency. "Communities up and down the New Jersey coast have experienced significant damage from surf, floods and tides. New Jersey's beaches have taken a pounding. Beach fronts and dunes have been washed away."  </p>
<p class="articleParagraph enarticleParagraph" >Photo: Debris from destroyed beach-front homes littered Dune Road yesterday in Westhampton, L.I., in the aftermath of a devastating Atlantic storm. (Associated Press) (pg. B1); In Sea Gate, Brooklyn, emergency workers helped clean up a severely eroded beach yesterday. (Nick Malter for The New York Times) (pg. B4) Map of the East Coast showing path of Hurricane Grace (pg. B4)  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>gdis : Disasters/Accidents | gwea : Weather | ghurr : Storms | gcat : Political/General News | gflood : Floods/Tidal Waves | gwere : Routine Weather Reports | gntdis : Natural Disasters/Catastrophes | grisk : Risk News | ncat : Content Types | nrgn : Routine General News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>nyc : New York City | usa : United States | usny : New York | namz : North America | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>WEATHER  | DROWNINGS  | FLOODS  | MISSING PERSONS  | MCFADDEN, ROBERT D </td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>New York Times Digital (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020050415dnb100y49</td></tr></table><br/></div></div><br/><span></span><div id="article-rec0000020011110dn6c00sjf" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/recLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>CLM</b>&nbsp;</td><td>BUSINESS PEOPLE</td></tr>
<tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>SOUTHWEST BERGEN YOUR TOWN RECORD</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>BUSINESS PEOPLE </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>OLIA SILEO </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>125 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>12 June 1991</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The Record</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>REC</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>© 1991 North Jersey Media Group Inc. Provided by ProQuest Information and Learning. All rights reserved.
 </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >{BUL} Robert Young of Wayne recently was honored by the Bendix Guidance Systems Division of Allied-Signal Inc. as the division's first employee with 55 years of service. George Gartner, president of the division, and Edward Hennessey, board chairman and chief executive officer of Allied-Signal, attended the ceremonies. Young, who started as a production-casting cleaner at Bendix Aviation's West Paterson facility, is currently lead man for the porters at division headquarters in Teterboro. </p>
<p class="articleParagraph enarticleParagraph" >{BUL} Peter F. Truberg of Rutherford has been elected president of the Mechanical Training Officers Association. Truberg is supervisor of training for the car-equipment division of the Port Authority Trans-Hudson (PATH) interstate rail-transit system.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td></td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>COLUMN | NEW JERSEY | BUSINESS | EXECUTIVE</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>Record / Hackensack, New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document rec0000020011110dn6c00sjf</td></tr></table><br/></div></div><br/><span></span><div id="article-rec0000020011110dn6c00sja" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/recLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>CLM</b>&nbsp;</td><td>BUSINESS PEOPLE</td></tr>
<tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>SOUTH BERGEN YOUR TOWN RECORD</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>BUSINESS PEOPLE </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>OLIA SILEO </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>125 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>12 June 1991</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The Record</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>REC</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>© 1991 North Jersey Media Group Inc. Provided by ProQuest Information and Learning. All rights reserved.
 </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >{BUL} Robert Young of Wayne recently was honored by the Bendix Guidance Systems Division of Allied-Signal Inc. as the division's first employee with 55 years of service. George Gartner, president of the division, and Edward Hennessey, board chairman and chief executive officer of Allied-Signal, attended the ceremonies. Young, who started as a production-casting cleaner at Bendix Aviation's West Paterson facility, is currently lead man for the porters at division headquarters in Teterboro. </p>
<p class="articleParagraph enarticleParagraph" >{BUL} Peter F. Truberg of Rutherford has been elected president of the Mechanical Training Officers Association. Truberg is supervisor of training for the car-equipment division of the Port Authority Trans-Hudson (PATH) interstate rail-transit system.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td></td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>COLUMN | NEW JERSEY | BUSINESS | EXECUTIVE</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>Record / Hackensack, New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document rec0000020011110dn6c00sja</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020050415dn3901dy1" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Editorial Desk; 1</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>Transit Authority Can Cut Its Costs</span>
</td></tr><tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>190 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>9 March 1991</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>22</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 1991 The New York Times Company.  All Rights Reserved.  </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >To the Editor:</p>
<p class="articleParagraph enarticleParagraph" >You report that the New York City Transit Authority says that its budget deficit has tripled in seven weeks (news article, Feb. 21). Before raising mass-transit fares again, the Transit Authority should be more imaginative in cutting its costs. Two free ideas:</p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >(1) Phase out tokens, token booths and their employees, replacing them with electronic tickets and passes like those used in the Paris Metro and the Port Authority Trans-Hudson trains. Have merchants sell them aboveground. This would undoubtedly save millions of dollars. There would also be no cash or other commodity in booths or machines to rob.</p>
<p class="articleParagraph enarticleParagraph" >(2) Increase paying ridership by competently designing a farebeater-proof turnstile. The so-called taller, "beater-resistant" models now installed in some subway stations are a joke: one can simply duck underneath. Anyone who has ridden the Paris Metro with its pneumatic gates, or even exited the New York subway through one of those revolving bar doors, knows the Transit Authority can do much better if it mounts a serious effort. BARBARA MARIN New York, Feb. 24, 1991 </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>CO</b>&nbsp;</td><td><br/>mta : Metropolitan Transportation Authority</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i721 : Urban/Commuter Transit | i72101 : Urban Mass Transit Systems | irailtr : Road/Rail Transport | itsp : Transportation/Logistics</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>ccat : Corporate/Industrial News | nlte : Letters to the Editor (Discontinued from 15th March 2016) | ncat : Content Types | nfact : Factiva Filters | niwe : IWE Filter | nlet : Letters | nrgn : Routine General News | nfce : C&E Exclusion Filter</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>nyc : New York City | usny : New York State | namz : North America | usa : United States | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>Letter  | FINANCES  | BUDGETS AND BUDGETING  | TRANSIT SYSTEMS  | NEW YORK CITY TRANSIT AUTHORITY (NYCTA) </td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>The New York Times Company</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020050415dn3901dy1</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020050524dmbi0036a" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Real Estate Desk; 10</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>STREETSCAPES: The Hudson Tubes Powerhouse; A Majestic, Aging Giant  </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By CHRISTOPHER GRAY  </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>889 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>18 November 1990</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>PG</b>&nbsp;</td><td>6</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 1990 The New York Times Company.  All Rights Reserved.  </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >THERE is an inherent dilemma in the appreciation of historic buildings. Is what we value the pristine, original structure, as the designer conceived it? Or is our principal interest the structure as it has aged, even if it is far from its original state?  </p>
<p class="articleParagraph enarticleParagraph" >The 1908 Hudson & Manhattan Railroad powerhouse in Jersey City, now in a state of gradual and perhaps terminal decay, is a useful case study in the matter of our tolerance for the unresolved, the unrestored, the unpackaged.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >What we now call the PATH tunnels -- the acronym means Port Authority Trans Hudson -- were opened in 1908 and known as the Hudson tubes. They were the first fixed Hudson River crossings to New York City. Most of the railroads serving New York deadended in Jersey City or Hoboken, requiring awkward ferry transfers to reach Manhattan.  </p>
<p class="articleParagraph enarticleParagraph" >The new $70 million Hudson & Manhattan Railroad system ran in a U-shape beginning at the Hudson Terminal in Manhattan -- the site of the present World Trade Center -- under the Hudson to Jersey City, up the shore to Hoboken and back into Manhattan again up to Greeley Square at 33d Street.  </p>
<p class="articleParagraph enarticleParagraph" >To power the trains a giant generating station was built near the Jersey City waterfront at 1 Bay Street. Robins & Oakman, a newly established firm, designed the structure -- 200 by 200 feet and 100 feet high, ungainly in form, but exquisite in detail.  </p>
<p class="articleParagraph enarticleParagraph" >The south section had four giant iron stacks and held the steam boiler apparatus with a central tower used for lifting coal. Behind a giant window, the north side contained the turbines which converted the steam into electricity.  </p>
<p class="articleParagraph enarticleParagraph" >Although these parts are awkwardly joined, the shell that covers them is a masterpiece of brickwork. Soaring arched windows run up the sides of the building and a brick cornice is produced by two sets of corbel details woven together, decorated with Greek crosses and square bosses. The unusual cant to the roof of the central tower gives the building a Tuscan or perhaps Egyptian feeling.  </p>
<p class="articleParagraph enarticleParagraph" >On the inside the boiler side was given a rough finish of plain red brick but the cleaner, turbine side was finished in light yellow face brick with an ornamental wrought iron and marble stairway and a glazed observation balcony as elegant as any Fifth Avenue storefront of the period. The station was designed to produce 18,000 kilowatts to run the 140 cars of the rail line.  </p>
<p class="articleParagraph enarticleParagraph" >According to Jerald Weinstein, an industrial archeologist who has studied the power station, the machinery was typical of its period, a time when the first generation of reciprocal engine generators were being replaced by turbines, the predominant type today.  </p>
<p class="articleParagraph enarticleParagraph" >The Hudson & Manhattan started buying rather than generating its power after World War II and, after the Port Authority took over the system in 1962, the machinery was sold for scrap. The building has been largely untended for over 20 years.  </p>
<p class="articleParagraph enarticleParagraph" >INSIDE, the vast rooms are silent. The ash-covered floor of the boiler room is overrun with invading plants, rusting ironwork criss-crosses high above and partly demolished reinforced concrete slabs hang in lumps. The turbine room is a giant, empty space, a gantry crane silently waiting for a new set of turbines to set into place.  </p>
<p class="articleParagraph enarticleParagraph" >Birds wheel in and out -- the windows are long gone -- and the sun pours in through the giant openings illuminating the remains of rusty beams 90 feet up. It is like some ancient, partly ruined cathedral.  </p>
<p class="articleParagraph enarticleParagraph" >Outside, the building is surrounded by vacant lots and low industrial buildings -- it's an elegant surprise in what seems like a forgotten neighborhood. Water leaking down through the walls has left blooms of efflorescence on the elegant brickwork. One of the black iron smokestacks is gone but lacy catwalks still surround the other three. A giant crack is widening at the top of the center tower but the brickwork looks otherwise sound -- not a piece of the sophisticated corbel work is out of place.  </p>
<p class="articleParagraph enarticleParagraph" >The building is now jointly owned by the Jersey City and the Port Authority, neither of which have any plans for it. The automatic response to a situation like this is typically a grand new use and an expensive restoration program.  </p>
<p class="articleParagraph enarticleParagraph" >The brick is cleaned, windows replaced, weeds cleaned out and a shopping mall or arts center is installed under a rubric like "The Power Plant." The building is saved -- finis.  </p>
<p class="articleParagraph enarticleParagraph" >But such a course bottles up, for neat consumption, the raw, messy life cycle of the building -- its design, its function, its obsolescence, its decay. It supposes that the building is an isolated artifact, needing only museological restoration to its newborn state -- that the interplay of light and weather and deterioration and silence and discovery is of minor importance.  </p>
<p class="articleParagraph enarticleParagraph" >In the case of the Hudson & Manhattan powerhouse, neither the issue of demolition nor restoration is likely to be joined soon. The area is an empty one, notwithstanding the Path station and some office development several blocks away.  </p>
<p class="articleParagraph enarticleParagraph" >For now at least it will remain a majestic ruin and it probably will never look any better than it does now.  </p>
<p class="articleParagraph enarticleParagraph" >Photo: The Hudson & Manhattan Railroad Powerhouse in Jersey City in the 1960's. (Port Authority Archives)  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i1 : Energy | i72101 : Urban Mass Transit Systems | i85 : Real Estate Transactions | i721 : Urban/Commuter Transit | icre : Construction/Real Estate | irailtr : Road/Rail Transport | ireest : Real Estate | itsp : Transportation/Shipping</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>ccat : Corporate/Industrial News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>nyc : New York City | namz : North America | usa : United States | use : Northeast U.S. | usny : New York</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>TRANSIT SYSTEMS  | ELECTRIC LIGHT AND POWER  | HUDSON TUBES POWERHOUSE (JERSEY CITY)  | GRAY, CHRISTOPHER </td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>New York Times Digital (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020050524dmbi0036a</td></tr></table><br/></div></div><br/><span></span><div id="article-rec0000020011115dm7u00ykd" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/recLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>NEWS</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>TOLL-ROAD BOSSES STILL WHITE, MALE AGENCIES PROMOTE `THEIR OWN' </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By Eugene Kiely, Record Staff Writer </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>1192 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>30 July 1990</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The Record</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>REC</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>© 1990 North Jersey Media Group Inc. Provided by ProQuest Information and Learning. All rights reserved.
 </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >Like the knights of King Arthur's court, the commissioners and top executives of the New Jersey Highway Authority convene monthly at a round table, giving the impression that each individual has equal power. </p>
<p class="articleParagraph enarticleParagraph" >But it's not only who sits at the table, but who doesn't. </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >Of the 16 top officials at the last round-table meeting in Woodbridge, there were no Hispanics, only one woman, and just two blacks. </p>
<p class="articleParagraph enarticleParagraph" >With last week's firing of Deputy Director Odella Welch _ the first black executive in the agency's history _ there will be no women, no Hispanics, and one black, commissioner Julian Robinson, who holds a part-time, unpaid position. </p>
<p class="articleParagraph enarticleParagraph" >"It is the old-boy network at its worst," Robinson said, explaining how a state agency has come to resemble an all-white, all-male club. </p>
<p class="articleParagraph enarticleParagraph" >Regardless of whether the administration in Trenton was Democratic or Republican, liberal or conservative, the Highway Authority and Turnpike Authority over the years have operated in one way. They have run a patronage system that dispenses jobs, especially high-paying jobs, to people much like themselves. </p>
<p class="articleParagraph enarticleParagraph" >In the 1990s _ a generation after the civil rights movement _ the agencies are slowly integrating, and with it they are experiencing growing pains. </p>
<p class="articleParagraph enarticleParagraph" >It wasn't until May 1987 that the Highway Authority adopted an affirmative action plan. The Turnpike Authority waited until August 1988 to adopt its plan. </p>
<p class="articleParagraph enarticleParagraph" >The late move to bring a racial balance to the toll-road authorities stands in stark contrast to some other independent authorities, including the New Jersey Sports and Exposition Authority and the Port Authority of New York and New Jersey. Both agencies acted in the early 1970s to improve hiring practices, and both have a higher percentage of minority employees. </p>
<p class="articleParagraph enarticleParagraph" >Since adopting affirmative action plans, the Highway Authority has made slow progress, while at the turnpike the situation has worsened. </p>
<p class="articleParagraph enarticleParagraph" >In two years, the percentage of white employees at the turnpike has increased, up from 1,811 (90.5 percent) to 1,913 (91 percent). But the percentage of Hispanics dropped from 2 percent to 1.5 percent, and the percentage of blacks declined from 7.3 percent to 7.2 percent. </p>
<p class="articleParagraph enarticleParagraph" >Moreover, the Turnpike Authority's affirmative action plan set a goal to reduce the number of black males by three _ claiming that 115 black males, which was 5.75 percent of its total work force, was more than adequate. </p>
<p class="articleParagraph enarticleParagraph" >"That doesn't make any sense," David Goldberg, the agency's chairman, said. "It certainly does suggest that it is timely to review the plan." </p>
<p class="articleParagraph enarticleParagraph" >Executive Director Donald Watson, who became the first black administrator in the Turnpike Authority's history two months ago, said he wants the turnpike "to be representative of the workforce in the state." </p>
<p class="articleParagraph enarticleParagraph" >By that measure, both the turnpike and the highway authorities fail. </p>
<p class="articleParagraph enarticleParagraph" >David Felsheim, an analyst at the state Department of Labor, said the state's workforce in 1989 was 84.5 percent white, 11.8 percent black, and 7.5 percent Hispanic. The totals add up to more than 100 percent because Hispanics are sometimes counted twice, both as Hispanic and again as white or black. </p>
<p class="articleParagraph enarticleParagraph" >Florio administration officials _ pointing to recent hirings of Watson and Shirley DeLibero as executive director of NJ Transit _ say they are moving to rectify past mistakes. </p>
<p class="articleParagraph enarticleParagraph" >State Transportation Commissioner Thomas Downs, who is a member of all the state's independent transportation authorities, claims Welch's firing at the Highway Authority was not racist, as Robinson has charged, but necessary to help streamline a bloated, out-of-control agency. </p>
<p class="articleParagraph enarticleParagraph" >Down admitted, however, that the authority "has been derelict in the past" when it came to equal employment practices and vowed to make changes. </p>
<p class="articleParagraph enarticleParagraph" >"The need to change the complexion of this authority is paramount," Downs said. </p>
<p class="articleParagraph enarticleParagraph" >But Robinson, a Highway Authority commissioner for 12 years, has heard it before and isn't willing to sit still. He said he plans to meet with the few other blacks who sit on independent authorities to "compare notes." </p>
<p class="articleParagraph enarticleParagraph" >"There is an unconscious racism at the authority . . . that causes people to get passed over in promotions and raises, or getting hired in the first place," Robinson said. </p>
<p class="articleParagraph enarticleParagraph" >A chief complaint of Robinson's is that the Highway Authority rarely goes outside the agency to hire executives _ with the notable exception of Welch. </p>
<p class="articleParagraph enarticleParagraph" >"The Highway Authority, through its spokesperson, will tell you, `We pride ourselves on promoting from within.' But if you talk about an agency that since its inception had no blacks, you don't have blacks and women to move up. </p>
<p class="articleParagraph enarticleParagraph" >"I'm not willing to wait 15, 20 years for these people to move up," he added. "If you want some blacks, you have to go out and find them." </p>
<p class="articleParagraph enarticleParagraph" >Although the agency has raised the percentage of black employees from 7.9 percent to 11.5 percent, blacks represent only 8.7 percent of the professional staff, just eight out of 92 employees. There are no Hispanic professionals. </p>
<p class="articleParagraph enarticleParagraph" >Willie Jones, the agency's affirmative action manager, said the "resource pool is not that strong" at the authority for blacks and other minorities. </p>
<p class="articleParagraph enarticleParagraph" >But a black Highway Authority employee, who asked not to be identified for fear of retaliation, said a lack of qualified minorities isn't the problem. </p>
<p class="articleParagraph enarticleParagraph" >"It's not so much being black, but who you know," the employee said. "There is a lot of cronyism that goes on here." </p>
<p class="articleParagraph enarticleParagraph" >Commissioner Robinson said the firing of Welch reaffirms the belief that cronyism, racism, and sexism are alive and well at the Highway Authority. </p>
<p class="articleParagraph enarticleParagraph" >"I dare say the termination of someone like Odella Welch confirms for all blacks who work at the authority that it's racist," he said. "They'll say, `If at that level she can be dealt with summarily like that, what hope do I have?'" </p>
<p class="articleParagraph enarticleParagraph" >(TABLE, Page a06) NEW JERSEY HIGHWAY AUTHORITY JUNE 30, 1990 WHITES 1,092 86.9% BLACKS 144 11.5% HISPANICS 18 1.4% OTHER 5 ** Total 1,257 </p>
<p class="articleParagraph enarticleParagraph" >NEW JERSEY TURNPIKE AUTHORITY JUNE 30, 1990 </p>
<p class="articleParagraph enarticleParagraph" >Whites 1,913 91.0% Blacks 152 7.2% Hispanics 28 1.4% Other{CT} 10 ** Total 2,103 </p>
<p class="articleParagraph enarticleParagraph" >NEW JERSEY SPORTS AND EXPOSITION AUTHORITY JUNE 30, 1990 </p>
<p class="articleParagraph enarticleParagraph" >WHITES 911 79.6% BLACKS 148 12.9% HISPANICS 78 6.8% OTHER 8 ** TOTAL 1,145 </p>
<p class="articleParagraph enarticleParagraph" >PORT AUTHORITY OF NEW YORK AND NEW JERSEY DEC. 31, 1989 WHITES 5,771 68.4% BLACKS 1,827 21.7% HISPANICS 566 6.7% OTHER 270 3.2% Total 8,434 </p>
<p class="articleParagraph enarticleParagraph" >NEW JERSEY POPULATION Dec. 31, 1989 WHITES 3,989,000 84.5% BLACKS 470,000 11.8% HISPANICS 302,000 7.5% </p>
<p class="articleParagraph enarticleParagraph" >Port Authority figures do not include Port Authority Trans-Hudson (PATH). The percentage for the total workforce in New Jersey adds up to more than 100 percent because some Hispanics are counted twice, both as Hispanic and again as White or Black. </p>
<p class="articleParagraph enarticleParagraph" >*=Asian-Americans, Native Americans **=Less than 1 percent </p>
<p class="articleParagraph enarticleParagraph" >Source: Highway Authority, Turnpike Authority, Sports Authority and Port Authority, New Jersey Department of Labor.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>MAJOR STORY | NEW JERSEY | EXECUTIVE | MAN | OFFICIAL | GOVERNMENT | BLACK | HISPANIC | WOMAN | EMPLOYMENT | DISCRIMINATION | RACE | SEX | MINORITY</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>Record / Hackensack, New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document rec0000020011115dm7u00ykd</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020050419dm6600af7" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Metropolitan Desk; B</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>Executive Director to Leave Port Authority  </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By CALVIN SIMS  </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>737 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>6 June 1990</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 1990 The New York Times Company.  All Rights Reserved.  </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >Stephen Berger, executive director of the Port Authority of New York and New Jersey, said yesterday that he planned to leave his post in July after five years with the agency to take a job in the private sector.  </p>
<p class="articleParagraph enarticleParagraph" >Stephen Berger, executive director of the Port Authority of New York and New Jersey, said yesterday that he planned to leave his post in July after five years with the agency to take a job in the private sector.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >His departure was widely expected in government and local transit agency circles since the appointment in April of Richard C. Leone as the authority's new chairman. People familiar with the authority said Mr. Leone, a former New Jersey State Treasurer, wanted a new executive director.  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Berger will become president and chief executive of the Financial Guaranty Insurance Company, a leading insurer of municipal bonds, where he is expected to make considerably more than his current salary of $170,000 a year.  </p>
<p class="articleParagraph enarticleParagraph" >His departure comes at a time when the Port Authority is dealing with several growing problems. Revenues are sliding, traffic at the airports run by the authority is declining, its marine terminals are losing customers and traffic across its Hudson River bridges and tunnels is at a lull. And the region is facing weak economic conditions.  </p>
<p class="articleParagraph enarticleParagraph" >In the last year, Mr. Berger found himself at sharp odds with Gov. Mario M. Cuomo of New York and Gov. Jim Florio of New Jersey, each of whom can veto the authority's actions. Mr. Berger's proposal for sharp increases in tolls on bridges, tunnels and the PATH train service was abandoned in April after both Governors resisted the plan.  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Berger is credited with generating support among officials in New York and New Jersey to finance a $6 billion capital plan for overhauling the Port Authority's immense operations, which include airports, river crossings, docks, container ports and terminals, and reducing the sprawling bureaucracy of the organization.  </p>
<p class="articleParagraph enarticleParagraph" >But Mr. Berger will be leaving his post before most of those plans are carried out.  </p>
<p class="articleParagraph enarticleParagraph" >''While the past five years have been among the most rewarding of my professional life,'' Mr. Berger said, ''the opportunity that has been presented to me is, from a professional and personal standpoint, one I could not turn down.''  </p>
<p class="articleParagraph enarticleParagraph" >Financial Guaranty, which is owned by G.E. Capital, a division of the General Electric Company, insured more than $2 billion in securities in the first quarter of 1990.  </p>
<p class="articleParagraph enarticleParagraph" >Several potential candidates are said to be under consideration to replace Mr. Berger, with the leading ones being Stanley Brezenoff, a former first deputy mayor now with the Commonwealth Fund, and Vincent Tese, head of the State Urban Development Corporation and a close personal friend of Governor Cuomo.  </p>
<p class="articleParagraph enarticleParagraph" >At a news conference yesterday at the World Trade Center, Mr. Berger, a balding man with a pointed beard, said that among his major accomplishments at the Port Authority were a privately operated ferry service from Hoboken, N.J., to Manhattan, the refurbishing of the PATH system and stations, the overhaul of La Guardia Airport and increased cooperation by management and labor.  </p>
<p class="articleParagraph enarticleParagraph" >Mitchel Moss, director of the Urban Research Center at New York University, applauded Mr. Berger for having reduced the size of the Port Authority's bureaucracy and appointing ''quality people'' in its middle ranks.  </p>
<p class="articleParagraph enarticleParagraph" >Mayor David N. Dinkins said Mr. Berger's years at the had been ''characterized by fiscal discipline and a visionary commitment to bringing the region's transportation infrastructure into the 21st century.''  </p>
<p class="articleParagraph enarticleParagraph" >But Mr. Moss criticized Mr. Berger for his strained relationships with the two Governors and for not finding funds to continue the renovation of Kennedy International Airport, which has been scaled back. ''He forgot that his authority was derived from the Governors and not his title,'' he said.  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Berger's successor will inherit an agency facing some major problems. Shipping at the Port of New York is lagging far behind many other ports in major cities on both the East and West Coasts. In 1989, the value of cargo shipped through New York was exceeded for the first time by the value of imports and exports shipped through the Port of Los Angeles.  </p>
<p class="articleParagraph enarticleParagraph" >At the same time, fewer people are traveling through the region's airports and using the Port Authority Trans-Hudson system known as PATH.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>CO</b>&nbsp;</td><td><br/>fingui : Financial Guaranty Insurance Company | panynj : Port Authority of New York and New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i82 : Insurance | i763 : Port/Harbor Operations | i82003 : Non-life Insurance | iguins : Guarantee Insurance | iship : Water Transport/Shipping | itsp : Transportation/Shipping</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>ccat : Corporate/Industrial News | gcat : Political/General News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>nyc : New York City | usny : New York | namz : North America | usa : United States | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>SUSPENSIONS, DISMISSALS AND RESIGNATIONS  | BIOGRAPHICAL INFORMATION  | PORT AUTHORITY OF NY AND NJ  | FINANCIAL GUARANTY INSURANCE CO  | BERGER, STEPHEN  | LEONE, RICHARD C  | SIMS, CALVIN </td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>New York Times Digital (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020050419dm6600af7</td></tr></table><br/></div></div><br/><span></span><div id="article-rec0000020011115dm5f00mh4" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/recLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>NEWS</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>COURT UPHOLDS PA TOLLS RULES DRIVERS CAN HELP FUND PATH TRAINS </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>From staff and news service reports </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>424 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>15 May 1990</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The Record</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>REC</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>© 1990 North Jersey Media Group Inc. Provided by ProQuest Information and Learning. All rights reserved.
 </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >The U.S. Supreme Court on Monday upheld a lower court ruling that the Port Authority of New York and New Jersey can use bridge and tunnel tolls to subsidize the financially ailing PATH trains. </p>
<p class="articleParagraph enarticleParagraph" >The justices, without comment, rejected an appeal by motorist clubs seeking to roll back a 1987 toll increase and to block a proposed increase. </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >The decision means the Port Authority will continue operating as usual. </p>
<p class="articleParagraph enarticleParagraph" >"We are extremely pleased with the decision," said Arthur Berg, attorney for the Port Authority. "The decision leaves in place the decision by the U.S Court of Appeals for the 2nd Circuit that the tolls for the Port Authority's tunnels and bridges are reasonable and just." </p>
<p class="articleParagraph enarticleParagraph" >The 2nd U.S. Circuit Court of Appeals last October ruled, 2-1, that the Port Authority may use bridge and tunnel tolls to help operate and improve the Port Authority Trans-Hudson Railroad linking Newark and Hoboken with Manhattan. </p>
<p class="articleParagraph enarticleParagraph" >"There is a spillover effect by which an interruption of service on PATH would disrupt the flow of traffic on the bridges and tunnels," the appeals court said. </p>
<p class="articleParagraph enarticleParagraph" >Without the railroad, the appeals court said, about 70,000 riders would have to find another way to get to work in New York City. Inevitably, they would add to the already massive rush-hour traffic jams in the tunnels and on the bridges between New Jersey and New York. </p>
<p class="articleParagraph enarticleParagraph" >But the Automobile Club of New York and five AAA clubs of New Jersey claimed that the tolls were unjust because they are used to subsidize the PATH trains. </p>
<p class="articleParagraph enarticleParagraph" >"We are extremely disappointed," said Anthony Genovese, AAA attorney. "It means that the motorists will continue to have to pay higher tolls to subsidize the Transit Authority's [train! operations." </p>
<p class="articleParagraph enarticleParagraph" >The appeal to the Supreme Court was the last resort, Genovese said. But the groups will continue to monitor the Port Authority, he added. </p>
<p class="articleParagraph enarticleParagraph" >"We'll be around the next time the Port Authority raises tolls," Genovese said. "We'll take a careful look." </p>
<p class="articleParagraph enarticleParagraph" >The Port Authority raised round-trip bridge and tunnel tolls in 1987 from $2 per car to $3. Motorists traveling east into New York pay the toll. No toll is paid heading west. </p>
<p class="articleParagraph enarticleParagraph" >The Port Authority last year considered a plan to increase tolls to $6 by July 1993. The authority says it has dropped that plan, but tolls still may be raised to $4 next year.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>COURT | NEW JERSEY | RAILROAD | COST | USA | NEW YORK STATE | NEW YORK CITY | BRIDGE | TUNNEL | FINANCE</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>Record / Hackensack, New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document rec0000020011115dm5f00mh4</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020050416dm46006ky" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Metropolitan Desk; B</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>Port Authority Drops Plan For Rise in Tolls and Fares  </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By ROBERT D. McFADDEN  </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>672 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>6 April 1990</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 1990 The New York Times Company.  All Rights Reserved.  </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >Facing opposition from Trenton, the Port Authority of New York and New Jersey has dropped plans for an immediate $1 increase in its bridge and tunnel tolls and a 25-cent fare hike on PATH trains later this year, the executive director of the bistate agency said yesterday.  </p>
<p class="articleParagraph enarticleParagraph" >Facing opposition from Trenton, the Port Authority of New York and New Jersey has dropped plans for an immediate $1 increase in its bridge and tunnel tolls and a 25-cent fare hike on PATH trains later this year, the executive director of the bistate agency said yesterday.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >The official, Stephen Berger, said he would recommend no toll or fare increases, but would propose budget cuts of $333 million in a 1990 budget of $2.6 billion to be considered by the 12-member board of the authority at the annual meeting on Thursday.  </p>
<p class="articleParagraph enarticleParagraph" >The increases, proposed in December, would have raised tolls last Sunday to $4, from $3, for the George Washington Bridge, Lincoln and Holland Tunnels, Bayonne and Goethals Bridges and Outerbridge Crossing. Fares on the Port Authority Trans-Hudson trains linking New Jersey and Manhattan would have increased, to $1.25 from $1, in December.  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Berger did not say why the plans for the increases had been dropped.  </p>
<p class="articleParagraph enarticleParagraph" >But Port Authority budgets are subject to approval by the Governors of New York and New Jersey, and Jim Florio of New Jersey has made no secret of his opposition to toll and fare increases.  </p>
<p class="articleParagraph enarticleParagraph" >Stances of 2 Governors  </p>
<p class="articleParagraph enarticleParagraph" >''The Governor is pleased that the Port Authority felt there was no need for any toll increase,'' Mr. Florio's press secretary, Emma Byrne, said late yesterday in Trenton. Mr. Florio had never openly threatened a veto. But had said he could only support ''clearly justified'' increases.  </p>
<p class="articleParagraph enarticleParagraph" >Gov. Mario M. Cuomo of New York had also been cool toward plans for the increases and had called for a thorough review of the agency's finances to determine whether such increases were justified.  </p>
<p class="articleParagraph enarticleParagraph" >Officials of the authority refused to discuss pressures by the governors. The officials said the abscence of increases this year would make the need for them even greater next year.  </p>
<p class="articleParagraph enarticleParagraph" >The authority, a quasigovernmental agency chartered by New York and New Jersey to promote commerce and transportation, operates and maintains most of the metropolitan region's port, its three major airports and many bridges, tunnels and transportation terminals.  </p>
<p class="articleParagraph enarticleParagraph" >Midway in Capital Program  </p>
<p class="articleParagraph enarticleParagraph" >The agency - which raises money from tolls and fares, bonds sold to the public and fees and rentals from airlines for airport gates and terminals -is not financed directly by the states. It is midway through a five-year $5.8 billion capital program that includes rehabilitating its Hudson River crossings and modernizing the airports.  </p>
<p class="articleParagraph enarticleParagraph" >Last year, in assessing its long-range plans, Port Authority officials said that bridge and tunnel tolls would have to be raised in stages to $6 by 1993 and that PATH fares would eventually have to be raised to $1.50. That brought a response of stern oppositon from Gov. Thomas H. Kean of New Jersey, and Mr. Florio was no more sanguine.  </p>
<p class="articleParagraph enarticleParagraph" >Confronted with declining revenues and no immediate prospect of toll or fare increases that would have brought in an additional $93 million a year, the authority faces sharp cutbacks in plans for capital spending this year, Mr. Berger said.  </p>
<p class="articleParagraph enarticleParagraph" >''It is necessary to revise the original budget recommendations to reflect reductions of $300 million in capital and $33 million in operating spending,'' Mr. Berger said. The cuts would pare the capital budget to $974 million and the operating budget to $1.3 billion.  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Berger did not say what capital projects were scheduled for cutbacks. The heaviest reductions are expected to fall on a long-range plan to modernize Kennedy International Airport. Capital projects scheduled for this year include automated people movers at Kennedy and Newark International Airports and improvements at the Hudson crossings.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>CO</b>&nbsp;</td><td><br/>panynj : Port Authority of New York and New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i72101 : Urban Mass Transit Systems | i721 : Urban/Commuter Transit | i763 : Port/Harbor Operations | irailtr : Road/Rail Transport | iship : Water Transport/Shipping | itsp : Transportation/Shipping</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>ccat : Corporate/Industrial News | gcat : Political/General News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>nyc : New York City | usny : New York | namz : North America | usa : United States | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>BRIDGES AND TUNNELS  | FARES  | TRANSIT SYSTEMS  | PORT AUTHORITY OF NY AND NJ  | PORT AUTHORITY TRANS-HUDSON CORP (PATH)  | MCFADDEN, ROBERT D  | FLORIO, JAMES J (GOV) </td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>New York Times Digital (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020050416dm46006ky</td></tr></table><br/></div></div><br/><span></span><div id="article-rec0000020011116dlca01c4b" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/recLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>OPINION</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>THE PA HAS CLAY FEET </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By James Ahearn </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>878 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>10 December 1989</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The Record</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>REC</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>© 1989 North Jersey Media Group Inc. Provided by ProQuest Information and Learning. All rights reserved.
 </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >The first reaction to the proposed $1 toll increase on the George Washington Bridge and the Lincoln and Holland Tunnels was surprise, incredulity, and skepticism. The anger will come later. </p>
<p class="articleParagraph enarticleParagraph" >People asked, "How could they be losing money?" Good question. The answer is that they aren't losing money. But PATH _ the Port Authority Trans-Hudson system _ is losing money, by the bucketful. </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >Drivers who have to wait in line to cross the Hudson every day may snort, "What do I care about PATH?" Well, they should care. As Stephen Berger, the executive director of the Port Authority, points out, trans-Hudson transportation is a unified system. </p>
<p class="articleParagraph enarticleParagraph" >If you raise fares on PATH high enough to really cut its deficit, people will stop taking the trains and start driving to work, or trying to drive, and that will affect other motorists. </p>
<p class="articleParagraph enarticleParagraph" >It's a good point. Still, when you look at the Port Authority's numbers, it is clear that revenues for the whole trans-Hudson system, including the bridge, the tunnels, and the money-gobbling PATH line, are much higher than operating expenses. </p>
<p class="articleParagraph enarticleParagraph" >So you go back to the opening question: Why does the authority need more money? The answer is to pay for capital improvements, $5.8 billion worth, nearly half of it to be spent at the region's three major airports. </p>
<p class="articleParagraph enarticleParagraph" >The capital program was launched in 1987, when Berger took office, and it is a 5-year plan. It is ambitious. Next year alone the authority is scheduled to invest $1.3 billion. By comparison, before Berger took over at the agency, it was spending only $200 million a year for capital projects. </p>
<p class="articleParagraph enarticleParagraph" >The Port Authority's facilities are not in bad shape and will be better when the capital program is completed. As Berger points out, the agency has had no fiascos like New York City's near-catastrophe with the Williamsburg Bridge. </p>
<p class="articleParagraph enarticleParagraph" >But couldn't the spending be phased out over a longer period than 5 years? Berger says now is the time to invest, because contractors are hungry for work. Bids are coming in lower than expected. </p>
<p class="articleParagraph enarticleParagraph" >Again, a reasonable argument. But not dispositive. Maybe the authority could defer some projects while doing others now. And then you start to look at particular projects and you begin to wonder. </p>
<p class="articleParagraph enarticleParagraph" >The authority is planning to lay out something like $300 million, for instance, on a "people mover" at Newark Airport. This would be a sort of monorail. It would take passengers from airport parking lots to the terminals and back. Buses ply this route now, quite satisfactorily. Why replace them? </p>
<p class="articleParagraph enarticleParagraph" >First, Berger says, the airport is going to get a lot busier. Ground access to the terminals has to be improved. All right, but is a $300 million monorail the best solution? Would a modest buses-only lane work? Is another, unacknowledged agenda at work here? The monorail would supplement and enhance the feasibility of a second, connecting monorail line, from Newark to the airport and on to Elizabeth. The second line has been proposed as a tool with which to encourage development in the two cities. </p>
<p class="articleParagraph enarticleParagraph" >And you wonder about the projections on which the authority is making its plans. In 1987 the airport handled 29 million passengers and the authority was predicting 45 million by the year 2000. Today the airport is down to 21 million passengers a year. True, some of the decline is attributable to the demise of People Express, a one-shot phenomenon. Still, the year 2000 is only a decade away. Is usage of Newark Airport going to double by then? </p>
<p class="articleParagraph enarticleParagraph" >The Port Authority, which used to have an image as the one governmental entity in the Metropolitan Region that really worked, has not been doing well lately. It put $25 million into a state-of-the-art fish processing and wholesaling plant in Brooklyn, and not a single fishmonger or dealer left the Fulton Market for the new quarters. The plant was closed ignominously last spring. </p>
<p class="articleParagraph enarticleParagraph" >The authority pioneered a "park and fly" lot in Ridgewood for travelers bound for Newark Airport. Business didn't come close to the break-even point, even though the authority paid for the buses that a private operator used for the run to the airport. The buses didn't work, as it turned out. </p>
<p class="articleParagraph enarticleParagraph" >Recently the authority has been embroiled in two embarrassing episodes. To hold down toll collectors' overtime it closed some toll lanes on the Hudson River crossings _ in rush hours. And then it stopped selling commuter discount tickets at toll plazas and gas stations. Commuters now have to mail in their orders, and some orders have gone astray. Compounding the snafu, the authority is incomprehensibly giving toll collectors bonuses for not selling the tickets. </p>
<p class="articleParagraph enarticleParagraph" >The agency no longer seems infallible. The best course for Governor Kean and his successor, Jim Florio, is to defer approval of the toll increase until a fresh examination can be made of the authority and its plans. If we're going to pay more every time we cross the Hudson, let's make sure we want the things the extra money will buy. James Ahearn is The Record's associate editor.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>OP-ED | COLUMN | NEW YORK STATE | NEW JERSEY | TRANSIT | BRIDGE | TUNNEL | CONSUMER | COST</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>Record / Hackensack, New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document rec0000020011116dlca01c4b</td></tr></table><br/></div></div><br/><span></span><div id="article-rec0000020011116dlbg01a9u" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/recLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>NEWS</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>N.J. IS HOOKED ON CIGARETTE ADS </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By Matthew J. Doherty, Record Staff Writer </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>982 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>16 November 1989</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The Record</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>REC</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>© 1989 North Jersey Media Group Inc. Provided by ProQuest Information and Learning. All rights reserved.
 </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >On a day when millions will join the Great American Smokeout to try to quit smoking, the state of New Jersey is profiting from the habit through tobacco advertising on its buses and trains and in its sports arenas. </p>
<p class="articleParagraph enarticleParagraph" >In state-operated facilities such as Giants Stadium and the Brendan Byrne Arena, huge billboards for Marlboro cigarettes beckon thousands of fans and television viewers every weekend. On the sides of NJ Transit buses, ads featuring attractive young models tout True cigarettes. </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >The cigarette advertising yields three agencies _ NJ Transit, the New Jersey Sports and Exposition Authority, and the Port Authority of New York and New Jersey _ tens of thousands of dollars each year. </p>
<p class="articleParagraph enarticleParagraph" >The practice is not unusual. Many other states, including New York, welcome tobacco advertising as a source of revenue. New York City's Metropolitan Transportation Authority accepts cigarette ads, and the Marlboro sign is a permanent fixture in Shea Stadium's outfield. </p>
<p class="articleParagraph enarticleParagraph" >But foes of smoking say New Jersey should butt out of tobacco advertising once and for all. Their rationale: It is estimated that each year 10,000 New Jerseyans die from smoking-related diseases such as heart disease, lung cancer, and emphysema. </p>
<p class="articleParagraph enarticleParagraph" >"I don't think the state should be profiting from the death and addiction of our citizens," said Regina Carlson, executive director of New Jersey GASP, a citizens group against smoking. </p>
<p class="articleParagraph enarticleParagraph" >Thousands of smokers in the state are trying to break the habit today as part of the annual Great American Smokeout, sponsored by the American Cancer Society. </p>
<p class="articleParagraph enarticleParagraph" >The tobacco industry is campaigning to recruit new smokers, particularly young people, members of minority groups, and women, health officials say. Tobacco ads invariably portray smoking as seductive, glamorous, and even healthy. </p>
<p class="articleParagraph enarticleParagraph" >"The tobacco companies have to replace all these smokers who are dying off. It's just good business," said Norman Hymowitz, a psychology professor at the New Jersey Medical School in Newark and principal investigator in a study of smoking in Paterson and Trenton. </p>
<p class="articleParagraph enarticleParagraph" >Legislation that would ban outdoor billboard advertising of tobacco and alcohol products was introduced Tuesday by state Sen. Frank X. Graves Jr., D-Paterson. "Kids need better role models than the smokers and boozers they see on billboards," said Graves, who is Paterson's mayor. </p>
<p class="articleParagraph enarticleParagraph" >Despite such strong objections, state agencies view cigarette billboards as money in the bank. The billboard space on transportation systems and in arenas is sold by private advertising agencies, and the state gets a piece of the action. </p>
<p class="articleParagraph enarticleParagraph" >"It's a good source of operating funds for us," said Sandra Check, a spokeswoman for NJ Transit. The agency, which reaped about $200,000 from tobacco-related advertising on its bus and rail system last year, has received no written complaints about the policy, she said. </p>
<p class="articleParagraph enarticleParagraph" >Check said any loss of tobacco advertising revenues would have to be offset by raising fares or state aid. </p>
<p class="articleParagraph enarticleParagraph" >Tobacco promotions on the Port Authority Trans-Hudson line account for 14 percent of all PATH advertisements, said spokesman John Hughes. He could not say exactly how much the Port Authority makes from the ads. </p>
<p class="articleParagraph enarticleParagraph" >John Samerjan, a spokesman for the New Jersey Sports and Exposition Authority, which operates the meadowlands complex, said that Marlboro ads are placed prominently near arena scoreboards and that tobacco promotions account for 10 percent of the authority's advertising revenues. </p>
<p class="articleParagraph enarticleParagraph" >Hymowitz said the stadium billboards and tobacco sponsorship of such sporting events as the Marlboro Grand Prix reflect a marketing strategy. </p>
<p class="articleParagraph enarticleParagraph" >"Sporting events remain a major opportunity for tobacco companies to portray smoking as very contemporary, something for the active person," said Hymowitz, whose research is part of a smoking-cessation study funded by the National Cancer Institute. </p>
<p class="articleParagraph enarticleParagraph" >He added that the billboards "create a positive association between smoking and health, and smoking and athletics." </p>
<p class="articleParagraph enarticleParagraph" >A state health official, Janice Marshall, criticized the bus ads, saying they reach "exactly the target population that tobacco companies want to reach," mostly inner-city minorities and blue-collar workers. </p>
<p class="articleParagraph enarticleParagraph" >Marshall, coordinator of the Cancer and Tobacco Use Control Program of the state Health Department, said the ads place NJ Transit in a contradictory position. "You can't smoke on the buses," she noted. </p>
<p class="articleParagraph enarticleParagraph" >Hymowitz said the state should reject all tobacco advertising "so it won't, by association, be promoting something that is so harmful." </p>
<p class="articleParagraph enarticleParagraph" >But a spokeswoman for the Washington-based Tobacco Institute, a trade association of cigarette manufacturers, said that would be censorship. </p>
<p class="articleParagraph enarticleParagraph" >"The tobacco industry has a right to speak to its customers," said Brennan Dawson, who insisted the dangers of smoking are unproven. </p>
<p class="articleParagraph enarticleParagraph" >Dawson denied that billboards are a conscious attempt to get young, impressionable people to light up, although she acknowledged that the ads are seen by children. </p>
<p class="articleParagraph enarticleParagraph" >"I don't think there is anything that anyone can do to prevent one child from seeing them," she said. </p>
<p class="articleParagraph enarticleParagraph" >Nationally, the percentage of Americans who smoke has declined from 40 percent in 1964 to 29 percent, according to a report issued this year by the surgeon general. The percentage of male smokers declined, but women were less likely to stop smoking, and blacks were the heaviest smokers. </p>
<p class="articleParagraph enarticleParagraph" >About 400,000 deaths each year are attributed to cigarette smoking. </p>
<p class="articleParagraph enarticleParagraph" >The federal government banned cigarette advertising on radio and television in 1971. Hymowitz predicted that billboard advertising of tobacco will someday meet the same fate. </p>
<p class="articleParagraph enarticleParagraph" >"It's just a question of time," he said. "There is a very high acceptance in the community for curbing cigarette smoking. Even smokers don't want to see their children smoke."  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>ART</b>&nbsp;</td><td><br/>1 - COLOR PHOTO - STEVE HOCKSTEIN / THE RECORD - Ads

like this one in Hoboken brought NJ Transit $200,000 in 1988.

2 - PHOTO - RICH GIGLI / THE RECORD - A bus at the Broadway terminal

in  Paterson. Cigarette advertisements target the young, minority

groups, and women, officials say. </td></tr>
<tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>NEW JERSEY | TOBACCO | ADVERTISING</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>Record / Hackensack, New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document rec0000020011116dlbg01a9u</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020050421dlbg00gy8" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Home Desk; C</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>A Railroad Yard Where All the Trains Run to Reminiscence  </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By JAMES BARRON  </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>1193 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>16 November 1989</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 1989 The New York Times Company.  All Rights Reserved.  </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >A BAY SHORE CENTRAL freight train loaded with timber derailed near a busy signal tower in Brooklyn the other day, scattering logs in the path of a Verrazano line locomotive barreling down an adjacent track.  </p>
<p class="articleParagraph enarticleParagraph" >A BAY SHORE CENTRAL freight train loaded with timber derailed near a busy signal tower in Brooklyn the other day, scattering logs in the path of a Verrazano line locomotive barreling down an adjacent track.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >But Adam Wanio remained calm. With one hand he scooped up the logs. With the other he righted the two overturned freight cars. He turned the juice back on. The two trains sped off as if nothing had happened.  </p>
<p class="articleParagraph enarticleParagraph" >If only Amtrak had it so easy. But then, Amtrak does not run model trains. Adam Wanio, a full-grown, full-size man in a railroader's flannel shirt, does. From his command post deep in the basement of a six-story apartment building at 28 Marine Avenue, Mr. Wanio controls a world of tiny tracks, tiny tunnels, tiny stations, even tiny sheep on tiny hillsides.  </p>
<p class="articleParagraph enarticleParagraph" >It is a world of fantasizing about Casey Jones and the days of fire-breathing, huffing-and-puffing locomotives. It is the world of the Bay Ridge Model Railroad Club, which is holding its 40th annual open house. It began last weekend and continues tomorrow from 8 to 10 P.M., on Saturday and Sunday from 2 P.M. to 5:30 P.M., and again next Friday, Saturday and Sunday. The $2 admission helps the nonprofit group pay the rent on its one-room clubhouse and, perhaps more important, the electric bill.  </p>
<p class="articleParagraph enarticleParagraph" >What a way to run a railroad. Most of the trains on the Bay Ridge tracks hark back to the days of whistle-stop campaigning and Pullman cars. The locomotives have been clacking and whirring and whizzing along for far longer than Amtrak has been in existence. Some of them date back to the club's founding, 42 years ago.  </p>
<p class="articleParagraph enarticleParagraph" >This world of little metal wheels gliding over little metal tracks appeals to little boys and girls of all ages. A thousand visitors crowded into the narrow viewing area around the tracks at the open house last year. Many were regulars, and they are back this year. Last weekend, David Decclesiis, a 35-year-old specialist in chemical marketing who lives in the neighborhood, brought his 5-year-old son, Michael. It was Michael's first visit to the club, his father's 30th.  </p>
<p class="articleParagraph enarticleParagraph" >''It was always over too fast,'' Mr. Decclesiis said. He said when he tried to set up a similar layout at home, ''My parents ripped my head off.''  </p>
<p class="articleParagraph enarticleParagraph" >George Corbin, a program analyst for the Treasury Department, looked envious. ''I'm putting in a subway system at home,'' Mr. Corbin said. It takes up half of his Brooklyn basement, he said. His wife will not cede him the rights to the other half.  </p>
<p class="articleParagraph enarticleParagraph" >A few feet down the tracks, Mike Esposito, 11, was intrigued by the pitch-black tunnels that the trains dart in and out of at frightening speeds. Frank Long, also 11, marveled at the electricity it takes to run the trains, the signals and the roundhouse with its rotating platform.  </p>
<p class="articleParagraph enarticleParagraph" >Chris DeLosa, 5, was in awe of the yellow trolley car because yellow is his favorite color. Chris's mother, Evelyn DeLosa, was in awe, too.  </p>
<p class="articleParagraph enarticleParagraph" >''I have to say something intelligent because I'm an adult,'' she said. ''Well, I think it's extraordinary what they did with this room. I love it. It's not just any room.'' Indeed it is not. The club began in one room and expanded into another to create today's 30-by-60-foot space.  </p>
<p class="articleParagraph enarticleParagraph" >Meeting every Friday since 1947, and on a lot of weekends, the club has a dozen members who have built most of the rolling stock, faithfully copying no-longer-manufactured locomotives, cabooses and everything in between. George Schmidt, a retired printer, made a copy of the longest steam locomotive ever built, a hulking relic of the Union Pacific that they call the big boy. Even in miniature it weighs 20 pounds.  </p>
<p class="articleParagraph enarticleParagraph" >Some of the old toy engines are now worth big money. ''A $75 engine in 1938 goes for $5,000 today,'' said Cono Bianco, the club president. ''Guys like Sinatra, they pay that kind of money. Guys like us, we bought them years ago when nobody wanted them.''  </p>
<p class="articleParagraph enarticleParagraph" >Elaborate wiring and a network of switches and signals lets club members run as many as four trains in both directions at once. The signals are automated so club members do not have to flip the switches and watch the tiny lights on the tiny towers wink yellow and green and red. But no one just stands idly by. Gliding around on little dollies in the dim spaces beneath the waist-high tracks and popping up through manhole-like gaps in the layout, they flip the switches and watch the tiny lights on the tiny towers, just as they did before automation.  </p>
<p class="articleParagraph enarticleParagraph" >Chugging around the tracks are trains with made-up names - the Bay Shore Central, the Verrazano line and the Brooklyn Northern, for example, as well as the trolleys of the Central City Traction Company. But the trains themselves are authentic, and so are the stations. One of the club's founding members, Robert Kretzschmar, is at work on an el. To make the platforms and stairways just so, he goes to real stations to take measurements.  </p>
<p class="articleParagraph enarticleParagraph" >Some of the club members are real railroaders - subway workers, for example. Some are relatives of railroaders - Mr. Kretzschmar's father was a signalman for the old New York Central. Mr. Kretzschmar himself owns a machine shop that has worked on Port Authority Trans-Hudson trains and the Chicago subways.  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Wanio said he joined the club because his three-bedroom house in Bensonhurst was not big enough for his dreams.  </p>
<p class="articleParagraph enarticleParagraph" >''Trains seem to be the last thing to be put up and the first thing taken down,'' he said, recalling how other family members would point to his trains and say, ''Gee, we could put the sofa here or the refrigerator there.''  </p>
<p class="articleParagraph enarticleParagraph" >But the club is always there. And though the members do not always like to talk about it, they have had their share of head-on collisions and rear-enders. The worst accident in memory occurred the day a club member decided to make a movie. The camera was rolling, Mr. Kretzschmar recalled, and ''we were concentrating on getting that nice, smooth motion.''  </p>
<p class="articleParagraph enarticleParagraph" >Which is why no one noticed that two trains were heading down the same track from opposite directions. Only after they slammed into each other did the cameraman discover that the film had run out.  </p>
<p class="articleParagraph enarticleParagraph" >Photos of club members, left, running trains; Robert Kretzschmar and, Cono Bianco, club president under the tracks; Vern Gillman, who works in the yard; Chris DeLosa, Eddie Long, Mike Esposito and Frank Long watch; an elevated train in miniature; Sam Snelling, a club member, in dispatcher's tower. (NYT/Bill Swersey)  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i362 : Railroad Rolling Stock | i71 : Railroads | irailtr : Road/Rail Transport | itsp : Transportation/Shipping</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>gdis : Disasters/Accidents | ccat : Corporate/Industrial News | gcat : Political/General News | glife : Living/Lifestyle | grisk : Risk News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>nyc : New York City | namz : North America | usa : United States | use : Northeast U.S. | usny : New York</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>RAILROADS  | MODELS AND REPLICAS  | BAY RIDGE MODEL RAILROAD CLUB  | BARRON, JAMES </td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>New York Times Digital (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020050421dlbg00gy8</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020050421dlav00cwz" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>National Desk; D</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>Supreme Court Roundup; Product-Liability Ruling Left Standing by Justices  </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By LINDA GREENHOUSE, Special to The New York Times  </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>961 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>31 October 1989</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 1989 The New York Times Company.  All Rights Reserved.  </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >WASHINGTON, Oct. 30 -- The Supreme Court today let stand one of the most far-reaching product-liability rulings ever issued by an American court.  </p>
<p class="articleParagraph enarticleParagraph" >The Supreme Court today let stand one of the most far-reaching product-liability rulings ever issued by an American court.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >That ruling, by New York's highest court last April, made all manufacturers of the drug DES, once widely prescribed in pregnancy and now linked to serious medical problems, potentially liable for damages in proportion to the share they had of the national market.  </p>
<p class="articleParagraph enarticleParagraph" >Several pharmaceutical companies, of the hundreds that produced the synthetic estrogen over a period of more than two decades, had appealed the ruling by the New York Court of Appeals. E. R. Squibb & Sons, Eli Lilly & Company and the Rexall Drug Company, which is no longer in business, told the Justices that the decision violated their constitutional right to due process of law because it left them open to liability for injuries that they did not cause.  </p>
<p class="articleParagraph enarticleParagraph" >Under the New York ruling, a woman bringing suit against one or more DES manufacturers need not show that she took a version of the drug manufactured by a particular company. Further, a manufacturer that is sued can be held liable, in proportion to its market share, even if it is clear from the evidence that the plaintiff could not have taken its drug. Appeal on Liability  </p>
<p class="articleParagraph enarticleParagraph" >Rexall, which manufactured a generic version sold only in its pharmacies, argued that it should not face liability in two cases in which it was conceded that the DES prescription was not filled in a Rexall pharmacy.  </p>
<p class="articleParagraph enarticleParagraph" >In his opinion for the New York Court of Appeals, Chief Judge Sol Wachtler said that the basis of liability was not the sale of a particular pill to a particular patient, but rather ''marketing the product'' in the first place. ''It is merely a windfall for a producer to escape liability solely because it manufactured a more identifiable pill, or sold only to certain drugstores,'' the opinion said.  </p>
<p class="articleParagraph enarticleParagraph" >The drug has been shown to cause cancer and fertility problems in the daughters of women who took it. DES was prescribed to about two million women from 1941 to 1970. A synthetic estrogen with the chemical name diethylstilbestrol, it helped prevent miscarriages and eased morning sickness.  </p>
<p class="articleParagraph enarticleParagraph" >There are as many as 1,000 DES lawsuits pending in the New York courts, with damage claims in the billions of dollars. They will now be resolved under the rules set out in the Court of Appeals decision.  </p>
<p class="articleParagraph enarticleParagraph" >The Supreme Court has traditionally been reluctant to review decisions by state courts applying state law in areas like product liability. Several other state courts had earlier adopted a version of the market-share approach taken by the New York court. But under those other rulings, a manufacturer can escape liability if it can prove it did not make the drug the plaintiff took.  </p>
<p class="articleParagraph enarticleParagraph" >Justices Antonin Scalia and Sandra Day O'Connor did not take part in the decision not to hear the case, Rexall v. Tigue, No. 89-168. Justice Scalia declined to say why he did not participate. Through the Court's press office, Justice O'Connor said she and her husband own stock in a drug company, Bristol-Myers, that might have a financial stake in the case.  </p>
<p class="articleParagraph enarticleParagraph" >There were also these other developments at the Court today: Warrantless Search Accepting an appeal from the state of Illinois, the Court agreed to decide whether the failure of the police to obtain a search warrant before entering a home can be excused if the police believe, mistakenly but in good faith, that a third person has given them valid permission to enter.  </p>
<p class="articleParagraph enarticleParagraph" >In this case, Illinois v. Rodriguez, Chicago police answered a complaint from a woman who said her boyfriend had beaten her. She took them to what she described as ''our'' apartment and let them in with a key. There the police arrested the man and found cocaine and other drug paraphernalia. The search, for which they had no warrant, was found to be invalid by the Illinois state courts because the woman had actually moved out of the apartment a month earlier and had taken the key without permission. Port Authority Liability  </p>
<p class="articleParagraph enarticleParagraph" >The Port Authority of New York and New Jersey is an interstate agency that straddles two Federal appellate circuits, the Second and the Third. The Third Circuit, which sits in Philadelphia, has ruled that the agency has the same constitutional immunity from suits in Federal court that a state enjoys under the 11th Amendment.  </p>
<p class="articleParagraph enarticleParagraph" >But the Second Circuit, in New York, ruled earlier this year that the 11th Amendment does not shield the Port Authority because judgments against the agency are paid from its own money and not from the treasuries of New York or New Jersey. The Court today agreed to hear the agency's appeal from the Second Circuit's ruling, Port Authority Trans-Hudson v. Feeney, No. 89-386. The agency told the Court that the two opposing rulings had placed it in a ''difficult and unusual situation.'' AIDS Testing  </p>
<p class="articleParagraph enarticleParagraph" >The Court refused to review a ruling by a Federal appeals court that prohibited a Nebraska state agency from requiring employees in a program for the mentally retarded to submit to blood tests for the virus that causes AIDS. The United States Court of Appeals for the Eighth Circuit ruled that the mandatory testing policy violated the Fourth Amendment's prohibition against unreasonable searches because there was an extremely slight risk that an infected employee could ever transmit the virus to a patient.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>gcrim : Crime/Courts | gvsup : Judicial Branch | gcancr : Cancer | ghea : Health | ghome : Law Enforcement | gcat : Political/General News | gcns : National Security | gmed : Medical Conditions | gpir : Politics/International Relations | gpol : Domestic Politics | gvbod : Government Bodies</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>usa : United States | namz : North America</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>PREGNANCY AND OBSTETRICS  | DES (DIETHYSTILBESTROL)  | CANCER  | LIABILITY FOR PRODUCTS  | SUITS AND LITIGATION  | SUPREME COURT (US)  | GREENHOUSE, LINDA </td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>New York Times Digital (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020050421dlav00cwz</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020050426dl9r0084n" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Metropolitan Desk; B</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>Taiwan Picks Yonkers Company for Subway Cars  </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By JAMES FERON  </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>801 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>27 September 1989</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late Edition - Final</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 1989 The New York Times Company.  All Rights Reserved.  </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >YONKERS, Sept. 26 -- A Japanese-owned rail car manufacturer here that has built subway cars for the New York City and PATH systems was awarded a $170 million contract today to build the first 132 subway cars for a new mass-transit system under construction in Taipei, Taiwan.  </p>
<p class="articleParagraph enarticleParagraph" >A Japanese-owned rail car manufacturer here that has built subway cars for the New York City and PATH systems was awarded a $170 million contract today to build the first 132 subway cars for a new mass-transit system under construction in Taipei, Taiwan.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >The company, U.R.C., enlisted leaders of the Yonkers government as well as of the Port Authority of New York and New Jersey, which leases the rail car manufacturing plant, to travel to Taiwan as part of an intense campaign for the subway car contract.  </p>
<p class="articleParagraph enarticleParagraph" >The contract, with the first cars to be delivered in 1992, will mean the employment of up to 250 skilled laborers in a city that has seen its once-vibrant manufacturing diminished by the movement of companies to the South. Mayor Nicholas C. Wasicsko said he was happy to hear ''some good news for a change in Yonkers.''  </p>
<p class="articleParagraph enarticleParagraph" >He was referring to the city's division over a housing desegregation plan imposed by a Federal judge. City officials have feared that the dispute, which attracted national attention for its constitutional implications, would keep new business away. 'Too Excited to Sleep'  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Wasicsko announced the awarding of the subway car contract flanked by beaming officials of the Nissho Iwai American Corporation, a subsidiary of the Japanese conglomerate Nissho Iwai, and Kawasaki Rail Cars, a division of Kawasaki, which jointly own U.R.C., which stands for United Rail Car.  </p>
<p class="articleParagraph enarticleParagraph" >News that U.R.C. had won the contract arrived in Yonkers late Monday night, according to Japanese company officials who had arrived for what they hoped would be a celebration. Noboru Sakamoto, a vice president of Nissho Iwai, said ''we were too excited to sleep.''  </p>
<p class="articleParagraph enarticleParagraph" >Hats Kageyama, general manager of the Transit Systems and Equipment Division of the Nissho Iwai American Corporation, said two companies, U.R.C. and an American-Canadian consortium that included a Canadian company, ''had qualified as American companies'' for the purpose of the contract Taiwan has an $18 billion trade surplus with the United States, and the State Commerce Departments have been pressing Taiwan to reduce that imbalance. Taiwan indicated that it would do so, virtually eliminating foreign rail car makers as bidders.  </p>
<p class="articleParagraph enarticleParagraph" >Yonkers itself has been seeking to reverse an economic drain, most recently exemplified by the move of the manufacturing division of the Polychrome Corporation, which makes printing supplies, to Atlanta. Mr. Wasicsko said he hoped U.R.C. would be able to hire some Polychrome workers and retrain them to build subway cars. Space Is Leased  </p>
<p class="articleParagraph enarticleParagraph" >The contract awarded to Yonkers by the Department of Rapid Transit in Taiwan is for the Tamshui-Hsintein line, one of the radial spokes on the semicircular system being built in Taipei. Two other rail car contracts will be put out for bid before the system is finished.  </p>
<p class="articleParagraph enarticleParagraph" >U.R.C. leases space in a structure that was the original home of the Otis Elevator Company and the site of the first safety elevator. It was purchased by the Port Authority in 1985 from United Technologies Corporation of Hartford, which had acquired Otis. The city helped United Technologies modernize the structure and its leaders were furious when the company moved to Connecticut.  </p>
<p class="articleParagraph enarticleParagraph" >Kawasaki's first contract for subway cars in the United States came in the 1970's with the export of 125 units from Japan to Philadelphia for its subway system. The Yonkers plant was opened for rail-car production in 1986 and its first job was the manufacture of subway cars for the PATH (Port Authority Trans-Hudson) system.  </p>
<p class="articleParagraph enarticleParagraph" >It built 95 cars in the Yonkers plant. U.R.C., under the Kawasaki name, assembled 325 R-62 cars used on New York City's Lexington Avenue IRT line.  </p>
<p class="articleParagraph enarticleParagraph" >Company officials said a ''cadre'' of 100 skilled workers was now employed in the Yonkers plant, rehabilitating cars for the New York City Transit Authority and building ''people mover'' rail cars, which are much smaller than subway cars, for the Las Vegas airport.  </p>
<p class="articleParagraph enarticleParagraph" >The U.R.C. officials said while engineering studies were being completed for the Taiwan system, U.R.C. would be producing 75 double-decker rail commuter coaches for the Massachusetts Bay Transit Authority in a $105 million contract. The company also expects to build ''people movers'' for Kennedy International Airport, when that system is introduced there.  </p>
<p class="articleParagraph enarticleParagraph" >photo of Nicholas C. Wasicsko, Lee Singer, Masakazu Fudo and Noboru Sakamoto (NYT/Jim Estrin)  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>CO</b>&nbsp;</td><td><br/>panynj : Port Authority of New York and New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i721 : Urban/Commuter Transit | i72101 : Urban Mass Transit Systems | i763 : Port/Harbor Operations | irailtr : Road/Rail Transport | iship : Water Transport/Shipping | itsp : Transportation/Shipping</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>ccat : Corporate/Industrial News | gcat : Political/General News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>taiwan : Taiwan | nyc : New York City | usny : New York | asiaz : Asia | chinaz : Greater China | devgcoz : Emerging Market Countries | easiaz : Eastern Asia | namz : North America | usa : United States | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>TRANSIT SYSTEMS  | URC (YONKERS CO)  | KAWASAKI HEAVY INDUSTRIES LTD  | NISSHO IWAI CORP  | WASICSKO, NICHOLAS C (MAYOR)  | FERON, JAMES </td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>New York Times Digital (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020050426dl9r0084n</td></tr></table><br/></div></div><br/><span></span><div id="article-rec0000020011116dl5e00ik9" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/recLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>NEWS</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>NARROW VOTE AVERTS PATH STRIKE </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By Eugene Kiely and Kathleen O'Brien, Record Staff

Writers </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>566 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>14 May 1989</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The Record</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>REC</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>© 1989 North Jersey Media Group Inc. Provided by ProQuest Information and Learning. All rights reserved.
 </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >Sparing commuters the headaches of a transit strike, PATH train engineers approved a new contract Friday night. The union had set a strike deadline of Monday. </p>
<p class="articleParagraph enarticleParagraph" >The Brotherhood of Locomotive Engineers Local 497, which had worked without a contract for more than three years, had twice rejected previous settlements, the last one by a vote of 130-5 two weeks ago. </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >The margin of approval was much closer: 92-74. </p>
<p class="articleParagraph enarticleParagraph" >"Just by the close vote, I can tell you people are not happy with it," said Roy Lobb, Local 497 general chairman. Particularly annoying are the wage increases of 4 percent and 4.5 percent in the final two years of the five-year contract, he said. The contract is retroactive, providing long-awaited raises for each year since 1985. </p>
<p class="articleParagraph enarticleParagraph" >It expires in September of 1990. "But it'll take us another 3 1/2 years to get to this point again," Lobb said, referring to the lengthy negotiations process. </p>
<p class="articleParagraph enarticleParagraph" >The Port Authority Trans Hudson trains carry 200,000 riders daily between Newark, Jersey City, Hoboken, and Manhattan. </p>
<p class="articleParagraph enarticleParagraph" >The last PATH strike, in June of 1980, produced major rush-hour traffic snarls as commuters scrambled to find other ways of crossing the Hudson River. Train commuters faced a 2 1/2-hour wait to board special shuttle buses into New York. Thousands of commuters waited in line for buses; hundreds of buses waited in traffic that clogged the Holland and Lincoln tunnels. </p>
<p class="articleParagraph enarticleParagraph" >That strike _ by the carmen's union _ lasted a record-breaking 79 days. </p>
<p class="articleParagraph enarticleParagraph" >The voting, which was at the Ukrainian Community Center in Jersey City, was conducted three times during the day to accommodate the morning, afternoon, and evening shifts of workers. Union members favored accepting the contract by a margin of 52-39 during the morning session. The afternoon session was nearly evenly split, 23-21. </p>
<p class="articleParagraph enarticleParagraph" >Union members interviewed at the center said they felt the Port Authority had stalled negotiations over the years. "The longer they stall us, the more money they make" by withholding retroactive raises, said a motorman who identified himself only as "Jimmy from Jersey City." A veteran union member from Bayonne, who said he voted against the settlement, said: "We've been waiting 4 1/2 years." </p>
<p class="articleParagraph enarticleParagraph" >The key to ratification appeared to be a new provision addressing the motormen's concern that new licensing provisions might force some of them out of work. The regulations, under consideration by the Federal Railroad Administration, might prevent engineers with poor automobile driving records from operating trains. </p>
<p class="articleParagraph enarticleParagraph" >The negotiated clause promises that alternate employment would be offered to any motormen put out of work by the new requirement. "If that was not in there, that agreement would've gone down big time," Lobb said. </p>
<p class="articleParagraph enarticleParagraph" >"To my knowledge, PATH may well be the first rail system in this country to make a contractual commitment on employee job security prior to enactment of any federal licensing requirements," said PATH Chairman Philip Kaltenbacher. </p>
<p class="articleParagraph enarticleParagraph" >Friday's vote was the culmination of a negotiating process that started almost four years ago when the engineers union in July 1985 filed notice under the federal Railway Labor Act that it would seek a new agreement. The last contract expired in September 1985. </p>
<p class="articleParagraph enarticleParagraph" >This article contains material from The Record's news services.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>STRIKE | TRANSIT | RAILROAD | EMPLOYMENT | UNION | CONTRACT | SALARY | NEW JERSEY</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>Record / Hackensack, New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document rec0000020011116dl5e00ik9</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020050425dl5c006ng" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Metropolitan Desk; B</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>PATH and Engineers in Tentative Accord  </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>225 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>12 May 1989</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late City Final Edition</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 1989 The New York Times Company.  All Rights Reserved.  </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >The National Mediation Board announced yesterday that negotiators for PATH and Local 497 of the Brotherhood of Locomotive Engineers had reached a tentative agreement after more than three years of talks.  </p>
<p class="articleParagraph enarticleParagraph" >The National Mediation Board announced yesterday that negotiators for PATH and Local 497 of the Brotherhood of Locomotive Engineers had reached a tentative agreement after more than three years of talks.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >A possible strike Monday affecting 100,000 commuters will be averted if the 190 motormen and switchmen approve the settlement with the Port Authority Trans-Hudson trains, said a spokesman for the mediation board, David Bushnell. The union votes on the contract today in Jersey City.  </p>
<p class="articleParagraph enarticleParagraph" >The membership has rejected settlements twice, most recently May 1, when it turned down a pact that would have raised salaries, to $48,000 a year from $38,000. That agreement would have run from September 1985 to March 1990. The tentative accord reached yesterday would cover the same period.  </p>
<p class="articleParagraph enarticleParagraph" >All parties agreed not to discuss the terms of the new agreement until the union vote, Mr. Bushnell said. The tentative settlement was reached shortly after 3 P.M. in a ''congenial and upbeat'' atmosphere after four days of renewed negotiations under the auspices of the board, he said.  </p>
<p class="articleParagraph enarticleParagraph" >The PATH directors will also vote on the contract.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i72101 : Urban Mass Transit Systems | i721 : Urban/Commuter Transit | irailtr : Road/Rail Transport | itsp : Transportation/Shipping</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>c42 : Labor/Personnel Issues | clabdi : Labor Disputes | cwrkpa : Workers Pay | ccat : Corporate/Industrial News | gcat : Political/General News | gjob : Labor Issues | gcom : Society/Community/Work | ncat : Content Types | nfact : Factiva Filters | nfcpin : FC&E Industry News Filter</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>usny : New York | namz : North America | usa : United States | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>TRANSIT SYSTEMS  | CONTRACTS  | LABOR  | PORT AUTHORITY TRANS-HUDSON CORP (PATH)  | BROTHERHOOD OF LOCOMOTIVE ENGINEERS </td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>New York Times Digital (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020050425dl5c006ng</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020050425dl55004zi" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Metropolitan Desk; B</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>Alternate Routes Set in Event of PATH Strike  </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By GEORGE JAMES  </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>522 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>5 May 1989</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late City Final Edition</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 1989 The New York Times Company.  All Rights Reserved.  </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >The Port Authority of New York and New Jersey released a contingency plan yesterday for transporting 100,000 commuters between New Jersey and Manhattan if PATH motormen carry out their threat to strike.  </p>
<p class="articleParagraph enarticleParagraph" >The Port Authority of New York and New Jersey released a contingency plan yesterday for transporting 100,000 commuters between New Jersey and Manhattan if PATH motormen carry out their threat to strike.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >Although the union has set May 15 as the strike date, some 190 motormen and motor switchmen would be legally free to strike as of 12:01 A.M. Monday. This is when the second of two federally mandated cooling-off periods expires, a spokesman for the National Mediation Board, David Bushnell, said yesterday.  </p>
<p class="articleParagraph enarticleParagraph" >Late yesterday a spokesman for the Port Authority, Bill Cahill, said the agency had received a message from the mediation board in Washington saying that the union, Local 497 of the Brotherhood of Locomotive Engineers, had agreed that it ''would not strike, picket or commit any job action'' before May 15.  </p>
<p class="articleParagraph enarticleParagraph" >Efforts yesterday to reach Roy Lobb, general chairman of the local, for comment were unsuccessful. New Talks Scheduled  </p>
<p class="articleParagraph enarticleParagraph" >Negotiations between the Port Authority Trans-Hudson Railroad and the union are to resume Monday in Washington under the auspices of the National Mediation Board.  </p>
<p class="articleParagraph enarticleParagraph" >Union members, who have been without a contract for more than three years, overwhelmingly rejected a tentative agreement last Monday.  </p>
<p class="articleParagraph enarticleParagraph" >''We are still hopeful a strike can be averted and look forward to a resumption of discussions that could result in a mutually acceptable agreement,'' said Stephen Berger, executive director of the Port Authority.  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Berger said that in the event of a strike, the authority would increase N.J. Transit train service from Newark to Pennsylvania Station in Manhattan and run shuttle buses from Hoboken Terminal and the Journal Square Transportation Center in Jersey City to downtown and midtown Manhattan.  </p>
<p class="articleParagraph enarticleParagraph" >In addition, ferry service would be offered from Hoboken and Exchange Place in Jersey City to Battery Park.  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Berger warned that if automobile congestion at the Lincoln and Holland Tunnels became ''intolerable,'' the authority would consider barring vehicles with only one occupant.  </p>
<p class="articleParagraph enarticleParagraph" >The port agency plans to place advertisements in today's newspapers detailing alternate routes, Mr. Cahill said.  </p>
<p class="articleParagraph enarticleParagraph" >Michael Scott, assistant general superintendent of PATH operations, said that up to 150 N.J. Transit buses would carry commuters between Hoboken Terminal and the Port Authority Bus Terminal in midtown Manhattan, as well as to points on Canal Street in lower Manhattan.  </p>
<p class="articleParagraph enarticleParagraph" >A yet-to-be-determined number of buses would also carry commuters to those points from the transportation center in Journal Square.  </p>
<p class="articleParagraph enarticleParagraph" >Although the bus and train plan could be put into effect Monday, if necessary, with some inconvenience to commuters, ferry service would not be operating until mid-week, Mr. Scott said.  </p>
<p class="articleParagraph enarticleParagraph" >Talks between the authority and the union reached a stalemate in 1985. The authority's current offer would raise salaries to $48,000 a year from $38,000 over the period from September 1985 to March 1990.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>CO</b>&nbsp;</td><td><br/>panynj : Port Authority of New York and New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i721 : Urban/Commuter Transit | i72101 : Urban Mass Transit Systems | i763 : Port/Harbor Operations | irailtr : Road/Rail Transport | iship : Water Transport/Shipping | itsp : Transportation/Shipping</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>c42 : Labor/Personnel Issues | clabdi : Labor Disputes | ccat : Corporate/Industrial News | gcat : Political/General News | gjob : Labor Issues | gcom : Society/Community/Work | ncat : Content Types | nfact : Factiva Filters | nfcpin : FC&E Industry News Filter</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>nyc : New York City | usny : New York | namz : North America | usa : United States | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>TRANSIT SYSTEMS  | STRIKES  | LABOR  | SUBWAYS  | PORT AUTHORITY OF NY AND NJ  | PORT AUTHORITY TRANS-HUDSON CORP (PATH)  | BROTHERHOOD OF LOCOMOTIVE ENGINEERS  | JAMES, GEORGE </td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>New York Times Digital (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020050425dl55004zi</td></tr></table><br/></div></div><br/><span></span><div id="article-rec0000020011116dl5400igv" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/recLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>CLM</b>&nbsp;</td><td>LETTERS</td></tr>
<tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>NEWS</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>OBITUARIES </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>2651 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>4 May 1989</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The Record</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>REC</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>© 1989 North Jersey Media Group Inc. Provided by ProQuest Information and Learning. All rights reserved.
 </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >VINCENT R. BIZIK, 65, of Maywood died Monday. Before retiring, he was a printer for Independent Printers, Wood-Ridge, for 30 years. He was an Army veteran of World War II. Surviving are four sons, Vincent of Grove, Okla., Richard J. Sr. of Newton, Raymond of Honolulu, and Norman D. of Paramus, and four grandchildren. A brother, Steven, died in March. Services will be Friday at 10:30 a.m. at Kamienski Funeral Home, Wallington, with burial in George Washington Memorial Park, Paramus. Visiting is today from 2 to 4 and 7 to 9 p.m. </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >ELIZABETH T. BRACKEN, 89, of Milford, Del., formerly of Passaic, died Wednesday. She was a parishioner of Holy Trinity R.C. Church and a member of the Police and Firemen's Widows Association, both in Passaic. Surviving are a daughter, Anna Lorenz of Milford, five grandchildren, and six great-grandchildren. Her husband, Hugh, is deceased. Mass will be said Saturday at 9 a.m. at Holy Trinity Church, with burial in St. Nicholas Cemetery, Lodi. Visiting is Friday from 2 to 4 and 7 to 9 p.m. at Powell-Marrocco Funeral Home, Passaic. </p>
<p class="articleParagraph enarticleParagraph" >JOHN J. BROWN, 84, of Cliffside Park died Monday. Before retiring, he was a foreman for Lightolier Inc., Secaucus. Surviving are his wife, Margaret Zuchva Brown; two daughters, Barbara Cancalosi of Ramsey and Patricia Bradley of Woodcliff Lake; a son, James of Willard, Ohio, 13 grandchildren, and a great-grandchild. Mass will be said Friday at 9:45 a.m. at Epiphany R.C. Church, Cliffside Park, with entombment at All Saints Mausoleum, Paramus. Visiting is today from 2 to 4 and 7 to 9 p.m. at A. K. Macagna Funeral Home, Cliffside Park. Donations to Hackensack Medical Center Hospice Program would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >KATHLEEN SUSKI EASTWOOD, 40, of Englewood Cliffs died Sunday. Before retiring last year, she was a head nurse at Bergen Pines County Hospital, Paramus. She was a parishioner of Holy Trinity R.C. Church, Fort Lee. Surviving are her husband, Edwin C. Jr.; her parents, Michael and Loretta Suski; two stepdaughters, Lauren Eastwood and Lisa Eastwood; a stepson, Edwin III; and a brother, Edward. A memorial Mass will be said Saturday at 2 p.m. at Bergen Pines County Hospital Auditorium East. Donations to the Kathleen Suski Eastwood Memorial Fund at Bergen Pines would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >REMO "RAYMOND" O. FAUSTINI, 66, of Maywood, formerly of Hackensack, died Tuesday. He was born in Italy and came to the United States in 1939. He was a partner in the Ray and Dom Construction Corp. in Bergen County and previously owned Cleveland Asphalt Co. He was an Army veteran of World War II and a member of American Legion Post 170, Rochelle Park. Surviving are his wife, Theresa Cavalli Faustini; a daughter, Vincenzina Matina, and a son, Domenico, both of Paramus; a sister, Margaret Faustini of Milford, Conn., and three grandchildren. A brother, Fioravanti, died in 1984. Mass will be said Saturday at 10:15 a.m. at St. Francis R.C. Church, Lodi, with entombment at Madonna Mausoleum, Fort Lee. Visiting will be today and Friday from 3 to 5 and 7 to 9 p.m. at Trinka-Faustini Funeral Home, Maywood. </p>
<p class="articleParagraph enarticleParagraph" >KATHRYN GALLAGHER, 75, of Wayne, formerly of Neptune, died Wednesday. She worked for Community Bus Lines, Passaic. She was a parishioner of St. Nicholas R.C. Church, Passaic, and a member of the Chilton Memorial Hospital Auxiliary. Surviving are three sons, John Jr. of Wayne, Frank of Kinnelon, and Stephen of Vernon; a daughter, Alice Gallagher of Clifton; two brothers, Francis Fraser of Passaic and William Fraser of Wayne; a sister, Eleanor Maycrink of Rutherford, and three grandchildren. Her husband, John, died in 1982. Mass will be said Saturday at 10 a.m. at St. Nicholas Church, with burial in St. Nicholas Cemetery, Lodi. Visiting will be Friday from 2 to 4 and 7 to 9 p.m. at Powell-Marrocco Funeral Home, Passaic. </p>
<p class="articleParagraph enarticleParagraph" >CAROLYN ANN GILES, 31, of Clifton died Tuesday. She was a marketing manager for Revlon Corp. in New York City for two years and previously worked for American Cyanamid, Wayne, and Shulton Co., Clifton. She was a 1988 graduate of Montclair State College and a parishioner of St. Clare R.C. Church, Clifton. Surviving are her parents, Gerard C. and Catherine McGivney Giles of Clifton; and her maternal grandmother, Margaret McGivney of Jersey City. Mass will be said Saturday at 9:30 a.m. at St. Clare Church, with burial in East Ridgelawn Cemetery, Clifton. Visiting will be Friday from 2 to 4 and 7 to 9 p.m. at Allwood Funeral Home, Clifton. </p>
<p class="articleParagraph enarticleParagraph" >GREGORY HORDIENKO, 89, of Garfield, formerly of Clifton and Passaic, died Wednesday. Born in the Ukraine, he came to the United States in 1949. Before retiring in 1960, he was a warehouse worker for Thomas Electronics, Passaic, for 15 years. He was a parishioner of Assumption of the Holy Virgin Orthodox Church, Clifton. Surviving are a daughter, Katia Nasarenko of Garfield, and two grandchildren. His wife, Maria, is deceased. Services will be Friday at 9:30 a.m. at Assumption Church, with burial in East Ridgelawn Cemetery, Clifton. Visiting is today from 2 to 4 and 7 to 9 p.m. at Shook Funeral Home, Clifton, with a parastas at 2:30. </p>
<p class="articleParagraph enarticleParagraph" >WALTER ANGELO HOWARD, 30, of Paterson died Tuesday. He worked for Concerned Parents for Head Start, Paterson, and attended Grace Bible Class, Passaic. Surviving are a daughter, Shontella McCutchen, his mother, Nora Lewis Howard, his father and stepmother, Walter B. and Marilyn Howard, and a brother, Dennis, all of Paterson; six sisters, Jacqueline, Tracy, Faith, and Stacy, all of Paterson, Maxine of Norfolk, Va., and April of Annapolis, Md.; and his maternal grandfather, George Lewis of Ridgewood. A memorial service will be Saturday at 1 p.m. at Bragg Funeral Home, Paterson. </p>
<p class="articleParagraph enarticleParagraph" >ROSE HOROWITZ JACOBS, 85, of Clifton, formerly of the Bronx, died Saturday. She was a life member of Hadassah and a member of the Zionist Organization of America. Surviving are a son, Ted of Fair Lawn; a brother, Saul of the Bronx, and a granddaughter. Her husband, Ralph, died in 1966. Arrangements were by Robert Schoem's Menorah Chapel, Paramus. Donations to Fair Lawn Hadassah or to Congregation Shomrei Torah would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >PETER KUVARA, 93, of Bergenfield, formerly of Teaneck, died Tuesday. He was born in Yugoslavia. Before retiring in 1967, he was a painter for the Port Authority Trans-Hudson rail line. He was a parishioner of St. John's R.C. Church, Bergenfield, and a member of its Senior Citizens and Club 50. Surviving are his wife, Rosalind Barilotti Kuvara; and a sister, Dora in Yugoslavia. Two brothers, Nicholas and John, are deceased. Mass will be said Friday at 11 a.m. at St. John's Church, with burial in Holy Cross Cemetery, North Arlington. Visiting is today from 2 to 4 and 7 to 9 p.m. at Riewerts Memorial Home, Bergenfield. </p>
<p class="articleParagraph enarticleParagraph" >EVELYN JACOBSON MATTHEWS of Fair Lawn died Tuesday. She had worked for the Fair Lawn school system since 1952 as an elementary school teacher, reading instructor, and guidance specialist, and as a guidance counselor in the high school for the past 13 years. A graduate of Hunter College, New York City, she earned a master's degree from Columbia University Teachers College. She was a member of Barnert Memorial Temple, Franklin Lakes, and the New Jersey and Fair Lawn education associations. Surviving are two sons, Steven J. of the Bronx and David J. of Red Bank; two sisters, Martha Kellman of New York City and Shirley Brickman of Boca Raton, Fla., and four grandchildren. Her husband, Emil A., died in 1952. Services are today at 10:30 a.m. at Louis Suburban Chapel, Fair Lawn, with burial in Cedar Park Cemetery, Emerson. A mourning period will be observed at the family residence. Donations to the Evelyn J. Matthews Scholarship Fund, in care of the Fair Lawn High School Guidance Department, Fair Lawn, N.J. 07410, would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >CHARLES R. MOORE, 76, of Ridgefield Park died Tuesday. Before retiring in 1977, he was a mechanic with Knapps Express, Ridgefield Park. He was an Army veteran of World War II, a former member of the Ridgefield Park Rescue Squad, and a member of the AARP. Surviving are a daughter, Kathleen Diamond of Ridgefield Park, and two grandsons. His wife, Edna Olssen Moore, died last year, and a sister, Elsie Malloy, died in 1977. Mass will be said Friday at 10 a.m. at St. Francis R.C. Church, with visiting today from 2 to 4 and 7 to 9 p.m. at Vorhees Funeral Home, both in Ridgefield Park. </p>
<p class="articleParagraph enarticleParagraph" >EDWARD PAUL MOSLOSKIE SR., 63, of Millsboro, Del., formerly of Paterson, died Wednesday. Before retiring in 1986, he was a steelworker for Athenia Steel Co., Clifton, for 22 years. He was a member of United Steelworkers Local 163 and the John McDonald Sportsmen's League, Paterson. Surviving are his wife, Josephine Ostan Mosloskie; four sons, Edward Jr. of Northampton, Pa., and Robert, Frank, and Ronald, all of Clifton; two daughters, Josephine Bradbury of Rehoboth Beach, Del., and Diane Mosloskie of Oakland; two brothers, Marion of Scottsdale, Ariz., and James of Aston, Pa., and 12 grandchildren. Mass will be said Saturday at 9:30 a.m. at St. Brendan's R.C. Church, Clifton, with burial in St. Mary's Cemetery, Saddle Brook. Visiting will be Friday from 2 to 4 and 7 to 9 p.m. at Shook Funeral Home, Clifton. </p>
<p class="articleParagraph enarticleParagraph" >HENRIETTA EHLEN PHEULPIN, 71, of Mahwah died Wednesday. Before retiring, she was a saleswoman and model for Saks Fifth Avenue in New York City. Surviving are her husband, Fred J.; a son, Fred of Burlington, Vt.; a sister, Eleanor Keisebach of St. Petersburg, Fla.; and two brothers, Patrick Ehlen and Fenton Ehlen, both in New York. Mass will be said Saturday at 10 a.m. at Immaculate Conception R.C. Church, with burial in Maryrest Cemetery, both in Mahwah. Visiting will be today and Friday from 2 to 4 and 7 to 9 p.m. at Van Emburgh-Sneider Funeral Home, Ramsey. </p>
<p class="articleParagraph enarticleParagraph" >MINNIE LEDERMAN PLATT, 91, of Paterson died Wednesday. She was born in Poland and came to the United States in 1914. She had operated a grocery store in Paterson and had worked in real estate. Surviving are three sons, Albert of Fort Lee, and Gerald and Theodore, both of Fair Lawn; 11 grandchildren, and two great-grandchildren. Her husband, Samuel, died in 1955, and a son, Arnold I., died in February. Services will be Friday at 11 a.m. at Louis Suburban Chapel, Fair Lawn, with burial in Workmen's Circle Cemetery, Saddle Brook. A mourning period will be observed at the Gerald Platt residence and at the Sharon Platt residence in Cliffside Park. </p>
<p class="articleParagraph enarticleParagraph" >DR. MICHAEL PLESHETTE, 53, of Teaneck died Wednesday. He was an endodontist and a professor at New York University. He was a member of the American Dental Association, the American Board of Endodontists, the American Association of Endodontics, and Congregation Beth Am, Teaneck. Surviving are two daughters, Stephanne Pleshette of Olivebridge, N.Y., and Elizabeth Pleshette of Teaneck. Services are today at noon at Robert Schoem's Menorah Chapel, Paramus, with burial in Beth-El Cemetery, Washington Township. </p>
<p class="articleParagraph enarticleParagraph" >JOSEPH J. PLUCINSKY, 63, of Moonachie died Wednesday. Before retiring, he was a mechanic at Hills Bros., Edgewater. He was an Air Force veteran of World War II, and a member of Teamsters Local 641, Secaucus, and American Federation of Musicians Local 248, Paterson. He was a parishioner of St. Margaret of Cortona R.C. Church, Little Ferry. Surviving are his wife, Arlene Van Saders Plucinsky; a daughter, Joanne of Moonachie; and a brother, William of Wayne. Mass will be said Friday at 9:30 a.m. at St. Margaret Church, with burial in George Washington Memorial Park, Paramus. Visiting is today from 2 to 4 and 7 to 9 p.m. at Hackensack Funeral Home. Donations to the Moonachie First Aid Squad or to the Hackensack Medical Center Hospice would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >CATHERINE TURNER RADZINSKI, 70, of Carlstadt, formerly of Bayonne and Hoboken, died Tuesday. She was a parishioner of St. Joseph's R.C. Church, East Rutherford. Surviving are two sons, William W. of West New York and Michael G. of Westwood; two daughters, Ann Baker and Catherine Lipari, both in West Germany; two brothers, John Turner of Livingston and Frederick Turner of East Hanover; three sisters, Ann Zowistowski of Rutland, Vt., and Patricia Conzery and Florence Horn, both of Whiting, and four grandchildren. Her husband, William, died in 1980, and a daughter, Donna Radzinski, died in 1977. Mass will be said Saturday at 10 a.m. at St. Joseph's Church, with burial in Holy Cross Cemetery, North Arlington. Visiting is tonight from 7 to 9 and Friday from 2 to 4 and 7 to 9 p.m. at the Thomas J. Diffily Funeral Home, Rutherford. </p>
<p class="articleParagraph enarticleParagraph" >SUZANNE M. COUGHLIN SAMMON, 75, of Bergenfield, formerly of Dumont and the Bronx, died Tuesday. She was assistant to the dean of the department of radiology at Albert Einstein College of Medicine, New York City. She was a parishioner of St. John's R.C. Church, Bergenfield, and a member of the Bergenfield Senior Citizens and the AARP. Surviving are her husband, Thomas J.; a daughter, Patricia Germinario of Bergenfield, and two grandchildren. Mass will be said Friday at 10 a.m. at St. John's Church, with entombment at Holy Cross Mausoleum, North Arlington. Visiting is today from 2 to 4 and 7 to 9 p.m. at Riewerts Memorial Home, Bergenfield. </p>
<p class="articleParagraph enarticleParagraph" >PAUL C. SCHMIDT, 82, of Wantage Township, formerly of Teaneck, died Tuesday. Before retiring 17 years ago, he was a construction supervisor for Con Edison in New York City. He was a vestry of Christ Episcopal Church, Teaneck. Surviving is a son-in-law, Edward Rutsch of Newton. His wife, Helen McBride Schmidt, and a daughter, Mary Jane Rutsch, died in April. A memorial service is scheduled May 13 at 11 a.m. at the Unitarian Fellowship Hall, Newton. Arrangements were by Pinkel Funeral Home, Sussex. Donations to the American Cancer Society would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >JOSEPH V. SHIELDS, 67, of Lodi died Tuesday. Before retiring nine years ago, he was a warehouse checker for Colgate Palmolive Co. in Jersey City, where he worked for 35 years. He was an Army veteran of World War II and a member of the Holy Name Society of St. Francis de Sales R.C. Church, Lodi. Surviving are his wife, Lucy Jaronczyk Shields; a daughter, Helen Garibaldi of Middletown, N.Y.; three brothers, William in New York, Lawrence of Richmondville, N.Y., and Francis of Sparta; three sisters, Agnes Dugan of Toms River, Dorothy Jamieson of Houston, and Genevieve Raganelli of Atlantic Highlands, and four grandchildren. Mass will be said Friday at 11 a.m. at St. Francis de Sales Church, with burial in Holy Cross Cemetery, North Arlington. Visiting is today from 2 to 4 and 7 to 9 p.m. at Alesso Funeral Home, Lodi. </p>
<p class="articleParagraph enarticleParagraph" >CAROLYN ENGELS WEIMANN, 74, of New Milford died Sunday. Before retiring, she was a teller for Central Bergen Savings and Loan Association in Teaneck. She was a member of the Hackensack Elks, the Lt. Louis Faller VFW Post 4290 Ladies Auxiliary, and the Senior Citizens, the Republican Club, and the Lady Foresters, all in New Milford. She was a former member of the New Milford Fire Department Ladies Auxiliary, an honorary member of the Aviation Hall of Fame, and a volunteer at Hackensack and Holy Name hospitals. Surviving are two sons, William of Florida, N.Y., and Alan of Westwood, and four grandchildren. Her husband, William, died in 1971. Arrangements were by Boulevard Funeral Home, New Milford. Donations to the New Milford Presbyterian Church would be appreciated.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>COLUMN | OBIT</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>Record / Hackensack, New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document rec0000020011116dl5400igv</td></tr></table><br/></div></div><br/><span></span><div id="article-rec0000020011116dl1n003vb" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/recLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>CLM</b>&nbsp;</td><td>OBITUARIES</td></tr>
<tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>NEWS</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>OBITUARIES </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>3637 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>23 January 1989</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The Record</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>REC</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>© 1989 North Jersey Media Group Inc. Provided by ProQuest Information and Learning. All rights reserved.
 </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >MARVIN BARNES, 67, of Boca Raton, Fla., and New York City died Saturday. Before retiring two weeks ago, he was an advertising director for Porky Products, Jersey City, where he worked for many years. Surviving are his wife, Pearl Schapiro Barnes; a son, Robert, and a daughter, Joan Axelrod, both of New York City; his mother, Anna Rosenblum Bernstein of Englewood; a brother, Jerome Bernstein of New York City, and two grandchildren. Services will be Tuesday at 11:15 a.m. at Riverside Chapel, New York City, with burial in Beth David Cemetery, Elmont, N.Y. Arrangements are by Wien &Wien, Englewood. </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >JACOB BAUM, 82, of Waldwick died Saturday. Before retiring in 1970, he was a salesman for Paterson Supply &Engineering Co., Paterson, where he worked for 10 years. He was a member of the Clifton Congregation of the Netherlands Reformed Church. Surviving are two sons, Allan Jay of Clifton and Kenneth Richard of Allendale; two sisters, Anna Van Goudswaard of Waldwick and Henrietta DeKorte of Hawthorne, and seven grandchildren. His wife, Marie Peters Baum, died in 1987. Services will be Wednesday at 1:30 p.m. at Allwood Funeral Home, Clifton, with burial in Cedar Lawn Cemetery, Paterson. Visiting is tonight from 7 to 9 and Tuesday from 2 to 4 and 7 to 9 p.m. Donations to the Netherlands Reformed Christian School, 164 Jacksonville Road, Pompton Plains, N.J. 07444, would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >JESSICA BERKERY, 13, of Alpine, formerly of Closter, died Saturday. Surviving are her parents, Al and Val Gallasso Berkery; a brother, Michael; and her maternal grandparents, Peter and Nina Gallasso of Leonia. Services will be Tuesday at 10 a.m. at Moritz Funeral Home, Closter, with burial in George Washington Memorial Park, Paramus. Visiting is today from 2 to 4 and 7 to 9 p.m. </p>
<p class="articleParagraph enarticleParagraph" >LEON BESS of Clifton, formerly of Paterson, died Saturday. He was a contract manager with the United States Defense Department at International Telephone and Telegraph Corp., Nutley, for 31 years. A graduate of Lowell (Mass.) Textile Institute, he was an Army veteran of World War II, a two-time recipient of the Purple Heart, and a member of the Disabled American Veterans. He was a member of the Clifton Jewish Center and the Hebrew Free School, Paterson, and was a lifetime delegate of B'nai B'rith and past president of Lodge 143, Paterson. Surviving are his wife, Ruth Wise Bess; a daughter, Eileen Chafel of Boston; and a brother, Ben of Fair Lawn. Services are today at 10 a.m. at Jewish Memorial Chapel, Passaic, with burial in King Solomon Memorial Park, Clifton. Donations to B'nai B'rith would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >GRACE LaPUMA CAPPELLINE, 79, of Northvale died Saturday. Before retiring, she was a seamstress and a member of the International Ladies' Garment Workers Union. She was a parishioner of St. Anthony's R.C. Church and a member of the Senior Citizens, both in Northvale. Surviving are a son, Victor Cappellini of Norwood; two daughters, Carol Truchan of Cape Cod, Mass., and Patricia Kasper of Phoenix; two sisters, Elizabeth Capasso of Piermont, N.Y., and Pauline Bergunzi of Northvale, nine grandchildren, and two great-grandchildren. Mass will be said Thursday at 9:30 a.m. at St. Anthony's Church, with burial in Rockland Cemetery, Sparkill, N.Y. Visiting is Tuesday and Wednesday from 2 to 4 and 7 to 9 p.m. at Anthony R. Pizzi Funeral Home, Northvale. </p>
<p class="articleParagraph enarticleParagraph" >OLGA NEUPERT CHRIST, 87, of Little Falls, formerly of Wayne, died Sunday. She was a member of the United Methodist Church, Little Falls, and the United Methodist Women's Association. There are no immediate survivors. Her husband, Emil, died in 1968. Services will be Wednesday at 1 p.m. at Norman A. Parker Funeral Home, Little Falls, with burial in George Washington Memorial Park, Paramus. </p>
<p class="articleParagraph enarticleParagraph" >JENNIE YORIO CUICCI, 81, of Jersey City died Friday. She was a parishioner of Our Lady of Mount Carmel R.C. Church, Jersey City, and a member of its St. Irene Society, Rosary Society, and Senior Citizens Club. She was a member of 50-Plus Club, Jersey City. Surviving are two sons, Peter of Secaucus and Gerard of Jersey City; five daughters, Daura DeSiena of Lyndhurst, Theresa M. Esposito of Secaucus, Madeline Buda of Ridgefield, Carol Miller of Hong Kong, and Carmela M. Arzigliano of Jersey City, 20 grandchildren, and seven great-grandchildren. Mass will be said Tuesday at 10 a.m. at Our Lady of Mount Carmel Church, with burial in Holy Cross Cemetery, North Arlington. Visiting is today from 2 to 4 and 7 to 9 p.m. at McLaughlin Funeral Home, Jersey City. Donations to the American Diabetes Association, 312 N. Adamsville Road, Bridgewater, N.J. 08807, would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >HERBERT E. DAVISON, 58, of Ridgefield Park died Saturday. He was a custodian at the Civic Center, Ridgefield Park. He served in the Navy and was a member of Elks Lodge 1506, Ridgefield Park, the Ridgewood Country Club, and the New Jersey Golfers Association. Surviving are two brothers, Halbert of North Bergen and F. James of Ridgefield Park. Services will be Wednesday at 1 p.m. at Vorhees Funeral Home, Ridgefield Park, with burial in East Ridgelawn Cemetery, Clifton. Visiting is Tuesday from 2 to 4 and 7 to 9 p.m. </p>
<p class="articleParagraph enarticleParagraph" >MILDRED ANTONUCCI DECKER, 84, of Lodi died Sunday. She was a teacher's aide for Columbus School Nursery, Lodi, in the 1940s. She was a member of the Lodi Senior Citizen's Club and International Ladies' Garment Workers Union Local 145, Passaic. Surviving are a daughter, Joan Genneken of Clifton, and three grandchildren. Her husband, Louis, died in 1937. Mass will be said Wednesday at 9:30 a.m. at St. Joseph's R.C. Church, with burial in St. Nicholas Cemetery, both in Lodi. Visiting is Tuesday from 2 to 4 and 7 to 9 p.m. at Santangelo Funeral Home, Lodi. </p>
<p class="articleParagraph enarticleParagraph" >JOHN T. FELDMIER, 88, of Paterson, formerly of West Paterson, died Sunday. Before retiring in 1980, he was a maintenance worker for Jacob's department store, Paterson, for 15 years. Previously, he worked at the former Quakenbush department store and the former Dolphin Jute Mills, both in Paterson. He was a member of the New Jersey National Guard and a parishioner of St. Mary's R.C. Church, Paterson. Surviving are his wife, Louise Serico Feldmier; and a sister, Mamie Buratte of Valencia, Calif. Mass will be said Wednesday at 9:30 a.m. at St. Mary's Church, with burial in Holy Sepulchre Cemetery, Totowa. Visiting is Tuesday from 2 to 4 and 7 to 9 p.m. at Moore's Home for Funerals, Paterson. </p>
<p class="articleParagraph enarticleParagraph" >CONSTANCE R. FLORIO of Palisades Park, formerly of Union City, died Saturday. Before retiring, she was a teacher at St. Anthony's School, East Newark. Surviving are two sisters, Margaret Florio of Palisades Park and Helen Sgherzi of Burbank, Calif. A sister, Antoinette, and two brothers, Ralph and Carlo, are deceased. Mass will be said Tuesday at 9 a.m. at Epiphany R.C. Church, Cliffside Park, with burial in Madonna Cemetery, Fort Lee. Visiting is today from 2 to 4 and 7 to 9 p.m. at McCorry Brothers, Funeral Home, Cliffside Park. </p>
<p class="articleParagraph enarticleParagraph" >MELVIN GEISSENBERGER, 42, of Dumont died Friday. He was a self-employed insurance broker. Surviving are his wife, Janice Orsino Geissenberger; a daughter, Jessica, and two sons, Adam and Jarrod, all at home; his mother, Rose Geissenberger in Florida; and a brother, Stanley of Fair Lawn. His father, Lothar, is deceased. Services are today at 1:30 p.m. at Wien &Wien, Englewood. </p>
<p class="articleParagraph enarticleParagraph" >CHRISTOPHER GIBBONS, 34, of Paterson died Friday. He was a member of the New Christian Missionary Baptist Church, Paterson. Surviving, all of Paterson, are his wife, Chancena Staples Gibbons; a son, Jeremy; two daughters, Tyneeca and Lateanya; and his father, Paul Gibbons. His mother, Joann Gibbons, is deceased. Services will be Wednesday at 1 p.m. at the Bragg Funeral Home, Paterson, with burial in Fair Lawn Memorial Cemetery. Visiting is Tuesday from 7 to 9 p.m. </p>
<p class="articleParagraph enarticleParagraph" >FRED "FRITZ" P. HASTICK SR., 55, of Clifton, formerly of Passaic, died Saturday. He worked for Athenia Steel Co., Clifton, for 20 years. He was treasurer of United Steel Workers of America Local 1394, Clifton. He was a player-coach in the Athenia Steel Softball Team, Clifton, and was active in the Passaic County Bowling League. Surviving are his wife, Lillian Ladanyi Hastick; and two sons, Peter and Fred Jr., two stepsons, James Carroll and Daniel Carroll, and a daughter, Mary-Ellen Newby, all of Clifton; a stepdaughter, Linda Tonei of Bethlehem, Pa.; a brother, Peter of Brick; three sisters, Barbara Aguiar of San Jose, Calif., and Betty Crescenta and Janet Tresca, both of Passaic, and four grandchildren. Services will be Wednesday at 10:30 a.m. at Kamienski Funeral Home, Wallington, with burial in Lodi Cemetery. Visiting is tonight from 7 to 9 and Tuesday from 2 to 4 and 7 to 9 p.m. </p>
<p class="articleParagraph enarticleParagraph" >FRANCES N. KEATING, 88, of Fairview died Saturday. Before retiring, she was a registered nurse in West New York. Surviving are a sister-in-law Kathe Keating; two nieces, and two nephews. Services will be Wednesday at 10 a.m. at the Episcopal Church of the Good Shepherd, Fort Lee, with burial in Fairview Cemetery. Visiting is tonight from 7 to 9 and Tuesday from 2 to 4 and 7 to 9 p.m. at McCorry Brothers Funeral Home, Cliffside Park. </p>
<p class="articleParagraph enarticleParagraph" >MARY CONNOLLY KELLY, 78, of Guttenberg, formerly of New York City, died Friday. Surviving are a daughter, Joan Kelly of Guttenberg; and a sister, Alice Wright of New York City. Her husband, George W., died in 1964. Mass will be said Tuesday at 10:15 a.m. at Epiphany R.C. Church, Cliffside Park, with burial in Long Island National Cemetery, Farmingdale, N.Y. Visiting is today from 2 to 4 and 7 to 9 p.m. at McCorry Brothers Funeral Home, Cliffside Park. </p>
<p class="articleParagraph enarticleParagraph" >THOMAS G. MASTROIANNI, 68, of Spring Hill, Fla., formerly of East Rutherford, died Thursday. Before retiring, he worked for the Carlstadt Public Works Department. He was a member of the Knights of Columbus, Spring Hill, and VFW Post 3149 and American Legion Post 69, both in Carlstadt. Surviving are his wife, Edna Columbo Mastroianni; and a sister, Virginia Mastroianni of Spring Hill. A memorial mass will be said Saturday at 10 a.m. at St. Joseph's R.C. Church, Rutherford. Arrangements were by Turner Funeral Home, Spring Hill. </p>
<p class="articleParagraph enarticleParagraph" >EUGENE B. McSHANE, 94, of Harrison, N.Y., formerly of Yonkers, N.Y., died Sunday. Before retiring in 1959, he was a maintenance worker for Consolidated Edison Co. in New York City for 25 years. Previously, he was a member of Plumbers Union Local 86, Yonkers. He was an Army veteran of World War I and a member of American Legion Post 7, Yonkers. Surviving are five nieces and five nephews. His wife, Maude Needham McShane, died in 1979. Mass will be said Tuesday at noon at St. Pius X R.C. Church, Old Tappan, with burial in Gate of Heaven Cemetery, Hawthorne, N.Y. Visiting is today from 2 to 4 and 7 to 9 p.m. at Becker Funeral Home, Westwood. </p>
<p class="articleParagraph enarticleParagraph" >WARREN R. MOCK, 26, of Westwood died Sunday. He worked for the Westwood Road Department for eight years. He was a 1980 graduate of Westwood High School and a member of Zion Evangelical Lutheran Church, Westwood. Surviving are his wife, Stephanie Zion Mock; and, all of Westwood, his father, Russell; a brother, William R.; a sister, Wendy R. Mock; and his paternal grandmother, Henrietta Mock. His mother, Jane Bomm Mock, died in 1986. Services will be Tuesday at 12:30 p.m. at Zion Church, with burial in George Washington Memorial Park, Paramus. Visiting is today from 2 to 4 and 7 to 9 p.m. at Becker Funeral Home, Westwood. Donations to Zion Lutheran School, Elm Street, Westwood, N.J. 07675, or to the American Cancer Society would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >PAUL L. MURPHY SR., 67, of Bergenfield, formerly of the Bronx, died Saturday. Before retiring in 1983, he was a photoengraver at the New York Daily News and a member of the New York Lithographers and Photoengravers Union Local 1, New York City. He was an Army veteran of World War II and past commander of Gen. George Goethals American Legion Post 90, Bergenfield. Surviving are his wife, Catherine D. Nolan Murphy; five sons, Douglas, Paul Jr., Peter, and Robert, all of Bergenfield, and John of Palisades Park; a daughter, Cathy Murphy of Bergenfield; a sister, Virginia Webber of Palisades, N.Y.; two brothers, Donald and Robert, both of New York City, and four grandchildren. A brother, Frank, died in 1975. Mass will be said Wednesday at 10 a.m. at St. John's R.C. Church, Bergenfield, with burial in George Washington Memorial Park, Paramus. Visiting is Tuesday from 2 to 4 and 7 to 9 p.m. at Riewerts Memorial Home, Bergenfield. </p>
<p class="articleParagraph enarticleParagraph" >VIRGINIA ANN PATTERSON, 24, of Paterson died Friday. She was a 1984 graduate of Eastside High School, Paterson. Surviving are her mother and stepfather, Elizabeth Patterson and Iasaac McDuffie of Paterson; her father, Otis Lee Whitaker of Rich Square, N.C.; a sister, Cynthia Patterson of Paterson; two stepsisters, Lisa Renee McDuffie and Volante McDuffie, both of the Bronx; her maternal grandmother, Willie Mae Patterson of Chapin, S.C.; and her paternal grandmother, Helen Whitaker of Rich Square. Services will be Thursday at 1 p.m. at Bragg Funeral Home, with burial in Cedar Lawn Cemetery, both in Paterson. Visiting is Wednesday from 7 to 9 p.m. </p>
<p class="articleParagraph enarticleParagraph" >ELIZABETH MURPHY PETROVICH, 86, of Kissimmee, Fla., formerly of West New York, died Saturday. Surviving are a son, Donald of Ridgefield Park; a daughter, Evelyn Russo of Kissimmee; a brother, George Murphy, and a sister, Mary Frigeni, both of Atlantic Highlands, three grandchildren, and four great-grandchildren. Her husband, Julius, died in 1977. Mass will be said Wednesday at 9:45 a.m. at Holy Trinity R.C. Church, Hackensack, with burial in Madonna Cemetery, Fort Lee. Visiting is Tuesday from 2 to 4 and 7 to 9 p.m. at Vorhees Funeral Home, Ridgefield Park. </p>
<p class="articleParagraph enarticleParagraph" >PAUL A. POGLIO, 89, of West New York died Sunday. Before retiring 20 years ago, he was a mechanic for E. J. Worhley Co., Fairview, for many years. Surviving are four daughters, Mildred Vichiconti of Franklin Lakes, Bernice Nackey of Fort Lee, Janet Wilkens of West New York, and Paula Burke of North Bergen, and eight grandchildren. His wife, May Nagel Poglio, died in 1968. Mass will be said Wednesday at Our Lady of Libra R.C. Church, West New York, with burial in Flower Hill Cemetery, North Bergen. Visiting is tonight from 7 to 9 and Tuesday from 2 to 4 and 7 to 9 p.m. at Leber Funeral Home, Union City. </p>
<p class="articleParagraph enarticleParagraph" >LOUELLA KISHICK PORYAZIAN, 64, of North Bergen died Sunday. Surviving are two sons, Patrick Boone of Ringwood and Karnig Poryazian of North Bergen; a daughter, Paulette Milby of Mount Pocono, Pa.; a sister, Mary McMichael of Carteret, and six grandchildren. Her husband, John, died in 1975. Mass will be said Wednesday at 9:30 a.m. at St. John's Nepomucene R.C. Church, Guttenberg, with burial in George Washington Memorial Park, Paramus. Visiting is Tuesday from 2 to 4 and 7 to 9 p.m. at Vainieri Funeral Home, North Bergen. </p>
<p class="articleParagraph enarticleParagraph" >MARK A. REDMOND, 81, of Harrington Park died Sunday. Before retiring in 1972, he was a conductor for Port Authority Trans-Hudson Corp. (PATH) for 40 years. He was a member of the Brotherhood of Railroad Trainmen, New York City. Surviving are a son, Peter of Harrington Park; a daughter, Joan Eliseo of Beth Page, N.Y., eight grandchildren, and four great-grandchildren. His wife, Katherine, died in 1965. Mass will be said Tuesday at 10 a.m. at St. Pius X R.C. Church, Old Tappan, with burial in Gate of Heaven Cemetery, Valhalla, N.Y. Visiting is today from 2 to 4 and 7 to 9 p.m. at Anthony R. Pizzi Funeral Home, Northvale. Donations to Covenant House, 460 W. 41st St., New York, N.Y. 10036, or to the American Cancer Society would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >KATHERINE MARTIN ROCHE, 96, of Clifton died Sunday. Before retiring, she was warper for the Botany Mills Co., Passaic. Surviving are a son, Joseph of Mount Holly; a daughter, Geraldine Hornby of Clifton; four grandchildren, and eight great-grandchildren. Her husband, Frank, died in 1979. Services will be Wednesday at noon at Allwood Funeral Home, Clifton, with burial in Laurel Grove Memorial Park, Totowa. Visiting is Tuesday from 2 to 4 and 7 to 9 p.m. </p>
<p class="articleParagraph enarticleParagraph" >RICHARD EDWARD SARGENT, 48, of Laguna Hills, Calif., formerly of North Arlington, died Thursday. He had owned World Disc-Drive, Laguna Hills. Surviving are his wife, Dorothy Wilson Sargent; a son, Scott, and two daughters, Merritt and Kristen, all at home; his mother, Emma Boltz Sargent of North Arlington; two sisters, Joan Greatorex of Stanhope and Nancy Hotz of Union; and a brother, Raymond of North Arlington. His father, Clyde, is deceased. A memorial service is scheduled Wednesday at 7:30 p.m. at Advent Lutheran Church, Wyckoff. Arrangements were by O'Connor Laguna Hills Mortuary, Laguna Hills, Calif. Donations to a local diabetes association would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >LILLIAN K. SCHWAGER SCHULE, 91, of West Dennis, Mass., formerly of Saddle Brook, died Friday. Surviving are a daughter, Georgette Koenigof East Dennis, and a granddaughter. Her husband, George, died in 1972. Arrangements were by Sperry McHoul Funeral Home, North Attleboro, Mass. </p>
<p class="articleParagraph enarticleParagraph" >GIANOULA KIOUSIS TASSIGIANIS, 72, of Ridgefield, formerly of Jersey City, died Saturday. She was born in Greece. Surviving are her husband, Loukas; two sons, Pantazis of North Bergen and George of Ridgefield, and two grandchildren. Mass will be said Tuesday at 11 a.m. at Greek Ascension Church, Fairview, with burial in Fairview Cemetery. Visiting is today from 2 to 4 and 7 to 9 p.m. at A. K. Macagna Funeral Home, Cliffside Park. </p>
<p class="articleParagraph enarticleParagraph" >GERTRUDE LEWIS THOMAS, 78, of Paterson, formerly of McColl, S.C., died Friday. She was a member of the Northside Forces Senior Citizens Association, Paterson. Surviving are a son, Lawrence of New York City; three daughters, Edith Croom and Rosalind Reynolds, both of Paterson, and Mary HeleneThomas of New York City; a brother, Abraham Lewis of McColl, nine grandchildren, five great-grandchildren, and a great-great-grandchild. Her husband, David, died in 1974. Services will be Wednesday at 2 p.m. at Bragg Funeral Home, Paterson, with burial in Fair Lawn Memorial Cemetery. Visiting will be Tuesday from 7 to 9 p.m. </p>
<p class="articleParagraph enarticleParagraph" >WILLIAM B. WEILER JR., 44, of Englewood, formerly of Queens, died Saturday. He was a darkroom technician at Harlem Hospital and at Columbia University, both in New York City, and was a member of the New Jersey Radiology Society. He was a member of the Englewood Volunteer Ambulance Corps and a coach in the Englewood Little League. Surviving are his wife, Paula; a son, Isaac; and a sister, Shirley Williams of Brooklyn. Arrangements were by Wien &Wien, Englewood. Donations to the Englewood Volunteer Ambulance Corps would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >RUTH WILLIAMS, 94, of Little Falls died Sunday. Before retiring in 1987, she was a self-employed seamstress. She was a member of the First Reformed Church, Little Falls, where she taught Sunday School and was a member of its Ladies Guild. There are no immediate survivors. Services will be Wednesday at 10 a.m. at First Reformed Church, with burial in Cedar Lawn Cemetery, Paterson. Visiting is Tuesday from 3 to 5 and 7 to 9 p.m. at Norman A. Parker Funeral Home, Little Falls. </p>
<p class="articleParagraph enarticleParagraph" >EDDIE WEY WONG, 49, of North Bergen died Saturday. He was a systems analyst for Cap Gemini America Inc., New York City. He was a member of the New Jersey Air National Guard and the North Bergen Chapter of the American Bowling Congress, a committee member of Boy Scout Pack 17, North Bergen, and had been a drag racer at Englishtown Raceway. Surviving, all of North Bergen, are his wife, Sue Chong Wong; two sons, James and John; his mother, Sheung Hay Gong; and a sister, Peggy Wong. Services will be Thursday at Leber Funeral Home, Union City, with burial in Fairview Cemetery. Visiting is Tuesday and Wednesday from 2 to 4 and 7 to 9 p.m. </p>
<p class="articleParagraph enarticleParagraph" >LILLIAN LEVY WORTSMAN, 99, of River Vale, formerly of New York City, died Sunday. She was a millinery designer for Best &Co., New York City, in the 1920s. She was a volunteer for the American Red Cross during both world wars. Surviving are a daughter, Mary Jacobs of Englewood, two grandchildren, and four great-grandchildren. Services will be Tuesday at 10 a.m. at Wien &Wien, Englewood, with burial in Washington Cemetery, Brooklyn. Donations to the Jewish Home and Rehabilitation Center, 685 Westwood Ave., River Vale, N.J. 07675, would be appreciated. </p>
<p class="articleParagraph enarticleParagraph" >MARIE BOGERT ZOECKLEIN of Hawthorne died Saturday. Before retiring in 1985, she was a nurse at Roosevelt School, Hawthorne, for 18 years. She was a graduate of St. Joseph's Hospital School of Nursing, Paterson, and was a member of the First Reformed Church, Hawthorne. Surviving are a son, Carl of Clifton; a sister, Lillian Bogert of Hawthorne, and a granddaughter. Services will be Tuesday at 1 p.m. at Browning-Forshay Funeral Home, Hawthorne, with burial in Laurel Grove Cemetery, Totowa. Visiting is today from 3 to 5 and 7 to 9 p.m. Donations to St. Joseph's Hospital and Medical Center Building Fund, Paterson, N.J., would be appreciated.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>OBIT | COLUMN</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>Record / Hackensack, New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document rec0000020011116dl1n003vb</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020050427dkb200l6i" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Metropolitan Desk; B</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>4 Held in PATH Fight, And Cameras Get Credit  </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By JAMES BARRON  </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>635 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>2 November 1988</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late City Final Edition</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 1988 The New York Times Company.  All Rights Reserved.  </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >When a group of ''skinheads'' began beating a 31-year-old New Jersey man in a Manhattan PATH station on Saturday, supervisors at the railroad's operations center four miles away in Jersey City saw the whole thing.  </p>
<p class="articleParagraph enarticleParagraph" >When a group of ''skinheads'' began beating a 31-year-old New Jersey man in a Manhattan PATH station on Saturday, supervisors at the railroad's operations center four miles away in Jersey City saw the whole thing.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >Sitting in front of 65 television monitors connected to closed-circuit cameras mounted in Port Authority Trans-Hudson stations in New York and New Jersey, a PATH communications agent watched as the fight began. The agent switched on a videocassette recorder and alerted the Port Authority police.  </p>
<p class="articleParagraph enarticleParagraph" >Police supervisors dispatched a Port Authority police car that was less than two blocks from the station, at Ninth Street and Sixth Avenue. The police arrived in time to arrest four men on charges of second-degree robbery and attempted second-degree assault.  </p>
<p class="articleParagraph enarticleParagraph" >The four were identified as John Himmelstein, 23 years old, and Matthew Andrews, 20, both of Philadelphia; Harry Wilson, 17, of Morrisville, Pa., and John Cook, 22, of Nutley, N.J. All four were arraigned before Judge Leona Freedman, a civil court judge sitting in New York City Criminal Court.  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Wilson was released on his own recognizance. The others are being held on bail as high as $25,000. 'Combined Pieces of Luck'  </p>
<p class="articleParagraph enarticleParagraph" >The authorities said the incident began as William Stump, 31, of Bayonne, N.J., was on his way to catch a train with his wife, Marianne, and their infant daughter. Several men dressed in black leather jackets, sporting shaved heads or crew cuts and shouting racial and neo-Nazi epithets, confronted Mr. Stump.  </p>
<p class="articleParagraph enarticleParagraph" >In the struggle that followed, the authorities said, the assailants pushed Mr. Stump down a stairway and threatened his child. Both Mr. Stump and his assailants are white.  </p>
<p class="articleParagraph enarticleParagraph" >''The combined pieces of luck in this situation,'' said Lynn Tierney, a Port Authority spokesman, ''were that someone was alert, that the police were close by and that there was enough manpower on the scene to handle the situation.''  </p>
<p class="articleParagraph enarticleParagraph" >Even before the police officers arrived, PATH workers at the communications center - a hushed, darkened situation room that has the look and feel of a control room at a television station - were taking other actions to try to stop the fight.  </p>
<p class="articleParagraph enarticleParagraph" >The agent at the communications center began yelling, ''Stop that!'' through a speaker system only a few feet from where the men were fighting. Cameras Installed in 1980  </p>
<p class="articleParagraph enarticleParagraph" >PATH supervisors at the Jersey City center were also preparing to shut electrical power to the third rail if the fight continued past the turnstiles and onto the platform. The concern, Michael Scott, assistant general superintendent of operations for the PATH trains, said, was that passengers might be pushed onto the tracks and electrocuted. The fight was contained in the stairwell.  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Scott said the closed-circuit cameras, which give PATH workers at the Jersey City center a glimpse of each station along the railroad's 14 miles of tracks, were installed in 1980 to help make the system run more smoothly by showing when crowds are flooding into certain stations. But he said the cameras have also made the system safer.  </p>
<p class="articleParagraph enarticleParagraph" >''The idea is to make the right moves, almost like a game of chess,'' Mr. Scott said. ''The monitors help us decide if we should send a police officer in on a train, if that's faster than going in by a patrol car. It works out very, very well.''  </p>
<p class="articleParagraph enarticleParagraph" >He said that videotapes made at the communications center have been introduced as evidence in trials of PATH assailants.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i72101 : Urban Mass Transit Systems | i97411 : Broadcasting | imed : Media | i721 : Urban/Commuter Transit | irailtr : Road/Rail Transport | itsp : Transportation/Shipping</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>gcrim : Crime/Courts | ccat : Corporate/Industrial News | gcat : Political/General News | gcns : National Security</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>nyc : New York City | usny : New York | namz : North America | usa : United States | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>TRANSIT SYSTEMS  | TELEVISION  | CLOSED-CIRCUIT TV  | PORT AUTHORITY TRANS-HUDSON CORP (PATH)  | COOK, JOHN (ASSAULT SUSPECT)  | WILSON, HARRY  | ANDREWS, MATTHEW  | HIMMELSTEIN, JOHN  | STUMP, WILLIAM  | BARRON, JAMES </td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>New York Times Digital (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020050427dkb200l6i</td></tr></table><br/></div></div><br/><span></span><div id="article-rec0000020011118dkar015k1" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/recLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>NEWS</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>PAY RAISES ADVISED TO END PATH DISPUTE </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>United Press International </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>249 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>27 October 1988</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The Record</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>REC</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>© 1988 North Jersey Media Group Inc. Provided by ProQuest Information and Learning. All rights reserved.
 </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >A presidential board recommended Wednesday that the PATH interstate commuter rail line retroactively raise engineers' wages by 5 percent a year to settle a dispute with one of its unions. </p>
<p class="articleParagraph enarticleParagraph" >The presidential board, appointed by President Reagan to help resolve the labor dispute with the Brotherhood of Locomotive Engineers, said the wage increases should be effective for each of the three years beginning September 1985, September 1986, and September 1987. </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >The three-member board was appointed Sept. 9 at the request of the Port Authority Trans-Hudson Corp., the operator of PATH and a unit of the Port Authority of New York and New Jersey. </p>
<p class="articleParagraph enarticleParagraph" >The dispute began July 8, 1985, when the union sought improvements in wages, rules, and benefits. The National Mediation Board was unable to resolve the conflict. </p>
<p class="articleParagraph enarticleParagraph" >After a formal hearing and review, the emergency board recommended that the parties follow the wage pattern of 5 percent each year for three years accepted by six other PATH unions. </p>
<p class="articleParagraph enarticleParagraph" >The board recommended the granting of the Martin Luther King Jr. holiday, an improvement in meal allowances, and an improvement in major medical and group life insurance. </p>
<p class="articleParagraph enarticleParagraph" >The commuter line carries more than 200,000 passengers daily from Newark, Jersey City, and Hoboken in New Jersey and the World Trade Center and Penn Station in New York City. </p>
<p class="articleParagraph enarticleParagraph" >A PATH official said Wednesday that fares will probably not be affected since the increases had been anticipated.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>NEW JERSEY | TRANSIT | RAILROAD | REAGAN | EMPLOYMENT | UNION</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>Record / Hackensack, New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document rec0000020011118dkar015k1</td></tr></table><br/></div></div><br/><span></span><div id="article-rec0000020011118dk7a00rj0" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/recLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>CLM</b>&nbsp;</td><td>NJ / NY BRIEFS</td></tr>
<tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>NEWS</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>REAGAN STEPS INTO PATH DISPUTE </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>FROM RECORD AND WIRE SERVICE REPORTS </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>106 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>10 July 1988</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The Record</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>REC</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>© 1988 North Jersey Media Group Inc. Provided by ProQuest Information and Learning. All rights reserved.
 </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >President Reagan announced Saturday the formation of a presidential emergency board in the the dispute between the Port Authority Trans-Hudson Corp. and 215 PATH car cleaners, inspectors, and mechanics represented by Local 6330 of the Transportation Communications Union-Carmen Division. </p>
<p class="articleParagraph enarticleParagraph" >The announcement triggers legal procedures designed to prevent or delay a strike. </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >The corporation operates the PATH trains connecting Newark, Jersey City, and Hoboken with New York City. Local 6330 has been without a contract since June 1985. Last month it rejected a package recommended by the National Mediation Board.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>COLUMN | REAGAN | UNION | STRIKE | EMPLOYMENT | RAILROAD | NEW YORK CITY | NEWARK | JERSEY CITY | HOBOKEN | CONTRACT</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>Record / Hackensack, New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document rec0000020011118dk7a00rj0</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020050427dk3d017tm" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Westchester Weekly Desk; 12WC</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>Yonkers Group Finds Taiwan Encouraging On Subway Bid  </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By JAMES FERON  </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>1377 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>13 March 1988</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late City Final Edition</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 1988 The New York Times Company.  All Rights Reserved.  </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >A DELEGATION of Yonkers officials learned quickly when they got to Taiwan this month why the capital, Taipei, needed a subway system. They found the burgeoning city of 2.5 million suffering from severe traffic congestion.  </p>
<p class="articleParagraph enarticleParagraph" >A DELEGATION of Yonkers officials learned quickly when they got to Taiwan this month why the capital, Taipei, needed a subway system. They found the burgeoning city of 2.5 million suffering from severe traffic congestion.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >Taxi drivers displayed such inventive driving techniques that Michael Browning, a member of the Yonkers party, quoted a Yonkers official as telling the Taiwanese that ''in the United States, we pay for entertainment like that.''  </p>
<p class="articleParagraph enarticleParagraph" >The delegation was not in Taipei for fun, however. It was there to persuade Taiwan that the United Rail Car Company, which uses the former Otis Elevator plant in Yonkers to build subway cars for the Port Authority Trans-Hudson system and the New York City Transit Authority, could build subway cars for Taipei's planned mass-transit system.  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Browning, director of sales engineering for United Rail Car, said that the Yonkers delegation found the Taiwanese open and informed and especially interested in buying American-made equipment to help reduce an enormous trade imbalance. A major United States consortium of industries is also bidding for the contract.  </p>
<p class="articleParagraph enarticleParagraph" >Mayor Nicholas C. Wasicsko of Yonkers made the trip, along with State Senator Nicholas A. Spano and Assemblyman Terence M. Zaleski, to emphasize another point: that a skilled workforce was available and that the $200 million contract for the first 200 cars was crucial to the economic viability of Yonkers.  </p>
<p class="articleParagraph enarticleParagraph" >Eventually, 800 subway cars would be required for Taipei's first such rail system, at a cost of $1 billion. Senator Spano said the initial contract alone ''would represent 200 jobs in addition to the 180 employees currently working there.''  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Wasicsko, noting that the Taiwanese ''were pleased to have another bidder in the contract,'' said the ''United Rail people made the technical pitches and the local elected officials were there to allay Taiwanese fears that this might be a Japanese company or that Japanese workers would be involved. They were mainly interested in buying American, to help an imbalance of trade.''  </p>
<p class="articleParagraph enarticleParagraph" >The United Rail plant was financed by Nissho Iwai, a major Japanese trading company, in a joint venture with Kawasaki, which is also Japanese. Mr. Zaleski said ''one of Taiwan's major concerns was their perception of United Rail Car as a wholly owned subsidiary of two Japanese entities.''  </p>
<p class="articleParagraph enarticleParagraph" >Assemblyman Zaleski said, ''The Taiwanese were very impressed that, with us, they would be dealing with a manufacturer of rail cars rather than a dealer, or broker.'' He was referring to the United States Taiwan Transit Group, the consortium that represents Yonkers's major domestic rival for the bid.  </p>
<p class="articleParagraph enarticleParagraph" >The consortium, created solely for the purpose of pursuing the Taipei contract, according to Larry E. Salci, president of the group, consists of subsidiaries of the General Electric Company, the Westinghouse Electric Company, the General Railway Signal Company, Pullman Technology and Wabco Westinghouse, which is owned by the American Standard Corporation.  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Salci has said that all of the companies of the consortium were United States corporations and that the rail cars would be assembled or manufactured in Pullman Technology's plant in Barre, Vt. Pullman is owned by the Bombardier Corporation, a Canadian company.  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Browning said he told the Taiwanese that ''to our knowledge, the fabrication facility'' for the consortium's car bodies ''is in Quebec, Canada, with the final assembly in Vermont.'' He said that United builds its cars in Yonkers, ''taking sheet metal and steel and welding them together into rail cars.''  </p>
<p class="articleParagraph enarticleParagraph" >''Our point was that the manufacturing was highly labor intensive,'' Mr. Browning said, ''and since that would be done in Yonkers we could achieve a higher U.S. content,'' or participation, in the rail cars than their competitors.  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Spano said it was ''clear from the Taiwanese they are looking at a substantial percentage of American participation to lower the trade deficit of $19 billion.''  </p>
<p class="articleParagraph enarticleParagraph" >''They wondered if they would be dealing with the Japanese economy, with the benefits going to Japan,'' Mr. Zaleski said. ''We made the case that the principal benefits would go to the local economy, to the New York State economy and the United States generally.''  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Browning said that the American delegation inluded six officials of United Rail as well as Francis T. Vaughan, director of the state's Far Eastern office in Tokyo, and three officials of the Port Authority of New York and New Jersey - Philip LaRocco, director of world trade and economic development, Keiji Imai, manager of the Tokyo development office and Mike Potterf, manager of the Washington office.  </p>
<p class="articleParagraph enarticleParagraph" >Officials of two Yonkers subsidiaries, the Garrett Corporation, manufacturers of propulsion systems, and Wabco, the Westinghouse Air Brake Company, also were present. Mr. Browning said that the Taiwan Transit Group, the consortium, ''claim they represent the universe of subsuppliers, so we wanted to show we can also provide subsuppliers.''  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Browning said, ''New York State is interested in promoting exports from New York as well as seeking direct investment into New York from that region, and that's the purpose of the Port Authority being there - to promote trade in both directions.'' The Port Authority is linked to the Yonkers project in another way: it owns the former Otis building that is used both by United Rail and by Wabco.  </p>
<p class="articleParagraph enarticleParagraph" >Mr. LaRocco issued an invitation to the Taiwanese authorities to visit the PATH system because United Rail has supplied new cars for PATH, as well as refurbishing old ones, ''and because the PATH fleet of about 350 cars will be similar in size to Taipei's,'' Mr. Browning said.  </p>
<p class="articleParagraph enarticleParagraph" >The PATH commuter subway system links midtown Manhattan and the Wall Street area with Hoboken and Newark in New Jersey. United Rail supplied 95 new cars for PATH, the last one delivered in November 1987 and is overhauling 248 older cars.  </p>
<p class="articleParagraph enarticleParagraph" >The Americans met with S. T. Hsu, the Mayor of Taipei; Augustine Wu, assistant director of the Taiwan Board of Trade, which sets Government trade policy; Clement Chang, Speaker of the Taipei City Council, and Benjamin Chi, director general of Dorts, the Department of Rapid Transit Services, Taipei's transit authority.  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Browning said United Rail had asked Mr. Chi when he would be advertising for bids and what kinds of subway cars he would be seeking. The Taiwanese official said that the bids would be offered next June, with the schedule of awards announced in December. The price, he said, would be roughly $1 million a car.  </p>
<p class="articleParagraph enarticleParagraph" >''As for the specifications of the cars, he told us the design would be based most closely on the subway cars in Singapore,'' also a new system. ''It was good news,'' Mr. Browning said, ''because they were manufactured by a Japanese consortium headed by Kawasaki, one of the partners in United Rail.''  </p>
<p class="articleParagraph enarticleParagraph" >Singapore's cars are 75 feet long, similar in length to about 1,000 new cars in service on the BMT and Independent lines in New York City. ''We are providing 200 of those cars, which are known as R-68A's,'' Mr. Browning said, ''with delivery starting in mid-1988 and ending in mid-89.''  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Wasicsko said he was hopeful that ''we will get at least a portion of the contract.'' He said Taipei would be offering other contracts for ''people movers'' as part of the city's new transportation infrastructure as well as rail cars for other sections of the rail system.  </p>
<p class="articleParagraph enarticleParagraph" >The Americans were welcomed by the American Institute of Taiwan, a quasi-diplomatic agency that ''represents United States interests,'' one visitor said, in the absence of official diplomatic recognition. Washington recognizes the Chinese Government in Peking.  </p>
<p class="articleParagraph enarticleParagraph" >Delegation members said that their visit was major news on the evening television newscasts the day they held a news conference and that the Yonkers bid also was given space in Taipei newspapers. Mr. Zaleski said the visit ''put us on the map, where previously we were an unknown quantity.''  </p>
<p class="articleParagraph enarticleParagraph" >Photo of subway car being built at the Unitd Rail Car plant (NYT)  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>CO</b>&nbsp;</td><td><br/>mta : Metropolitan Transportation Authority | otel : Otis Elevator Company | utc : United Technologies Corporation</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i721 : Urban/Commuter Transit | i72101 : Urban Mass Transit Systems | i32 : Machinery/Industrial Goods | i3255 : Handling Equipment | irailtr : Road/Rail Transport | itsp : Transportation/Shipping</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>ccat : Corporate/Industrial News | gcat : Political/General News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>china : China | taiwan : Taiwan | nyc : New York City | usny : New York | asiaz : Asia | chinaz : Greater China | devgcoz : Emerging Market Countries | dvpcoz : Developing Economies | easiaz : Eastern Asia | namz : North America | usa : United States | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>TRANSIT SYSTEMS  | SUBWAYS  | NEW YORK CITY TRANSIT AUTHORITY (NYCTA)  | UNITED RAIL CAR CO  | PORT AUTHORITY TRANS-HUDSON CORP (PATH)  | FERON, JAMES  | SPANO, NICHOLAS A (SEN)  | ZALESKI, TERENCE (ASSEMBLYMAN)  | WASICSKO, NICHOLAS C (MAYOR) </td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>New York Times Digital (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020050427dk3d017tm</td></tr></table><br/></div></div><br/><span></span><div id="article-NYTF000020050426dk1h00cb9" class="article" ><div class="article enArticle"><p><img src="https://logos.factiva.com/nytfLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>Westchester Weekly Desk; 11WC</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>Rail Car Plant In Yonkers Seeks $1 Billion Contract  </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By JAMES FERON  </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>1622 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>17 January 1988</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The New York Times</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>NYTF</td></tr><tr><td align="right" valign="top" class="index"><b>ED</b>&nbsp;</td><td>Late City Final Edition</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>Copyright 1988 The New York Times Company.  All Rights Reserved.  </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >IT was just a year ago that Yonkers officials were saying they hoped the city's new Kawasaki rail car plant, which was making subway cars for New York and New Jersey, would someday be manufacturing them for railroad and subway systems around the world.  </p>
<p class="articleParagraph enarticleParagraph" >IT was just a year ago that Yonkers officials were saying they hoped the city's new Kawasaki rail car plant, which was making subway cars for New York and New Jersey, would someday be manufacturing them for railroad and subway systems around the world.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >That day could be approaching. Mayor Nicholas C. Wasicsko will be leaving for Taiwan next Sunday with State Senator Nicholas A. Spano, Assemblyman Terence M. Zaleski and other officials to lobby for a $1 billion contract to build 800 subway cars.  </p>
<p class="articleParagraph enarticleParagraph" >The cars would be used for a subway being built in Taipei, the capital of Taiwan, according to Michael Browning, an official of Nissho Iwai, a major Japanese trading company that has financed the plant in a joint venture with Kawasaki.  </p>
<p class="articleParagraph enarticleParagraph" >''The first order of 200 cars, worth about $200 million, would be for the Tamshui-Hsintien line, one of the radial spokes on the semi-circular system being built in Taipei,'' Mr. Browning said. The Yonkers plant is currently working on other contracts through early 1989, but the initial Taiwan contract would guarantee work until 1991.  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Wasicsko, whose only previous foreign trip has been a Mexican vacation, said that the Taiwanese ''have a lot of respect for local officials, and they take it as a sign of great respect to have officials take the time to visit.''  </p>
<p class="articleParagraph enarticleParagraph" >The Mayor will also be accompanied by representatives of Kawasaki, which is paying for the trip, and by officials of the state's Department of Economic Development and of the Port Authority of New York and New Jersey, which owns the Yonkers plant. Port Authority officials asked Mayor Wasicsko to make the five-day trip, a spokesman for the Mayor said.  </p>
<p class="articleParagraph enarticleParagraph" >The plant, former home of the Otis Elevator Company, was purchased by the Port Authority in 1985 from the United Technologies Corporation of Hartford, which had acquired Otis. The Port Authority leased space to Kawasaki and Nissho Iwai Partnership, whose name is being changed to United Rail Car Partnership.  </p>
<p class="articleParagraph enarticleParagraph" >Several factors have combined to encourage Yonkers officials to believe that an American-based company stands a good chance of getting the contract. For one, Taiwan has an $18 billion trade surplus with the United States, and the State Department and the United States Department of Commerce have been pressing Taiwan to reduce that imbalance.  </p>
<p class="articleParagraph enarticleParagraph" >The Taiwanese, for their part, sent a trade mission to the United States last year ''to find things to buy,'' Mr. Browning said.  </p>
<p class="articleParagraph enarticleParagraph" >The Taiwanese, he said, ''are making strong and legitimate efforts to buy American, providing they can get a competitive bid from a U.S. manufacturer.''  </p>
<p class="articleParagraph enarticleParagraph" >''Given the weakness of the dollar,'' he added, ''we will be able to give them a competitive bid.''  </p>
<p class="articleParagraph enarticleParagraph" >He said that he assumed bids had also been received from companies in France, West Germany, Canada and Japan, as well as from a consortium in the United States known as the United States Taiwan Transit Group.  </p>
<p class="articleParagraph enarticleParagraph" >This consortium could represent a significant competitor to the Yonkers plant. It consists of subsidiaries of the General Electric Company, the Westinghouse Electric Company, the General Railway Signal Company, Pullman Technology Inc. and WABCO Westinghouse, which is owned by American Standard Corporation.  </p>
<p class="articleParagraph enarticleParagraph" >Larry E. Salci, president of the group, said that ''we are a consortium of five companies created solely for the purpose of pursuing the Taipai Mass Rapid Transit Project.''  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Salci said all of the companies in the consortium were United States corporations and that the rail cars would be assembled or manufactured in Pullman Technology's plant in Barre, Vt. Pullman is owned by the Bombardier Corporation, a Canadian company.  </p>
<p class="articleParagraph enarticleParagraph" >He said that the consortium was formed about three years ago and since then has been seeking ''to supply railway vehicles and signaling equipment to the Taipai program.'' According to Mr. Salci, the transit project is now in its final engineering stages.  </p>
<p class="articleParagraph enarticleParagraph" >He said that the Barre plant had been producing rail cars since the 1980's, ''for the New York City Transit Authority, for the Metro-North Commuter Railroad, for New Jersey Transit and for the Connecticut Department of Transportation.''  </p>
<p class="articleParagraph enarticleParagraph" >According to Mr. Salci, General Electric's transportation division is located in Erie, Pa.; General Railway Signal in Rochester, N.Y.; WABCO Westinghouse in North Carolina and Westinghouse Electric in Pittsburgh.  </p>
<p class="articleParagraph enarticleParagraph" >The Barre plant, Mr. Salci said, ''has both capabilities, of manufacturing the car body shells as well as doing the car assemblies.'' Another official of the consortium said that the Barre plant has been doing final assemblies.  </p>
<p class="articleParagraph enarticleParagraph" >''The group we represent is the core of the U.S. transit industry and has been for 50 years,'' Mr. Salci said. ''Bombardier owns all of the designs and technology that once belonged to both Budd and Pullman, or 80 percent of the U.S. transit car industry over the last 50 years.  </p>
<p class="articleParagraph enarticleParagraph" >''We've had many delegations, both technical and commercial, in Taiwan over the last three years,'' he added. ''We're working on this project very intensively.'' He said that the recent Export Trading Act permitted U.S. companies to compete as a consortium, something that had been difficult under U.S. anti-trust laws.  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Browning, who is director of sales engineering for the Kawasaki group, said that Taiwanese officials recently traveled to Singapore to inspect ''a new subway system that was put into service only three or four months ago.'' The cars there are Kawasakis that were ''built in a consortium with three or four Japanese manufacturers,'' he said.  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Browning said that Mr. Wasicsko would be telling the Taiwanese ''a simple story - that this plant is the only one of its type and that it has a significant impact on the region.'' He explained that he believed it was the only United States plant manufacturing the rail cars, rather than assembling them, and added that he did not think the Taiwanese were aware of the scope of the Kawasaki plant.  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Spano said the contract would represent ''a real shot in the arm for expansion of the plant.'' Yonkers has designated the area that includes the plant as an Economic Development Zone, part of an effort to compete for state assistance. If the Yonkers site is among those picked by a state commission, companies setting up business or expanding would receive a 10 percent reduction on state corporate taxes, credit on capital-growth taxes, full refunds of sales taxes on building materials, a 3 percent reduction of utility bills for nonretail businesses and an exemption from real property taxes on a sliding scale over 10 years.  </p>
<p class="articleParagraph enarticleParagraph" >''When Nissho Iwai makes its presenation,'' Mr. Spano said, ''it will be asking local officials to be with them to show the extent of government and economic support.''  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Zaleski said that ''very few major rail car contracts are available at any time around the world and this, therefore, is a prime opportunity to win a contract while also helping to redress the trade imbalance.''  </p>
<p class="articleParagraph enarticleParagraph" >''While this operation is a joint venture of two wholly owned American subsidiaries of Japanese companies, virtually all of the subsidiary contracting work will be done through American source subcontractors, and all construction work will be done in the United States,'' Mr. Zaleski said.  </p>
<p class="articleParagraph enarticleParagraph" >In addition to the 300 jobs that would be guaranteed at the Yonkers rail car plant, he said, ''at least 500 other jobs in the local economy are attributable to this plant's continued viability.''  </p>
<p class="articleParagraph enarticleParagraph" >Mr. Zaleski suggested that ''American-sourced suppliers should have an inside track because of the trade imbalance and the signals that have gone out to redress it.''  </p>
<p class="articleParagraph enarticleParagraph" >Kawasaki's first contract for subway cars was with the Philadelphia subway system in the late 1970's, exporting 125 units from Japan for the Broad Street subway and 141 light rail vehicles, ''which are like street cars,'' Mr. Browning said, for the Southeast Pennsylvania Transit Authority.  </p>
<p class="articleParagraph enarticleParagraph" >The Yonkers plant was opened for rail-car production in 1986, its first job the manufacture of subway cars for the PATH (Port Authority Trans-Hudson) System between 33d Street and the World Trade Center in Manhattan, and Hoboken and Newark in New Jersey.  </p>
<p class="articleParagraph enarticleParagraph" >It built 95 cars, Mr. Browning said, and the last one was delivered two months ago. ''We built most of the PATH cars in the Yonkers plant, instead of just assembling them,'' he said, adding that it was still the only United States plant capable of making cars from the ground up.  </p>
<p class="articleParagraph enarticleParagraph" >The first five PATH cars were built in Kawasaki's plant in Kobe, Japan, while the Yonkers plant contract was being negotiated. The next 33 were ''kit'' cars - car bodies from Kobe, assembly in Yonkers - and the final 57 cars were built completely in Yonkers.  </p>
<p class="articleParagraph enarticleParagraph" >''That would be our basis for the Taiwan contract, completely manufacturing the cars,'' according to Mr. Browning, who said it was the Port Authority's goal to encourage manufacturing in the port district, which he described as within 25 miles of the Statue of Liberty.  </p>
<p class="articleParagraph enarticleParagraph" >Kawasaki has since assembled 325 R-62 cars used on on New York City's Lexington Avenue IRT line. Mr. Browning said, ''It was a very successful order, producing cars with the best mean times between failures in the system.'' The plant is now assembling 200 cars for the Independent and BMT lines, the first ones to be delivered this year.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IN</b>&nbsp;</td><td><br/>i721 : Urban/Commuter Transit | i72101 : Urban Mass Transit Systems | irailtr : Road/Rail Transport | itsp : Transportation/Shipping</td></tr><tr><td align="right" valign="top" class="index"><br/><b>NS</b>&nbsp;</td><td><br/>ccat : Corporate/Industrial News | gcat : Political/General News</td></tr><tr><td align="right" valign="top" class="index"><br/><b>RE</b>&nbsp;</td><td><br/>nyc : New York City | taiwan : Taiwan | usny : New York | asiaz : Asia | chinaz : Greater China | devgcoz : Emerging Market Countries | easiaz : Eastern Asia | namz : North America | usa : United States | use : Northeast U.S.</td></tr><tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>TRANSIT SYSTEMS  | SUBWAYS  | LOBBYING AND LOBBYISTS  | KAWASAKI GROUP  | FERON, JAMES  | WASICSKO, NICHOLAS C (COUNCILMAN)  | SPANO, NICHOLAS A (SEN)  | ZALESKI, TERENCE (ASSEMBLYMAN) </td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>New York Times Digital (Full Text)</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document NYTF000020050426dk1h00cb9</td></tr></table><br/></div></div><br/><span></span><div id="article-rec0000020011118dj8o02fbk" class="lastarticle" ><div id="lastArticle" class="article enArticle"><p><img src="https://logos.factiva.com/recLogo.gif" onerror="this.style.display='none';"/></p>
<table cellpadding="1" cellspacing="1" border="0"><tr><td align="right" valign="top" class="index"><b>SE</b>&nbsp;</td><td>NEWS</td></tr>
<tr><td align="right" valign="top" class="index"><b>HD</b>&nbsp;</td><td><span class='enHeadline'>PATH ON RIGHT TRACK NEW CARS INCREASING SERVICE </span>
</td></tr><tr><td align="right" valign="top" class="index"><b>BY</b>&nbsp;</td><td>By Robert Polner, Record Staff Writer </td></tr>
<tr><td align="right" valign="top" class="index"><b>WC</b>&nbsp;</td><td>1197 words</td></tr><tr><td align="right" valign="top" class="index"><b>PD</b>&nbsp;</td><td>24 August 1987</td></tr><tr><td align="right" valign="top" class="index"><b>SN</b>&nbsp;</td><td>The Record</td></tr><tr><td align="right" valign="top" class="index"><b>SC</b>&nbsp;</td><td>REC</td></tr><tr><td align="right" valign="top" class="index"><b>LA</b>&nbsp;</td><td>English</td></tr><tr><td align="right" valign="top" class="index"><b>CY</b>&nbsp;</td><td>© 1987 North Jersey Media Group Inc. Provided by ProQuest Information and Learning. All rights reserved.
 </td></tr>
<tr><td align="right" valign="top" class="index"><p><b>LP</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >It has taken a quarter century of sporadic train delays and equipment failures, but the Port Authority Trans-Hudson rail system _ PATH _ is gaining the confidence of riders and critics. </p>
<p class="articleParagraph enarticleParagraph" >"The system is beginning to improve, noticeably so," said Douglas John Bowen, vice-president of the New Jersey Association of Railroad Passengers, a West Orange-based rider advocacy group, with more than 200 members. </p>
</td></tr><tr><td align="right" valign="top" class="index"><p><b>TD</b>&nbsp;</p></td><td><p class="articleParagraph enarticleParagraph" >On the eve of the 25th anniversary of the Port Authority's purchase of PATH _ the rail link that carries 210,000 commuters per day between northern New Jersey and Lower and Midtown Manhattan _ critics say the widespread deterioration that afflicted the system is a thing of the past. </p>
<p class="articleParagraph enarticleParagraph" >Critics also say worsening traffic at Hudson River bridge and tunnel crossings operated by the Port Authority of New York and New Jersey has prompted a monetary commitment to improving PATH. </p>
<p class="articleParagraph enarticleParagraph" >The bistate agency is spending $200 million for 95 modern silvery cars with double doors and added seating. Two thirds of the new cars have been placed in service, and an extra car is being added to every train, bringing 20 percent more capacity. </p>
<p class="articleParagraph enarticleParagraph" >In addition, the fleet of 248 PATH cars dating back to 1965 is being refurbished. The program, however, is a year behind schedule and will not be completed before next spring. About 80 of the rehabilitated cars are in service. </p>
<p class="articleParagraph enarticleParagraph" >Figures show the Port Authority may be putting the skids on delays and breakdowns, but officials acknowledged it's too early to tell for sure. </p>
<p class="articleParagraph enarticleParagraph" >In 1986, the incidents resulting in delays or temporary removal of trains rose to 1,431 _ 190 more than the number experienced in 1984. An "incident" can be anything from a broken door _ a common malady of overcrowded trains _ to problems with brakes, air-conditioning, and lighting. </p>
<p class="articleParagraph enarticleParagraph" >The latest figures show problems leveling off _ an average 121 incidents a month between January and July, compared with 120 during the same period in 1986. </p>
<p class="articleParagraph enarticleParagraph" >"For a while there, we were in the worst position a rail operation could find itself, with a downward trend of available and reliable cars and a steady increase in ridership," said Richard R. Kelly, vice- president and general manager of the PATH Corporation. </p>
<p class="articleParagraph enarticleParagraph" >"But we're getting back on track." PATH officials acknowledged that problems could have been avoided with a schedule of preventive maintenance, phased in, instead of a crash program requiring reliance on 38 vintage-1958 cars. </p>
<p class="articleParagraph enarticleParagraph" >But Kelly said the work could not be done until a quick and reliable service shop was opened by the Port Authority two years ago. The shop, in Jersey City, is a companion to an older, less efficient shop, also in Jersey City. Another factor was the contracting of private companies to overhaul trains, a procedure that began four years ago. </p>
<p class="articleParagraph enarticleParagraph" >"I find PATH generally reliable," said Hyman Sprekman, an attorney for Equitable Life, as he waited on the Hoboken platform one morning last week for the train to West 33rd Street in Manhattan. </p>
<p class="articleParagraph enarticleParagraph" >Among a half-dozen passengers interviewed, none voiced major complaints about service, although a few said blackouts and loss of air-conditioning is still too common for comfort. </p>
<p class="articleParagraph enarticleParagraph" >PATH, a 14-mile, 1,130-employee operation, runs trains above ground, below ground, and under the Hudson River _ in Newark, Harrison, Jersey City, and Hoboken. The Manhattan stops are at the World Trade Center and between Christopher Street to 33rd Street on the West Side. </p>
<p class="articleParagraph enarticleParagraph" >The old Hudson and Manhattan Railroad _ the Hudson Tubes _ was acquired by the Port Authority on Sept. 1, 1962, after the bistate agency received contractual assurance that the debt-ridden operation would not hurt its credit rating or ability to finance other projects. </p>
<p class="articleParagraph enarticleParagraph" >PATH has experienced a twofold rise in patronage over the last days of the Hudson Tube. Its passenger volume is comparable to that of the larger Bay Area Rapid Transit in San Francisco. PATH officials expect 2,000 to 4,000 new riders a year through the mid-1990's, a heavy burden for a system that has reached peak-period capacity. </p>
<p class="articleParagraph enarticleParagraph" >Officials also are counting on new forms of transit, outside of PATH, to absorb the 80,000 more New Jersey commuters to Manhattan expected by the turn of the century. </p>
<p class="articleParagraph enarticleParagraph" >Options include a planned Port Authority ferry service, at least three years from reality; a proposed trolley along the New Jersey waterfront; and direct train service from the Hackensack meadowlands to Penn Station in New York, which is envisioned by NJ Transit. </p>
<p class="articleParagraph enarticleParagraph" >For now, critics say PATH could do more to encourage people to use mass transit to Manhattan, particularly during the off-peak times. </p>
<p class="articleParagraph enarticleParagraph" >On weekends, no direct service operates between Hoboken and the World Trade Center, despite the proximity to South Street Seaport, Battery Park, and other downtown attractions. </p>
<p class="articleParagraph enarticleParagraph" >Weekend travel to Pavonia station in Jersey City also remains difficult even though a new shopping mall has opened above ground. </p>
<p class="articleParagraph enarticleParagraph" >"The lack of off-peak service is a disincentive," Bowen said. "With new commercial developments like the Pavonia mall and Newport {a massive commercial and residential project in Jersey City}, PATH has a golden opportunity to attract weekend riders with more frequent trains." While rush-hour service is every three minutes, Bowen said, off-peak service ranges from one train every 30 minutes _ a "disaster" _ to one every 15 minutes, "which is only in the ballpark of acceptability for a major rail system." Stephen B. Dobrow, president of the Committee for Better Transit, a commuter lobby concerned with New York and New Jersey travel, said the Port Authority is more devoted to autos and lucrative economic development projects than to PATH. </p>
<p class="articleParagraph enarticleParagraph" >The Port Authority, Dobrow said, has resisted extending tracks to Newark International Airport. The airport, which is run by the agency, is one of the fastest-growing in the country and offers limited public transportation. </p>
<p class="articleParagraph enarticleParagraph" >"I think you could also argue that the Port Authority has done the minimum to make it work," he said. </p>
<p class="articleParagraph enarticleParagraph" >Lloyd Schwalb, a spokesman for the Port Authority, countered: </p>
<p class="articleParagraph enarticleParagraph" >"We took a bankrupt heap that could barely run and made it into one of the best transit systems in the country. It's handling twice as many people as then, and over $1 billion has been invested in it by the Port Authority. That's putting your money where your mouth is. </p>
<p class="articleParagraph enarticleParagraph" >"Now we're putting another billion dollars into the system, and it's 90 percent our own funding. No other passenger railroad in the country can say that." The original $1.2-billion investment reflects $800 million in operating subsidies to cover the deficit and $400 million in deferred maintenance, officials said. </p>
<p class="articleParagraph enarticleParagraph" >The Port Authority is now investing $1 billion in the new and rehabilitated cars. It also is using the money to modernize and lengthen platforms at PATH's six oldest stations, mount a new entrance pavilion at Exchange Place in Jersey City, and open a garage and yard in Harrison.  </p>
</td></tr><tr><td align="right" valign="top" class="index"><br/><b>ART</b>&nbsp;</td><td><br/>PHOTO - AL PAGLIONE / THE RECORD - New Jersey

commuters jamming PATH platform at the Hoboken station. PATH is

celebrating its 25th year. </td></tr>
<tr><td align="right" valign="top" class="index"><br/><b>IPD</b>&nbsp;</td><td><br/>NEW JERSEY | TRANSPORTATION | RAILROAD | EQUIPMENT</td></tr><tr><td align="right" valign="top" class="index"><br/><b>PUB</b>&nbsp;</td><td><br/>Record / Hackensack, New Jersey</td></tr><tr><td align="right" valign="top" class="index"><br/><b>AN</b>&nbsp;</td><td><br/>Document rec0000020011118dj8o02fbk</td></tr></table><br/></div></div><br/></div></div><span><div id="pageFooter"><table width="100%" cellspacing="0" cellpadding="0" border="0" class="footerBG">
	<tr>
		<td nowrap="nowrap" width="100%" align="right"><span class="copyright">&copy; 2024 Factiva, Inc.  All rights reserved.</span></td>
		<td><div class="ftright">&nbsp;</div></td>
	</tr>
</table>
<span class='shadowL'></span><span class='shadowR'></span></div><noscript><img src="http://om.dowjoneson.com/b/ss/djfactivatesting/1/H.22.1--NS/0" height="1" width="1" border="0" alt="" /></noscript></span><script type="text/javascript">
//<![CDATA[
jQuery(document).ready (function(){
try { InitializeOmniture('djfactiva');}catch (ex) {}
try {DJOmniture.Property.SessionId = "pbvi2WwW_MU2TONJRHFRDGMRXGIZTIYRUGNRDAMZZGBSDMOBSGVSGGMJZHE4A";DJOmniture.Property.UserId_Ns = "E6OO2HVCQHVPFGQJS4YZ7OYDUQ";DJOmniture.Property.AccountId = "";DJOmniture.Property.FullURL = "https://global.factiva.com/hp/printsavews.aspx?ppstype=Article&pp=Print&hc=All";DJOmniture.Property.AccessCode = "0086";DJOmniture.Property.PageName = "PageNameNotSet";DJOmniture.Property.SearchType = "";DJOmniture.Property.FilterType = "";DJOmniture.Property.FilterValue = "";DJOmniture.Property.Type = "";DJOmniture.Property.ProfileName = "";DJOmniture.Property.ReportType = "";DJOmniture.Property.DataRange = "";DJOmniture.Property.FormatType = "";DJOmniture.Property.AccessionNumber = "";DJOmniture.Property.ContentType = "";DJOmniture.Property.ArticleType = "";DJOmniture.Property.Headline = "";DJOmniture.Property.Author = "";DJOmniture.Property.WordCount = "";DJOmniture.Property.PublicationDate = "";DJOmniture.Property.Source = "";DJOmniture.Property.BaseLanguage = "";DJOmniture.Property.ScreeningType = "";DJOmniture.Property.ScreeningValue = "";DJOmniture.Property.FactivaPageId = "";DJOmniture.Property.Channel = "";DJOmniture.Property.Area = "";DJOmniture.Property.Section = "";DJOmniture.Property.SearchQueryLength = "";}catch (ex) {}
});
//]]>
</script><table id="tmpJQueryTarget" border="0">
	<tr>

	</tr>
</table>

<script type="text/javascript">
//<![CDATA[
(function() {var fn = function() {$get('PageScriptManager_HiddenField').value = '';Sys.Application.remove_init(fn);};Sys.Application.add_init(fn);})();//]]>
</script>

<script src="https://global.factiva.com/CombineScriptsHandler.ashx?_TSM_HiddenField_=PageScriptManager_HiddenField&amp;_TSM_CombinedScripts_=%3b%3bAjaxControlToolkit%2c+Version%3d3.0.30930.28736%2c+Culture%3dneutral%2c+PublicKeyToken%3d28f01b0e84b6d53e%3aen-US%3ab0eefc76-0092-471b-ab62-f3ddc8240d71%3a865923e8%3bfactiva.com.ui%3aen-US%3a3bc5af08-e27e-4f9f-a631-cfc41eadfddc%3a84690f9d%3bEMG.Toolkit.Web%2c+Version%3d2.0.0.22%2c+Culture%3dneutral%2c+PublicKeyToken%3dnull%3aen-US%3ade23b191-cd1d-4c31-b7c2-c71e4d22ee9a%3a78e334ee%3a28617db2%3a235de37a%3a35bc484f%3a360cd5dc%3bAjaxControlToolkit%2c+Version%3d3.0.30930.28736%2c+Culture%3dneutral%2c+PublicKeyToken%3d28f01b0e84b6d53e%3aen-US%3ab0eefc76-0092-471b-ab62-f3ddc8240d71%3a91bd373d%3bEMG.Toolkit.Web%2c+Version%3d2.0.0.22%2c+Culture%3dneutral%2c+PublicKeyToken%3dnull%3aen-US%3ade23b191-cd1d-4c31-b7c2-c71e4d22ee9a%3a312433fe%3aa74d42b0" type="text/javascript"></script>
<script type="text/javascript">
//<![CDATA[
Sys.Application.add_init(function() {
    $create(EMG.Toolkit.Web.TableSorterBehavior, {"debug":true,"headers":"{0: { sorter: false}, 3: {sorter: false}}","id":"_jqueryPlgn","sortList":"[0,1], [1,1]","widgetZebra":"[\u0027zebra\u0027]"}, null, null, $get("tmpJQueryTarget"));
});
//]]>
</script>
</form><script type='text/javascript'>if(document.getElementById('carryOverHeadlines')){document.getElementById('carryOverHeadlines').style.display = 'block';}$(document).ready(function(){setTimeout(function(){window.print();}, 1000);});</script><script type='text/javascript'>framesViewNotReqd = false;modalEnabled = true;RequestFromModal=false;RequestFromIPad=false;SnapshotBaseUrl='https://Snapshot.factiva.com';</script>
</body>
</html>
"""

Next the author saved the contents of html_code into a .htm file.

In [7]:
# Write the HTML code to the file
with open(f"{path}/factiva/factiva.htm", 'w') as file: #replace with your path
    file.write(html_code)


# # For a list of variables with HTML content
# html_list = [html_code1, html_code2, html_code3]  # Replace with your actual variables

# # Iterate over the list and write each HTML content to a separate file
# for i, html_code in enumerate(html_list):
#     file_path = f"/content/drive/MyDrive/Factiva/factiva_{i}.htm"  # Replace with your path
#     with open(file_path, 'w') as file:
#         file.write(html_code)

Then the author used the glob.glob function to find all the .htm files and save them as the variable “files”.

In [8]:
files = glob.glob(f"{path}/factiva/*.htm", recursive = True) #replace with your path

In [ ]:
files

Next the author used a for loop create an empty list then filled it with any tables found in the HTML flies. The extend method merges these dataframes into empty_list so that it contains all the dataframes in one flat structure.

In [10]:
empty_list = []
for file in files:
    data = pd.read_html(file, index_col = 0) #reads the HTML content of the file and tries to find any tables inside it.
    empty_list.extend(data) # The extend() method is used to add the data (which is a list of dataframes) from the current file to empty_list.

In [ ]:
empty_list

The next for loop turns the data in empty_list into a single dataframe transposing the data so that each row of data is divided into the standard column headers provided by the html data (i.e. HD for headline, BY for author, PD for published date etc.)

In [12]:
frames = pd.concat([l for l in empty_list if 'HD' in l.index.values], axis=1).T

In [ ]:
frames

In [14]:
df = frames.reset_index()

In [ ]:
frames.columns

The following code renames several column headers so that they’re easier to read.

In [16]:
frames.rename(columns = {'HD': 'Headline',
                         'PD': 'Publication_Date','SN': 'Source_Name', 'LP': 'Lead Paragraph',
                          'TD': 'Body',
                         'BY':'Author_Name'}, inplace=True)

frames = frames[['Headline', 'Publication_Date', 'Source_Name', 'Lead Paragraph', 'Body', 'Author_Name']]
#renames the columns to easier to understand column titles
#looks like it also dropped the columns that didn't get re-named

The following code ensures the Publication_Date column is  in datetime format.

In [ ]:
frames['Publication_Date'] = pd.to_datetime(frames['Publication_Date'])
frames.sort_values(by='Publication_Date', inplace=True)
#Makes sure the publication_Date column is in a format that pyton recognizes as a date

In [ ]:
frames

The following code combines the text from the Lead Paragraph and Body columns into a new column with the full text of each article.

In [ ]:
frames['CombinedText'] = frames['Lead Paragraph'] + " " + frames['Body']
#combine the text from the Lead Paragraph and the Body columns so we have the full article in a single cell.

In [ ]:
frames

This line of code counts how many times the data contain "PATH" in uppercase, and how many times it contains the word in lowercase. The data has 413 instances of PATH, and 9 instances of path, indicating it is appropriate for this study. The author generated this code by asking Gemini to count the mentions of PATH and path.

In [ ]:
import pandas as pd

# Assuming your DataFrame is named 'frames' and the column is named 'combined text'

# Count lowercase occurrences
lowercase_count = frames['CombinedText'].str.count('path').sum()

# Count uppercase occurrences
uppercase_count = frames['CombinedText'].str.count('PATH').sum()

# Print the results
print(f"Lowercase 'path' count: {lowercase_count}")
print(f"Uppercase 'PATH' count: {uppercase_count}")

The author asked Gemini to produce the following two lines of code, which count the rows (articles) that mention both PATH and path. The results show that though the original Factiva search was for articles that mention Port Authority Trans Hudson, only 75 out of 100 articles go on to mention PATH. Review of a random sample of 5 of the remaining 25 articles shows that those that don't mention PATH, do in fact mention Port Authority Trans Hudson, but only once and therefore don't go on to use the acronym. This shows further need for the ability to search for PATH specifically.

In [ ]:
# Assuming your DataFrame is named 'frames' and the column is named 'CombinedText'

# Count rows with at least one mention of "PATH"
path_count = frames[frames['CombinedText'].str.contains('PATH', na=False)].shape[0]

# Print the result
print(f"Number of rows with at least one mention of 'PATH': {path_count}")

In [ ]:
# Assuming your DataFrame is named 'frames' and the column is named 'CombinedText'

# Count rows with at least one mention of "path"
path_count = frames[frames['CombinedText'].str.contains('path', na=False)].shape[0]

# Print the result
print(f"Number of rows with at least one mention of 'path': {path_count}")

The following lines of code are adapted from [Geeks for Geeks 2021 tutorial](https://www.geeksforgeeks.org/generating-word-cloud-python/) on generating word clouds in Python, with additional input form Gemini.

In [ ]:
!pip install wordcloud==1.8.1
!pip install matplotlib==3.7.1

In [25]:
import pandas as pd
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

In [26]:
text = ' '.join(frames['CombinedText'].astype(str).tolist())
#This code converts the 'CombinedText' column to string type, converts it to a list, and
#then joins the list elements into a single string with spaces in between.

In [ ]:
!pip install Pillow==9.0.0
#solved below issue with min font size

The following two lines of code remove stopwords and generate a word cloud. The word cloud shows that many of the terms in the data are relevant to PATH, technical terms like tunnel, Amtrak, or bridge. Removing terms that are obviously related to PATH, including PATH itself, Port Authority, or place names could shed greater light on what technical terms are in the data.

However, the word cloud also shows that the data includes many terms related to funerals. This is because several of the articles are obituary pages that include an individual who once worked for PATH. These articles tend to be long, and have only one mention of PATH (only one of the many individuals honored in the obituary worked for PATH). Being able to identify PATH specifically could help researchers remove extra data from these articles.

In [32]:
stopwords = set(STOPWORDS)
wordcloud = WordCloud(width = 800, height = 800,
                background_color ='white',
                stopwords = stopwords,
                min_font_size = 10).generate(text) #this line caused an issue
                #issue fixed by downgrading pillow to v 9 (above code)

In [ ]:
plt.figure(figsize = (8, 4), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)

plt.show()

This code resets the index so it is in ascending order starting at 0.

In [30]:
df = frames.reset_index()

The following code loops through each row of the DataFrame and creates a unique text file for each based on the content in the CombinedText column. The code generates the filenames dynamically based on the row index, which the above code set in ascending order from 0.

In [31]:
for index, row in df.iterrows():
    file_name = f"{path}/factiva/text_file_{index + 1}.txt"  # Create a unique filename for each row
    with open(file_name, 'w') as file:
        text_content = str(row['CombinedText']) if pd.notnull(row['CombinedText']) else ''  # Convert to string and handle NaN
        file.write(text_content)  # Write the text content to the file

This concludes the data collection section of this project. Explanation of the data analysis and the creation of a custom spaCy model capable of recognizing lowercase instances of ‘path’ can be found in the **PATH_NER_Analysis_CustomModel_Codebook**.